# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 84 new papers today
          31 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/30 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.08667


extracting tarball to tmp_2212.08667...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.08670


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.08667/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

Retrieving document from  https://arxiv.org/e-print/2212.09675


extracting tarball to tmp_2212.09675...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.09675/OH119z6QSOsOutflows.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure figures/Cartoon
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.08702-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.08702) | **The shocked molecular layer in RCW 120**  |
|| M. S. Kirsanova, et al. -- incl., <mark>A. O. H. Olofsson</mark>, <mark>D. A. Semenov</mark> |
|*Appeared on*| *2022-12-20*|
|*Comments*| *accepted for publication in MNRAS*|
|**Abstract**| Expansion of wind-blown bubbles or HII regions lead to formation of shocks in the interstellar medium, which compress surrounding gas into dense layers. We made spatially and velocity-resolved observations of the RCW~120 PDR and nearby molecular gas with CO(6-5) and 13CO(6-5) lines and distinguished a bright CO-emitting layer, which we related with the dense shocked molecular gas moving away from the ionizing star due to expansion of HII region. Simulating gas density and temperature, as well as brightness of several CO and C+ emission lines from the PDR, we found reasonable agreement with the observed values. Analysing gas kinematics, we revealed the large-scale shocked PDR and also several dense environments of embedded protostars and outflows. We observe the shocked layer as the most regular structure in the CO(6-5) map and in the velocity space, when the gas around YSOs is dispersed by the outflows. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.09159-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.09159) | **PHANGS-JWST First Results: Destruction of the PAH molecules in HII  regions probed by JWST and MUSE**  |
|| Oleg V. Egorov, et al. -- incl., <mark>Kathryn Kreckel</mark>, <mark>Daizhong Liu</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2022-12-20*|
|*Comments*| *16 pages, 7 figures, 1 table. Accepted for publication in ApJL as part of a PHANGS-JWST First Results Focus issue*|
|**Abstract**| Polycyclic aromatic hydrocarbons (PAHs) play a critical role in the reprocessing of stellar radiation and in balancing the heating and cooling processes in the interstellar medium (ISM), but appear to be destroyed in HII regions. However, the mechanisms driving their destruction are still not completely understood. Using PHANGS-JWST and PHANGS-MUSE observations, we investigate how the PAH fraction changes in about 1500 HII regions across four nearby star-forming galaxies (NGC 628, NGC 1365, NGC 7496, IC 5332). We find a strong anti-correlation between the PAH fraction and the ionization parameter (the ratio between the ionizing photon flux and the hydrogen density) of HII regions. This relation becomes steeper for more luminous HII regions. The metallicity of HII regions has only a minor impact on these results in our galaxy sample. We find that the PAH fraction decreases with the H$\alpha$ equivalent width - a proxy for the age of the HII regions - although this trend is much weaker than the one identified using the ionization parameter. Our results are consistent with a scenario where hydrogen-ionizing UV radiation is the dominant source of PAH destruction in star-forming regions. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.09168-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.09168) | **PHANGS-JWST First Results: Rapid Evolution of Star Formation in the  Central Molecular Gas Ring of NGC1365**  |
|| <mark>Eva Schinnerer</mark>, et al. -- incl., <mark>Jonathan D. Henshaw</mark>, <mark>Daizhong Liu</mark>, <mark>Stephen Hannon</mark>, <mark>Justus Neumann</mark>, <mark>Sophia K. Stuber</mark> |
|*Appeared on*| *2022-12-20*|
|*Comments*| *24 pages, 8 figures, accepted for publications as part of PHANGS-JWST ApJL Focus Issue*|
|**Abstract**| Large-scale bars can fuel galaxy centers with molecular gas, often leading to the development of dense ring-like structures where intense star formation occurs, forming a very different environment compared to galactic disks. We pair ~0.3" (30pc) resolution new JWST/MIRI imaging with archival ALMA CO(2-1) mapping of the central ~5kpc of the nearby barred spiral galaxy NGC1365, to investigate the physical mechanisms responsible for this extreme star formation. The molecular gas morphology is resolved into two well-known bright bar lanes that surround a smooth dynamically cold gas disk (R_gal ~ 475pc) reminiscent of non-star-forming disks in early type galaxies and likely fed by gas inflow triggered by stellar feedback in the lanes. The lanes host a large number of JWST-identified massive young star clusters. We find some evidence for temporal star formation evolution along the ring. The complex kinematics in the gas lanes reveal strong streaming motions and may be consistent with convergence of gas streamlines expected there. Indeed, the extreme line-widths are found to be the result of inter-`cloud' motion between gas peaks; ScousePy decomposition reveals multiple components with line widths of <sigma_CO,scouse> ~ 19km/s and surface densities of <Sigma_H2,scouse> ~ 800M_sun/pc^2, similar to the properties observed throughout the rest of the central molecular gas structure. Tailored hydro-dynamical simulations exhibit many of the observed properties and imply that the observed structures are transient and highly time-variable. From our study of NGC1365, we conclude that it is predominantly the high gas inflow triggered by the bar that is setting the star formation in its CMZ. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.09307-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.09307) | **ALMA-IMF VII -- Investigating the origin of stellar masses: CMF  evolution in the W43-MM2&MM3 mini-starburst**  |
|| Y. Pouteau, et al. -- incl., <mark>P. Hennebelle</mark>, <mark>F. Herpin</mark>, <mark>H.-L. Liu</mark> |
|*Appeared on*| *2022-12-20*|
|*Comments*| *Accepted for publication in A&A (december, 5th 2022), 24 pages, 14 figures*|
|**Abstract**| Among the most central open questions on the initial mass function (IMF) of stars is the impact of the environment on the shape of the core mass function (CMF) and thus potentially on the IMF. The ALMA-IMF Large Program aims to investigate the variations of the core distributions with cloud characteristics, as diagnostic observables of the formation process and evolution. The present study focuses on the W43-MM2&MM3 mini-starburst, whose CMF has been recently found to be top-heavy with respect to the Salpeter slope. W43-MM2&MM3 is a useful test case for environmental studies because it harbors a rich cluster containing a statistically significant number of cores, previously characterized in paper III. We applied a multiscale decomposition technique on the ALMA 1.3 mm and 3 mm continuum images to define six subregions. For each subregion we characterize the high column density probability distribution function, n-PDF, shape of the cloud gas using the 1.3 mm image. Using the core catalog, we investigate correlations between the CMF, n-PDF and core mass segregation. We classify the subregions into different stages of evolution, from quiescent to burst to post-burst, based on the surface number density of cores, number of outflows, and UCHii presence. The high-mass end of the subregion CMFs vary from being close to the Salpeter slope (quiescent) to top-heavy (burst and post-burst). Moreover, the second tail of the n-PDF varies from steep, to flat like observed for the high mass star-forming clouds. We found that subregions with flat second n-PDF tails display top-heavy CMFs. The CMF may evolve from Salpeter to top-heavy throughout the star formation process from the quiescent to the burst phase. This scenario raises the question of if the CMF might revert again to Salpeter as the cloud approaches the end of its star formation stage, a hypothesis that remains to be tested. |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.09630-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.09630) | **Radiative Transfer in Lyα Nebulae: I. Modeling a Continuous or  Clumpy Spherical Halo with a Central Source**  |
|| Seok-Jun Chang, et al. -- incl., <mark>Hee-Won Lee</mark> |
|*Appeared on*| *2022-12-20*|
|*Comments*| *42 pages, 27 figures, accepted for publication in ApJ, Comments welcome!*|
|**Abstract**| To understand the mechanism behind high-$z$ Ly${\alpha}$ nebulae, we simulate the scattering of Ly${\alpha}$ in a $\rm H\,I$ halo about a central Ly${\alpha}$ source. For the first time, we consider both smooth and clumpy distributions of halo gas, as well as a range of outflow speeds, total $\rm H\,I$ column densities, $\rm H\,I$ spatial concentrations, and central source galaxies (e.g., with Ly${\alpha}$ line widths corresponding to those typical of AGN or star-forming galaxies). We compute the spatial-frequency diffusion and the polarization of the Ly${\alpha}$ photons scattered by atomic hydrogen. Our scattering-only model reproduces the typical size of Ly${\alpha}$ nebulae ($\sim 100\,$kpc) at total column densities $N_{\rm HI} \geq 10^{20} \rm cm^{-2}$ and predicts a range of positive, flat, and negative polarization radial gradients. We also find two general classes of Ly${\alpha}$ nebula morphologies: with and without bright cores. Cores are seen when $N_{\rm HI}$ is low, i.e., when the central source is directly visible, and are associated with a polarization jump, a steep increase in the polarization radial profile just outside the halo center. Of all the parameters tested in our smooth or clumpy medium model, $N_{\rm HI}$ dominates the trends. The radial behaviors of the Ly${\alpha}$ surface brightness, spectral line shape, and polarization in the clumpy model with covering factor $f_c \gtrsim 5$ approach those of the smooth model at the same $N_{\rm HI}$. A clumpy medium with high $N_{\rm HI}$ and low $f_c \lesssim 2$ generates Ly${\alpha}$ features via scattering that the smooth model cannot: a bright core, symmetric line profile, and polarization jump. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.09590-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.09590) | **Ultra-Low-Frequency Radio Astronomy Observations from a Selenocentric  Orbit: first results of the Longjiang-2 experiment**  |
|| Jingye Yan, et al. -- incl., <mark>Lin Wu</mark>, <mark>Li Deng</mark>, <mark>Fei Zhao</mark>, <mark>Li Zhou</mark>, <mark>Jinsheng Guo</mark> |
|*Appeared on*| *2022-12-20*|
|*Comments*| *Accepted for publication in Experimental Astronomy; 22 pages, 11 figures*|
|**Abstract**| This paper introduces the first results of observations with the Ultra-Long-Wavelength (ULW) -- Low Frequency Interferometer and Spectrometer (LFIS) on board the selenocentric satellite Longjiang-2. We present a brief description of the satellite and focus on the LFIS payload. The in-orbit commissioning confirmed a reliable operational status of the instrumentation. We also present results of a transition observation, which offers unique measurements on several novel aspects. We estimate the RFI suppression required for such a radio astronomy instrumentation at the Moon distances from Earth to be of the order of 80 dB. We analyse a method of separating Earth- and satellite-originated radio frequency interference (RFI). It is found that the RFI level at frequencies lower than a few MHz is smaller than the receiver noise floor. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.09016-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.09016) | **Multi-band Optical Variability of the TeV Blazar PG 1553+113 in 2019**  |
|| Vinit Dhiman, et al. -- incl., <mark>Beatriz Villarroel</mark>, <mark>Zhongli Zhang</mark> |
|*Appeared on*| *2022-12-20*|
|*Comments*| *13 pages,8 figures, 7 tables, Accepted for publication in MNRAS*|
|**Abstract**| We report the flux and spectral variability of PG 1553+113 on intra-night (IDV) to short-term timescales using BVRI data collected over 91 nights from 28 February to 8 November 2019 employing ten optical telescopes: three in Bulgaria, two each in India and Serbia, and one each in Greece, Georgia, and Latvia. We monitored the blazar quasi-simultaneously for 16 nights in the V and R bands and 8 nights in the V, R, I bands and examined the light curves (LCs) for intra-day flux and colour variations using two powerful tests: the power-enhanced F-test and the nested ANOVA test. The source was found to be significantly (> 99%) variable in 4 nights out of 27 in R-band, 1 out of 16 in V-band, and 1 out of 6 nights in I-band. No temporal variations in the colours were observed on IDV timescale. During the course of these observations the total variation in R-band was 0.89 mag observed. We also investigated the spectral energy distribution (SED) using B, V, R, and I band data. We found optical spectral indices in the range of 0.878+-0.029 to 1.106+-0.065 by fitting a power law to these SEDs of PG 1553+113. We found that the source follows a bluer-when-brighter trend on IDV timescales. We discuss possible physical causes of the observed spectral variability. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.08947-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.08947) | **Sulfur Ion Implantations Into Condensed CO2: Implications for Europa**  |
|| D. V. Mifsud, et al. -- incl., <mark>P. Herczku</mark> |
|*Appeared on*| *2022-12-20*|
|*Comments*| *Published in Geophysical Research Letters*|
|**Abstract**| The ubiquity of sulfur ions within the Jovian magnetosphere has led to suggestions that the implantation of these ions into the surface of Europa may lead to the formation of SO2. However, previous studies on the implantation of sulfur ions into H2O ice (the dominant species on the Europan surface) have failed to detect SO2 formation. Other studies concerned with similar implantations into CO2 ice, which is also known to exist on Europa, have offered seemingly conflicting results. In this letter, we describe the results of a study on the implantation of 290 keV S+ ions into condensed CO2 at 20 and 70 K. Our results demonstrate that SO2 is observed after implantation at 20 K, but not at the Europa-relevant temperature of 70 K. We conclude that this process is likely not a reasonable mechanism for SO2 formation on Europa, and that other mechanisms should be explored instead. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.09471-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.09471) | **Exploring the Universe via the Wide, Deep Near-infrared Imaging ESO  Public Survey SHARKS**  |
|| <mark>Helmut Dannerbauer</mark>, Aurelio Carnero, Nicholas Cross, Carlos M. Gutierrez |
|*Appeared on*| *2022-12-20*|
|*Comments*| *Published in the ESO messenger #187: this https URL*|
|**Abstract**| The ESO Public Survey Southern H-ATLAS Regions Ks-band Survey (SHARKS) comprises 300 square degrees of deep imaging at 2.2 microns (the Ks band) with the VISTA InfraRed CAMera (VIRCAM) at the 4-metre Visible and Infrared Survey Telescope for Astronomy (VISTA). The first data release of the survey, comprising 5% of the data, was published via the ESO database on 31 January 2022. We describe the strategy and status of the first data release and present the data products. We discuss briefly different scientific areas being explored with the SHARKS data and conclude with an outline of planned data releases. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.08702.md
    + _build/html/tmp_2212.08702/./fig2a.png
    + _build/html/tmp_2212.08702/./fig2b.png
    + _build/html/tmp_2212.08702/./fig1.png
    + _build/html/tmp_2212.08702/./fig5.png
exported in  _build/html/2212.09159.md
    + _build/html/tmp_2212.09159/./pah_plot_1_rev1.png
    + _build/html/tmp_2212.09159/./pah_plot_2_rev1.png
    + _build/html/tmp_2212.09159/./pah_plot_3_rev1.png
exported in  _build/html/2212.09168.md
    + _build/html/tmp_2212.09168/./figures/scouse2x2.png
    + _build/html/tmp_2212.09168/./figures/scouse2x2.png
    + _build/html/tmp_2212.09168/./figures/MIR_images_v5.png
exported in  _build/html/2212.09307.md
    + _build/html/tmp_2212.09307/./NH2_image+cores.jpg
    + _build/html/tmp_2212.09307/./MnGSeg-scales-1mm.png
    + _build/html/tmp_2212.09307/./MnGSeg-scales-3mm.png
    + _build/html/tmp_2212.09307/./fig-resume.png
exported in  _build/html/2212.09443.md
    + _build/html/tmp_2212.09443/./2017_04_16_MEAN_int_ALMA_sample_s

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\hii}{H {\sc{ii}}}$
$\newcommand{\cii}{C {\sc{ii}}}$
$\newcommand{\kms}{ km s^{-1}}$
$\newcommand{\co}{CO(6--5)}$
$\newcommand{\tco}{^{13}CO(6--5)}$
$\newcommand{\tmb}{T_{\tm mb}}$
$\newcommand{\offsets}{(\Delta \alpha, \Delta \delta)}$
$\newcommand{\innersize}{0.9~pc}$
$\newcommand{\distance}{1.3~kpc}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\hii$}{H {\sc{ii}}}$
$\newcommand{$\cii$}{C {\sc{ii}}}$
$\newcommand{$\kms$}{ km s^{-1}}$
$\newcommand{$\co$}{CO(6--5)}$
$\newcommand{$\tco$}{^{13}CO(6--5)}$
$\newcommand{$\tmb$}{T_{\tm mb}}$
$\newcommand{$\offsets$}{(\Delta \alpha, \Delta \delta)}$
$\newcommand{$\innersize$}{0.9~pc}$
$\newcommand{$\distance$}{1.3~kpc}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The shocked molecular layer in RCW 120

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.08702-b31b1b.svg)](https://arxiv.org/abs/2212.08702)<mark>Appeared on: 2022-12-20</mark> - _accepted for publication in MNRAS_

</div>
<div id="authors">

M. S. Kirsanova, et al. -- incl., <mark><mark>A. O. H. Olofsson</mark></mark>, <mark><mark>D. A. Semenov</mark></mark>

</div>
<div id="abstract">

**Abstract:** Expansion of wind-blown bubbles or$\hii$regions lead to formation of shocks in the interstellar medium, which compress surrounding gas into dense layers. We made spatially and velocity-resolved observations of the RCW 120 PDR and nearby molecular gas with$\co$and$\tco$lines and distinguished a bright CO-emitting layer, which we related with the dense shocked molecular gas moving away from the ionizing star due to expansion of$\hii$region. Simulating gas density and temperature, as well as brightness of several CO and C+ emission lines from the PDR, we found reasonable agreement with the observed values. Analysing gas kinematics, we revealed the large-scale shocked PDR and also several dense environments of embedded protostars and outflows. We observe the shocked layer as the most regular structure in the$\co$map and in the velocity space, when the gas around YSOs is dispersed by the outflows.

</div>

<div id="div_fig1">

<img src="tmp_2212.08702/./fig2a.png" alt="Fig7.1" width="50%"/><img src="tmp_2212.08702/./fig2b.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Left: map of the $\co$  peak intensity in the RCW 120 PDR shown by colour scale. Contours show brightness of the [$\cii$] line emission at 20, 30 and 40 K. White crosses show positions of the $\tco$  spectra shown in Fig. \ref{fig:individual_spectra}, where position 1 corresponds to illuminated side of the PDR. Right: map of the $\co$  integrated intensity shown by colour scale. The 870 $\rm \mu$m contours for 0.4, 2.0 and 10.0 Jy/beam are shown by contours. The APEX beam at 691 GHz is shown by black circle. The black stars show the locations of YSOs from \citet{Figueira_2017} in both plots and in following figures. (*fig:obsemission*)

</div>
<div id="div_fig2">

<img src="tmp_2212.08702/./fig1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Image of RCW 120 at 70\micron . Color scale shows emission of neutral material around the ionized gas. The position of the ionizing star of RCW 120 is marked as a red star. The 843 MHz radio continuum emission corresponding to free-free emission of the ionized gas is shown with white contours linearly spaced from from 0.1 to 0.4 Jy/beam. The 870 $\rm \mu$m contours for 0.4, 2.0 and 10.0 Jy/beam are shown in black. Black circles show the locations of compact sources described by \citet{Figueira_2017}(their Table 5). The sizes of the circles depend linearly on the source masses \citep[$M_{\rm env}$ from][]{Figueira_2017} here and on the following figures. The area, observed in the $\co$  line, is shown by the red rectangle. (*fig:obsregion*)

</div>
<div id="div_fig3">

<img src="tmp_2212.08702/./fig5.png" alt="Fig3" width="100%"/>

**Figure 3. -** Simulated (lines) and observed (colour rectangles and horizontal straight lines) properties of the RCW 120 PDR. Simulated physical conditions and chemical composition in the dense neutral layer around the $\hii$  region obtained with the MARION model are shown on the two top panels. The ionizing star is located outside the left border of the panels in the beginning of the horizontal axe. The red rectangles and the red lines show the $\pm \sigma$ significance intervals and the best-fit solution of the non-LTE analysis with the observed and archival data (see Sec. \ref{sec:analysis}). Simulated brightness of the $\cii$, CO and $^{13}$CO emission in different lines obtained with the RADEX software is shown in two bottom panels. Green and black horizontal colour bold lines and rectangles show the brightness and $\pm \sigma$ intervals for the archival $^{13}$CO(2--1) and (3--2) lines. The observed range of $\co$  and $\tco$  brightness are shown by the red rectangle. The $\tco$  brightness used for simulations with RADEX is shown by the red horizontal line. (*fig:radexmodel*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\OSU}{\label{OSU} Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{\Alberta}{\label{Alberta} Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\newcommand{\ANU}{\label{ANU} Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\IPAC}{\label{IPAC} Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}$
$\newcommand{\Carnegie}{\label{Carnegi} Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}$
$\newcommand{\CCAPP}{\label{CCAPP} Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\CfA}{\label{CfA} Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\CITEVA}{\label{CITEVA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CNRS}{\label{CNRS} CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}$
$\newcommand{\ESO}{\label{ESO} European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei M\"{u}nchen, Germany}$
$\newcommand{\HD}{\label{HD} Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\ICRAR}{\label{ICRAR} International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}$
$\newcommand{\IRAM}{\label{IRAM} Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}$
$\newcommand{\ITA}{\label{ITA} Universit\"{a}t Heidelberg, Zentrum f\"{u}r Astronomie, Institut f\"{u}r Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\IWR}{\label{IWR} Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\JHU}{\label{JHU} Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\Leiden}{\label{Leiden} Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}$
$\newcommand{\Maryland}{\label{Maryland} Department of Astronomy, University of Maryland, College Park, MD 20742, USA}$
$\newcommand{\MPE}{\label{MPE} Max-Planck-Institut f\"{u}r extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}$
$\newcommand{\MPIA}{\label{MPIA} Max-Planck-Institut f\"{u}r Astronomie, K\"{o}nigstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\Nagoya}{\label{Nagoya} Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}$
$\newcommand{\NRAO}{\label{NRAO} National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}$
$\newcommand{\OAN}{\label{OAN} Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\ObsParis}{\label{ObsParis} Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}$
$\newcommand{\Oxford}{\label{Oxford} Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\Princeton}{\label{Princeton} Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}$
$\newcommand{\UToledo}{\label{UToledo} University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}$
$\newcommand{\Toulouse}{\label{Toulouse} Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}$
$\newcommand{\UBonn}{\label{UBonn} Argelander-Institut f\"ur Astronomie, Universit\"at Bonn, Auf dem H\"ugel 71, 53121 Bonn, Germany}$
$\newcommand{\UChile}{\label{UChile} Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}$
$\newcommand{\UConn}{\label{UConn} Department of Physics, University of Connecticut, Storrs, CT, 06269, USA}$
$\newcommand{\UGent}{\label{UGent} Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\ULyon}{\label{ULyon} Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\F-69230 Saint-Genis-Laval, France}$
$\newcommand{\UMass}{\label{UMass} University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}$
$\newcommand{\UWyoming}{\label{UWyoming} Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LAM}{\label{LAM} Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille), Marseille, France}$
$\newcommand{\UHawaii}{\label{UHawaii} Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UCM}{\label{UCM} Departamento de F\'{\i}sica de la Tierra y Astrof\'{\i}sica, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\IPARC}{\label{IPARC} Instituto de F\'{\i}sica de Part\'{\i}culas y del Cosmos IPARCOS, Facultad de Ciencias F\'{\i}sicas, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\STScI}{\label{STScI} Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\STScIESA}{\label{STScIESA} AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\McMaster}{\label{McMaster} Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{\INAF}{\label{INAF} INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Florence, Italy}$
$\newcommand{\Sydney}{\label{Sydney} Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}$
$\newcommand{\UA}{\label{UA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CITA}{\label{CITA} Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George St, Toronto, ON M5S 3H8, Canada}$
$\newcommand{\ASIAA}{\label{ASIAA} Institute of Astronomy and Astrophysics, Academia Sinica, No. 1, Sec. 4, Roosevelt Road, Taipei 10617, Taiwan}$
$\newcommand{\TKU}{\label{TKU} Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}$
$\newcommand{\PSMA}{\label{PSMA} Penn State Mont Alto, 1 Campus Drive, Mont Alto, PA  17237, USA}$
$\newcommand{\eso}{\label{ESO} European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\newcommand{\ILL}{\label{ILL} ILL}$
$\newcommand{\stromlo}{\label{stromlo} Research School of Astronomy and Astrophysics, Australian National University, Mt Stromlo Observatory, Weston Creek, ACT 2611, Australia}$
$\newcommand{\UCSD}{\label{UCSD} Center for Astrophysics \& Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, San Diego, CA 92093, USA}$
$\newcommand{\USorbonne}{\label{Usorbonne} Sorbonne Universit\'e, Observatoire de Paris, Universit\'e PSL, \' Ecole normale sup\'erieure, CNRS, LERMA, F-75005, Paris, France}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\sigmaHa}{\sigma(\mathrm{H\alpha})}$
$\newcommand{\Ha}{\mathrm{H\alpha}}$
$\newcommand{\Hb}{\mathrm{H\beta}}$
$\newcommand{\SIIHa}{[S~\textsc{ii}]/\mathrm{H\alpha}}$
$\newcommand{\NIIHa}{[N~\textsc{ii}]/\mathrm{H\alpha}}$
$\newcommand{\OIIIHb}{[O~\textsc{iii}]/\mathrm{H\beta}}$
$\newcommand{\OIII}{[O~\textsc{iii}]}$
$\newcommand{\SII}{[S~\textsc{ii}]}$
$\newcommand{\SIII}{[S~\textsc{iii}]}$
$\newcommand{\SIIISII}{[S~\textsc{iii}]/[S~\textsc{ii}]}$
$\newcommand{\HII}{H~\textsc{ii}}$
$\newcommand{\HI}{H~\textsc{i}}$
$\newcommand{\kms}{\rm km s^{-1}}$
$\newcommand{\JWST}{\rm{\it JWST}}$
$\newcommand{\Herschel}{\rm{\it Herschel}}$
$\newcommand{\Spitzer}{\rm{\it Spitzer}}$
$\newcommand{\HST}{\rm{\it HST}}$
$\newcommand{\qPAH}{q_\mathrm{PAH}}$
$\newcommand{\nreg}{1529}$
$\newcommand{\red}{\textcolor{red}}$
$\newcommand{\revone}{\}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\OSU$}{\label{OSU} Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{$\Alberta$}{\label{Alberta} Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\newcommand{$\ANU$}{\label{ANU} Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{$\IPAC$}{\label{IPAC} Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}$
$\newcommand{$\Carnegie$}{\label{Carnegi} Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}$
$\newcommand{$\CCAPP$}{\label{CCAPP} Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{$\CfA$}{\label{CfA} Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{$\CITEVA$}{\label{CITEVA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{$\CNRS$}{\label{CNRS} CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}$
$\newcommand{$\ESO$}{\label{ESO} European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei M\"{u}nchen, Germany}$
$\newcommand{$\HD$}{\label{HD} Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{$\ICRAR$}{\label{ICRAR} International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}$
$\newcommand{$\IRAM$}{\label{IRAM} Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}$
$\newcommand{$\ITA$}{\label{ITA} Universit\"{a}t Heidelberg, Zentrum f\"{u}r Astronomie, Institut f\"{u}r Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{$\IWR$}{\label{IWR} Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{$\JHU$}{\label{JHU} Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{$\Leiden$}{\label{Leiden} Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}$
$\newcommand{$\Maryland$}{\label{Maryland} Department of Astronomy, University of Maryland, College Park, MD 20742, USA}$
$\newcommand{$\MPE$}{\label{MPE} Max-Planck-Institut f\"{u}r extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}$
$\newcommand{$\MPIA$}{\label{MPIA} Max-Planck-Institut f\"{u}r Astronomie, K\"{o}nigstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{$\Nagoya$}{\label{Nagoya} Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}$
$\newcommand{$\NRAO$}{\label{NRAO} National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}$
$\newcommand{$\OAN$}{\label{OAN} Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{$\ObsParis$}{\label{ObsParis} Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}$
$\newcommand{$\Oxford$}{\label{Oxford} Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{$\Princeton$}{\label{Princeton} Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}$
$\newcommand{$\UToledo$}{\label{UToledo} University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}$
$\newcommand{$\Toulouse$}{\label{Toulouse} Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}$
$\newcommand{$\UBonn$}{\label{UBonn} Argelander-Institut f\"ur Astronomie, Universit\"at Bonn, Auf dem H\"ugel 71, 53121 Bonn, Germany}$
$\newcommand{$\UChile$}{\label{UChile} Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}$
$\newcommand{$\UConn$}{\label{UConn} Department of Physics, University of Connecticut, Storrs, CT, 06269, USA}$
$\newcommand{$\UGent$}{\label{UGent} Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{$\ULyon$}{\label{ULyon} Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\F-69230 Saint-Genis-Laval, France}$
$\newcommand{$\UMass$}{\label{UMass} University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}$
$\newcommand{$\UWyoming$}{\label{UWyoming} Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{$\LAM$}{\label{LAM} Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille), Marseille, France}$
$\newcommand{$\UHawaii$}{\label{UHawaii} Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{$\UCM$}{\label{UCM} Departamento de F\'{\i}sica de la Tierra y Astrof\'{\i}sica, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{$\IPARC$}{\label{IPARC} Instituto de F\'{\i}sica de Part\'{\i}culas y del Cosmos IPARCOS, Facultad de Ciencias F\'{\i}sicas, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{$\STScI$}{\label{STScI} Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{$\STScI$ESA}{\label{STScIESA} AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{$\McMaster$}{\label{McMaster} Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{$\INAF$}{\label{INAF} INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Florence, Italy}$
$\newcommand{$\Sydney$}{\label{Sydney} Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}$
$\newcommand{$\UA$}{\label{UA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{$\CITA$}{\label{CITA} Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George St, Toronto, ON M5S 3H8, Canada}$
$\newcommand{$\ASIAA$}{\label{ASIAA} Institute of Astronomy and Astrophysics, Academia Sinica, No. 1, Sec. 4, Roosevelt Road, Taipei 10617, Taiwan}$
$\newcommand{$\TKU$}{\label{TKU} Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}$
$\newcommand{$\PSMA$}{\label{PSMA} Penn State Mont Alto, 1 Campus Drive, Mont Alto, PA  17237, USA}$
$\newcommand{$\eso$}{\label{ESO} European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\newcommand{$\ILL$}{\label{ILL} ILL}$
$\newcommand{$\stromlo$}{\label{stromlo} Research School of Astronomy and Astrophysics, Australian National University, Mt Stromlo Observatory, Weston Creek, ACT 2611, Australia}$
$\newcommand{$\UCSD$}{\label{UCSD} Center for Astrophysics \& Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, San Diego, CA 92093, USA}$
$\newcommand{$\USorbonne$}{\label{Usorbonne} Sorbonne Universit\'e, Observatoire de Paris, Universit\'e PSL, \' Ecole normale sup\'erieure, CNRS, LERMA, F-75005, Paris, France}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\sigmaHa$}{\sigma(\mathrm{H\alpha})}$
$\newcommand{$\Ha$}{\mathrm{H\alpha}}$
$\newcommand{$\Hb$}{\mathrm{H\beta}}$
$\newcommand{$\SIIHa$}{[S~$\textsc${ii}]/\mathrm{H\alpha}}$
$\newcommand{$\NIIHa$}{[N~$\textsc${ii}]/\mathrm{H\alpha}}$
$\newcommand{$\OIIIHb$}{[O~$\textsc${iii}]/\mathrm{H\beta}}$
$\newcommand{$\OIII$}{[O~$\textsc${iii}]}$
$\newcommand{$\SII$}{[S~$\textsc${ii}]}$
$\newcommand{$\SII$I}{[S~$\textsc${iii}]}$
$\newcommand{$\SII$ISII}{[S~$\textsc${iii}]/[S~$\textsc${ii}]}$
$\newcommand{$\HII$}{H~$\textsc${ii}}$
$\newcommand{$\HI$}{H~$\textsc${i}}$
$\newcommand{$\kms$}{\rm km s^{-1}}$
$\newcommand{$\JWST$}{\rm{\it JWST}}$
$\newcommand{$\Herschel$}{\rm{\it Herschel}}$
$\newcommand{$\Spitzer$}{\rm{\it Spitzer}}$
$\newcommand{$\HST$}{\rm{\it HST}}$
$\newcommand{$\qPAH$}{q_\mathrm{PAH}}$
$\newcommand{$\nreg$}{1529}$
$\newcommand{$\red$}{\textcolor{red}}$
$\newcommand{\revone}{\}$</div>



<div id="title">

# PHANGS-JWST First Results: Destruction of the PAH molecules in HII regions probed by JWST and MUSE

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.09159-b31b1b.svg)](https://arxiv.org/abs/2212.09159)<mark>Appeared on: 2022-12-20</mark> - _16 pages, 7 figures, 1 table. Accepted for publication in ApJL as part of a PHANGS-JWST First Results Focus issue_

</div>
<div id="authors">

Oleg V. Egorov, et al. -- incl., <mark>Kathryn Kreckel</mark>, <mark>Daizhong Liu</mark>, <mark>Eva Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Polycyclic aromatic hydrocarbons (PAHs) play a critical role in the reprocessing of stellar radiation and in balancing the heating and cooling processes in the interstellar medium (ISM), but appear to be destroyed in$\HII$regions.However, the mechanisms driving their destruction are still not completely understood. Using  PHANGS-JWST and PHANGS-MUSE observations,we investigate how the PAH fraction changes in about 1500$\HII$regions across four nearby star-forming galaxies (NGC 628, NGC 1365, NGC 7496, IC 5332). We find a strong anti-correlation between the PAH fraction and the ionization parameter (the ratio between the ionizing photon flux and the hydrogen density)  of$\HII$regions. This relation becomes steeper for more luminous$\HII$regions. The metallicity of$\HII$regions has only a minor impact on these results in our galaxy sample. We find that the PAH fraction decreases with the$\Ha$equivalent width -- a proxy for the age of the$\HII$regions -- although this trend is much weaker than the one identified using the ionization parameter. Our results are consistent with a scenario where hydrogen-ionizing UV radiation is the dominant source of PAH destruction in star-forming regions.

</div>

<div id="div_fig1">

<img src="tmp_2212.09159/./pah_plot_1_rev1.png" alt="Fig2" width="100%"/>

**Figure 2. -** Correlation of PAH abundance, traced by $R_{\rm PAH}$, with  ionization parameter, traced by $\log$($\SII$ISII) (Panels a, b), or ionization fraction $f_{\rm ion}$(Panels c, d). Color encodes the oxygen abundance $12+\log(\mathrm{O/H})$(Panel a), ratio of F1130W/F770W -- a proxy for the ionization state of PAHs (Panel b), number of hydrogen ionizing photons $Q^0$(Panel c), and effective radii of the $\HII$ regions relative to the PSF of the MUSE data (Panel d). Dashed lines show a linear fit for the $\HII$ regions (Eq. \ref{eq:corr1} for Panels a,b). Faint ($Q^0 < 10^{50} {\rm s}^{-1}$; green dashed line, Eq. \ref{eq:corr_fnt}) and bright ($Q^0 > 10^{50} {\rm s}^{-1}$; magenta dashed line, Eq. \ref{eq:corr_brt}) $\HII$ regions are considered separately on Panels c and d. The Spearman's correlation coefficient ($\rho$) is given in the corner of each plot. \revone{Coefficients $\rho_1$ and $\rho_2$ are calculated separately for the black and cyan points (Panels b and d), or for the faint and bright regions (Panel c), respectively. The errors on $\rho$ are derived from 1000 Monte Carlo realization with the data randomly distributed around the measured values within the uncertainties.} (*fig:Rpah_vs_ip*)

</div>
<div id="div_fig2">

<img src="tmp_2212.09159/./pah_plot_2_rev1.png" alt="Fig3" width="100%"/>

**Figure 3. -** Secondary dependence of the PAH fraction on: the oxygen abundance of $\HII$ regions, measured as $12+\log\mathrm{(O/H)}$(Panel a); the F1130W/F770W flux ratio, sensitive to the relative fraction of neutral to ionized PAH (Panel b); the number of hydrogen ionizing photons $Q^0$(Panel c); the equivalent width of $\Ha$, corrected for the contribution of the underlying old stellar population (Panel d). The 2D histograms on each panel show the distribution of  $\Delta R_{\rm PAH}$, obtained after subtracting of the values derived by Eq. \ref{eq:corr1} from the measured $R_{\rm PAH}$. The red dashed line shows a zero-difference level. The light-blue dashed line on Panel (a) corresponds to a linear fit defined by Eq. \ref{eq:corr_secondary}, tracing the secondary relation of $R_{\rm PAH}$ on metallicity.  The Spearman's correlation coefficient ($\rho$) is given in the corner of each plot. \revone{Its uncertainty is derived in the same way as in Fig. \ref{fig:Rpah_vs_ip}. The error bars in the right-hand corner correspond to the 95th-percentile uncertainties of the measured values.} (*fig:Rpah_secondary*)

</div>
<div id="div_fig3">

<img src="tmp_2212.09159/./pah_plot_3_rev1.png" alt="Fig4" width="100%"/>

**Figure 4. -** Correlation of the PAH fraction, traced by $R_{\rm PAH}$, with different physical conditions. Panel a: correlation with oxygen abundance, $12+\log(\mathrm{O/H})$, with red symbols showing the mean value for each galaxy.
    Panel b: equivalent width of $\Ha$, $EW(\mathrm{H}\alpha)$, corrected for the contribution of the underlying old stellar population, and a tracer of $\HII$ region age. Note that because of the large uncertainty introduced by this background continuum subtraction, we consider here only regions with relative error $\delta(EW(\mathrm{H}\alpha)) < 10\%$.
    Panel c: $\log$($\OIIIHb$), a tracer of the hardness of the UV radiation. \revone{The Spearman's correlation coefficient ($\rho$) is given in the corner of each plot. Its uncertainty is derived in the same way as in Fig. \ref{fig:Rpah_vs_ip}. The error bars in the right-hand corner correspond to the 95th-percentile uncertainties of the measured values.} (*fig:Rpah_other_corr*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\comm}[1]{{\color{red} \textbf{#1}}}$
$\newcommand{\Msun}{\mathrm{M}_{\sun}}$
$\newcommand{\fr}[1]{{\color{green} \textbf{#1}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\comm}[1]{{\color{red} \textbf{#1}}}$
$\newcommand{\Msun}{\mathrm{M}_{\sun}}$
$\newcommand{\fr}[1]{{\color{green} \textbf{#1}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\comm$}[1]{{\color{red} \textbf{#1}}}$
$\newcommand{$\Msun$}{\mathrm{M}_{\sun}}$
$\newcommand{$\fr$}[1]{{\color{green} \textbf{#1}}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\comm$}[1]{{\color{red} \textbf{#1}}}$
$\newcommand{$\Msun$}{\mathrm{M}_{\sun}}$
$\newcommand{$\fr$}[1]{{\color{green} \textbf{#1}}}$</div>



<div id="title">

# PHANGS-JWST First Results: Rapid Evolution of Star Formation in the Central Molecular Gas Ring of NGC 1365

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.09168-b31b1b.svg)](https://arxiv.org/abs/2212.09168)<mark>Appeared on: 2022-12-20</mark> - _24 pages, 8 figures, accepted for publications as part of PHANGS-JWST ApJL Focus Issue_

</div>
<div id="authors">

<mark><mark>Eva Schinnerer</mark></mark>, et al. -- incl., <mark><mark>Jonathan D. Henshaw</mark></mark>, <mark><mark>Daizhong Liu</mark></mark>, <mark><mark>Stephen Hannon</mark></mark>, <mark><mark>Justus Neumann</mark></mark>, <mark><mark>Sophia K. Stuber</mark></mark>

</div>
<div id="abstract">

**Abstract:** Large-scale bars can fuel galaxy centers with molecular gas, often leading to the development of dense ring-like structures where intense star formation occurs, forming a very different environment compared to galactic disks. We pair$\sim$0.3$\arcsec$(30 pc) resolution new JWST/MIRI imaging with archival ALMA CO(2-1) mapping of the central$\sim$5 kpc of the nearby barred spiral galaxy NGC 1365, to investigate the physical mechanisms responsible for this extreme star formation.The molecular gas morphology is resolved into two well-known bright bar lanes that surround a smooth dynamically cold gas disk ($\rm R_{gal} \sim 475 pc$) reminiscent of non-star-forming disks in early type galaxies and likely fed by gas inflow triggered by stellar feedback in the lanes. The lanes host a large number of JWST-identified massive young star clusters. We find some evidence for temporal star formation evolution along the ring.The complex kinematics in the gas lanes reveal strong streaming motions and may be consistent with convergence of gas streamlines expected there. Indeed, the extreme line-widths are found to be the result of inter-`cloud' motion between gas peaks;{\sc ScousePy}decomposition reveals multiple components with line widths of$\rm \langle\sigma_{CO,scouse}\rangle \approx 19 km s^{-1}$and surface densities of$\rm \langle \Sigma_{H_2,scouse}\rangle \approx  800 M_{\odot} pc^{-2}$, similar to the properties observed throughout the rest of the central molecular gas structure.  Tailored hydro-dynamical simulations exhibit many of the observed properties and imply that the observed structures are transient and highly time-variable.  From our study of NGC 1365, we conclude that it is predominantly the high gas inflow triggered by the bar that is setting the star formation in its CMZ.

</div>

<div id="div_fig1">

<img src="tmp_2212.09168/./figures/scouse2x2.png" alt="Fig4" width="100%"/>

**Figure 4. -** {\sc ScousePy} decomposition of the 30 pc resolution molecular CO(2-1) gas emission. Multiple Gaussian components are needed to fit $\sim31\%$ of the data. This is demonstrated most clearly in the spiral arms, where two and three component models are common (up to six components in the southwestern arm; \textit{top left}). Spectra from selected regions with the resulting {\sc ScousePy} fits typically exhibit well separated line components (\textit{top right}). The inferred peak CO temperature and velocity dispersions of the individual {\sc ScousePy} components (\textit{bottom left}) occupy a similar region of parameter space independent of whether they are derived from single or multiple components fits (color-coding). The grey shaded regions here correspond to the limits of our fitting (a signal-to-noise ratio of 2 and a single channel width).
    The three-dimensional representation of the individual {\sc ScousePy} components located within the white dashed box in the top left panel, displayed in $ppv$ space (\textit{bottom right}), reveals an inner smoothly rotating disk (highlighted in black; extracted from the ellipse in the top left panel) while the bar lanes show strong local fluctuations in velocity indicating an increasing complexity of the kinematics in the molecular gas there (the color here refers to the peak CO temperature of the individual components). The physical spacing of the $pp$ grid in this image is $\sim475$ pc. (*fig:scouse*)

</div>
<div id="div_fig2">

<img src="tmp_2212.09168/./figures/scouse2x2.png" alt="Fig12" width="100%"/>

**Figure 12. -** {\sc ScousePy} decomposition of the 30 pc resolution molecular CO(2-1) gas emission. Multiple Gaussian components are needed to fit $\sim31\%$ of the data. This is demonstrated most clearly in the spiral arms, where two and three component models are common (up to six components in the southwestern arm; \textit{top left}). Spectra from selected regions with the resulting {\sc ScousePy} fits typically exhibit well separated line components (\textit{top right}). The inferred peak CO temperature and velocity dispersions of the individual {\sc ScousePy} components (\textit{bottom left}) occupy a similar region of parameter space independent of whether they are derived from single or multiple components fits (color-coding). The grey shaded regions here correspond to the limits of our fitting (a signal-to-noise ratio of 2 and a single channel width).
    The three-dimensional representation of the individual {\sc ScousePy} components located within the white dashed box in the top left panel, displayed in $ppv$ space (\textit{bottom right}), reveals an inner smoothly rotating disk (highlighted in black; extracted from the ellipse in the top left panel) while the bar lanes show strong local fluctuations in velocity indicating an increasing complexity of the kinematics in the molecular gas there (the color here refers to the peak CO temperature of the individual components). The physical spacing of the $pp$ grid in this image is $\sim475$ pc. (*fig:scouse*)

</div>
<div id="div_fig3">

<img src="tmp_2212.09168/./figures/MIR_images_v5.png" alt="Fig3" width="100%"/>

**Figure 3. -** Distribution of the emission from PAHs and hot dust emission as probed by JWST/MIRI observations at $\sim$0.3$\arcsec$ resolution, which is comparable to the resolution of the ALMA CO(2-1) data (North is up, East to the left.).
    \textit{Left:} The F770W filter with a $\sim$0.24$\arcsec$ resolution reveals the overall distribution of the neutral ISM as its emission is dominated by the  7.7$\mu$m  PAH feature. For reference the distribution of the CO emission is shown by a single blue contour. Note that the central position is significantly affected by saturation due to the bright AGN and dominated by its PSF causing the artefacts. Also the three brightest compact sources north of the AGN are saturated.
    \textit{Right:} Several embedded young star-forming clusters are evident in the 10 $\mu$m emission and stand out as bright compact sources. Our 10 $\mu$m-selected sources are highlighted by the cyan diamond symbols and are only found in the region shown. About 57\% of these objects coincide with the young ($<$10 Myr) star clusters studied by \citet[][shown as green crosses]{WHITMORE_PHANGSJWST} and have estimated stellar masses of $\rm 10^6  M_{\odot}$ or higher. The 21 $\mu$m-selected sources from \cite{HASSANI_PHANGSJWST} that are classified as ISM emitting sources are shown as cyan circles. The dearth of clusters in the bar lane southwest of the nucleus is notable. (*fig:jwst*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\Msol}{\ensuremath{M_{\odot}}}$
$\newcommand{\Lsol}{\ensuremath{L_{\odot}}}$
$\newcommand{\kms}{\ensuremath{\rm km s^{-1} }}$
$\newcommand{\hii}{H\mbox{\sc ii} }$
$\newcommand{\bsens}{\texttt{bsens}\xspace}$
$\newcommand{\cleanest}{\texttt{cleanest}\xspace}$
$\newcommand{\denoised}{\texttt{denoised}\xspace}$
$\newcommand{\original}{\texttt{denoised}\xspace}$
$\newcommand{\Av}{A_{\rm v}}$
$\newcommand{\NHtwo}{N_{\rm H_2}}$
$\newcommand{\NHtwoUnits}{\times 10^{21} cm^{-2}}$
$\newcommand{\st}{^{\star}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\}{mean}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\Msol$}{$\ensuremath${M_{\odot}}}$
$\newcommand{$\Lsol$}{$\ensuremath${L_{\odot}}}$
$\newcommand{$\kms$}{$\ensuremath${\rm km s^{-1} }}$
$\newcommand{$\hii$}{H\mbox{\sc ii} }$
$\newcommand{$\bsens$}{\texttt{bsens}$\xspace$}$
$\newcommand{$\cleanest$}{\texttt{cleanest}$\xspace$}$
$\newcommand{$\denoised$}{\texttt{denoised}$\xspace$}$
$\newcommand{$\original$}{\texttt{denoised}$\xspace$}$
$\newcommand{$\Av$}{A_{\rm v}}$
$\newcommand{$\NHtwo$}{N_{\rm H_2}}$
$\newcommand{$\NHtwo$Units}{\times 10^{21} cm^{-2}}$
$\newcommand{$\st$}{^{$\st$ar}}$
$\newcommand{$\arraystretch$}{1.5}$
$\newcommand{$\arraystretch$}{1.2}$
$\newcommand{\}{mean}$</div>



<div id="title">

# MM3 mini-starburst

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.09307-b31b1b.svg)](https://arxiv.org/abs/2212.09307)<mark>Appeared on: 2022-12-20</mark> - _Accepted for publication in A&A (december, 5th 2022), 24 pages, 14 figures_

</div>
<div id="authors">

Y. Pouteau, et al. -- incl., <mark><mark>P. Hennebelle</mark></mark>, <mark><mark>F. Herpin</mark></mark>, <mark><mark>H.-L. Liu</mark></mark>

</div>
<div id="abstract">

**Abstract:** Among the most central open questions on the initial mass function (IMF) of stars is the impact of the environment on the shape of the core mass function (CMF) and thus potentially on the IMF.The ALMA-IMF Large Program aims to investigate the variations of the core distributions (CMF and mass segregation) with cloud characteristics such as the density and kinematic of the gas, as diagnostic observables of the formation process and evolution. The present study focuses on the W43-MM2\&MM3 mini-starburst, whose CMF has been recently found to be top-heavy with respect to the Salpeter slope of the canonical IMF.W43-MM2\&MM3 is a useful test case for environmental studies because it harbors a rich cluster containing a statistically significant number of cores (specifically, 205 cores), previously characterized in paper III. We applied a multiscale decomposition technique on the ALMA 1.3 mm and 3 mm continuum images of W43-MM2\&MM3 to define six subregions each of 0.5-1 pc in size. For each subregion we characterize the high column density probability distribution function,$\eta$-PDF, shape of the cloud gas using the 1.3 mm image. Using the core catalog, we investigate correlations between the CMF and, cloud and core properties such as the$\eta$-PDF and core mass segregation.We classify the W43-MM2\&MM3 subregions into different stages of evolution, from quiescent to burst to post-burst, based on the surface number density of cores, number of outflows, and UC$\hii$presence. The high-mass end ($>$1$\Msol$) of the subregion CMFs vary from being close to the Salpeter slope (quiescent) to top-heavy (burst and post-burst). Moreover, the second tail of the$\eta$-PDF varies from steep (quiescent), to flat (burst and post-burst) like observed for the high mass star-forming clouds. We found that subregions with flat second$\eta$-PDF tails display top-heavy CMFs.In dynamical environments such as W43-MM2\&MM3, the high-mass end of the CMF appears to be rooted in the cloud structure at high (column) densities that surrounds cores. This connection stems from the fact that cores and their immediate surroundings are both determined and shaped by cloud formation process, current evolutionary state of the cloud, and broader the star formation history. The CMF may evolve from Salpeter to top-heavy throughout the star formation process from the quiescent to the burst phase. This scenario raises the question of if the CMF might revert again to Salpeter as the cloud approaches the end of its star formation stage, a hypothesis that remains to be tested.

</div>

<div id="div_fig1">

<img src="tmp_2212.09307/./NH2_image+cores.jpg" alt="Fig2" width="100%"/>

**Figure 2. -** The W43-MM2\&MM3 mini-starburst ridge traced by its column density (color scale), derived from the ALMA 1.3 mm continuum image of \cite{pouteau2022}. It hosts a rich cluster of 205 cores (gray crosses) reported by \cite{pouteau2022} and the ultra-compact $\hii$(UC$\hii$) region of W43-MM3, traced by its H41$\alpha$ emission line (Galv\'an-Madrid et al. in prep., gray scale) and where the column density is not computed. Green polygons outline the W43-MM2\&MM3 subregions defined in Sect. \ref{sect:subregions def}. The ellipse in the lower left corner indicates the $0.46$\arcsec$$ angular resolution of the column density image while the scale bar indicates the length scale at a distance of 5.5 kpc. \textit{a)} White dashed boxes outline the zooms-ins of \textit{b--c}. (*fig:nh2 and cores*)

</div>
<div id="div_fig2">

<img src="tmp_2212.09307/./MnGSeg-scales-1mm.png" alt="Fig4.1" width="50%"/><img src="tmp_2212.09307/./MnGSeg-scales-3mm.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Subregions of $0.5-1$ pc sizes identified in the W43-MM2\&MM3 ridge from the ALMA 12 m array $\denoised$$\bsens$ continuum images of \cite{pouteau2022} at 1.3 mm (\textit{panel a}) and 3 mm (\textit{panel b}).
A multi-scale decomposition was performed, decomposition scales of $\sim$4$-7$\arcsec$$ were chosen and subregion outlines (green contours) were set to follow inflection points in the column density distribution. Subregions are labelled MM2, MM3, MM10, MM12 and MM51 following the names of fragments extracted from $\sim$10$\arcsec$ resolution (sub)millimeter images \citep{motte2003}. The remaining subregion at 1.3 mm is labelled Outskirts. The white contour outlines the 1.3 mm coverage in panel \textit{(b)} and a 1-pc scale bar is shown in the lower-left corner of each image. (*fig:mngseg scales*)

</div>
<div id="div_fig3">

<img src="tmp_2212.09307/./fig-resume.png" alt="Fig7" width="100%"/>

**Figure 7. -** Schematic evolutionary diagram of subregions in dynamical star-forming clouds, which are qualified as either quiescent or by following the different phases before, during, and after their main star formation event: pre-burst, main burst and post-burst. Their characteristics, including cloud column density $\eta$-PDF, CMF, and core mass segregation are illustrated. \textit{Top panels:} Subregions in quiescent or pre-burst regimes present a $\eta$-PDF tail close to that found in low-mass star-forming regions and a CMF high-mass end close to the Salpeter slope. \textit{Central panels:} Subregions in their main burst regime harbor an enhanced star-formation activity and strong core mass segregation. They display an $\eta$-PDF with a much flatter second tail as well as a top-heavy CMF. \textit{Bottom panels:} Post-burst subregions, under the influence of stellar feedback, have less sustained star formation activity and more complex core mass segregation. They also present an $\eta$-PDF with a flat second tail, and a CMF which is both top-heavy and bottom-light. The location on the time sequence of the W43-MM2\&MM3 subregions are given to summarize the result of present article. (*fig:summary*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# Differences in physical properties of coronal bright points and their ALMA counterparts within and outside coronal holes

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.09443-b31b1b.svg)](https://arxiv.org/abs/2212.09443)<mark>Appeared on: 2022-12-20</mark> - _27 pages, 28 figures, accepted in Astronomy and Astrophysics_

</div>
<div id="authors">

F. Matković, et al. -- incl., <mark><mark>S. G. Heinemann</mark></mark>

</div>
<div id="abstract">

**Abstract:** This study investigates and compares brightness and area of coronal bright points (CBPs) inside and outside of coronal holes (CHs) using the single-dish Band 6 observations by the Atacama Large Millimeter/submillimeter Array (ALMA), combined with extreme-ultraviolet (EUV) 193 $\overset{\circ}{\mathrm{A}}$ filtergrams obtained by the Atmospheric Imaging Assembly (AIA) and magnetograms obtained by the Helioseismic and Magnetic Imager (HMI), both on board Solar Dynamics Observatory (SDO). The CH boundaries were extracted from the SDO/AIA images using the Collection of Analysis Tools for Coronal Holes (CATCH) and CBPs were identified in the SDO/AIA, SDO/HMI, and ALMA data. Measurements of brightness and areas in both ALMA and SDO/AIA images were conducted for CBPs within CHs and quiet Sun regions outside CHs. A statistical analysis of the measured physical properties resulted in a lower average CBP brightness in both ALMA and SDO/AIA data for CBPs within the CHs. Depending on the CBP sample size, the difference in intensity for the SDO/AIA data, and brightness temperature for the ALMA data, between the CBPs inside and outside CHs ranged from between 2$\sigma$ and 4.5$\sigma$, showing a statistically significant difference between those two CBP groups. For CBP areas, CBPs within the CH boundaries showed smaller areas on average, with the observed difference between the two CBP groups between 1$\sigma$ and 2$\sigma$ for the SDO/AIA data, and up to 3.5$\sigma$ for the ALMA data, indicating that CBP areas are also significantly different. Given the measured properties, we conclude that the CBPs inside CHs tend to be less bright on average, but also smaller in comparison to those outside of CHs. This conclusion might point to the specific physical conditions and properties of the local CH region around a CBP limiting the maximum achievable intensity (temperature) and size of a CBP. 

</div>

<div id="div_fig1">

<img src="tmp_2212.09443/./2017_04_16_MEAN_int_ALMA_sample_size_5_1.png" alt="Fig13.1" width="25%"/><img src="tmp_2212.09443/./2017_04_16_MEAN_int_ALMA_sample_size_5_Nsigma_level_1.png" alt="Fig13.2" width="25%"/><img src="tmp_2212.09443/./2017_04_16_MEAN_int_ALMA_sample_size_5_t_test_statistic_1.png" alt="Fig13.3" width="25%"/><img src="tmp_2212.09443/./2017_04_16_MEAN_int_ALMA_sample_size_5_p_value_1.png" alt="Fig13.4" width="25%"/>

**Figure 13. -** Statistical analysis of mean ALMA Band 6 CBP brightness temperature. Top row: Left panel shows the mean values of the mean CBP intensities in the ALMA Band 6 image with corresponding standard errors of 200 randomly chosen equal size CBP sample pairs, with one sample containing CBPs within (red) and the other outside (blue) the CH1. The right panel shows histogram of the largest N for which the relation (\ref{Nsigma}) holds true. Bottom row: Left panel shows histogram of the $t$-test statistic values ($t$-values) and the right panel shows the histogram of the $p$-values obtained for the mean values of CBP mean intensities in the ALMA Band 6 image. Individual CBP sample contains 5 randomly chosen CBPs out of the many selected CBPs either within or outside the CH1. (*mean_int_fig_ch1*)

</div>
<div id="div_fig2">

<img src="tmp_2212.09443/./2017_04_22_MEAN_int_ALMA_sample_size_18_1.png" alt="Fig4.1" width="25%"/><img src="tmp_2212.09443/./2017_04_22_MEAN_int_ALMA_sample_size_18_Nsigma_level_1.png" alt="Fig4.2" width="25%"/><img src="tmp_2212.09443/./2017_04_22_MEAN_int_ALMA_sample_size_18_t_test_statistic_1.png" alt="Fig4.3" width="25%"/><img src="tmp_2212.09443/./2017_04_22_MEAN_int_ALMA_sample_size_18_p_value_1.png" alt="Fig4.4" width="25%"/>

**Figure 4. -** Statistical analysis of mean ALMA Band 6 CBP brightness temperature. Top row: Left panel shows the mean values of the mean CBP intensities in the ALMA Band 6 image with corresponding standard errors of 200 randomly chosen equal size CBP sample pairs, with one sample containing CBPs within (red) and the other outside (blue) the CH2. The right panel shows histogram of the largest N for which the relation (\ref{Nsigma}) holds true. Bottom row: Left panel shows histogram of the $t$-test statistic values ($t$-values) and the right panel shows the histogram of the $p$-values obtained for the mean values of CBP mean intensities in the ALMA Band 6 image. Individual CBP sample contains 18 randomly chosen CBPs out of the many selected CBPs either within or outside the CH2. (*mean_int_fig*)

</div>
<div id="div_fig3">

<img src="tmp_2212.09443/./2017_04_22_AREA_ALMA_sample_size_18_1.png" alt="Fig6.1" width="25%"/><img src="tmp_2212.09443/./2017_04_22_AREA_ALMA_sample_size_18_Nsigma_level_1.png" alt="Fig6.2" width="25%"/><img src="tmp_2212.09443/./2017_04_22_AREA_ALMA_sample_size_18_t_test_statistic_1.png" alt="Fig6.3" width="25%"/><img src="tmp_2212.09443/./2017_04_22_AREA_ALMA_sample_size_18_p_value_1.png" alt="Fig6.4" width="25%"/>

**Figure 6. -** Statistical analysis of projected ALMA Band 6 CBP area. Top row: Left panel shows the mean values of the CBP areas in the ALMA Band 6 image with corresponding standard errors of 200 randomly chosen equal size CBP sample pairs, with one sample containing CBPs within (red) and the other outside (blue) the CH2. The right panel shows histogram of the largest N for which the relation (\ref{Nsigma}) holds true. Bottom row: Left panel shows histograms of the $t$-test statistic values ($t$-values) and the right panel shows the histogram of the $p$-values obtained for the mean values of CBP areas in the ALMA Band 6 image. Individual CBP sample contains 18 randomly chosen CBPs out of the many selected CBPs either within or outside the CH2. (*area_fig*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand{\TODO}[2][]{$
$\ifthenelse{\isempty{#1}}{\textbf{\textcolor{red}{TODO: #2}}}{\textbf{\textcolor{red}{TODO (#1) #2}}}$
$}$
$\newcommand{\J}[2][]{$
$	\ifthenelse{\isempty{#1}}{\begingroup$
$\newcommand{\JlowerN}{\}ADD{#2}{-1}{\JlowerN}$
$		J=#2{\to}\JlowerN$
$		\endgroup$
$	}{J=#1{\to}#2}$
$}$
$\newcommand{\rom}[1]{{{\uppercase\expandafter{\romannumeral #1}}}}$
$\newcommand{\romup}[1]{{\textup{\uppercase\expandafter{\romannumeral #1}}}}$
$\newcommand{\incode}[1]{{\raggedright\lstinline|#1|}}$
$\newcommand{\incodep}[1]{{\raggedright\lstinline|"#1"|}}$
$\newcommand{\neutralcarbon}{\ifmmode \text{C\textsc{i}} \else{\sc C i}\fi}$
$\newcommand{\ionizedcarbon}{\ifmmode \text{C\textsc{ii}} \else{\sc C ii}\fi}$
$\newcommand{\neutralhydrogen}{\ifmmode \text{H\textsc{i}} \else{\sc H i}\fi}$
$\newcommand{\ionizedhydrogen}{\ifmmode \text{H\textsc{ii}} \else{\sc H ii}\fi}$
$\newcommand{\carbonmonoxide}{\ifmmode \text{\textsc{CO}} \else{\sc CO}\fi}$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{\Remark}[1]{\csname rmk-#1\endcsname}$
$\newcommand$
$\newcommand{\figurename}{Fig.}$
$\newcommand{\gtsima}{\; \buildrel > \over \sim \;}$
$\newcommand{\ltsima}{\; \buildrel < \over \sim \;}$
$\newcommand{\prosima}{\; \buildrel \propto \over \sim \;}$
$\newcommand{\gsim}{\lower.5ex\hbox{\gtsima}}$
$\newcommand{\lsim}{\lower.5ex\hbox{\ltsima}}$
$\newcommand{\simgt}{\lower.5ex\hbox{\gtsima}}$
$\newcommand{\simlt}{\lower.5ex\hbox{\ltsima}}$
$\newcommand{\simpr}{\lower.5ex\hbox{\prosima}}$
$\newcommand{\la}{\lsim}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\8mu}{8 \mu{\rm m}}$
$\newcommand{\16mu}{16 \mu{\rm m}}$
$\newcommand{\24mu}{24 \mu{\rm m}}$
$\newcommand{\70mu}{70 \mu{\rm m}}$
$\newcommand{\Umean}{\left<U\right>}$
$\newcommand{\Umin}{U_{\mathrm{min}}}$
$\newcommand{\Umax}{U_{\mathrm{max}}}$
$\newcommand{\SNR}{\mathrm{S/N}}$
$\newcommand{\alphaCO}{\alpha_{\mathrm{CO}}}$
$\newcommand{\deltaGDzero}{\delta_{\mathrm{GDR,0}}}$
$\newcommand{\deltaGDR}{\delta_{\mathrm{GDR}}}$
$\newcommand{\deltaDGR}{\delta_{\mathrm{DGR}}}$
$\newcommand{\deltaISM}{\delta_{\mathrm{ISM}}}$
$\newcommand{\deltaGas}{\mu_{\mathrm{molgas}}}$
$\newcommand{\deltaTotalGas}{\mu_{\mathrm{totalgas}}}$
$\newcommand{\deltaTotalGDR}{\delta_{\mathrm{totalGDR}}}$
$\newcommand{\deltaMS}{\delta{\mathrm{MS}}}$
$\newcommand{\DeltaMS}{\Delta{\mathrm{MS}}}$
$\newcommand{\GDR}{\mathrm{GDR}}$
$\newcommand{\DGR}{\mathrm{DGR}}$
$\newcommand{\Tkin}{T_{\mathrm{kin}}}$
$\newcommand{\nmean}{\left<n_{\mathrm{H}_2}\right>}$
$\newcommand{\nthresh}{n_{\mathrm{H_2,thresh}}}$
$\newcommand{\nH2}{n_{\mathrm{H}_2}}$
$\newcommand{\nHtwo}{n_{\mathrm{H}_2}}$
$\newcommand{\NH2_}{N_{\mathrm{H}_2}}$
$\newcommand{\percmcubic}{\mathrm{cm}^{-3}}$
$\newcommand{\percmsquare}{\mathrm{cm}^{-2}}$
$\newcommand{\dvddr}{\mathrm{d}v/\mathrm{d}r}$
$\newcommand{\lgNH2_}{\mathrm{log}_{10} (N_{\mathrm{H}_2} / \mathrm{cm^{-2}})}$
$\newcommand{\lgnmean}{\mathrm{log}_{10} \left<n_{\mathrm{H_2}}\right>}$
$\newcommand{\lgnHtwo}{\mathrm{log} n_{\mathrm{H_2}}}$
$\newcommand{\lgnthresh}{\mathrm{log}_{10} (n_{\mathrm{H_2,thresh}})}$
$\newcommand{\alphavir}{\alpha_{\mathrm{vir}}}$
$\newcommand{\HIrom}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{\romannumeral 1}}}}$
$\newcommand{\HIIrom}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{\romannumeral 2}}}}$
$\newcommand{\CO}{\mathrm{CO}}$
$\newcommand{\metalZOH}{12+\log_{10}(\mathrm{O/H})}$
$\newcommand{\Msun}{\mathrm{M}_{\odot}}$
$\newcommand{\Lsun}{\mathrm{L}_{\odot}}$
$\newcommand{\Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\newcommand{\kms}{\mathrm{km s^{-1}}}$
$\newcommand{\SFR}{\mathrm{SFR}}$
$\newcommand{\LIR}{L_{\mathrm{IR}}}$
$\newcommand{\Mstar}{M_{\star}}$
$\newcommand{\Mdust}{M_{\mathrm{dust}}}$
$\newcommand{\Tdust}{T_{\mathrm{dust}}}$
$\newcommand{\Mgas}{M_{\mathrm{gas}}}$
$\newcommand{\Mmolgas}{M_{\mathrm{mol gas}}}$
$\newcommand{\Mtotalgas}{M_{\mathrm{total gas}}}$
$\newcommand{\Matomicgas}{M_{\mathrm{atomic gas}}}$
$\newcommand{\arcsec}{"}$
$\newcommand{\R52}{R_{52}}$
$\newcommand{\fPDR}{f_{\mathrm{PDR}}}$
$\newcommand{\fmol}{f_{\mathrm{mol}}}$
$\newcommand{\fH2_}{f_{\mathrm{H_2}}}$
$\newcommand{\MH2_}{M_{\mathrm{H_2}}}$
$\newcommand{\XCICO}{[\mathrm{C}{\text{\sc{i}}/\mathrm{CO}]}}$
$\newcommand{\micron}{\mu\mathrm{m}}$
$\newcommand{\Jykms}{\mathrm{Jy km/s}}$
$\newcommand{\Kkms_}{\mathrm{K km/s}}$
$\newcommand{\LprimeCO}{L^{\prime}_{\mathrm{CO}}}$
$\newcommand{\MAGPHYS}{\textsc{MAGPHYS}}$
$\newcommand{\michi2}{\textsc{MiChi2}}$
$\newcommand{\um}{\ifmmode{\mu\mathrm{m}}\else{\mu\mathrm{m}}\fi}$
$\newcommand{\cofour}{CO(4--3)}$
$\newcommand{\cotwo}{CO(2--1)}$
$\newcommand{\J}[2][]{$
$	\ifthenelse{\isempty{#1}}{\begingroup$
$\newcommand{\JlowerN}{\}ADD{#2}{-1}{\JlowerN}$
$		J=#2{\to}\JlowerN$
$		\endgroup$
$	}{J=#1{\to}#2}$
$}$
$\newcommand{\rom}[1]{{{\uppercase\expandafter{\romannumeral #1}}}}$
$\newcommand{\romup}[1]{{\textup{\uppercase\expandafter{\romannumeral #1}}}}$
$\newcommand{\incode}[1]{{\raggedright\lstinline|#1|}}$
$\newcommand{\incodep}[1]{{\raggedright\lstinline|"#1"|}}$
$\newcommand{\neutralcarbon}{\ifmmode \text{C\textsc{i}} \else{\sc C i}\fi}$
$\newcommand{\ionizedcarbon}{\ifmmode \text{C\textsc{ii}} \else{\sc C ii}\fi}$
$\newcommand{\neutralhydrogen}{\ifmmode \text{H\textsc{i}} \else{\sc H i}\fi}$
$\newcommand{\ionizedhydrogen}{\ifmmode \text{H\textsc{ii}} \else{\sc H ii}\fi}$
$\newcommand{\carbonmonoxide}{\ifmmode \text{\textsc{CO}} \else{\sc CO}\fi}$
$\newcommand{\HII}{\ionizedhydrogen}$
$\newcommand{\CI}{\neutralcarbon}$
$\newcommand{\CO}{\carbonmonoxide}$
$\newcommand{\}{alphaCO}{\relax\ifmmode$
$    \alpha_\mathrm{CO}$
$    \else{\alpha_\mathrm{CO}}\fi$
$    }$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{\Remark}[1]{\csname rmk-#1\endcsname}$
$\DefineRemark{CI3P13P0}{[\mathrm{C}{\text{\sc{i}}}]\;(^{3}P_{1}\textnormal{--}^{3}P_{0})}$
$\DefineRemark{CI}{\mathrm{C}{\text{\sc{i}}}}$
$\DefineRemark{CI10}{[\mathrm{C}{\text{\sc{i}}}] (1\textnormal{--}0)}$
$\DefineRemark{CI21}{[\mathrm{C}{\text{\sc{i}}}] (2\textnormal{--}1)}$
$\DefineRemark{CO}{\mathrm{CO}}$
$\DefineRemark{CO10}{\mathrm{CO} (1\textnormal{--}0)}$
$\DefineRemark{CO21}{\mathrm{CO} (2\textnormal{--}1)}$
$\DefineRemark{CO32}{\mathrm{CO} (3\textnormal{--}2)}$
$\DefineRemark{CO43}{\mathrm{CO} (4\textnormal{--}3)}$
$\DefineRemark{COJ10}{\mathrm{CO} (J=1 \to 0)}$
$\DefineRemark{COJ21}{\mathrm{CO} (J=2 \to 1)}$
$\DefineRemark{COJ32}{\mathrm{CO} (J=3 \to 2)}$
$\DefineRemark{COJ43}{\mathrm{CO} (J=4 \to 3)}$
$\DefineRemark{RCI10CO10}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{10}}}$
$}$
$\DefineRemark{RCI10CO21}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{21}}}$
$}$
$\DefineRemark{RCI10CO32}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{32}}}$
$}$
$\DefineRemark{RCI10CO43}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{43}}}$
$}$
$\DefineRemark{LPrimeCO21}{L^{\prime}_{\Remark{CO21}}}$
$\DefineRemark{LPrimeCI10}{L^{\prime}_{\Remark{CI10}}}$
$\DefineRemark{ICO10}{I_{\Remark{CO10}}}$
$\DefineRemark{ICO21}{I_{\Remark{CO21}}}$
$\DefineRemark{ICO32}{I_{\Remark{CO32}}}$
$\DefineRemark{ICO43}{I_{\Remark{CO43}}}$
$\DefineRemark{ICI10}{I_{\Remark{CI10}}}$
$\DefineRemark{kms}{\mathrm{km s^{-1}}}$
$\DefineRemark{Kkms}{\mathrm{K km s^{-1}}}$
$\DefineRemark{Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\DefineRemark{MJysr}{\mathrm{MJy sr^{-1}}}$
$\DefineRemark{R42}{\ifmmode{R_{42}}\else{R_{42}}\fi}$
$\DefineRemark{R41}{\ifmmode{R_{41}}\else{R_{41}}\fi}$
$\DefineRemark{R31}{\ifmmode{R_{31}}\else{R_{31}}\fi}$
$\DefineRemark{R21}{\ifmmode{R_{21}}\else{R_{21}}\fi}$
$\DefineRemark{R02}{\ifmmode{R_{\rm C\text{\sc{i}}CO}}\else{R_{\rm C\text{\sc{i}}CO}}\fi}$
$\DefineRemark{RCICO}{\ifmmode{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\else{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\fi}$
$\newcommand{\figurename}{Fig.}$
$\accepted{December 7, 2022}$
$\submitjournal{ApJL}$
$\shorttitle{Stellar Feedback, Molecular Excitation and Dissociation in NGC~1365}$
$\shortauthors{Liu et al.}$
$\begin{document}$
$\title{PHANGS-JWST First Results: Stellar Feedback-Driven Excitation and Dissociation of Molecular Gas in the Starburst Ring of NGC 1365?$
$}$
$\correspondingauthor{Daizhong Liu}$
$\email{dzliu@mpe.mpg.de, astro.dzliu@gmail.com}$
$\author[0000-0001-9773-7479]{Daizhong Liu}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-3933-7677]{Eva Schinnerer}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0001-5301-1326]{Yixian Cao}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-2545-1700]{Adam Leroy}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\author[0000-0003-1242-505X]{Antonio Usero}$
$\affiliation{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\author[0000-0002-5204-2259]{Erik Rosolowsky}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\author[0000-0002-6155-7166]{Eric Emsellem}$
$\affiliation{European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\affiliation{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\author[0000-0002-8804-0212]{J.~M.~Diederik~Kruijssen}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0002-5635-5180]{M\'elanie Chevance}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0001-6708-1317]{Simon C. O. Glover}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0001-6113-6241]{Mattia C. Sormani}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5480-5686]{Alberto D. Bolatto}$
$\affiliation{Department of Astronomy and Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}$
$\author[0000-0003-0378-4667]{Jiayi~Sun}$
$\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\author[0000-0002-9333-387X]{Sophia K. Stuber}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-4209-1599]{Yu-Hsuan Teng}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0003-0166-9745]{Frank Bigiel}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0003-0583-7363]{Ivana Be\v{s}li\'c}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-3247-5321]{Kathryn~Grasha}$
$\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\author[0000-0001-9656-7682]{Jonathan~D.~Henshaw}$
$\affiliation{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-0410-4504]{Ashley.~T.~Barnes}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-8760-6157]{Jakob~S.~den Brok}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-2501-9328]{Toshiki Saito}$
$\affiliation{National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo, 181-8588, Japan}$
$\author[0000-0002-5782-9093]{Daniel~A.~Dale}$
$\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\author[0000-0002-7365-5791]{Elizabeth~J.~Watkins}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0002-1370-6964]{Hsi-An Pan}$
$\affiliation{Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan}$
$\author[0000-0002-0560-3172]{Ralf S. Klessen}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Universit\"at Heidelberg, Interdisziplin\"ares Zentrum f\"ur Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5259-2314]{Gagandeep~S. Anand}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1943-723X]{Sinan Deger}$
$\affiliation{The Oskar Klein Centre for Cosmoparticle Physics, Department of Physics, Stockholm University, AlbaNova, Stockholm, SE-106 91, Sweden}$
$\author[0000-0002-4755-118X]{Oleg V. Egorov}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0001-5310-467X]{Christopher~M. Faesi}$
$\affiliation{University of Massachusetts-Amherst, 710 North Pleasant Street, Amherst, MA 01003, USA}$
$\author[0000-0002-8806-6308]{Hamid Hassani}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, Alberta, T6G 2E1, Canada}$
$\author[0000-0003-3917-6460]{Kirsten L. Larson}$
$\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2278-9407]{Janice~C. Lee}$
$\affiliation{Gemini Observatory/NSF's NOIRLab, 950 N. Cherry Avenue, Tucson, AZ 85719, USA; Caltech/IPAC, California Institute of Technology, Pasadena, CA 91125, USA}$
$\author[0000-0002-1790-3148]{Laura A. Lopez}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\affiliation{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\affiliation{Flatiron Institute, Center for Computational Astrophysics, NY 10010, USA}$
$\author[0000-0003-3061-6546]{Jér\^{o}me Pety}$
$\affiliation{IRAM, 300 rue de la Piscine, 38400 Saint Martin d'H\`eres, France}$
$\affiliation{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris}$
$\author[0000-0002-4378-8534]{Karin Sandstrom}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0002-8528-7340]{David~A. Thilker}$
$\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0002-3784-7032]{Bradley~C. Whitmore}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0012-2142]{Thomas G. Williams}$
$\affiliation{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\suppressAffiliations$
$\begin{abstract}$
$We compare embedded young massive star clusters (YMCs) to (sub-)millimeter line observations tracing the excitation and dissociation of molecular gas in the starburst ring of NGC~1365. This galaxy hosts one of the strongest nuclear starbursts and richest populations of YMCs within 20~Mpc.$
$Here we combine near-/mid-IR PHANGS-JWST imaging with new ALMA multi-J CO(1--0, 2--1 and 4--3) and [\CI](1--0) mapping, which we use to trace CO excitation via \Remark{R42}=\Remark{ICO43}/\Remark{ICO21} and \Remark{R21}=\Remark{ICO21}/\Remark{ICO10} and dissociation via \Remark{RCICO}=\Remark{ICI10}/\Remark{ICO21} at 330~pc resolution.$
$We find that the gas flowing into the starburst ring from northeast to southwest appears strongly affected by stellar feedback, showing decreased excitation (lower \Remark{R42}) and increased signatures of dissociation (higher \Remark{RCICO}) in the downstream regions.$
$There, radiative transfer modelling suggests that the molecular gas density decreases and temperature and \XCICO abundance ratio increase.$
$We compare \Remark{R42} and \Remark{RCICO} with local conditions across the regions and find that both correlate with near-IR 2~\um emission tracing the YMCs and with both PAH (11.3~\um) and dust continuum (21~\um) emission.$
$In general, \Remark{RCICO} exhibits \sim 0.1~dex tighter correlations than \Remark{R42}, suggesting \CI  to be a more sensitive tracer of changing physical conditions in the NGC~1365 starburst than CO(4--3).$
$Our results are consistent with a scenario where gas flows into the two arm regions along the bar, becomes condensed/shocked, forms YMCs, and then these YMCs heat and dissociate the gas.$
$\end{abstract}$
$\keywords{galaxies: ISM --- galaxies: star formation --- ISM: molecules --- ISM: atoms}$
$\n\end{document}}}}\end{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\8mu}{8 \mu{\rm m}}$
$\newcommand{\16mu}{16 \mu{\rm m}}$
$\newcommand{\24mu}{24 \mu{\rm m}}$
$\newcommand{\70mu}{70 \mu{\rm m}}$
$\newcommand{\Umean}{\left<U\right>}$
$\newcommand{\Umin}{U_{\mathrm{min}}}$
$\newcommand{\Umax}{U_{\mathrm{max}}}$
$\newcommand{\SNR}{\mathrm{S/N}}$
$\newcommand{\alphaCO}{\alpha_{\mathrm{CO}}}$
$\newcommand{\deltaGDzero}{\delta_{\mathrm{GDR,0}}}$
$\newcommand{\deltaGDR}{\delta_{\mathrm{GDR}}}$
$\newcommand{\deltaDGR}{\delta_{\mathrm{DGR}}}$
$\newcommand{\deltaISM}{\delta_{\mathrm{ISM}}}$
$\newcommand{\deltaGas}{\mu_{\mathrm{molgas}}}$
$\newcommand{\deltaTotalGas}{\mu_{\mathrm{totalgas}}}$
$\newcommand{\deltaTotalGDR}{\delta_{\mathrm{totalGDR}}}$
$\newcommand{\deltaMS}{\delta{\mathrm{MS}}}$
$\newcommand{\DeltaMS}{\Delta{\mathrm{MS}}}$
$\newcommand{\GDR}{\mathrm{GDR}}$
$\newcommand{\DGR}{\mathrm{DGR}}$
$\newcommand{\Tkin}{T_{\mathrm{kin}}}$
$\newcommand{\nmean}{\left<n_{\mathrm{H}_2}\right>}$
$\newcommand{\nthresh}{n_{\mathrm{H_2,thresh}}}$
$\newcommand{\nH2}{n_{\mathrm{H}_2}}$
$\newcommand{\nHtwo}{n_{\mathrm{H}_2}}$
$\newcommand{\NH2_}{N_{\mathrm{H}_2}}$
$\newcommand{\percmcubic}{\mathrm{cm}^{-3}}$
$\newcommand{\percmsquare}{\mathrm{cm}^{-2}}$
$\newcommand{\dvddr}{\mathrm{d}v/\mathrm{d}r}$
$\newcommand{\lgNH2_}{\mathrm{log}_{10} (N_{\mathrm{H}_2} / \mathrm{cm^{-2}})}$
$\newcommand{\lgnmean}{\mathrm{log}_{10} \left<n_{\mathrm{H_2}}\right>}$
$\newcommand{\lgnHtwo}{\mathrm{log} n_{\mathrm{H_2}}}$
$\newcommand{\lgnthresh}{\mathrm{log}_{10} (n_{\mathrm{H_2,thresh}})}$
$\newcommand{\alphavir}{\alpha_{\mathrm{vir}}}$
$\newcommand{\HIrom}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{\romannumeral 1}}}}$
$\newcommand{\HIIrom}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{\romannumeral 2}}}}$
$\newcommand{\CO}{\mathrm{CO}}$
$\newcommand{\metalZOH}{12+\log_{10}(\mathrm{O/H})}$
$\newcommand{\Msun}{\mathrm{M}_{\odot}}$
$\newcommand{\Lsun}{\mathrm{L}_{\odot}}$
$\newcommand{\Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\newcommand{\kms}{\mathrm{km s^{-1}}}$
$\newcommand{\SFR}{\mathrm{SFR}}$
$\newcommand{\LIR}{L_{\mathrm{IR}}}$
$\newcommand{\Mstar}{M_{\star}}$
$\newcommand{\Mdust}{M_{\mathrm{dust}}}$
$\newcommand{\Tdust}{T_{\mathrm{dust}}}$
$\newcommand{\Mgas}{M_{\mathrm{gas}}}$
$\newcommand{\Mmolgas}{M_{\mathrm{mol gas}}}$
$\newcommand{\Mtotalgas}{M_{\mathrm{total gas}}}$
$\newcommand{\Matomicgas}{M_{\mathrm{atomic gas}}}$
$\newcommand{\arcsec}{"}$
$\newcommand{\R52}{R_{52}}$
$\newcommand{\fPDR}{f_{\mathrm{PDR}}}$
$\newcommand{\fmol}{f_{\mathrm{mol}}}$
$\newcommand{\fH2_}{f_{\mathrm{H_2}}}$
$\newcommand{\MH2_}{M_{\mathrm{H_2}}}$
$\newcommand{\XCICO}{[\mathrm{C}{\text{\sc{i}}/\mathrm{CO}]}}$
$\newcommand{\micron}{\mu\mathrm{m}}$
$\newcommand{\Jykms}{\mathrm{Jy km/s}}$
$\newcommand{\Kkms_}{\mathrm{K km/s}}$
$\newcommand{\LprimeCO}{L^{\prime}_{\mathrm{CO}}}$
$\newcommand{\MAGPHYS}{\textsc{MAGPHYS}}$
$\newcommand{\michi2}{\textsc{MiChi2}}$
$\newcommand{\um}{\ifmmode{\mu\mathrm{m}}\else{\mu\mathrm{m}}\fi}$
$\newcommand{\cofour}{CO(4--3)}$
$\newcommand{\cotwo}{CO(2--1)}$
$\newcommand{\J}[2][]{$
$	\ifthenelse{\isempty{#1}}{\begingroup$
$\newcommand{\JlowerN}{\}ADD{#2}{-1}{\JlowerN}$
$		J=#2{\to}\JlowerN$
$		\endgroup$
$	}{J=#1{\to}#2}$
$}$
$\newcommand{\rom}[1]{{{\uppercase\expandafter{\romannumeral #1}}}}$
$\newcommand{\romup}[1]{{\textup{\uppercase\expandafter{\romannumeral #1}}}}$
$\newcommand{\incode}[1]{{\raggedright\lstinline|#1|}}$
$\newcommand{\incodep}[1]{{\raggedright\lstinline|"#1"|}}$
$\newcommand{\neutralcarbon}{\ifmmode \text{C\textsc{i}} \else{\sc C i}\fi}$
$\newcommand{\ionizedcarbon}{\ifmmode \text{C\textsc{ii}} \else{\sc C ii}\fi}$
$\newcommand{\neutralhydrogen}{\ifmmode \text{H\textsc{i}} \else{\sc H i}\fi}$
$\newcommand{\ionizedhydrogen}{\ifmmode \text{H\textsc{ii}} \else{\sc H ii}\fi}$
$\newcommand{\carbonmonoxide}{\ifmmode \text{\textsc{CO}} \else{\sc CO}\fi}$
$\newcommand{\HII}{\ionizedhydrogen}$
$\newcommand{\CI}{\neutralcarbon}$
$\newcommand{\CO}{\carbonmonoxide}$
$\newcommand{\}{alphaCO}{\relax\ifmmode$
$    \alpha_\mathrm{CO}$
$    \else{\alpha_\mathrm{CO}}\fi$
$    }$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{\Remark}[1]{\csname rmk-#1\endcsname}$
$\DefineRemark{CI3P13P0}{[\mathrm{C}{\text{\sc{i}}}]\;(^{3}P_{1}\textnormal{--}^{3}P_{0})}$
$\DefineRemark{CI}{\mathrm{C}{\text{\sc{i}}}}$
$\DefineRemark{CI10}{[\mathrm{C}{\text{\sc{i}}}] (1\textnormal{--}0)}$
$\DefineRemark{CI21}{[\mathrm{C}{\text{\sc{i}}}] (2\textnormal{--}1)}$
$\DefineRemark{CO}{\mathrm{CO}}$
$\DefineRemark{CO10}{\mathrm{CO} (1\textnormal{--}0)}$
$\DefineRemark{CO21}{\mathrm{CO} (2\textnormal{--}1)}$
$\DefineRemark{CO32}{\mathrm{CO} (3\textnormal{--}2)}$
$\DefineRemark{CO43}{\mathrm{CO} (4\textnormal{--}3)}$
$\DefineRemark{COJ10}{\mathrm{CO} (J=1 \to 0)}$
$\DefineRemark{COJ21}{\mathrm{CO} (J=2 \to 1)}$
$\DefineRemark{COJ32}{\mathrm{CO} (J=3 \to 2)}$
$\DefineRemark{COJ43}{\mathrm{CO} (J=4 \to 3)}$
$\DefineRemark{RCI10CO10}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{10}}}$
$}$
$\DefineRemark{RCI10CO21}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{21}}}$
$}$
$\DefineRemark{RCI10CO32}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{32}}}$
$}$
$\DefineRemark{RCI10CO43}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{43}}}$
$}$
$\DefineRemark{LPrimeCO21}{L^{\prime}_{\Remark{CO21}}}$
$\DefineRemark{LPrimeCI10}{L^{\prime}_{\Remark{CI10}}}$
$\DefineRemark{ICO10}{I_{\Remark{CO10}}}$
$\DefineRemark{ICO21}{I_{\Remark{CO21}}}$
$\DefineRemark{ICO32}{I_{\Remark{CO32}}}$
$\DefineRemark{ICO43}{I_{\Remark{CO43}}}$
$\DefineRemark{ICI10}{I_{\Remark{CI10}}}$
$\DefineRemark{kms}{\mathrm{km s^{-1}}}$
$\DefineRemark{Kkms}{\mathrm{K km s^{-1}}}$
$\DefineRemark{Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\DefineRemark{MJysr}{\mathrm{MJy sr^{-1}}}$
$\DefineRemark{R42}{\ifmmode{R_{42}}\else{R_{42}}\fi}$
$\DefineRemark{R41}{\ifmmode{R_{41}}\else{R_{41}}\fi}$
$\DefineRemark{R31}{\ifmmode{R_{31}}\else{R_{31}}\fi}$
$\DefineRemark{R21}{\ifmmode{R_{21}}\else{R_{21}}\fi}$
$\DefineRemark{R02}{\ifmmode{R_{\rm C\text{\sc{i}}CO}}\else{R_{\rm C\text{\sc{i}}CO}}\fi}$
$\DefineRemark{RCICO}{\ifmmode{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\else{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\fi}$
$\newcommand{\figurename}{Fig.}$
$\accepted{December 7, 2022}$
$\submitjournal{ApJL}$
$\shorttitle{Stellar Feedback, Molecular Excitation and Dissociation in NGC~1365}$
$\shortauthors{Liu et al.}$
$\begin{document}$
$\title{PHANGS-JWST First Results: Stellar Feedback-Driven Excitation and Dissociation of Molecular Gas in the Starburst Ring of NGC 1365?$
$}$
$\correspondingauthor{Daizhong Liu}$
$\email{dzliu@mpe.mpg.de, astro.dzliu@gmail.com}$
$\author[0000-0001-9773-7479]{Daizhong Liu}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-3933-7677]{Eva Schinnerer}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0001-5301-1326]{Yixian Cao}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-2545-1700]{Adam Leroy}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\author[0000-0003-1242-505X]{Antonio Usero}$
$\affiliation{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\author[0000-0002-5204-2259]{Erik Rosolowsky}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\author[0000-0002-6155-7166]{Eric Emsellem}$
$\affiliation{European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\affiliation{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\author[0000-0002-8804-0212]{J.~M.~Diederik~Kruijssen}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0002-5635-5180]{M\'elanie Chevance}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0001-6708-1317]{Simon C. O. Glover}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0001-6113-6241]{Mattia C. Sormani}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5480-5686]{Alberto D. Bolatto}$
$\affiliation{Department of Astronomy and Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}$
$\author[0000-0003-0378-4667]{Jiayi~Sun}$
$\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\author[0000-0002-9333-387X]{Sophia K. Stuber}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-4209-1599]{Yu-Hsuan Teng}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0003-0166-9745]{Frank Bigiel}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0003-0583-7363]{Ivana Be\v{s}li\'c}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-3247-5321]{Kathryn~Grasha}$
$\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\author[0000-0001-9656-7682]{Jonathan~D.~Henshaw}$
$\affiliation{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-0410-4504]{Ashley.~T.~Barnes}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-8760-6157]{Jakob~S.~den Brok}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-2501-9328]{Toshiki Saito}$
$\affiliation{National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo, 181-8588, Japan}$
$\author[0000-0002-5782-9093]{Daniel~A.~Dale}$
$\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\author[0000-0002-7365-5791]{Elizabeth~J.~Watkins}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0002-1370-6964]{Hsi-An Pan}$
$\affiliation{Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan}$
$\author[0000-0002-0560-3172]{Ralf S. Klessen}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Universit\"at Heidelberg, Interdisziplin\"ares Zentrum f\"ur Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5259-2314]{Gagandeep~S. Anand}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1943-723X]{Sinan Deger}$
$\affiliation{The Oskar Klein Centre for Cosmoparticle Physics, Department of Physics, Stockholm University, AlbaNova, Stockholm, SE-106 91, Sweden}$
$\author[0000-0002-4755-118X]{Oleg V. Egorov}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0001-5310-467X]{Christopher~M. Faesi}$
$\affiliation{University of Massachusetts-Amherst, 710 North Pleasant Street, Amherst, MA 01003, USA}$
$\author[0000-0002-8806-6308]{Hamid Hassani}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, Alberta, T6G 2E1, Canada}$
$\author[0000-0003-3917-6460]{Kirsten L. Larson}$
$\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2278-9407]{Janice~C. Lee}$
$\affiliation{Gemini Observatory/NSF's NOIRLab, 950 N. Cherry Avenue, Tucson, AZ 85719, USA; Caltech/IPAC, California Institute of Technology, Pasadena, CA 91125, USA}$
$\author[0000-0002-1790-3148]{Laura A. Lopez}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\affiliation{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\affiliation{Flatiron Institute, Center for Computational Astrophysics, NY 10010, USA}$
$\author[0000-0003-3061-6546]{Jér\^{o}me Pety}$
$\affiliation{IRAM, 300 rue de la Piscine, 38400 Saint Martin d'H\`eres, France}$
$\affiliation{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris}$
$\author[0000-0002-4378-8534]{Karin Sandstrom}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0002-8528-7340]{David~A. Thilker}$
$\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0002-3784-7032]{Bradley~C. Whitmore}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0012-2142]{Thomas G. Williams}$
$\affiliation{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\suppressAffiliations$
$\begin{abstract}$
$We compare embedded young massive star clusters (YMCs) to (sub-)millimeter line observations tracing the excitation and dissociation of molecular gas in the starburst ring of NGC~1365. This galaxy hosts one of the strongest nuclear starbursts and richest populations of YMCs within 20~Mpc.$
$Here we combine near-/mid-IR PHANGS-JWST imaging with new ALMA multi-J CO(1--0, 2--1 and 4--3) and [\CI](1--0) mapping, which we use to trace CO excitation via \Remark{R42}=\Remark{ICO43}/\Remark{ICO21} and \Remark{R21}=\Remark{ICO21}/\Remark{ICO10} and dissociation via \Remark{RCICO}=\Remark{ICI10}/\Remark{ICO21} at 330~pc resolution.$
$We find that the gas flowing into the starburst ring from northeast to southwest appears strongly affected by stellar feedback, showing decreased excitation (lower \Remark{R42}) and increased signatures of dissociation (higher \Remark{RCICO}) in the downstream regions.$
$There, radiative transfer modelling suggests that the molecular gas density decreases and temperature and \XCICO abundance ratio increase.$
$We compare \Remark{R42} and \Remark{RCICO} with local conditions across the regions and find that both correlate with near-IR 2~\um emission tracing the YMCs and with both PAH (11.3~\um) and dust continuum (21~\um) emission.$
$In general, \Remark{RCICO} exhibits \sim 0.1~dex tighter correlations than \Remark{R42}, suggesting \CI  to be a more sensitive tracer of changing physical conditions in the NGC~1365 starburst than CO(4--3).$
$Our results are consistent with a scenario where gas flows into the two arm regions along the bar, becomes condensed/shocked, forms YMCs, and then these YMCs heat and dissociate the gas.$
$\end{abstract}$
$\keywords{galaxies: ISM --- galaxies: star formation --- ISM: molecules --- ISM: atoms}$
$\n\end{document}}}}$
$\newcommand{\Umin}{U_{\mathrm{min}}}$
$\newcommand{\Umax}{U_{\mathrm{max}}}$
$\newcommand{\SNR}{\mathrm{S/N}}$
$\newcommand{\alphaCO}{\alpha_{\mathrm{CO}}}$
$\newcommand{\deltaGDzero}{\delta_{\mathrm{GDR,0}}}$
$\newcommand{\deltaGDR}{\delta_{\mathrm{GDR}}}$
$\newcommand{\deltaDGR}{\delta_{\mathrm{DGR}}}$
$\newcommand{\deltaISM}{\delta_{\mathrm{ISM}}}$
$\newcommand{\deltaGas}{\mu_{\mathrm{molgas}}}$
$\newcommand{\deltaTotalGas}{\mu_{\mathrm{totalgas}}}$
$\newcommand{\deltaTotalGDR}{\delta_{\mathrm{totalGDR}}}$
$\newcommand{\deltaMS}{\delta{\mathrm{MS}}}$
$\newcommand{\DeltaMS}{\Delta{\mathrm{MS}}}$
$\newcommand{\GDR}{\mathrm{GDR}}$
$\newcommand{\DGR}{\mathrm{DGR}}$
$\newcommand{\Tkin}{T_{\mathrm{kin}}}$
$\newcommand{\nmean}{\left<n_{\mathrm{H}_2}\right>}$
$\newcommand{\nthresh}{n_{\mathrm{H_2,thresh}}}$
$\newcommand{\nH2}{n_{\mathrm{H}_2}}$
$\newcommand{\nHtwo}{n_{\mathrm{H}_2}}$
$\newcommand{\NH2_}{N_{\mathrm{H}_2}}$
$\newcommand{\percmcubic}{\mathrm{cm}^{-3}}$
$\newcommand{\percmsquare}{\mathrm{cm}^{-2}}$
$\newcommand{\dvddr}{\mathrm{d}v/\mathrm{d}r}$
$\newcommand{\lgNH2_}{\mathrm{log}_{10} (N_{\mathrm{H}_2} / \mathrm{cm^{-2}})}$
$\newcommand{\lgnmean}{\mathrm{log}_{10} \left<n_{\mathrm{H_2}}\right>}$
$\newcommand{\lgnHtwo}{\mathrm{log} n_{\mathrm{H_2}}}$
$\newcommand{\lgnthresh}{\mathrm{log}_{10} (n_{\mathrm{H_2,thresh}})}$
$\newcommand{\alphavir}{\alpha_{\mathrm{vir}}}$
$\newcommand{\HIrom}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{\romannumeral 1}}}}$
$\newcommand{\HIIrom}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{\romannumeral 2}}}}$
$\newcommand{\CO}{\mathrm{CO}}$
$\newcommand{\metalZOH}{12+\log_{10}(\mathrm{O/H})}$
$\newcommand{\Msun}{\mathrm{M}_{\odot}}$
$\newcommand{\Lsun}{\mathrm{L}_{\odot}}$
$\newcommand{\Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\newcommand{\kms}{\mathrm{km s^{-1}}}$
$\newcommand{\SFR}{\mathrm{SFR}}$
$\newcommand{\LIR}{L_{\mathrm{IR}}}$
$\newcommand{\Mstar}{M_{\star}}$
$\newcommand{\Mdust}{M_{\mathrm{dust}}}$
$\newcommand{\Tdust}{T_{\mathrm{dust}}}$
$\newcommand{\Mgas}{M_{\mathrm{gas}}}$
$\newcommand{\Mmolgas}{M_{\mathrm{mol gas}}}$
$\newcommand{\Mtotalgas}{M_{\mathrm{total gas}}}$
$\newcommand{\Matomicgas}{M_{\mathrm{atomic gas}}}$
$\newcommand{\arcsec}{"}$
$\newcommand{\R52}{R_{52}}$
$\newcommand{\fPDR}{f_{\mathrm{PDR}}}$
$\newcommand{\fmol}{f_{\mathrm{mol}}}$
$\newcommand{\fH2_}{f_{\mathrm{H_2}}}$
$\newcommand{\MH2_}{M_{\mathrm{H_2}}}$
$\newcommand{\XCICO}{[\mathrm{C}{\text{\sc{i}}/\mathrm{CO}]}}$
$\newcommand{\micron}{\mu\mathrm{m}}$
$\newcommand{\Jykms}{\mathrm{Jy km/s}}$
$\newcommand{\Kkms_}{\mathrm{K km/s}}$
$\newcommand{\LprimeCO}{L^{\prime}_{\mathrm{CO}}}$
$\newcommand{\MAGPHYS}{\textsc{MAGPHYS}}$
$\newcommand{\michi2}{\textsc{MiChi2}}$
$\newcommand{\um}{\ifmmode{\mu\mathrm{m}}\else{\mu\mathrm{m}}\fi}$
$\newcommand{\cofour}{CO(4--3)}$
$\newcommand{\cotwo}{CO(2--1)}$
$\newcommand{\J}[2][]{$
$	\ifthenelse{\isempty{#1}}{\begingroup$
$\newcommand{\JlowerN}{\}ADD{#2}{-1}{\JlowerN}$
$		J=#2{\to}\JlowerN$
$		\endgroup$
$	}{J=#1{\to}#2}$
$}$
$\newcommand{\rom}[1]{{{\uppercase\expandafter{\romannumeral #1}}}}$
$\newcommand{\romup}[1]{{\textup{\uppercase\expandafter{\romannumeral #1}}}}$
$\newcommand{\incode}[1]{{\raggedright\lstinline|#1|}}$
$\newcommand{\incodep}[1]{{\raggedright\lstinline|"#1"|}}$
$\newcommand{\neutralcarbon}{\ifmmode \text{C\textsc{i}} \else{\sc C i}\fi}$
$\newcommand{\ionizedcarbon}{\ifmmode \text{C\textsc{ii}} \else{\sc C ii}\fi}$
$\newcommand{\neutralhydrogen}{\ifmmode \text{H\textsc{i}} \else{\sc H i}\fi}$
$\newcommand{\ionizedhydrogen}{\ifmmode \text{H\textsc{ii}} \else{\sc H ii}\fi}$
$\newcommand{\carbonmonoxide}{\ifmmode \text{\textsc{CO}} \else{\sc CO}\fi}$
$\newcommand{\HII}{\ionizedhydrogen}$
$\newcommand{\CI}{\neutralcarbon}$
$\newcommand{\CO}{\carbonmonoxide}$
$\newcommand{\}{alphaCO}{\relax\ifmmode$
$    \alpha_\mathrm{CO}$
$    \else{\alpha_\mathrm{CO}}\fi$
$    }$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{\Remark}[1]{\csname rmk-#1\endcsname}$
$\DefineRemark{CI3P13P0}{[\mathrm{C}{\text{\sc{i}}}]\;(^{3}P_{1}\textnormal{--}^{3}P_{0})}$
$\DefineRemark{CI}{\mathrm{C}{\text{\sc{i}}}}$
$\DefineRemark{CI10}{[\mathrm{C}{\text{\sc{i}}}] (1\textnormal{--}0)}$
$\DefineRemark{CI21}{[\mathrm{C}{\text{\sc{i}}}] (2\textnormal{--}1)}$
$\DefineRemark{CO}{\mathrm{CO}}$
$\DefineRemark{CO10}{\mathrm{CO} (1\textnormal{--}0)}$
$\DefineRemark{CO21}{\mathrm{CO} (2\textnormal{--}1)}$
$\DefineRemark{CO32}{\mathrm{CO} (3\textnormal{--}2)}$
$\DefineRemark{CO43}{\mathrm{CO} (4\textnormal{--}3)}$
$\DefineRemark{COJ10}{\mathrm{CO} (J=1 \to 0)}$
$\DefineRemark{COJ21}{\mathrm{CO} (J=2 \to 1)}$
$\DefineRemark{COJ32}{\mathrm{CO} (J=3 \to 2)}$
$\DefineRemark{COJ43}{\mathrm{CO} (J=4 \to 3)}$
$\DefineRemark{RCI10CO10}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{10}}}$
$}$
$\DefineRemark{RCI10CO21}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{21}}}$
$}$
$\DefineRemark{RCI10CO32}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{32}}}$
$}$
$\DefineRemark{RCI10CO43}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{43}}}$
$}$
$\DefineRemark{LPrimeCO21}{L^{\prime}_{\Remark{CO21}}}$
$\DefineRemark{LPrimeCI10}{L^{\prime}_{\Remark{CI10}}}$
$\DefineRemark{ICO10}{I_{\Remark{CO10}}}$
$\DefineRemark{ICO21}{I_{\Remark{CO21}}}$
$\DefineRemark{ICO32}{I_{\Remark{CO32}}}$
$\DefineRemark{ICO43}{I_{\Remark{CO43}}}$
$\DefineRemark{ICI10}{I_{\Remark{CI10}}}$
$\DefineRemark{kms}{\mathrm{km s^{-1}}}$
$\DefineRemark{Kkms}{\mathrm{K km s^{-1}}}$
$\DefineRemark{Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\DefineRemark{MJysr}{\mathrm{MJy sr^{-1}}}$
$\DefineRemark{R42}{\ifmmode{R_{42}}\else{R_{42}}\fi}$
$\DefineRemark{R41}{\ifmmode{R_{41}}\else{R_{41}}\fi}$
$\DefineRemark{R31}{\ifmmode{R_{31}}\else{R_{31}}\fi}$
$\DefineRemark{R21}{\ifmmode{R_{21}}\else{R_{21}}\fi}$
$\DefineRemark{R02}{\ifmmode{R_{\rm C\text{\sc{i}}CO}}\else{R_{\rm C\text{\sc{i}}CO}}\fi}$
$\DefineRemark{RCICO}{\ifmmode{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\else{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\fi}$
$\newcommand{\figurename}{Fig.}$
$\accepted{December 7, 2022}$
$\submitjournal{ApJL}$
$\shorttitle{Stellar Feedback, Molecular Excitation and Dissociation in NGC~1365}$
$\shortauthors{Liu et al.}$
$\begin{document}$
$\title{PHANGS-JWST First Results: Stellar Feedback-Driven Excitation and Dissociation of Molecular Gas in the Starburst Ring of NGC 1365?$
$}$
$\correspondingauthor{Daizhong Liu}$
$\email{dzliu@mpe.mpg.de, astro.dzliu@gmail.com}$
$\author[0000-0001-9773-7479]{Daizhong Liu}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-3933-7677]{Eva Schinnerer}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0001-5301-1326]{Yixian Cao}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-2545-1700]{Adam Leroy}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\author[0000-0003-1242-505X]{Antonio Usero}$
$\affiliation{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\author[0000-0002-5204-2259]{Erik Rosolowsky}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\author[0000-0002-6155-7166]{Eric Emsellem}$
$\affiliation{European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\affiliation{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\author[0000-0002-8804-0212]{J.~M.~Diederik~Kruijssen}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0002-5635-5180]{M\'elanie Chevance}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0001-6708-1317]{Simon C. O. Glover}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0001-6113-6241]{Mattia C. Sormani}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5480-5686]{Alberto D. Bolatto}$
$\affiliation{Department of Astronomy and Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}$
$\author[0000-0003-0378-4667]{Jiayi~Sun}$
$\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\author[0000-0002-9333-387X]{Sophia K. Stuber}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-4209-1599]{Yu-Hsuan Teng}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0003-0166-9745]{Frank Bigiel}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0003-0583-7363]{Ivana Be\v{s}li\'c}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-3247-5321]{Kathryn~Grasha}$
$\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\author[0000-0001-9656-7682]{Jonathan~D.~Henshaw}$
$\affiliation{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-0410-4504]{Ashley.~T.~Barnes}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-8760-6157]{Jakob~S.~den Brok}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-2501-9328]{Toshiki Saito}$
$\affiliation{National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo, 181-8588, Japan}$
$\author[0000-0002-5782-9093]{Daniel~A.~Dale}$
$\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\author[0000-0002-7365-5791]{Elizabeth~J.~Watkins}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0002-1370-6964]{Hsi-An Pan}$
$\affiliation{Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan}$
$\author[0000-0002-0560-3172]{Ralf S. Klessen}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Universit\"at Heidelberg, Interdisziplin\"ares Zentrum f\"ur Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5259-2314]{Gagandeep~S. Anand}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1943-723X]{Sinan Deger}$
$\affiliation{The Oskar Klein Centre for Cosmoparticle Physics, Department of Physics, Stockholm University, AlbaNova, Stockholm, SE-106 91, Sweden}$
$\author[0000-0002-4755-118X]{Oleg V. Egorov}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0001-5310-467X]{Christopher~M. Faesi}$
$\affiliation{University of Massachusetts-Amherst, 710 North Pleasant Street, Amherst, MA 01003, USA}$
$\author[0000-0002-8806-6308]{Hamid Hassani}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, Alberta, T6G 2E1, Canada}$
$\author[0000-0003-3917-6460]{Kirsten L. Larson}$
$\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2278-9407]{Janice~C. Lee}$
$\affiliation{Gemini Observatory/NSF's NOIRLab, 950 N. Cherry Avenue, Tucson, AZ 85719, USA; Caltech/IPAC, California Institute of Technology, Pasadena, CA 91125, USA}$
$\author[0000-0002-1790-3148]{Laura A. Lopez}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\affiliation{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\affiliation{Flatiron Institute, Center for Computational Astrophysics, NY 10010, USA}$
$\author[0000-0003-3061-6546]{Jér\^{o}me Pety}$
$\affiliation{IRAM, 300 rue de la Piscine, 38400 Saint Martin d'H\`eres, France}$
$\affiliation{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris}$
$\author[0000-0002-4378-8534]{Karin Sandstrom}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0002-8528-7340]{David~A. Thilker}$
$\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0002-3784-7032]{Bradley~C. Whitmore}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0012-2142]{Thomas G. Williams}$
$\affiliation{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\suppressAffiliations$
$\begin{abstract}$
$We compare embedded young massive star clusters (YMCs) to (sub-)millimeter line observations tracing the excitation and dissociation of molecular gas in the starburst ring of NGC~1365. This galaxy hosts one of the strongest nuclear starbursts and richest populations of YMCs within 20~Mpc.$
$Here we combine near-/mid-IR PHANGS-JWST imaging with new ALMA multi-J CO(1--0, 2--1 and 4--3) and [\CI](1--0) mapping, which we use to trace CO excitation via \Remark{R42}=\Remark{ICO43}/\Remark{ICO21} and \Remark{R21}=\Remark{ICO21}/\Remark{ICO10} and dissociation via \Remark{RCICO}=\Remark{ICI10}/\Remark{ICO21} at 330~pc resolution.$
$We find that the gas flowing into the starburst ring from northeast to southwest appears strongly affected by stellar feedback, showing decreased excitation (lower \Remark{R42}) and increased signatures of dissociation (higher \Remark{RCICO}) in the downstream regions.$
$There, radiative transfer modelling suggests that the molecular gas density decreases and temperature and \XCICO abundance ratio increase.$
$We compare \Remark{R42} and \Remark{RCICO} with local conditions across the regions and find that both correlate with near-IR 2~\um emission tracing the YMCs and with both PAH (11.3~\um) and dust continuum (21~\um) emission.$
$In general, \Remark{RCICO} exhibits \sim 0.1~dex tighter correlations than \Remark{R42}, suggesting \CI  to be a more sensitive tracer of changing physical conditions in the NGC~1365 starburst than CO(4--3).$
$Our results are consistent with a scenario where gas flows into the two arm regions along the bar, becomes condensed/shocked, forms YMCs, and then these YMCs heat and dissociate the gas.$
$\end{abstract}$
$\keywords{galaxies: ISM --- galaxies: star formation --- ISM: molecules --- ISM: atoms}$
$\n\end{document}}}$
$\newcommand{\nthresh}{n_{\mathrm{H_2,thresh}}}$
$\newcommand{\nH2}{n_{\mathrm{H}_2}}$
$\newcommand{\nHtwo}{n_{\mathrm{H}_2}}$
$\newcommand{\NH2_}{N_{\mathrm{H}_2}}$
$\newcommand{\percmcubic}{\mathrm{cm}^{-3}}$
$\newcommand{\percmsquare}{\mathrm{cm}^{-2}}$
$\newcommand{\dvddr}{\mathrm{d}v/\mathrm{d}r}$
$\newcommand{\lgNH2_}{\mathrm{log}_{10} (N_{\mathrm{H}_2} / \mathrm{cm^{-2}})}$
$\newcommand{\lgnmean}{\mathrm{log}_{10} \left<n_{\mathrm{H_2}}\right>}$
$\newcommand{\lgnHtwo}{\mathrm{log} n_{\mathrm{H_2}}}$
$\newcommand{\lgnthresh}{\mathrm{log}_{10} (n_{\mathrm{H_2,thresh}})}$
$\newcommand{\alphavir}{\alpha_{\mathrm{vir}}}$
$\newcommand{\HIrom}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{\romannumeral 1}}}}$
$\newcommand{\HIIrom}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{\romannumeral 2}}}}$
$\newcommand{\CO}{\mathrm{CO}}$
$\newcommand{\metalZOH}{12+\log_{10}(\mathrm{O/H})}$
$\newcommand{\Msun}{\mathrm{M}_{\odot}}$
$\newcommand{\Lsun}{\mathrm{L}_{\odot}}$
$\newcommand{\Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\newcommand{\kms}{\mathrm{km s^{-1}}}$
$\newcommand{\SFR}{\mathrm{SFR}}$
$\newcommand{\LIR}{L_{\mathrm{IR}}}$
$\newcommand{\Mstar}{M_{\star}}$
$\newcommand{\Mdust}{M_{\mathrm{dust}}}$
$\newcommand{\Tdust}{T_{\mathrm{dust}}}$
$\newcommand{\Mgas}{M_{\mathrm{gas}}}$
$\newcommand{\Mmolgas}{M_{\mathrm{mol gas}}}$
$\newcommand{\Mtotalgas}{M_{\mathrm{total gas}}}$
$\newcommand{\Matomicgas}{M_{\mathrm{atomic gas}}}$
$\newcommand{\arcsec}{"}$
$\newcommand{\R52}{R_{52}}$
$\newcommand{\fPDR}{f_{\mathrm{PDR}}}$
$\newcommand{\fmol}{f_{\mathrm{mol}}}$
$\newcommand{\fH2_}{f_{\mathrm{H_2}}}$
$\newcommand{\MH2_}{M_{\mathrm{H_2}}}$
$\newcommand{\XCICO}{[\mathrm{C}{\text{\sc{i}}/\mathrm{CO}]}}$
$\newcommand{\micron}{\mu\mathrm{m}}$
$\newcommand{\Jykms}{\mathrm{Jy km/s}}$
$\newcommand{\Kkms_}{\mathrm{K km/s}}$
$\newcommand{\LprimeCO}{L^{\prime}_{\mathrm{CO}}}$
$\newcommand{\MAGPHYS}{\textsc{MAGPHYS}}$
$\newcommand{\michi2}{\textsc{MiChi2}}$
$\newcommand{\um}{\ifmmode{\mu\mathrm{m}}\else{\mu\mathrm{m}}\fi}$
$\newcommand{\cofour}{CO(4--3)}$
$\newcommand{\cotwo}{CO(2--1)}$
$\newcommand{\J}[2][]{$
$	\ifthenelse{\isempty{#1}}{\begingroup$
$\newcommand{\JlowerN}{\}ADD{#2}{-1}{\JlowerN}$
$		J=#2{\to}\JlowerN$
$		\endgroup$
$	}{J=#1{\to}#2}$
$}$
$\newcommand{\rom}[1]{{{\uppercase\expandafter{\romannumeral #1}}}}$
$\newcommand{\romup}[1]{{\textup{\uppercase\expandafter{\romannumeral #1}}}}$
$\newcommand{\incode}[1]{{\raggedright\lstinline|#1|}}$
$\newcommand{\incodep}[1]{{\raggedright\lstinline|"#1"|}}$
$\newcommand{\neutralcarbon}{\ifmmode \text{C\textsc{i}} \else{\sc C i}\fi}$
$\newcommand{\ionizedcarbon}{\ifmmode \text{C\textsc{ii}} \else{\sc C ii}\fi}$
$\newcommand{\neutralhydrogen}{\ifmmode \text{H\textsc{i}} \else{\sc H i}\fi}$
$\newcommand{\ionizedhydrogen}{\ifmmode \text{H\textsc{ii}} \else{\sc H ii}\fi}$
$\newcommand{\carbonmonoxide}{\ifmmode \text{\textsc{CO}} \else{\sc CO}\fi}$
$\newcommand{\HII}{\ionizedhydrogen}$
$\newcommand{\CI}{\neutralcarbon}$
$\newcommand{\CO}{\carbonmonoxide}$
$\newcommand{\}{alphaCO}{\relax\ifmmode$
$    \alpha_\mathrm{CO}$
$    \else{\alpha_\mathrm{CO}}\fi$
$    }$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{\Remark}[1]{\csname rmk-#1\endcsname}$
$\DefineRemark{CI3P13P0}{[\mathrm{C}{\text{\sc{i}}}]\;(^{3}P_{1}\textnormal{--}^{3}P_{0})}$
$\DefineRemark{CI}{\mathrm{C}{\text{\sc{i}}}}$
$\DefineRemark{CI10}{[\mathrm{C}{\text{\sc{i}}}] (1\textnormal{--}0)}$
$\DefineRemark{CI21}{[\mathrm{C}{\text{\sc{i}}}] (2\textnormal{--}1)}$
$\DefineRemark{CO}{\mathrm{CO}}$
$\DefineRemark{CO10}{\mathrm{CO} (1\textnormal{--}0)}$
$\DefineRemark{CO21}{\mathrm{CO} (2\textnormal{--}1)}$
$\DefineRemark{CO32}{\mathrm{CO} (3\textnormal{--}2)}$
$\DefineRemark{CO43}{\mathrm{CO} (4\textnormal{--}3)}$
$\DefineRemark{COJ10}{\mathrm{CO} (J=1 \to 0)}$
$\DefineRemark{COJ21}{\mathrm{CO} (J=2 \to 1)}$
$\DefineRemark{COJ32}{\mathrm{CO} (J=3 \to 2)}$
$\DefineRemark{COJ43}{\mathrm{CO} (J=4 \to 3)}$
$\DefineRemark{RCI10CO10}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{10}}}$
$}$
$\DefineRemark{RCI10CO21}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{21}}}$
$}$
$\DefineRemark{RCI10CO32}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{32}}}$
$}$
$\DefineRemark{RCI10CO43}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{43}}}$
$}$
$\DefineRemark{LPrimeCO21}{L^{\prime}_{\Remark{CO21}}}$
$\DefineRemark{LPrimeCI10}{L^{\prime}_{\Remark{CI10}}}$
$\DefineRemark{ICO10}{I_{\Remark{CO10}}}$
$\DefineRemark{ICO21}{I_{\Remark{CO21}}}$
$\DefineRemark{ICO32}{I_{\Remark{CO32}}}$
$\DefineRemark{ICO43}{I_{\Remark{CO43}}}$
$\DefineRemark{ICI10}{I_{\Remark{CI10}}}$
$\DefineRemark{kms}{\mathrm{km s^{-1}}}$
$\DefineRemark{Kkms}{\mathrm{K km s^{-1}}}$
$\DefineRemark{Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\DefineRemark{MJysr}{\mathrm{MJy sr^{-1}}}$
$\DefineRemark{R42}{\ifmmode{R_{42}}\else{R_{42}}\fi}$
$\DefineRemark{R41}{\ifmmode{R_{41}}\else{R_{41}}\fi}$
$\DefineRemark{R31}{\ifmmode{R_{31}}\else{R_{31}}\fi}$
$\DefineRemark{R21}{\ifmmode{R_{21}}\else{R_{21}}\fi}$
$\DefineRemark{R02}{\ifmmode{R_{\rm C\text{\sc{i}}CO}}\else{R_{\rm C\text{\sc{i}}CO}}\fi}$
$\DefineRemark{RCICO}{\ifmmode{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\else{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\fi}$
$\newcommand{\figurename}{Fig.}$
$\accepted{December 7, 2022}$
$\submitjournal{ApJL}$
$\shorttitle{Stellar Feedback, Molecular Excitation and Dissociation in NGC~1365}$
$\shortauthors{Liu et al.}$
$\begin{document}$
$\title{PHANGS-JWST First Results: Stellar Feedback-Driven Excitation and Dissociation of Molecular Gas in the Starburst Ring of NGC 1365?$
$}$
$\correspondingauthor{Daizhong Liu}$
$\email{dzliu@mpe.mpg.de, astro.dzliu@gmail.com}$
$\author[0000-0001-9773-7479]{Daizhong Liu}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-3933-7677]{Eva Schinnerer}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0001-5301-1326]{Yixian Cao}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-2545-1700]{Adam Leroy}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\author[0000-0003-1242-505X]{Antonio Usero}$
$\affiliation{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\author[0000-0002-5204-2259]{Erik Rosolowsky}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\author[0000-0002-6155-7166]{Eric Emsellem}$
$\affiliation{European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\affiliation{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\author[0000-0002-8804-0212]{J.~M.~Diederik~Kruijssen}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0002-5635-5180]{M\'elanie Chevance}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0001-6708-1317]{Simon C. O. Glover}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0001-6113-6241]{Mattia C. Sormani}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5480-5686]{Alberto D. Bolatto}$
$\affiliation{Department of Astronomy and Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}$
$\author[0000-0003-0378-4667]{Jiayi~Sun}$
$\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\author[0000-0002-9333-387X]{Sophia K. Stuber}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-4209-1599]{Yu-Hsuan Teng}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0003-0166-9745]{Frank Bigiel}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0003-0583-7363]{Ivana Be\v{s}li\'c}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-3247-5321]{Kathryn~Grasha}$
$\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\author[0000-0001-9656-7682]{Jonathan~D.~Henshaw}$
$\affiliation{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-0410-4504]{Ashley.~T.~Barnes}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-8760-6157]{Jakob~S.~den Brok}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-2501-9328]{Toshiki Saito}$
$\affiliation{National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo, 181-8588, Japan}$
$\author[0000-0002-5782-9093]{Daniel~A.~Dale}$
$\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\author[0000-0002-7365-5791]{Elizabeth~J.~Watkins}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0002-1370-6964]{Hsi-An Pan}$
$\affiliation{Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan}$
$\author[0000-0002-0560-3172]{Ralf S. Klessen}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Universit\"at Heidelberg, Interdisziplin\"ares Zentrum f\"ur Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5259-2314]{Gagandeep~S. Anand}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1943-723X]{Sinan Deger}$
$\affiliation{The Oskar Klein Centre for Cosmoparticle Physics, Department of Physics, Stockholm University, AlbaNova, Stockholm, SE-106 91, Sweden}$
$\author[0000-0002-4755-118X]{Oleg V. Egorov}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0001-5310-467X]{Christopher~M. Faesi}$
$\affiliation{University of Massachusetts-Amherst, 710 North Pleasant Street, Amherst, MA 01003, USA}$
$\author[0000-0002-8806-6308]{Hamid Hassani}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, Alberta, T6G 2E1, Canada}$
$\author[0000-0003-3917-6460]{Kirsten L. Larson}$
$\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2278-9407]{Janice~C. Lee}$
$\affiliation{Gemini Observatory/NSF's NOIRLab, 950 N. Cherry Avenue, Tucson, AZ 85719, USA; Caltech/IPAC, California Institute of Technology, Pasadena, CA 91125, USA}$
$\author[0000-0002-1790-3148]{Laura A. Lopez}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\affiliation{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\affiliation{Flatiron Institute, Center for Computational Astrophysics, NY 10010, USA}$
$\author[0000-0003-3061-6546]{Jér\^{o}me Pety}$
$\affiliation{IRAM, 300 rue de la Piscine, 38400 Saint Martin d'H\`eres, France}$
$\affiliation{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris}$
$\author[0000-0002-4378-8534]{Karin Sandstrom}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0002-8528-7340]{David~A. Thilker}$
$\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0002-3784-7032]{Bradley~C. Whitmore}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0012-2142]{Thomas G. Williams}$
$\affiliation{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\suppressAffiliations$
$\begin{abstract}$
$We compare embedded young massive star clusters (YMCs) to (sub-)millimeter line observations tracing the excitation and dissociation of molecular gas in the starburst ring of NGC~1365. This galaxy hosts one of the strongest nuclear starbursts and richest populations of YMCs within 20~Mpc.$
$Here we combine near-/mid-IR PHANGS-JWST imaging with new ALMA multi-J CO(1--0, 2--1 and 4--3) and [\CI](1--0) mapping, which we use to trace CO excitation via \Remark{R42}=\Remark{ICO43}/\Remark{ICO21} and \Remark{R21}=\Remark{ICO21}/\Remark{ICO10} and dissociation via \Remark{RCICO}=\Remark{ICI10}/\Remark{ICO21} at 330~pc resolution.$
$We find that the gas flowing into the starburst ring from northeast to southwest appears strongly affected by stellar feedback, showing decreased excitation (lower \Remark{R42}) and increased signatures of dissociation (higher \Remark{RCICO}) in the downstream regions.$
$There, radiative transfer modelling suggests that the molecular gas density decreases and temperature and \XCICO abundance ratio increase.$
$We compare \Remark{R42} and \Remark{RCICO} with local conditions across the regions and find that both correlate with near-IR 2~\um emission tracing the YMCs and with both PAH (11.3~\um) and dust continuum (21~\um) emission.$
$In general, \Remark{RCICO} exhibits \sim 0.1~dex tighter correlations than \Remark{R42}, suggesting \CI  to be a more sensitive tracer of changing physical conditions in the NGC~1365 starburst than CO(4--3).$
$Our results are consistent with a scenario where gas flows into the two arm regions along the bar, becomes condensed/shocked, forms YMCs, and then these YMCs heat and dissociate the gas.$
$\end{abstract}$
$\keywords{galaxies: ISM --- galaxies: star formation --- ISM: molecules --- ISM: atoms}$
$\n\end{document}}$
$\newcommand{\lgnHtwo}{\mathrm{log} n_{\mathrm{H_2}}}$
$\newcommand{\lgnthresh}{\mathrm{log}_{10} (n_{\mathrm{H_2,thresh}})}$
$\newcommand{\alphavir}{\alpha_{\mathrm{vir}}}$
$\newcommand{\HIrom}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{\romannumeral 1}}}}$
$\newcommand{\HIIrom}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{\romannumeral 2}}}}$
$\newcommand{\CO}{\mathrm{CO}}$
$\newcommand{\metalZOH}{12+\log_{10}(\mathrm{O/H})}$
$\newcommand{\Msun}{\mathrm{M}_{\odot}}$
$\newcommand{\Lsun}{\mathrm{L}_{\odot}}$
$\newcommand{\Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\newcommand{\kms}{\mathrm{km s^{-1}}}$
$\newcommand{\SFR}{\mathrm{SFR}}$
$\newcommand{\LIR}{L_{\mathrm{IR}}}$
$\newcommand{\Mstar}{M_{\star}}$
$\newcommand{\Mdust}{M_{\mathrm{dust}}}$
$\newcommand{\Tdust}{T_{\mathrm{dust}}}$
$\newcommand{\Mgas}{M_{\mathrm{gas}}}$
$\newcommand{\Mmolgas}{M_{\mathrm{mol gas}}}$
$\newcommand{\Mtotalgas}{M_{\mathrm{total gas}}}$
$\newcommand{\Matomicgas}{M_{\mathrm{atomic gas}}}$
$\newcommand{\arcsec}{"}$
$\newcommand{\R52}{R_{52}}$
$\newcommand{\fPDR}{f_{\mathrm{PDR}}}$
$\newcommand{\fmol}{f_{\mathrm{mol}}}$
$\newcommand{\fH2_}{f_{\mathrm{H_2}}}$
$\newcommand{\MH2_}{M_{\mathrm{H_2}}}$
$\newcommand{\XCICO}{[\mathrm{C}{\text{\sc{i}}/\mathrm{CO}]}}$
$\newcommand{\micron}{\mu\mathrm{m}}$
$\newcommand{\Jykms}{\mathrm{Jy km/s}}$
$\newcommand{\Kkms_}{\mathrm{K km/s}}$
$\newcommand{\LprimeCO}{L^{\prime}_{\mathrm{CO}}}$
$\newcommand{\MAGPHYS}{\textsc{MAGPHYS}}$
$\newcommand{\michi2}{\textsc{MiChi2}}$
$\newcommand{\um}{\ifmmode{\mu\mathrm{m}}\else{\mu\mathrm{m}}\fi}$
$\newcommand{\cofour}{CO(4--3)}$
$\newcommand{\cotwo}{CO(2--1)}$
$\newcommand{\HII}{\ionizedhydrogen}$
$\newcommand{\CI}{\neutralcarbon}$
$\newcommand{\CO}{\carbonmonoxide}$
$\newcommand{\}{alphaCO}$
$\newcommand{\JlowerN}{\}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand{\TODO}[2][]{$
$\ifthenelse{\isempty{#1}}{\textbf{\textcolor{red}{TODO: #2}}}{\textbf{\textcolor{red}{TODO (#1) #2}}}$
$}$
$\newcommand{\J}[2][]{$
$	\ifthenelse{\isempty{#1}}{\begingroup$
$\newcommand{\JlowerN}{\}ADD{#2}{-1}{\JlowerN}$
$		J=#2{\to}\JlowerN$
$		\endgroup$
$	}{J=#1{\to}#2}$
$}$
$\newcommand{$\rom$}[1]{{{\uppercase\expandafter{$\rom$annumeral #1}}}}$
$\newcommand{$\rom$up}[1]{{\textup{\uppercase\expandafter{$\rom$annumeral #1}}}}$
$\newcommand{$\incode$}[1]{{\raggedright\lstinline|#1|}}$
$\newcommand{$\incode$p}[1]{{\raggedright\lstinline|"#1"|}}$
$\newcommand{$\neutralcarbon$}{\ifmmode \text{C$\textsc${i}} \else{\sc C i}\fi}$
$\newcommand{$\ion$izedcarbon}{\ifmmode \text{C$\textsc${ii}} \else{\sc C ii}\fi}$
$\newcommand{$\neutralhydrogen$}{\ifmmode \text{H$\textsc${i}} \else{\sc H i}\fi}$
$\newcommand{$\ion$izedhydrogen}{\ifmmode \text{H$\textsc${ii}} \else{\sc H ii}\fi}$
$\newcommand{$\carbonmonoxide$}{\ifmmode \text{$\textsc${CO}} \else{\sc CO}\fi}$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{$\Remark$}[1]{\csname rmk-#1\endcsname}$
$\newcommand$
$\newcommand{$\figurename$}{Fig.}$
$\newcommand{$\gtsima$}{\; \buildrel > \over \sim \;}$
$\newcommand{$\ltsima$}{\; \buildrel < \over \sim \;}$
$\newcommand{$\prosima$}{\; \buildrel \propto \over \sim \;}$
$\newcommand{$\gsim$}{\lower.5ex\hbox{$\gtsima$}}$
$\newcommand{$\lsim$}{\lower.5ex\hbox{$\ltsima$}}$
$\newcommand{$\simgt$}{\lower.5ex\hbox{$\gtsima$}}$
$\newcommand{$\simlt$}{\lower.5ex\hbox{$\ltsima$}}$
$\newcommand{$\simpr$}{\lower.5ex\hbox{$\prosima$}}$
$\newcommand{$\la$}{$\lsim$}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{$\eeq$}{\end{equation}}$
$\newcommand{\8mu}{8 \mu{\rm m}}$
$\newcommand{\16mu}{16 \mu{\rm m}}$
$\newcommand{\24mu}{24 \mu{\rm m}}$
$\newcommand{\70mu}{70 \mu{\rm m}}$
$\newcommand{\Umean}{\left<U\right>}$
$\newcommand{$\Umin$}{U_{\mathrm{min}}}$
$\newcommand{$\Umax$}{U_{\mathrm{max}}}$
$\newcommand{$\SNR$}{\mathrm{S/N}}$
$\newcommand{$\alphaCO$}{\alpha_{\mathrm{CO}}}$
$\newcommand{$\deltaGDzero$}{\delta_{\mathrm{GDR,0}}}$
$\newcommand{$\deltaGDR$}{\delta_{\mathrm{GDR}}}$
$\newcommand{$\deltaDGR$}{\delta_{\mathrm{DGR}}}$
$\newcommand{$\deltaISM$}{\delta_{\mathrm{ISM}}}$
$\newcommand{$\deltaGas$}{\mu_{\mathrm{molgas}}}$
$\newcommand{$\deltaTotalGas$}{\mu_{\mathrm{totalgas}}}$
$\newcommand{$\deltaTotalGDR$}{\delta_{\mathrm{totalGDR}}}$
$\newcommand{$\deltaMS$}{\delta{\mathrm{MS}}}$
$\newcommand{$\DeltaMS$}{\Delta{\mathrm{MS}}}$
$\newcommand{$\GDR$}{\mathrm{GDR}}$
$\newcommand{$\DGR$}{\mathrm{DGR}}$
$\newcommand{$\Tkin$}{T_{\mathrm{kin}}}$
$\newcommand{\nmean}{\left<n_{\mathrm{H}_2}\right>}$
$\newcommand{$\nthresh$}{n_{\mathrm{H_2,thresh}}}$
$\newcommand{$\nH$2}{n_{\mathrm{H}_2}}$
$\newcommand{$\nH$two}{n_{\mathrm{H}_2}}$
$\newcommand{$\NH$2_}{N_{\mathrm{H}_2}}$
$\newcommand{$\percmcubic$}{\mathrm{cm}^{-3}}$
$\newcommand{$\percmsquare$}{\mathrm{cm}^{-2}}$
$\newcommand{$\dvddr$}{\mathrm{d}v/\mathrm{d}r}$
$\newcommand{$\lgNH$2_}{\mathrm{log}_{10} (N_{\mathrm{H}_2} / \mathrm{cm^{-2}})}$
$\newcommand{\lgnmean}{\mathrm{log}_{10} \left<n_{\mathrm{H_2}}\right>}$
$\newcommand{$\lgnHtwo$}{\mathrm{log} n_{\mathrm{H_2}}}$
$\newcommand{$\lgnthresh$}{\mathrm{log}_{10} (n_{\mathrm{H_2,thresh}})}$
$\newcommand{$\alphavir$}{\alpha_{\mathrm{vir}}}$
$\newcommand{$\HIrom$}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{$\rom$annumeral 1}}}}$
$\newcommand{$\HIIrom$}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{$\rom$annumeral 2}}}}$
$\newcommand{$\CO$}{\mathrm{CO}}$
$\newcommand{$\metalZOH$}{12+\log_{10}(\mathrm{O/H})}$
$\newcommand{$\Msun$}{\mathrm{M}_{\odot}}$
$\newcommand{$\Lsun$}{\mathrm{L}_{\odot}}$
$\newcommand{$\Kkmspc$2}{\mathrm{K km s^{-1} pc^{2}}}$
$\newcommand{$\kms$}{\mathrm{km s^{-1}}}$
$\newcommand{$\SFR$}{\mathrm{SFR}}$
$\newcommand{$\LIR$}{L_{\mathrm{IR}}}$
$\newcommand{$\Mstar$}{M_{\star}}$
$\newcommand{$\Mdust$}{M_{\mathrm{dust}}}$
$\newcommand{$\Tdust$}{T_{\mathrm{dust}}}$
$\newcommand{$\Mgas$}{M_{\mathrm{gas}}}$
$\newcommand{$\Mmolgas$}{M_{\mathrm{mol gas}}}$
$\newcommand{$\Mtotalgas$}{M_{\mathrm{total gas}}}$
$\newcommand{$\Matomicgas$}{M_{\mathrm{atomic gas}}}$
$\newcommand{$\arcsec$}{"}$
$\newcommand{$\R$52}{R_{52}}$
$\newcommand{$\fPDR$}{f_{\mathrm{PDR}}}$
$\newcommand{$\fmol$}{f_{\mathrm{mol}}}$
$\newcommand{$\fH$2_}{f_{\mathrm{H_2}}}$
$\newcommand{$\MH$2_}{M_{\mathrm{H_2}}}$
$\newcommand{$\XCICO$}{[\mathrm{C}{\text{\sc{i}}/\mathrm{CO}]}}$
$\newcommand{$\micron$}{\mu\mathrm{m}}$
$\newcommand{$\Jykms$}{\mathrm{Jy km/s}}$
$\newcommand{$\Kkms$_}{\mathrm{K km/s}}$
$\newcommand{$\LprimeCO$}{L^{\prime}_{\mathrm{CO}}}$
$\newcommand{$\MAGPHYS$}{$\textsc${MAGPHYS}}$
$\newcommand{$\michi$2}{$\textsc${MiChi2}}$
$\newcommand{$\um$}{\ifmmode{\mu\mathrm{m}}\else{\mu\mathrm{m}}\fi}$
$\newcommand{$\cofour$}{CO(4--3)}$
$\newcommand{$\cotwo$}{CO(2--1)}$
$\newcommand{\J}[2][]{$
$	\ifthenelse{\isempty{#1}}{\begingroup$
$\newcommand{\JlowerN}{\}ADD{#2}{-1}{\JlowerN}$
$		J=#2{\to}\JlowerN$
$		\endgroup$
$	}{J=#1{\to}#2}$
$}$
$\newcommand{$\rom$}[1]{{{\uppercase\expandafter{$\rom$annumeral #1}}}}$
$\newcommand{$\rom$up}[1]{{\textup{\uppercase\expandafter{$\rom$annumeral #1}}}}$
$\newcommand{$\incode$}[1]{{\raggedright\lstinline|#1|}}$
$\newcommand{$\incode$p}[1]{{\raggedright\lstinline|"#1"|}}$
$\newcommand{$\neutralcarbon$}{\ifmmode \text{C$\textsc${i}} \else{\sc C i}\fi}$
$\newcommand{$\ion$izedcarbon}{\ifmmode \text{C$\textsc${ii}} \else{\sc C ii}\fi}$
$\newcommand{$\neutralhydrogen$}{\ifmmode \text{H$\textsc${i}} \else{\sc H i}\fi}$
$\newcommand{$\ion$izedhydrogen}{\ifmmode \text{H$\textsc${ii}} \else{\sc H ii}\fi}$
$\newcommand{$\carbonmonoxide$}{\ifmmode \text{$\textsc${CO}} \else{\sc CO}\fi}$
$\newcommand{$\HII$}{$\ion$izedhydrogen}$
$\newcommand{$\CI$}{$\neutralcarbon$}$
$\newcommand{$\CO$}{$\carbonmonoxide$}$
$\newcommand{\}{alphaCO}{\relax\ifmmode$
$    \alpha_\mathrm{CO}$
$    \else{\alpha_\mathrm{CO}}\fi$
$    }$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{$\Remark$}[1]{\csname rmk-#1\endcsname}$
$\DefineRemark{CI3P13P0}{[\mathrm{C}{\text{\sc{i}}}]\;(^{3}P_{1}\textnormal{--}^{3}P_{0})}$
$\DefineRemark{CI}{\mathrm{C}{\text{\sc{i}}}}$
$\DefineRemark{CI10}{[\mathrm{C}{\text{\sc{i}}}] (1\textnormal{--}0)}$
$\DefineRemark{CI21}{[\mathrm{C}{\text{\sc{i}}}] (2\textnormal{--}1)}$
$\DefineRemark{CO}{\mathrm{CO}}$
$\DefineRemark{CO10}{\mathrm{CO} (1\textnormal{--}0)}$
$\DefineRemark{CO21}{\mathrm{CO} (2\textnormal{--}1)}$
$\DefineRemark{CO32}{\mathrm{CO} (3\textnormal{--}2)}$
$\DefineRemark{CO43}{\mathrm{CO} (4\textnormal{--}3)}$
$\DefineRemark{COJ10}{\mathrm{CO} (J=1 \to 0)}$
$\DefineRemark{COJ21}{\mathrm{CO} (J=2 \to 1)}$
$\DefineRemark{COJ32}{\mathrm{CO} (J=3 \to 2)}$
$\DefineRemark{COJ43}{\mathrm{CO} (J=4 \to 3)}$
$\DefineRemark{RCI10CO10}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{10}}}$
$}$
$\DefineRemark{RCI10CO21}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{21}}}$
$}$
$\DefineRemark{RCI10CO32}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{32}}}$
$}$
$\DefineRemark{RCI10CO43}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{43}}}$
$}$
$\DefineRemark{LPrimeCO21}{L^{\prime}_{$\Remark${CO21}}}$
$\DefineRemark{LPrimeCI10}{L^{\prime}_{$\Remark${CI10}}}$
$\DefineRemark{ICO10}{I_{$\Remark${CO10}}}$
$\DefineRemark{ICO21}{I_{$\Remark${CO21}}}$
$\DefineRemark{ICO32}{I_{$\Remark${CO32}}}$
$\DefineRemark{ICO43}{I_{$\Remark${CO43}}}$
$\DefineRemark{ICI10}{I_{$\Remark${CI10}}}$
$\DefineRemark{kms}{\mathrm{km s^{-1}}}$
$\DefineRemark{Kkms}{\mathrm{K km s^{-1}}}$
$\DefineRemark{Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\DefineRemark{MJysr}{\mathrm{MJy sr^{-1}}}$
$\DefineRemark{R42}{\ifmmode{R_{42}}\else{R_{42}}\fi}$
$\DefineRemark{R41}{\ifmmode{R_{41}}\else{R_{41}}\fi}$
$\DefineRemark{R31}{\ifmmode{R_{31}}\else{R_{31}}\fi}$
$\DefineRemark{R21}{\ifmmode{R_{21}}\else{R_{21}}\fi}$
$\DefineRemark{R02}{\ifmmode{R_{\rm C\text{\sc{i}}CO}}\else{R_{\rm C\text{\sc{i}}CO}}\fi}$
$\DefineRemark{RCICO}{\ifmmode{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\else{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\fi}$
$\newcommand{$\figurename$}{Fig.}$
$\accepted{December 7, 2022}$
$\submitjournal{ApJL}$
$\shorttitle{Stellar Feedback, Molecular Excitation and Dissociation in NGC~1365}$
$\shortauthors{Liu et al.}$
$\begin{document}$
$\title{PHANGS-JWST First Results: Stellar Feedback-Driven Excitation and Dissociation of Molecular Gas in the Starburst Ring of NGC 1365?$
$}$
$\correspondingauthor{Daizhong Liu}$
$\email{dzliu@mpe.mpg.de, astro.dzliu@gmail.com}$
$\author[0000-0001-9773-7479]{Daizhong Liu}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-3933-7677]{Eva Schinnerer}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0001-5301-1326]{Yixian Cao}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-2545-1700]{Adam Leroy}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\author[0000-0003-1242-505X]{Antonio Usero}$
$\affiliation{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\author[0000-0002-5204-2259]{Erik Rosolowsky}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\author[0000-0002-6155-7166]{Eric Emsellem}$
$\affiliation{European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\affiliation{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\author[0000-0002-8804-0212]{J.~M.~Diederik~Kruijssen}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0002-5635-5180]{M\'elanie Chevance}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0001-6708-1317]{Simon C. O. Glover}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0001-6113-6241]{Mattia C. Sormani}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5480-5686]{Alberto D. Bolatto}$
$\affiliation{Department of Astronomy and Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}$
$\author[0000-0003-0378-4667]{Jiayi~Sun}$
$\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\author[0000-0002-9333-387X]{Sophia K. Stuber}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-4209-1599]{Yu-Hsuan Teng}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0003-0166-9745]{Frank Bigiel}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0003-0583-7363]{Ivana Be\v{s}li\'c}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-3247-5321]{Kathryn~Grasha}$
$\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\author[0000-0001-9656-7682]{Jonathan~D.~Henshaw}$
$\affiliation{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-0410-4504]{Ashley.~T.~Barnes}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-8760-6157]{Jakob~S.~den Brok}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-2501-9328]{Toshiki Saito}$
$\affiliation{National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo, 181-8588, Japan}$
$\author[0000-0002-5782-9093]{Daniel~A.~Dale}$
$\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\author[0000-0002-7365-5791]{Elizabeth~J.~Watkins}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0002-1370-6964]{Hsi-An Pan}$
$\affiliation{Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan}$
$\author[0000-0002-0560-3172]{Ralf S. Klessen}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Universit\"at Heidelberg, Interdisziplin\"ares Zentrum f\"ur Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5259-2314]{Gagandeep~S. Anand}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1943-723X]{Sinan Deger}$
$\affiliation{The Oskar Klein Centre for Cosmoparticle Physics, Department of Physics, Stockholm University, AlbaNova, Stockholm, SE-106 91, Sweden}$
$\author[0000-0002-4755-118X]{Oleg V. Egorov}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0001-5310-467X]{Christopher~M. Faesi}$
$\affiliation{University of Massachusetts-Amherst, 710 North Pleasant Street, Amherst, MA 01003, USA}$
$\author[0000-0002-8806-6308]{Hamid Hassani}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, Alberta, T6G 2E1, Canada}$
$\author[0000-0003-3917-6460]{Kirsten L. Larson}$
$\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2278-9407]{Janice~C. Lee}$
$\affiliation{Gemini Observatory/NSF's NOIRLab, 950 N. Cherry Avenue, Tucson, AZ 85719, USA; Caltech/IPAC, California Institute of Technology, Pasadena, CA 91125, USA}$
$\author[0000-0002-1790-3148]{Laura A. Lopez}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\affiliation{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\affiliation{Flatiron Institute, Center for Computational Astrophysics, NY 10010, USA}$
$\author[0000-0003-3061-6546]{Jér\^{o}me Pety}$
$\affiliation{IRAM, 300 rue de la Piscine, 38400 Saint Martin d'H\`eres, France}$
$\affiliation{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris}$
$\author[0000-0002-4378-8534]{Karin Sandstrom}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0002-8528-7340]{David~A. Thilker}$
$\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0002-3784-7032]{Bradley~C. Whitmore}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0012-2142]{Thomas G. Williams}$
$\affiliation{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\suppressAffiliations$
$\begin{abstract}$
$We compare embedded young massive star clusters (YMCs) to (sub-)millimeter line observations tracing the excitation and dissociation of molecular gas in the starburst ring of NGC~1365. This galaxy hosts one of the strongest nuclear starbursts and richest populations of YMCs within 20~Mpc.$
$Here we combine near-/mid-IR PHANGS-JWST imaging with new ALMA multi-J CO(1--0, 2--1 and 4--3) and [$\CI$](1--0) mapping, which we use to trace CO excitation via $\Remark${R42}=$\Remark${ICO43}/$\Remark${ICO21} and $\Remark${R21}=$\Remark${ICO21}/$\Remark${ICO10} and dissociation via $\Remark${RCICO}=$\Remark${ICI10}/$\Remark${ICO21} at 330~pc resolution.$
$We find that the gas flowing into the starburst ring from northeast to southwest appears strongly affected by stellar feedback, showing decreased excitation (lower $\Remark${R42}) and increased signatures of dissociation (higher $\Remark${RCICO}) in the downstream regions.$
$There, radiative transfer modelling suggests that the molecular gas density decreases and temperature and $\XCICO$ abundance ratio increase.$
$We compare $\Remark${R42} and $\Remark${RCICO} with local conditions across the regions and find that both correlate with near-IR 2~$\um$ emission tracing the YMCs and with both PAH (11.3~$\um$) and dust continuum (21~$\um$) emission.$
$In general, $\Remark${RCICO} exhibits \sim 0.1~dex tighter correlations than $\Remark${R42}, suggesting $\CI$  to be a more sensitive tracer of changing physical conditions in the NGC~1365 starburst than CO(4--3).$
$Our results are consistent with a scenario where gas flows into the two arm regions along the bar, becomes condensed/shocked, forms YMCs, and then these YMCs heat and dissociate the gas.$
$\end{abstract}$
$\keywords{galaxies: ISM --- galaxies: star formation --- ISM: molecules --- ISM: atoms}$
$\n\end{document}}}}\end{equation}}$
$\newcommand{$\eeq$}{\end{equation}}$
$\newcommand{\8mu}{8 \mu{\rm m}}$
$\newcommand{\16mu}{16 \mu{\rm m}}$
$\newcommand{\24mu}{24 \mu{\rm m}}$
$\newcommand{\70mu}{70 \mu{\rm m}}$
$\newcommand{\Umean}{\left<U\right>}$
$\newcommand{$\Umin$}{U_{\mathrm{min}}}$
$\newcommand{$\Umax$}{U_{\mathrm{max}}}$
$\newcommand{$\SNR$}{\mathrm{S/N}}$
$\newcommand{$\alphaCO$}{\alpha_{\mathrm{CO}}}$
$\newcommand{$\deltaGDzero$}{\delta_{\mathrm{GDR,0}}}$
$\newcommand{$\deltaGDR$}{\delta_{\mathrm{GDR}}}$
$\newcommand{$\deltaDGR$}{\delta_{\mathrm{DGR}}}$
$\newcommand{$\deltaISM$}{\delta_{\mathrm{ISM}}}$
$\newcommand{$\deltaGas$}{\mu_{\mathrm{molgas}}}$
$\newcommand{$\deltaTotalGas$}{\mu_{\mathrm{totalgas}}}$
$\newcommand{$\deltaTotalGDR$}{\delta_{\mathrm{totalGDR}}}$
$\newcommand{$\deltaMS$}{\delta{\mathrm{MS}}}$
$\newcommand{$\DeltaMS$}{\Delta{\mathrm{MS}}}$
$\newcommand{$\GDR$}{\mathrm{GDR}}$
$\newcommand{$\DGR$}{\mathrm{DGR}}$
$\newcommand{$\Tkin$}{T_{\mathrm{kin}}}$
$\newcommand{\nmean}{\left<n_{\mathrm{H}_2}\right>}$
$\newcommand{$\nthresh$}{n_{\mathrm{H_2,thresh}}}$
$\newcommand{$\nH$2}{n_{\mathrm{H}_2}}$
$\newcommand{$\nH$two}{n_{\mathrm{H}_2}}$
$\newcommand{$\NH$2_}{N_{\mathrm{H}_2}}$
$\newcommand{$\percmcubic$}{\mathrm{cm}^{-3}}$
$\newcommand{$\percmsquare$}{\mathrm{cm}^{-2}}$
$\newcommand{$\dvddr$}{\mathrm{d}v/\mathrm{d}r}$
$\newcommand{$\lgNH$2_}{\mathrm{log}_{10} (N_{\mathrm{H}_2} / \mathrm{cm^{-2}})}$
$\newcommand{\lgnmean}{\mathrm{log}_{10} \left<n_{\mathrm{H_2}}\right>}$
$\newcommand{$\lgnHtwo$}{\mathrm{log} n_{\mathrm{H_2}}}$
$\newcommand{$\lgnthresh$}{\mathrm{log}_{10} (n_{\mathrm{H_2,thresh}})}$
$\newcommand{$\alphavir$}{\alpha_{\mathrm{vir}}}$
$\newcommand{$\HIrom$}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{$\rom$annumeral 1}}}}$
$\newcommand{$\HIIrom$}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{$\rom$annumeral 2}}}}$
$\newcommand{$\CO$}{\mathrm{CO}}$
$\newcommand{$\metalZOH$}{12+\log_{10}(\mathrm{O/H})}$
$\newcommand{$\Msun$}{\mathrm{M}_{\odot}}$
$\newcommand{$\Lsun$}{\mathrm{L}_{\odot}}$
$\newcommand{$\Kkmspc$2}{\mathrm{K km s^{-1} pc^{2}}}$
$\newcommand{$\kms$}{\mathrm{km s^{-1}}}$
$\newcommand{$\SFR$}{\mathrm{SFR}}$
$\newcommand{$\LIR$}{L_{\mathrm{IR}}}$
$\newcommand{$\Mstar$}{M_{\star}}$
$\newcommand{$\Mdust$}{M_{\mathrm{dust}}}$
$\newcommand{$\Tdust$}{T_{\mathrm{dust}}}$
$\newcommand{$\Mgas$}{M_{\mathrm{gas}}}$
$\newcommand{$\Mmolgas$}{M_{\mathrm{mol gas}}}$
$\newcommand{$\Mtotalgas$}{M_{\mathrm{total gas}}}$
$\newcommand{$\Matomicgas$}{M_{\mathrm{atomic gas}}}$
$\newcommand{$\arcsec$}{"}$
$\newcommand{$\R$52}{R_{52}}$
$\newcommand{$\fPDR$}{f_{\mathrm{PDR}}}$
$\newcommand{$\fmol$}{f_{\mathrm{mol}}}$
$\newcommand{$\fH$2_}{f_{\mathrm{H_2}}}$
$\newcommand{$\MH$2_}{M_{\mathrm{H_2}}}$
$\newcommand{$\XCICO$}{[\mathrm{C}{\text{\sc{i}}/\mathrm{CO}]}}$
$\newcommand{$\micron$}{\mu\mathrm{m}}$
$\newcommand{$\Jykms$}{\mathrm{Jy km/s}}$
$\newcommand{$\Kkms$_}{\mathrm{K km/s}}$
$\newcommand{$\LprimeCO$}{L^{\prime}_{\mathrm{CO}}}$
$\newcommand{$\MAGPHYS$}{$\textsc${MAGPHYS}}$
$\newcommand{$\michi$2}{$\textsc${MiChi2}}$
$\newcommand{$\um$}{\ifmmode{\mu\mathrm{m}}\else{\mu\mathrm{m}}\fi}$
$\newcommand{$\cofour$}{CO(4--3)}$
$\newcommand{$\cotwo$}{CO(2--1)}$
$\newcommand{\J}[2][]{$
$	\ifthenelse{\isempty{#1}}{\begingroup$
$\newcommand{\JlowerN}{\}ADD{#2}{-1}{\JlowerN}$
$		J=#2{\to}\JlowerN$
$		\endgroup$
$	}{J=#1{\to}#2}$
$}$
$\newcommand{$\rom$}[1]{{{\uppercase\expandafter{$\rom$annumeral #1}}}}$
$\newcommand{$\rom$up}[1]{{\textup{\uppercase\expandafter{$\rom$annumeral #1}}}}$
$\newcommand{$\incode$}[1]{{\raggedright\lstinline|#1|}}$
$\newcommand{$\incode$p}[1]{{\raggedright\lstinline|"#1"|}}$
$\newcommand{$\neutralcarbon$}{\ifmmode \text{C$\textsc${i}} \else{\sc C i}\fi}$
$\newcommand{$\ion$izedcarbon}{\ifmmode \text{C$\textsc${ii}} \else{\sc C ii}\fi}$
$\newcommand{$\neutralhydrogen$}{\ifmmode \text{H$\textsc${i}} \else{\sc H i}\fi}$
$\newcommand{$\ion$izedhydrogen}{\ifmmode \text{H$\textsc${ii}} \else{\sc H ii}\fi}$
$\newcommand{$\carbonmonoxide$}{\ifmmode \text{$\textsc${CO}} \else{\sc CO}\fi}$
$\newcommand{$\HII$}{$\ion$izedhydrogen}$
$\newcommand{$\CI$}{$\neutralcarbon$}$
$\newcommand{$\CO$}{$\carbonmonoxide$}$
$\newcommand{\}{alphaCO}{\relax\ifmmode$
$    \alpha_\mathrm{CO}$
$    \else{\alpha_\mathrm{CO}}\fi$
$    }$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{$\Remark$}[1]{\csname rmk-#1\endcsname}$
$\DefineRemark{CI3P13P0}{[\mathrm{C}{\text{\sc{i}}}]\;(^{3}P_{1}\textnormal{--}^{3}P_{0})}$
$\DefineRemark{CI}{\mathrm{C}{\text{\sc{i}}}}$
$\DefineRemark{CI10}{[\mathrm{C}{\text{\sc{i}}}] (1\textnormal{--}0)}$
$\DefineRemark{CI21}{[\mathrm{C}{\text{\sc{i}}}] (2\textnormal{--}1)}$
$\DefineRemark{CO}{\mathrm{CO}}$
$\DefineRemark{CO10}{\mathrm{CO} (1\textnormal{--}0)}$
$\DefineRemark{CO21}{\mathrm{CO} (2\textnormal{--}1)}$
$\DefineRemark{CO32}{\mathrm{CO} (3\textnormal{--}2)}$
$\DefineRemark{CO43}{\mathrm{CO} (4\textnormal{--}3)}$
$\DefineRemark{COJ10}{\mathrm{CO} (J=1 \to 0)}$
$\DefineRemark{COJ21}{\mathrm{CO} (J=2 \to 1)}$
$\DefineRemark{COJ32}{\mathrm{CO} (J=3 \to 2)}$
$\DefineRemark{COJ43}{\mathrm{CO} (J=4 \to 3)}$
$\DefineRemark{RCI10CO10}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{10}}}$
$}$
$\DefineRemark{RCI10CO21}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{21}}}$
$}$
$\DefineRemark{RCI10CO32}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{32}}}$
$}$
$\DefineRemark{RCI10CO43}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{43}}}$
$}$
$\DefineRemark{LPrimeCO21}{L^{\prime}_{$\Remark${CO21}}}$
$\DefineRemark{LPrimeCI10}{L^{\prime}_{$\Remark${CI10}}}$
$\DefineRemark{ICO10}{I_{$\Remark${CO10}}}$
$\DefineRemark{ICO21}{I_{$\Remark${CO21}}}$
$\DefineRemark{ICO32}{I_{$\Remark${CO32}}}$
$\DefineRemark{ICO43}{I_{$\Remark${CO43}}}$
$\DefineRemark{ICI10}{I_{$\Remark${CI10}}}$
$\DefineRemark{kms}{\mathrm{km s^{-1}}}$
$\DefineRemark{Kkms}{\mathrm{K km s^{-1}}}$
$\DefineRemark{Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\DefineRemark{MJysr}{\mathrm{MJy sr^{-1}}}$
$\DefineRemark{R42}{\ifmmode{R_{42}}\else{R_{42}}\fi}$
$\DefineRemark{R41}{\ifmmode{R_{41}}\else{R_{41}}\fi}$
$\DefineRemark{R31}{\ifmmode{R_{31}}\else{R_{31}}\fi}$
$\DefineRemark{R21}{\ifmmode{R_{21}}\else{R_{21}}\fi}$
$\DefineRemark{R02}{\ifmmode{R_{\rm C\text{\sc{i}}CO}}\else{R_{\rm C\text{\sc{i}}CO}}\fi}$
$\DefineRemark{RCICO}{\ifmmode{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\else{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\fi}$
$\newcommand{$\figurename$}{Fig.}$
$\accepted{December 7, 2022}$
$\submitjournal{ApJL}$
$\shorttitle{Stellar Feedback, Molecular Excitation and Dissociation in NGC~1365}$
$\shortauthors{Liu et al.}$
$\begin{document}$
$\title{PHANGS-JWST First Results: Stellar Feedback-Driven Excitation and Dissociation of Molecular Gas in the Starburst Ring of NGC 1365?$
$}$
$\correspondingauthor{Daizhong Liu}$
$\email{dzliu@mpe.mpg.de, astro.dzliu@gmail.com}$
$\author[0000-0001-9773-7479]{Daizhong Liu}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-3933-7677]{Eva Schinnerer}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0001-5301-1326]{Yixian Cao}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-2545-1700]{Adam Leroy}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\author[0000-0003-1242-505X]{Antonio Usero}$
$\affiliation{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\author[0000-0002-5204-2259]{Erik Rosolowsky}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\author[0000-0002-6155-7166]{Eric Emsellem}$
$\affiliation{European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\affiliation{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\author[0000-0002-8804-0212]{J.~M.~Diederik~Kruijssen}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0002-5635-5180]{M\'elanie Chevance}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0001-6708-1317]{Simon C. O. Glover}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0001-6113-6241]{Mattia C. Sormani}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5480-5686]{Alberto D. Bolatto}$
$\affiliation{Department of Astronomy and Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}$
$\author[0000-0003-0378-4667]{Jiayi~Sun}$
$\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\author[0000-0002-9333-387X]{Sophia K. Stuber}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-4209-1599]{Yu-Hsuan Teng}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0003-0166-9745]{Frank Bigiel}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0003-0583-7363]{Ivana Be\v{s}li\'c}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-3247-5321]{Kathryn~Grasha}$
$\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\author[0000-0001-9656-7682]{Jonathan~D.~Henshaw}$
$\affiliation{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-0410-4504]{Ashley.~T.~Barnes}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-8760-6157]{Jakob~S.~den Brok}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-2501-9328]{Toshiki Saito}$
$\affiliation{National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo, 181-8588, Japan}$
$\author[0000-0002-5782-9093]{Daniel~A.~Dale}$
$\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\author[0000-0002-7365-5791]{Elizabeth~J.~Watkins}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0002-1370-6964]{Hsi-An Pan}$
$\affiliation{Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan}$
$\author[0000-0002-0560-3172]{Ralf S. Klessen}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Universit\"at Heidelberg, Interdisziplin\"ares Zentrum f\"ur Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5259-2314]{Gagandeep~S. Anand}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1943-723X]{Sinan Deger}$
$\affiliation{The Oskar Klein Centre for Cosmoparticle Physics, Department of Physics, Stockholm University, AlbaNova, Stockholm, SE-106 91, Sweden}$
$\author[0000-0002-4755-118X]{Oleg V. Egorov}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0001-5310-467X]{Christopher~M. Faesi}$
$\affiliation{University of Massachusetts-Amherst, 710 North Pleasant Street, Amherst, MA 01003, USA}$
$\author[0000-0002-8806-6308]{Hamid Hassani}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, Alberta, T6G 2E1, Canada}$
$\author[0000-0003-3917-6460]{Kirsten L. Larson}$
$\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2278-9407]{Janice~C. Lee}$
$\affiliation{Gemini Observatory/NSF's NOIRLab, 950 N. Cherry Avenue, Tucson, AZ 85719, USA; Caltech/IPAC, California Institute of Technology, Pasadena, CA 91125, USA}$
$\author[0000-0002-1790-3148]{Laura A. Lopez}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\affiliation{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\affiliation{Flatiron Institute, Center for Computational Astrophysics, NY 10010, USA}$
$\author[0000-0003-3061-6546]{Jér\^{o}me Pety}$
$\affiliation{IRAM, 300 rue de la Piscine, 38400 Saint Martin d'H\`eres, France}$
$\affiliation{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris}$
$\author[0000-0002-4378-8534]{Karin Sandstrom}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0002-8528-7340]{David~A. Thilker}$
$\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0002-3784-7032]{Bradley~C. Whitmore}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0012-2142]{Thomas G. Williams}$
$\affiliation{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\suppressAffiliations$
$\begin{abstract}$
$We compare embedded young massive star clusters (YMCs) to (sub-)millimeter line observations tracing the excitation and dissociation of molecular gas in the starburst ring of NGC~1365. This galaxy hosts one of the strongest nuclear starbursts and richest populations of YMCs within 20~Mpc.$
$Here we combine near-/mid-IR PHANGS-JWST imaging with new ALMA multi-J CO(1--0, 2--1 and 4--3) and [$\CI$](1--0) mapping, which we use to trace CO excitation via $\Remark${R42}=$\Remark${ICO43}/$\Remark${ICO21} and $\Remark${R21}=$\Remark${ICO21}/$\Remark${ICO10} and dissociation via $\Remark${RCICO}=$\Remark${ICI10}/$\Remark${ICO21} at 330~pc resolution.$
$We find that the gas flowing into the starburst ring from northeast to southwest appears strongly affected by stellar feedback, showing decreased excitation (lower $\Remark${R42}) and increased signatures of dissociation (higher $\Remark${RCICO}) in the downstream regions.$
$There, radiative transfer modelling suggests that the molecular gas density decreases and temperature and $\XCICO$ abundance ratio increase.$
$We compare $\Remark${R42} and $\Remark${RCICO} with local conditions across the regions and find that both correlate with near-IR 2~$\um$ emission tracing the YMCs and with both PAH (11.3~$\um$) and dust continuum (21~$\um$) emission.$
$In general, $\Remark${RCICO} exhibits \sim 0.1~dex tighter correlations than $\Remark${R42}, suggesting $\CI$  to be a more sensitive tracer of changing physical conditions in the NGC~1365 starburst than CO(4--3).$
$Our results are consistent with a scenario where gas flows into the two arm regions along the bar, becomes condensed/shocked, forms YMCs, and then these YMCs heat and dissociate the gas.$
$\end{abstract}$
$\keywords{galaxies: ISM --- galaxies: star formation --- ISM: molecules --- ISM: atoms}$
$\n\end{document}}}}$
$\newcommand{$\Umin$}{U_{\mathrm{min}}}$
$\newcommand{$\Umax$}{U_{\mathrm{max}}}$
$\newcommand{$\SNR$}{\mathrm{S/N}}$
$\newcommand{$\alphaCO$}{\alpha_{\mathrm{CO}}}$
$\newcommand{$\deltaGDzero$}{\delta_{\mathrm{GDR,0}}}$
$\newcommand{$\deltaGDR$}{\delta_{\mathrm{GDR}}}$
$\newcommand{$\deltaDGR$}{\delta_{\mathrm{DGR}}}$
$\newcommand{$\deltaISM$}{\delta_{\mathrm{ISM}}}$
$\newcommand{$\deltaGas$}{\mu_{\mathrm{molgas}}}$
$\newcommand{$\deltaTotalGas$}{\mu_{\mathrm{totalgas}}}$
$\newcommand{$\deltaTotalGDR$}{\delta_{\mathrm{totalGDR}}}$
$\newcommand{$\deltaMS$}{\delta{\mathrm{MS}}}$
$\newcommand{$\DeltaMS$}{\Delta{\mathrm{MS}}}$
$\newcommand{$\GDR$}{\mathrm{GDR}}$
$\newcommand{$\DGR$}{\mathrm{DGR}}$
$\newcommand{$\Tkin$}{T_{\mathrm{kin}}}$
$\newcommand{\nmean}{\left<n_{\mathrm{H}_2}\right>}$
$\newcommand{$\nthresh$}{n_{\mathrm{H_2,thresh}}}$
$\newcommand{$\nH$2}{n_{\mathrm{H}_2}}$
$\newcommand{$\nH$two}{n_{\mathrm{H}_2}}$
$\newcommand{$\NH$2_}{N_{\mathrm{H}_2}}$
$\newcommand{$\percmcubic$}{\mathrm{cm}^{-3}}$
$\newcommand{$\percmsquare$}{\mathrm{cm}^{-2}}$
$\newcommand{$\dvddr$}{\mathrm{d}v/\mathrm{d}r}$
$\newcommand{$\lgNH$2_}{\mathrm{log}_{10} (N_{\mathrm{H}_2} / \mathrm{cm^{-2}})}$
$\newcommand{\lgnmean}{\mathrm{log}_{10} \left<n_{\mathrm{H_2}}\right>}$
$\newcommand{$\lgnHtwo$}{\mathrm{log} n_{\mathrm{H_2}}}$
$\newcommand{$\lgnthresh$}{\mathrm{log}_{10} (n_{\mathrm{H_2,thresh}})}$
$\newcommand{$\alphavir$}{\alpha_{\mathrm{vir}}}$
$\newcommand{$\HIrom$}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{$\rom$annumeral 1}}}}$
$\newcommand{$\HIIrom$}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{$\rom$annumeral 2}}}}$
$\newcommand{$\CO$}{\mathrm{CO}}$
$\newcommand{$\metalZOH$}{12+\log_{10}(\mathrm{O/H})}$
$\newcommand{$\Msun$}{\mathrm{M}_{\odot}}$
$\newcommand{$\Lsun$}{\mathrm{L}_{\odot}}$
$\newcommand{$\Kkmspc$2}{\mathrm{K km s^{-1} pc^{2}}}$
$\newcommand{$\kms$}{\mathrm{km s^{-1}}}$
$\newcommand{$\SFR$}{\mathrm{SFR}}$
$\newcommand{$\LIR$}{L_{\mathrm{IR}}}$
$\newcommand{$\Mstar$}{M_{\star}}$
$\newcommand{$\Mdust$}{M_{\mathrm{dust}}}$
$\newcommand{$\Tdust$}{T_{\mathrm{dust}}}$
$\newcommand{$\Mgas$}{M_{\mathrm{gas}}}$
$\newcommand{$\Mmolgas$}{M_{\mathrm{mol gas}}}$
$\newcommand{$\Mtotalgas$}{M_{\mathrm{total gas}}}$
$\newcommand{$\Matomicgas$}{M_{\mathrm{atomic gas}}}$
$\newcommand{$\arcsec$}{"}$
$\newcommand{$\R$52}{R_{52}}$
$\newcommand{$\fPDR$}{f_{\mathrm{PDR}}}$
$\newcommand{$\fmol$}{f_{\mathrm{mol}}}$
$\newcommand{$\fH$2_}{f_{\mathrm{H_2}}}$
$\newcommand{$\MH$2_}{M_{\mathrm{H_2}}}$
$\newcommand{$\XCICO$}{[\mathrm{C}{\text{\sc{i}}/\mathrm{CO}]}}$
$\newcommand{$\micron$}{\mu\mathrm{m}}$
$\newcommand{$\Jykms$}{\mathrm{Jy km/s}}$
$\newcommand{$\Kkms$_}{\mathrm{K km/s}}$
$\newcommand{$\LprimeCO$}{L^{\prime}_{\mathrm{CO}}}$
$\newcommand{$\MAGPHYS$}{$\textsc${MAGPHYS}}$
$\newcommand{$\michi$2}{$\textsc${MiChi2}}$
$\newcommand{$\um$}{\ifmmode{\mu\mathrm{m}}\else{\mu\mathrm{m}}\fi}$
$\newcommand{$\cofour$}{CO(4--3)}$
$\newcommand{$\cotwo$}{CO(2--1)}$
$\newcommand{\J}[2][]{$
$	\ifthenelse{\isempty{#1}}{\begingroup$
$\newcommand{\JlowerN}{\}ADD{#2}{-1}{\JlowerN}$
$		J=#2{\to}\JlowerN$
$		\endgroup$
$	}{J=#1{\to}#2}$
$}$
$\newcommand{$\rom$}[1]{{{\uppercase\expandafter{$\rom$annumeral #1}}}}$
$\newcommand{$\rom$up}[1]{{\textup{\uppercase\expandafter{$\rom$annumeral #1}}}}$
$\newcommand{$\incode$}[1]{{\raggedright\lstinline|#1|}}$
$\newcommand{$\incode$p}[1]{{\raggedright\lstinline|"#1"|}}$
$\newcommand{$\neutralcarbon$}{\ifmmode \text{C$\textsc${i}} \else{\sc C i}\fi}$
$\newcommand{$\ion$izedcarbon}{\ifmmode \text{C$\textsc${ii}} \else{\sc C ii}\fi}$
$\newcommand{$\neutralhydrogen$}{\ifmmode \text{H$\textsc${i}} \else{\sc H i}\fi}$
$\newcommand{$\ion$izedhydrogen}{\ifmmode \text{H$\textsc${ii}} \else{\sc H ii}\fi}$
$\newcommand{$\carbonmonoxide$}{\ifmmode \text{$\textsc${CO}} \else{\sc CO}\fi}$
$\newcommand{$\HII$}{$\ion$izedhydrogen}$
$\newcommand{$\CI$}{$\neutralcarbon$}$
$\newcommand{$\CO$}{$\carbonmonoxide$}$
$\newcommand{\}{alphaCO}{\relax\ifmmode$
$    \alpha_\mathrm{CO}$
$    \else{\alpha_\mathrm{CO}}\fi$
$    }$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{$\Remark$}[1]{\csname rmk-#1\endcsname}$
$\DefineRemark{CI3P13P0}{[\mathrm{C}{\text{\sc{i}}}]\;(^{3}P_{1}\textnormal{--}^{3}P_{0})}$
$\DefineRemark{CI}{\mathrm{C}{\text{\sc{i}}}}$
$\DefineRemark{CI10}{[\mathrm{C}{\text{\sc{i}}}] (1\textnormal{--}0)}$
$\DefineRemark{CI21}{[\mathrm{C}{\text{\sc{i}}}] (2\textnormal{--}1)}$
$\DefineRemark{CO}{\mathrm{CO}}$
$\DefineRemark{CO10}{\mathrm{CO} (1\textnormal{--}0)}$
$\DefineRemark{CO21}{\mathrm{CO} (2\textnormal{--}1)}$
$\DefineRemark{CO32}{\mathrm{CO} (3\textnormal{--}2)}$
$\DefineRemark{CO43}{\mathrm{CO} (4\textnormal{--}3)}$
$\DefineRemark{COJ10}{\mathrm{CO} (J=1 \to 0)}$
$\DefineRemark{COJ21}{\mathrm{CO} (J=2 \to 1)}$
$\DefineRemark{COJ32}{\mathrm{CO} (J=3 \to 2)}$
$\DefineRemark{COJ43}{\mathrm{CO} (J=4 \to 3)}$
$\DefineRemark{RCI10CO10}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{10}}}$
$}$
$\DefineRemark{RCI10CO21}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{21}}}$
$}$
$\DefineRemark{RCI10CO32}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{32}}}$
$}$
$\DefineRemark{RCI10CO43}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{43}}}$
$}$
$\DefineRemark{LPrimeCO21}{L^{\prime}_{$\Remark${CO21}}}$
$\DefineRemark{LPrimeCI10}{L^{\prime}_{$\Remark${CI10}}}$
$\DefineRemark{ICO10}{I_{$\Remark${CO10}}}$
$\DefineRemark{ICO21}{I_{$\Remark${CO21}}}$
$\DefineRemark{ICO32}{I_{$\Remark${CO32}}}$
$\DefineRemark{ICO43}{I_{$\Remark${CO43}}}$
$\DefineRemark{ICI10}{I_{$\Remark${CI10}}}$
$\DefineRemark{kms}{\mathrm{km s^{-1}}}$
$\DefineRemark{Kkms}{\mathrm{K km s^{-1}}}$
$\DefineRemark{Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\DefineRemark{MJysr}{\mathrm{MJy sr^{-1}}}$
$\DefineRemark{R42}{\ifmmode{R_{42}}\else{R_{42}}\fi}$
$\DefineRemark{R41}{\ifmmode{R_{41}}\else{R_{41}}\fi}$
$\DefineRemark{R31}{\ifmmode{R_{31}}\else{R_{31}}\fi}$
$\DefineRemark{R21}{\ifmmode{R_{21}}\else{R_{21}}\fi}$
$\DefineRemark{R02}{\ifmmode{R_{\rm C\text{\sc{i}}CO}}\else{R_{\rm C\text{\sc{i}}CO}}\fi}$
$\DefineRemark{RCICO}{\ifmmode{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\else{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\fi}$
$\newcommand{$\figurename$}{Fig.}$
$\accepted{December 7, 2022}$
$\submitjournal{ApJL}$
$\shorttitle{Stellar Feedback, Molecular Excitation and Dissociation in NGC~1365}$
$\shortauthors{Liu et al.}$
$\begin{document}$
$\title{PHANGS-JWST First Results: Stellar Feedback-Driven Excitation and Dissociation of Molecular Gas in the Starburst Ring of NGC 1365?$
$}$
$\correspondingauthor{Daizhong Liu}$
$\email{dzliu@mpe.mpg.de, astro.dzliu@gmail.com}$
$\author[0000-0001-9773-7479]{Daizhong Liu}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-3933-7677]{Eva Schinnerer}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0001-5301-1326]{Yixian Cao}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-2545-1700]{Adam Leroy}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\author[0000-0003-1242-505X]{Antonio Usero}$
$\affiliation{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\author[0000-0002-5204-2259]{Erik Rosolowsky}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\author[0000-0002-6155-7166]{Eric Emsellem}$
$\affiliation{European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\affiliation{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\author[0000-0002-8804-0212]{J.~M.~Diederik~Kruijssen}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0002-5635-5180]{M\'elanie Chevance}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0001-6708-1317]{Simon C. O. Glover}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0001-6113-6241]{Mattia C. Sormani}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5480-5686]{Alberto D. Bolatto}$
$\affiliation{Department of Astronomy and Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}$
$\author[0000-0003-0378-4667]{Jiayi~Sun}$
$\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\author[0000-0002-9333-387X]{Sophia K. Stuber}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-4209-1599]{Yu-Hsuan Teng}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0003-0166-9745]{Frank Bigiel}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0003-0583-7363]{Ivana Be\v{s}li\'c}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-3247-5321]{Kathryn~Grasha}$
$\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\author[0000-0001-9656-7682]{Jonathan~D.~Henshaw}$
$\affiliation{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-0410-4504]{Ashley.~T.~Barnes}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-8760-6157]{Jakob~S.~den Brok}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-2501-9328]{Toshiki Saito}$
$\affiliation{National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo, 181-8588, Japan}$
$\author[0000-0002-5782-9093]{Daniel~A.~Dale}$
$\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\author[0000-0002-7365-5791]{Elizabeth~J.~Watkins}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0002-1370-6964]{Hsi-An Pan}$
$\affiliation{Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan}$
$\author[0000-0002-0560-3172]{Ralf S. Klessen}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Universit\"at Heidelberg, Interdisziplin\"ares Zentrum f\"ur Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5259-2314]{Gagandeep~S. Anand}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1943-723X]{Sinan Deger}$
$\affiliation{The Oskar Klein Centre for Cosmoparticle Physics, Department of Physics, Stockholm University, AlbaNova, Stockholm, SE-106 91, Sweden}$
$\author[0000-0002-4755-118X]{Oleg V. Egorov}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0001-5310-467X]{Christopher~M. Faesi}$
$\affiliation{University of Massachusetts-Amherst, 710 North Pleasant Street, Amherst, MA 01003, USA}$
$\author[0000-0002-8806-6308]{Hamid Hassani}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, Alberta, T6G 2E1, Canada}$
$\author[0000-0003-3917-6460]{Kirsten L. Larson}$
$\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2278-9407]{Janice~C. Lee}$
$\affiliation{Gemini Observatory/NSF's NOIRLab, 950 N. Cherry Avenue, Tucson, AZ 85719, USA; Caltech/IPAC, California Institute of Technology, Pasadena, CA 91125, USA}$
$\author[0000-0002-1790-3148]{Laura A. Lopez}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\affiliation{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\affiliation{Flatiron Institute, Center for Computational Astrophysics, NY 10010, USA}$
$\author[0000-0003-3061-6546]{Jér\^{o}me Pety}$
$\affiliation{IRAM, 300 rue de la Piscine, 38400 Saint Martin d'H\`eres, France}$
$\affiliation{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris}$
$\author[0000-0002-4378-8534]{Karin Sandstrom}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0002-8528-7340]{David~A. Thilker}$
$\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0002-3784-7032]{Bradley~C. Whitmore}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0012-2142]{Thomas G. Williams}$
$\affiliation{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\suppressAffiliations$
$\begin{abstract}$
$We compare embedded young massive star clusters (YMCs) to (sub-)millimeter line observations tracing the excitation and dissociation of molecular gas in the starburst ring of NGC~1365. This galaxy hosts one of the strongest nuclear starbursts and richest populations of YMCs within 20~Mpc.$
$Here we combine near-/mid-IR PHANGS-JWST imaging with new ALMA multi-J CO(1--0, 2--1 and 4--3) and [$\CI$](1--0) mapping, which we use to trace CO excitation via $\Remark${R42}=$\Remark${ICO43}/$\Remark${ICO21} and $\Remark${R21}=$\Remark${ICO21}/$\Remark${ICO10} and dissociation via $\Remark${RCICO}=$\Remark${ICI10}/$\Remark${ICO21} at 330~pc resolution.$
$We find that the gas flowing into the starburst ring from northeast to southwest appears strongly affected by stellar feedback, showing decreased excitation (lower $\Remark${R42}) and increased signatures of dissociation (higher $\Remark${RCICO}) in the downstream regions.$
$There, radiative transfer modelling suggests that the molecular gas density decreases and temperature and $\XCICO$ abundance ratio increase.$
$We compare $\Remark${R42} and $\Remark${RCICO} with local conditions across the regions and find that both correlate with near-IR 2~$\um$ emission tracing the YMCs and with both PAH (11.3~$\um$) and dust continuum (21~$\um$) emission.$
$In general, $\Remark${RCICO} exhibits \sim 0.1~dex tighter correlations than $\Remark${R42}, suggesting $\CI$  to be a more sensitive tracer of changing physical conditions in the NGC~1365 starburst than CO(4--3).$
$Our results are consistent with a scenario where gas flows into the two arm regions along the bar, becomes condensed/shocked, forms YMCs, and then these YMCs heat and dissociate the gas.$
$\end{abstract}$
$\keywords{galaxies: ISM --- galaxies: star formation --- ISM: molecules --- ISM: atoms}$
$\n\end{document}}}$
$\newcommand{$\nthresh$}{n_{\mathrm{H_2,thresh}}}$
$\newcommand{$\nH$2}{n_{\mathrm{H}_2}}$
$\newcommand{$\nH$two}{n_{\mathrm{H}_2}}$
$\newcommand{$\NH$2_}{N_{\mathrm{H}_2}}$
$\newcommand{$\percmcubic$}{\mathrm{cm}^{-3}}$
$\newcommand{$\percmsquare$}{\mathrm{cm}^{-2}}$
$\newcommand{$\dvddr$}{\mathrm{d}v/\mathrm{d}r}$
$\newcommand{$\lgNH$2_}{\mathrm{log}_{10} (N_{\mathrm{H}_2} / \mathrm{cm^{-2}})}$
$\newcommand{\lgnmean}{\mathrm{log}_{10} \left<n_{\mathrm{H_2}}\right>}$
$\newcommand{$\lgnHtwo$}{\mathrm{log} n_{\mathrm{H_2}}}$
$\newcommand{$\lgnthresh$}{\mathrm{log}_{10} (n_{\mathrm{H_2,thresh}})}$
$\newcommand{$\alphavir$}{\alpha_{\mathrm{vir}}}$
$\newcommand{$\HIrom$}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{$\rom$annumeral 1}}}}$
$\newcommand{$\HIIrom$}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{$\rom$annumeral 2}}}}$
$\newcommand{$\CO$}{\mathrm{CO}}$
$\newcommand{$\metalZOH$}{12+\log_{10}(\mathrm{O/H})}$
$\newcommand{$\Msun$}{\mathrm{M}_{\odot}}$
$\newcommand{$\Lsun$}{\mathrm{L}_{\odot}}$
$\newcommand{$\Kkmspc$2}{\mathrm{K km s^{-1} pc^{2}}}$
$\newcommand{$\kms$}{\mathrm{km s^{-1}}}$
$\newcommand{$\SFR$}{\mathrm{SFR}}$
$\newcommand{$\LIR$}{L_{\mathrm{IR}}}$
$\newcommand{$\Mstar$}{M_{\star}}$
$\newcommand{$\Mdust$}{M_{\mathrm{dust}}}$
$\newcommand{$\Tdust$}{T_{\mathrm{dust}}}$
$\newcommand{$\Mgas$}{M_{\mathrm{gas}}}$
$\newcommand{$\Mmolgas$}{M_{\mathrm{mol gas}}}$
$\newcommand{$\Mtotalgas$}{M_{\mathrm{total gas}}}$
$\newcommand{$\Matomicgas$}{M_{\mathrm{atomic gas}}}$
$\newcommand{$\arcsec$}{"}$
$\newcommand{$\R$52}{R_{52}}$
$\newcommand{$\fPDR$}{f_{\mathrm{PDR}}}$
$\newcommand{$\fmol$}{f_{\mathrm{mol}}}$
$\newcommand{$\fH$2_}{f_{\mathrm{H_2}}}$
$\newcommand{$\MH$2_}{M_{\mathrm{H_2}}}$
$\newcommand{$\XCICO$}{[\mathrm{C}{\text{\sc{i}}/\mathrm{CO}]}}$
$\newcommand{$\micron$}{\mu\mathrm{m}}$
$\newcommand{$\Jykms$}{\mathrm{Jy km/s}}$
$\newcommand{$\Kkms$_}{\mathrm{K km/s}}$
$\newcommand{$\LprimeCO$}{L^{\prime}_{\mathrm{CO}}}$
$\newcommand{$\MAGPHYS$}{$\textsc${MAGPHYS}}$
$\newcommand{$\michi$2}{$\textsc${MiChi2}}$
$\newcommand{$\um$}{\ifmmode{\mu\mathrm{m}}\else{\mu\mathrm{m}}\fi}$
$\newcommand{$\cofour$}{CO(4--3)}$
$\newcommand{$\cotwo$}{CO(2--1)}$
$\newcommand{\J}[2][]{$
$	\ifthenelse{\isempty{#1}}{\begingroup$
$\newcommand{\JlowerN}{\}ADD{#2}{-1}{\JlowerN}$
$		J=#2{\to}\JlowerN$
$		\endgroup$
$	}{J=#1{\to}#2}$
$}$
$\newcommand{$\rom$}[1]{{{\uppercase\expandafter{$\rom$annumeral #1}}}}$
$\newcommand{$\rom$up}[1]{{\textup{\uppercase\expandafter{$\rom$annumeral #1}}}}$
$\newcommand{$\incode$}[1]{{\raggedright\lstinline|#1|}}$
$\newcommand{$\incode$p}[1]{{\raggedright\lstinline|"#1"|}}$
$\newcommand{$\neutralcarbon$}{\ifmmode \text{C$\textsc${i}} \else{\sc C i}\fi}$
$\newcommand{$\ion$izedcarbon}{\ifmmode \text{C$\textsc${ii}} \else{\sc C ii}\fi}$
$\newcommand{$\neutralhydrogen$}{\ifmmode \text{H$\textsc${i}} \else{\sc H i}\fi}$
$\newcommand{$\ion$izedhydrogen}{\ifmmode \text{H$\textsc${ii}} \else{\sc H ii}\fi}$
$\newcommand{$\carbonmonoxide$}{\ifmmode \text{$\textsc${CO}} \else{\sc CO}\fi}$
$\newcommand{$\HII$}{$\ion$izedhydrogen}$
$\newcommand{$\CI$}{$\neutralcarbon$}$
$\newcommand{$\CO$}{$\carbonmonoxide$}$
$\newcommand{\}{alphaCO}{\relax\ifmmode$
$    \alpha_\mathrm{CO}$
$    \else{\alpha_\mathrm{CO}}\fi$
$    }$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{$\Remark$}[1]{\csname rmk-#1\endcsname}$
$\DefineRemark{CI3P13P0}{[\mathrm{C}{\text{\sc{i}}}]\;(^{3}P_{1}\textnormal{--}^{3}P_{0})}$
$\DefineRemark{CI}{\mathrm{C}{\text{\sc{i}}}}$
$\DefineRemark{CI10}{[\mathrm{C}{\text{\sc{i}}}] (1\textnormal{--}0)}$
$\DefineRemark{CI21}{[\mathrm{C}{\text{\sc{i}}}] (2\textnormal{--}1)}$
$\DefineRemark{CO}{\mathrm{CO}}$
$\DefineRemark{CO10}{\mathrm{CO} (1\textnormal{--}0)}$
$\DefineRemark{CO21}{\mathrm{CO} (2\textnormal{--}1)}$
$\DefineRemark{CO32}{\mathrm{CO} (3\textnormal{--}2)}$
$\DefineRemark{CO43}{\mathrm{CO} (4\textnormal{--}3)}$
$\DefineRemark{COJ10}{\mathrm{CO} (J=1 \to 0)}$
$\DefineRemark{COJ21}{\mathrm{CO} (J=2 \to 1)}$
$\DefineRemark{COJ32}{\mathrm{CO} (J=3 \to 2)}$
$\DefineRemark{COJ43}{\mathrm{CO} (J=4 \to 3)}$
$\DefineRemark{RCI10CO10}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{10}}}$
$}$
$\DefineRemark{RCI10CO21}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{21}}}$
$}$
$\DefineRemark{RCI10CO32}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{32}}}$
$}$
$\DefineRemark{RCI10CO43}{$
${R}_{\mathrm{C{\text{\sc{i}}}_{10}}/\mathrm{CO_{43}}}$
$}$
$\DefineRemark{LPrimeCO21}{L^{\prime}_{$\Remark${CO21}}}$
$\DefineRemark{LPrimeCI10}{L^{\prime}_{$\Remark${CI10}}}$
$\DefineRemark{ICO10}{I_{$\Remark${CO10}}}$
$\DefineRemark{ICO21}{I_{$\Remark${CO21}}}$
$\DefineRemark{ICO32}{I_{$\Remark${CO32}}}$
$\DefineRemark{ICO43}{I_{$\Remark${CO43}}}$
$\DefineRemark{ICI10}{I_{$\Remark${CI10}}}$
$\DefineRemark{kms}{\mathrm{km s^{-1}}}$
$\DefineRemark{Kkms}{\mathrm{K km s^{-1}}}$
$\DefineRemark{Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\DefineRemark{MJysr}{\mathrm{MJy sr^{-1}}}$
$\DefineRemark{R42}{\ifmmode{R_{42}}\else{R_{42}}\fi}$
$\DefineRemark{R41}{\ifmmode{R_{41}}\else{R_{41}}\fi}$
$\DefineRemark{R31}{\ifmmode{R_{31}}\else{R_{31}}\fi}$
$\DefineRemark{R21}{\ifmmode{R_{21}}\else{R_{21}}\fi}$
$\DefineRemark{R02}{\ifmmode{R_{\rm C\text{\sc{i}}CO}}\else{R_{\rm C\text{\sc{i}}CO}}\fi}$
$\DefineRemark{RCICO}{\ifmmode{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\else{R_{\mathrm{C}\text{\sc{i}}\mathrm{CO}}}\fi}$
$\newcommand{$\figurename$}{Fig.}$
$\accepted{December 7, 2022}$
$\submitjournal{ApJL}$
$\shorttitle{Stellar Feedback, Molecular Excitation and Dissociation in NGC~1365}$
$\shortauthors{Liu et al.}$
$\begin{document}$
$\title{PHANGS-JWST First Results: Stellar Feedback-Driven Excitation and Dissociation of Molecular Gas in the Starburst Ring of NGC 1365?$
$}$
$\correspondingauthor{Daizhong Liu}$
$\email{dzliu@mpe.mpg.de, astro.dzliu@gmail.com}$
$\author[0000-0001-9773-7479]{Daizhong Liu}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-3933-7677]{Eva Schinnerer}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0001-5301-1326]{Yixian Cao}$
$\affiliation{Max-Planck-Institut f\"ur Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\author[0000-0002-2545-1700]{Adam Leroy}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\author[0000-0003-1242-505X]{Antonio Usero}$
$\affiliation{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\author[0000-0002-5204-2259]{Erik Rosolowsky}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\author[0000-0002-6155-7166]{Eric Emsellem}$
$\affiliation{European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\affiliation{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\author[0000-0002-8804-0212]{J.~M.~Diederik~Kruijssen}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0002-5635-5180]{M\'elanie Chevance}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\author[0000-0001-6708-1317]{Simon C. O. Glover}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0001-6113-6241]{Mattia C. Sormani}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5480-5686]{Alberto D. Bolatto}$
$\affiliation{Department of Astronomy and Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}$
$\author[0000-0003-0378-4667]{Jiayi~Sun}$
$\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\author[0000-0002-9333-387X]{Sophia K. Stuber}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-4209-1599]{Yu-Hsuan Teng}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0003-0166-9745]{Frank Bigiel}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0003-0583-7363]{Ivana Be\v{s}li\'c}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-3247-5321]{Kathryn~Grasha}$
$\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\author[0000-0001-9656-7682]{Jonathan~D.~Henshaw}$
$\affiliation{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-0410-4504]{Ashley.~T.~Barnes}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-8760-6157]{Jakob~S.~den Brok}$
$\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}$
$\author[0000-0002-2501-9328]{Toshiki Saito}$
$\affiliation{National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo, 181-8588, Japan}$
$\author[0000-0002-5782-9093]{Daniel~A.~Dale}$
$\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\author[0000-0002-7365-5791]{Elizabeth~J.~Watkins}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0002-1370-6964]{Hsi-An Pan}$
$\affiliation{Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan}$
$\author[0000-0002-0560-3172]{Ralf S. Klessen}$
$\affiliation{Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\affiliation{Universit\"at Heidelberg, Interdisziplin\"ares Zentrum f\"ur Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\author[0000-0002-5259-2314]{Gagandeep~S. Anand}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1943-723X]{Sinan Deger}$
$\affiliation{The Oskar Klein Centre for Cosmoparticle Physics, Department of Physics, Stockholm University, AlbaNova, Stockholm, SE-106 91, Sweden}$
$\author[0000-0002-4755-118X]{Oleg V. Egorov}$
$\affiliation{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany}$
$\author[0000-0001-5310-467X]{Christopher~M. Faesi}$
$\affiliation{University of Massachusetts-Amherst, 710 North Pleasant Street, Amherst, MA 01003, USA}$
$\author[0000-0002-8806-6308]{Hamid Hassani}$
$\affiliation{Department of Physics, University of Alberta, Edmonton, Alberta, T6G 2E1, Canada}$
$\author[0000-0003-3917-6460]{Kirsten L. Larson}$
$\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2278-9407]{Janice~C. Lee}$
$\affiliation{Gemini Observatory/NSF's NOIRLab, 950 N. Cherry Avenue, Tucson, AZ 85719, USA; Caltech/IPAC, California Institute of Technology, Pasadena, CA 91125, USA}$
$\author[0000-0002-1790-3148]{Laura A. Lopez}$
$\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\affiliation{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\affiliation{Flatiron Institute, Center for Computational Astrophysics, NY 10010, USA}$
$\author[0000-0003-3061-6546]{Jér\^{o}me Pety}$
$\affiliation{IRAM, 300 rue de la Piscine, 38400 Saint Martin d'H\`eres, France}$
$\affiliation{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universit\'es, 75014 Paris}$
$\author[0000-0002-4378-8534]{Karin Sandstrom}$
$\affiliation{Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\author[0000-0002-8528-7340]{David~A. Thilker}$
$\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0002-3784-7032]{Bradley~C. Whitmore}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0012-2142]{Thomas G. Williams}$
$\affiliation{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\affiliation{Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany}$
$\suppressAffiliations$
$\begin{abstract}$
$We compare embedded young massive star clusters (YMCs) to (sub-)millimeter line observations tracing the excitation and dissociation of molecular gas in the starburst ring of NGC~1365. This galaxy hosts one of the strongest nuclear starbursts and richest populations of YMCs within 20~Mpc.$
$Here we combine near-/mid-IR PHANGS-JWST imaging with new ALMA multi-J CO(1--0, 2--1 and 4--3) and [$\CI$](1--0) mapping, which we use to trace CO excitation via $\Remark${R42}=$\Remark${ICO43}/$\Remark${ICO21} and $\Remark${R21}=$\Remark${ICO21}/$\Remark${ICO10} and dissociation via $\Remark${RCICO}=$\Remark${ICI10}/$\Remark${ICO21} at 330~pc resolution.$
$We find that the gas flowing into the starburst ring from northeast to southwest appears strongly affected by stellar feedback, showing decreased excitation (lower $\Remark${R42}) and increased signatures of dissociation (higher $\Remark${RCICO}) in the downstream regions.$
$There, radiative transfer modelling suggests that the molecular gas density decreases and temperature and $\XCICO$ abundance ratio increase.$
$We compare $\Remark${R42} and $\Remark${RCICO} with local conditions across the regions and find that both correlate with near-IR 2~$\um$ emission tracing the YMCs and with both PAH (11.3~$\um$) and dust continuum (21~$\um$) emission.$
$In general, $\Remark${RCICO} exhibits \sim 0.1~dex tighter correlations than $\Remark${R42}, suggesting $\CI$  to be a more sensitive tracer of changing physical conditions in the NGC~1365 starburst than CO(4--3).$
$Our results are consistent with a scenario where gas flows into the two arm regions along the bar, becomes condensed/shocked, forms YMCs, and then these YMCs heat and dissociate the gas.$
$\end{abstract}$
$\keywords{galaxies: ISM --- galaxies: star formation --- ISM: molecules --- ISM: atoms}$
$\n\end{document}}$
$\newcommand{$\lgnHtwo$}{\mathrm{log} n_{\mathrm{H_2}}}$
$\newcommand{$\lgnthresh$}{\mathrm{log}_{10} (n_{\mathrm{H_2,thresh}})}$
$\newcommand{$\alphavir$}{\alpha_{\mathrm{vir}}}$
$\newcommand{$\HIrom$}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{$\rom$annumeral 1}}}}$
$\newcommand{$\HIIrom$}{\mathrm{H}\textnormal{\textup{\uppercase\expandafter{$\rom$annumeral 2}}}}$
$\newcommand{$\CO$}{\mathrm{CO}}$
$\newcommand{$\metalZOH$}{12+\log_{10}(\mathrm{O/H})}$
$\newcommand{$\Msun$}{\mathrm{M}_{\odot}}$
$\newcommand{$\Lsun$}{\mathrm{L}_{\odot}}$
$\newcommand{$\Kkmspc$2}{\mathrm{K km s^{-1} pc^{2}}}$
$\newcommand{$\kms$}{\mathrm{km s^{-1}}}$
$\newcommand{$\SFR$}{\mathrm{SFR}}$
$\newcommand{$\LIR$}{L_{\mathrm{IR}}}$
$\newcommand{$\Mstar$}{M_{\star}}$
$\newcommand{$\Mdust$}{M_{\mathrm{dust}}}$
$\newcommand{$\Tdust$}{T_{\mathrm{dust}}}$
$\newcommand{$\Mgas$}{M_{\mathrm{gas}}}$
$\newcommand{$\Mmolgas$}{M_{\mathrm{mol gas}}}$
$\newcommand{$\Mtotalgas$}{M_{\mathrm{total gas}}}$
$\newcommand{$\Matomicgas$}{M_{\mathrm{atomic gas}}}$
$\newcommand{$\arcsec$}{"}$
$\newcommand{$\R$52}{R_{52}}$
$\newcommand{$\fPDR$}{f_{\mathrm{PDR}}}$
$\newcommand{$\fmol$}{f_{\mathrm{mol}}}$
$\newcommand{$\fH$2_}{f_{\mathrm{H_2}}}$
$\newcommand{$\MH$2_}{M_{\mathrm{H_2}}}$
$\newcommand{$\XCICO$}{[\mathrm{C}{\text{\sc{i}}/\mathrm{CO}]}}$
$\newcommand{$\micron$}{\mu\mathrm{m}}$
$\newcommand{$\Jykms$}{\mathrm{Jy km/s}}$
$\newcommand{$\Kkms$_}{\mathrm{K km/s}}$
$\newcommand{$\LprimeCO$}{L^{\prime}_{\mathrm{CO}}}$
$\newcommand{$\MAGPHYS$}{$\textsc${MAGPHYS}}$
$\newcommand{$\michi$2}{$\textsc${MiChi2}}$
$\newcommand{$\um$}{\ifmmode{\mu\mathrm{m}}\else{\mu\mathrm{m}}\fi}$
$\newcommand{$\cofour$}{CO(4--3)}$
$\newcommand{$\cotwo$}{CO(2--1)}$
$\newcommand{$\HII$}{$\ion$izedhydrogen}$
$\newcommand{$\CI$}{$\neutralcarbon$}$
$\newcommand{$\CO$}{$\carbonmonoxide$}$
$\newcommand{\}{alphaCO}$
$\newcommand{\JlowerN}{\}$</div>



<div id="title">

# PHANGS-JWST First Results: Stellar Feedback-Driven Excitation and Dissociation of Molecular Gas in the Starburst Ring of NGC 1365?

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.09652-b31b1b.svg)](https://arxiv.org/abs/2212.09652)<mark>Appeared on: 2022-12-20</mark> - _19 pages, 7 figures and 2 tables in total (12 pages and 6 figures in main text). Accepted as part of a PHANGS-JWST Focus Issue to appear in ApJL_

</div>
<div id="authors">

<mark>Daizhong Liu</mark>, et al. -- incl., <mark>Eva Schinnerer</mark>, <mark>Sophia K. Stuber</mark>, <mark>Frank Bigiel</mark>, <mark>Jonathan~D.~Henshaw</mark>, <mark>Janice~C. Lee</mark>

</div>
<div id="abstract">

**Abstract:** We compare embedded young massive star clusters (YMCs) to (sub-)millimeter line observations tracing the excitation and dissociation of molecular gas in the starburst ring of NGC 1365. This galaxy hosts one of the strongest nuclear starbursts and richest populations of YMCs within 20 Mpc.Here we combine near-/mid-IR PHANGS-JWST imaging with new ALMA multi-$J$CO(1--0, 2--1 and 4--3) and[$\CI$](1--0) mapping, which we use to trace CO excitation via$\Remark{R42}=$\Remark${ICO43}/$\Remark${ICO21}$and$\Remark{R21}=$\Remark${ICO21}/$\Remark${ICO10}$and dissociation via$\Remark{RCICO}=$\Remark${ICI10}/$\Remark${ICO21}$at 330 pc resolution.We find that the gas flowing into the starburst ring from northeast to southwest appears strongly affected by stellar feedback, showing decreased excitation (lower$\Remark{R42}$) and increased signatures of dissociation (higher$\Remark{RCICO}$) in the downstream regions.There, radiative transfer modelling suggests that the molecular gas density decreases and temperature and$\XCICO$abundance ratio increase.We compare$\Remark{R42}$and$\Remark{RCICO}$with local conditions across the regions and find that both correlate with near-IR 2$\um$emission tracing the YMCs and with both PAH (11.3$\um$) and dust continuum (21$\um$) emission.In general,$\Remark{RCICO}$exhibits$\sim 0.1$dex tighter correlations than$\Remark{R42}$, suggesting$\CI$to be a more sensitive tracer of changing physical conditions in the NGC 1365 starburst than CO(4--3).Our results are consistent with a scenario where gas flows into the two arm regions along the bar, becomes condensed/shocked, forms YMCs, and then these YMCs heat and dissociate the gas.

</div>

<div id="div_fig1">

<img src="tmp_2212.09652/./Plot_NGC1365_full_images.png" alt="Fig7" width="100%"/>

**Figure 7. -** ALMA and JWST images of NGC 1365.
    \textit{Left column (a, d, g):} Native-resolution ($0.3"$) ALMA CO(2-1) integrated intensity (mom0), line-of-sight velocity (mom1) and velocity dispersion (mom2); see also \citet{SCHINNERER_PHANGSJWST}.
    \textit{Middle column (b, e, h):} JWST stellar continuum dominated NIRCam 2 $\um$, PAH-dominated MIRI 11.3 $\um$, and dust continuum dominated MIRI 21 $\um$, all at native resolutions; see also \citet{WHITMORE_PHANGSJWST}.
    \textit{Right column (c, f, i):} Ratio maps in logarithmic scale for $\Remark{R21} = $\Remark${ICO21}/$\Remark${ICO10}$, $\Remark{R42} = $\Remark${ICO43}/$\Remark${ICO21}$ and $\Remark{RCICO} = $\Remark${ICI10}/$\Remark${ICO21}$ at the common resolution of $3.5"$(beam is shown in bottom left).
    Same contours of $\Remark{ICO21}$ at 400, 1000, 1500, 2000 and 3000 $\Remark{Kkms}$ are shown in all panels.
    Four regions mentioned in this work are highlighted in panel \textit{a)}.
    The cone in panel \textit{d)} represents the known
    ionized gas outflow with a half-opening angle $50^\circ$, cone axis inclination $35^{\circ}$ and PA $126^{\circ}$(\citealt{Hjelm1996}; see also \citealt{Sandqvist1995,Lindblad1999,Venturi2018,GaoYulong2021}) and the white line indicates the disk kinematic major axis at PA $220^\circ$(\citealt{Jorsater1995,Lindblad1999}).
     (*fig:1*)

</div>
<div id="div_fig2">

<img src="tmp_2212.09652/./Plot_NGC1365_full_images.png" alt="Fig11" width="100%"/>

**Figure 11. -** ALMA and JWST images of NGC 1365.
    \textit{Left column (a, d, g):} Native-resolution ($0.3"$) ALMA CO(2-1) integrated intensity (mom0), line-of-sight velocity (mom1) and velocity dispersion (mom2); see also \citet{SCHINNERER_PHANGSJWST}.
    \textit{Middle column (b, e, h):} JWST stellar continuum dominated NIRCam 2 $\um$, PAH-dominated MIRI 11.3 $\um$, and dust continuum dominated MIRI 21 $\um$, all at native resolutions; see also \citet{WHITMORE_PHANGSJWST}.
    \textit{Right column (c, f, i):} Ratio maps in logarithmic scale for $\Remark{R21} = $\Remark${ICO21}/$\Remark${ICO10}$, $\Remark{R42} = $\Remark${ICO43}/$\Remark${ICO21}$ and $\Remark{RCICO} = $\Remark${ICI10}/$\Remark${ICO21}$ at the common resolution of $3.5"$(beam is shown in bottom left).
    Same contours of $\Remark{ICO21}$ at 400, 1000, 1500, 2000 and 3000 $\Remark{Kkms}$ are shown in all panels.
    Four regions mentioned in this work are highlighted in panel \textit{a)}.
    The cone in panel \textit{d)} represents the known
    ionized gas outflow with a half-opening angle $50^\circ$, cone axis inclination $35^{\circ}$ and PA $126^{\circ}$(\citealt{Hjelm1996}; see also \citealt{Sandqvist1995,Lindblad1999,Venturi2018,GaoYulong2021}) and the white line indicates the disk kinematic major axis at PA $220^\circ$(\citealt{Jorsater1995,Lindblad1999}).
     (*fig:1*)

</div>
<div id="div_fig3">

<img src="tmp_2212.09652/./Plot_NGC1365_line_ratios.png" alt="Fig10" width="100%"/>

**Figure 10. -** \textit{Panels a) to f):}$\Remark${R42} in red and $\Remark${RCICO} in blue
    as functions of six variables at 330 pc resolution across the inner part of NGC 1365:
    \textit{a)}
    CO(1--0) line-integrated intensity,
    \textit{b)} velocity dispersion,
    \textit{c)}$\Remark${R21} line ratio,
    \textit{d)} 2 $\um$,
    \textit{e)} 11.3 $\um$, and
    \textit{f)} 21 $\um$ emission.
    Dashed lines are the best-fit trends and legends show the Spearman rank correlation coefficients, which are also reported in Table \ref{tab:1}.
    Each inset upper panel shows the rms distribution of the data after subtracting the best-fit trend in bins of the horizontal-axis variable.
     (*fig:scatter-plot*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\rom}[1]{{{\uppercase\expandafter{\romannumeral #1}}}}$
$\newcommand{\romup}[1]{{\textup{\uppercase\expandafter{\romannumeral #1}}}}$
$\newcommand{\incode}[1]{{\raggedright\lstinline|#1|}}$
$\newcommand{\incodep}[1]{{\raggedright\lstinline|"#1"|}}$
$\newcommand{\neutralcarbon}{\ifmmode \text{C\text{\sc{i}}} \else{\sc C i}\fi}$
$\newcommand{\ionizedcarbon}{\ifmmode \text{C\text{\sc{ii}}} \else{\sc C ii}\fi}$
$\newcommand{\neutralhydrogen}{\ifmmode \text{H\text{\sc{i}}} \else{\sc H i}\fi}$
$\newcommand{\ionizedhydrogen}{\ifmmode \text{H\text{\sc{ii}}} \else{\sc H ii}\fi}$
$\newcommand{\carbonmonoxide}{\ifmmode \text{\textsc{CO}} \else{\sc CO}\fi}$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{\Remark}[1]{\csname rmk-#1\endcsname}$
$\newcommand$
$\newcommand{\HII}{\ionizedhydrogen}$
$\newcommand{\CI}{\neutralcarbon}$
$\newcommand{\CO}{\carbonmonoxide}$
$\newcommand{\}{alphaCO}$
$\newcommand{\}{alphaCI}$
$\newcommand{\}{XCO}$
$\newcommand{\}{XCI}$
$\newcommand{\}{SNR}$
$\newcommand{\}{RCICO}$
$\newcommand{\}{tauCO}$
$\newcommand{\}{tauCI}$
$\newcommand{\}{XCICO}$
$\newcommand{\}{RNCINCO}$
$\newcommand{\}{Tkin}$
$\newcommand{\}{Tdust}$
$\newcommand{\}{CIabundance}$
$\newcommand{\}{CIcolumndensity}$
$\newcommand{\}{COabundance}$
$\newcommand{\}{CIabundance}$
$\newcommand{\}{COcolumndensity}$
$\newcommand{\}{CObrightness}$
$\newcommand{\}{HydrogenSurfaceDensity}$
$\newcommand{\}{Sigmol}$
$\newcommand{\}{nHtwo}$
$\newcommand{\}{dv}$
$\newcommand{\NoHe}{^{\neg\mathrm{He}}}$
$\newcommand{\Tprime}{T^{\prime}}$
$\newcommand{\Kkms}{\mathrm{K km s^{-1}}}$
$\newcommand{\Mspc2}{\mathrm{M_{\odot} pc^{-2}}}$
$\newcommand{\Mskpc2}{\mathrm{M_{\odot} kpc^{-2}}}$
$\newcommand{\kms}{\mathrm{km s^{-1}}}$
$\newcommand{\Msyr}{\mathrm{M_{\odot} yr^{-1}}}$
$\newcommand{\Lsun}{\mathrm{L_{\odot}}}$
$\newcommand{\Msun}{\mathrm{M_{\odot}}}$
$\newcommand{\Umean}{\left<U\right>}$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{\Remark}[1]{\csname rmk-#1\endcsname}$
$\DefineRemark{CI3P13P0}{[\mathrm{C}{\text{\sc{i}}}]\;(^{3}P_{1}\textnormal{--}^{3}P_{0})}$
$\DefineRemark{CI}{\mathrm{C}{\text{\sc{i}}}}$
$\DefineRemark{CI10}{[\mathrm{C}{\text{\sc{i}}}] (1\textnormal{--}0)}$
$\DefineRemark{CI21}{[\mathrm{C}{\text{\sc{i}}}] (2\textnormal{--}1)}$
$\DefineRemark{CO}{\mathrm{CO}}$
$\DefineRemark{CO10}{\mathrm{CO} (1\textnormal{--}0)}$
$\DefineRemark{CO21}{\mathrm{CO} (2\textnormal{--}1)}$
$\DefineRemark{CO32}{\mathrm{CO} (3\textnormal{--}2)}$
$\DefineRemark{CO43}{\mathrm{CO} (4\textnormal{--}3)}$
$\DefineRemark{COJ10}{\mathrm{CO} (J=1 \to 0)}$
$\DefineRemark{COJ21}{\mathrm{CO} (J=2 \to 1)}$
$\DefineRemark{COJ32}{\mathrm{CO} (J=3 \to 2)}$
$\DefineRemark{COJ43}{\mathrm{CO} (J=4 \to 3)}$
$\DefineRemark{RCI10CO10}{$
${R}_{\mathrm{C{\text{\sc{i}}}{10}}/\mathrm{CO{10}}}$
$}$
$\DefineRemark{RCI10CO21}{$
${R}_{\mathrm{C{\text{\sc{i}}}{10}}/\mathrm{CO{21}}}$
$}$
$\DefineRemark{RCI10CO32}{$
${R}_{\mathrm{C{\text{\sc{i}}}{10}}/\mathrm{CO{32}}}$
$}$
$\DefineRemark{RCI10CO43}{$
${R}_{\mathrm{C{\text{\sc{i}}}{10}}/\mathrm{CO{43}}}$
$}$
$\DefineRemark{LPrimeCO21}{L^{\prime}_{\Remark{CO21}}}$
$\DefineRemark{LPrimeCI10}{L^{\prime}_{\Remark{CI10}}}$
$\DefineRemark{ICO10}{I_{\Remark{CO10}}}$
$\DefineRemark{ICO21}{I_{\Remark{CO21}}}$
$\DefineRemark{ICO32}{I_{\Remark{CO32}}}$
$\DefineRemark{ICO43}{I_{\Remark{CO43}}}$
$\DefineRemark{ICI10}{I_{\Remark{CI10}}}$
$\DefineRemark{Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\begin{document}$
$\title{C~\textsc{i} and CO in Nearby Spiral Galaxies - I. Line Ratio and Abundance Variations at \sim 200~pc Scales}$
$\titlerunning{C~\textsc{i} \& CO - I. Line Ratio and Abundance Ratio}$
$\authorrunning{D. Liu et al.}$
$\author{$
$    Daizhong Liu \inst{\ref{MPE}}$
$\and$
$    Eva Schinnerer \inst{\ref{MPIA}}$
$\and$
$    Toshiki Saito \inst{\ref{NAOJ}}$
$\and$
$    Erik Rosolowsky \inst{\ref{UAlberta}}$
$\and$
$    Adam Leroy \inst{\ref{OSU}}$
$\and$
$    Antonio Usero \inst{\ref{Madrid}}$
$\and$
$    Karin Sandstrom \inst{\ref{UCSD}}$
$\and$
$    Ralf S. Klessen \inst{\ref{HeidelbergZfA},\ref{HeidelbergZfW}}$
$\and$
$    Simon C. O. Glover \inst{\ref{HeidelbergZfA}}$
$\and$
$    Yiping Ao \inst{\ref{PMO},\ref{USTC}}$
$\and$
$    Ivana Be\v{s}li\'c \inst{\ref{BonnAIfA}}$
$\and$
$    Frank Bigiel \inst{\ref{BonnAIfA}}$
$\and$
$    Yixian Cao \inst{\ref{MPE}}$
$\and$
$    Jérémy Chastenet \inst{\ref{UCSD},\ref{Gent}}$
$\and$
$    Mélanie Chevance \inst{\ref{HeidelbergARI},\ref{HeidelbergZfA}}$
$\and$
$    Daniel A. Dale \inst{\ref{Wyoming}}$
$\and$
$    Yu Gao \inst{\ref{XiamenUniv}}\thanks{Deceased.}$
$\and$
$    Annie Hughes \inst{\ref{Toulouse}}$
$\and$
$    Kathryn Kreckel \inst{\ref{HeidelbergARI}}$
$\and$
$    J.~M.~Diederik Kruijssen \inst{\ref{HeidelbergARI}}$
$\and$
$    Hsi-An Pan \inst{\ref{TamkangUniv}}$
$\and$
$    J\'er\^ome Pety \inst{\ref{IRAM},\ref{SorbonneUniv}}$
$\and$
$    Dragan Salak \inst{\ref{HokkaidoUniv},\ref{HokkaidoUnivGrad}}$
$\and$
$    Francesco Santoro \inst{\ref{MPIA}}$
$\and$
$    Andreas Schruba \inst{\ref{MPE}}$
$\and$
$    Jiayi Sun \inst{\ref{McMasterUniv},\ref{TorontoUniv}}$
$\and$
$    Yu-Hsuan Teng \inst{\ref{UCSD}}$
$\and$
$    Thomas Williams \inst{\ref{MPIA}}$
$}$
$\institute{$
$    Max-Planck-Institut f\"ur extraterrestrische Physik (MPE), Giessenbachstrasse 1, D-85748 Garching, Germany\\email{dzliu@mpe.mpg.de, astro.dzliu@gmail.com}$
$    \label{MPE}$
$\and$
$    Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany$
$    \label{MPIA}$
$\and$
$National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo, 181-8588, Japan$
$    \label{NAOJ}$
$\and$
$Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada$
$    \label{UAlberta}$
$\and$
$Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA$
$    \label{OSU}$
$\and$
$    Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain$
$    \label{Madrid}$
$\and$
$    Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA$
$    \label{UCSD}$
$\and$
$    Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany$
$    \label{HeidelbergZfA}$
$\and$
$    Universit\"at Heidelberg, Interdisziplin\"ares Zentrum f\"ur Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany$
$    \label{HeidelbergZfW}$
$\and$
$     Purple Mountain Observatory and Key Laboratory for Radio Astronomy, Chinese Academy of Sciences, Nanjing, China$
$     \label{PMO}$
$\and$
$    School of Astronomy and Space Science, University of Science and Technology of China, Hefei, China$
$    \label{USTC}$
$\and$
$    Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany$
$    \label{BonnAIfA}$
$\and$
$    Sterrenkundig Observatorium, Ghent University, Krijgslaan 281-S9, 9000 Gent, Belgium$
$    \label{Gent}$
$\and$
$    Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany$
$    \label{HeidelbergARI}$
$\and$
$    Department of Physics \& Astronomy, University of Wyoming, Laramie, WY 82071, USA$
$    \label{Wyoming}$
$\and$
$    Department of Astronomy, Xiamen University, Xiamen, Fujian 361005, China$
$    \label{XiamenUniv}$
$\and$
$    CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France; Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France$
$    \label{Toulouse}$
$\and$
$    Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan$
$    \label{TamkangUniv}$
$\and$
$    Institut de Radioastronomie Millimétrique (IRAM), 300 rue de la Piscine, F-38406 Saint-Martin-d'Hères, France$
$    \label{IRAM}$
$\and$
$    Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France$
$    \label{SorbonneUniv}$
$\and$
$    Institute for the Advancement of Higher Education, Hokkaido University, Kita 17 Nishi 8, Kita-ku, Sapporo, Hokkaido 060-0817, Japan$
$    \label{HokkaidoUniv}$
$\and$
$    Department of Cosmosciences, Graduate School of Science, Hokkaido University, Kita 10 Nishi 8, Kita-ku, Sapporo, Hokkaido 060-0817, Japan$
$    \label{HokkaidoUnivGrad}$
$\and$
$    Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada$
$    \label{McMasterUniv}$
$\and$
$    Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada$
$    \label{TorontoUniv}$
$}$
$\date{Accepted: December 13, 2022}$
$\abstract{$
$We present new neutral atomic carbon [C~\textsc{i}]~( ^3P_1~\to~ ^3P_0) mapping observations within the inner \sim 7~kpc and \sim 4~kpc of the disks of NGC~3627 and NGC~4321 at a spatial resolution of 190~pc and 270~pc, respectively, using the Atacama Large Millimeter/Submillimeter Array (ALMA) Atacama Compact Array (ACA).$
$We combine these with the CO(2--1) data from PHANGS-ALMA, and literature [C~\textsc{i}] and CO data for two other starburst and/or active galactic nucleus (AGN) galaxies (NGC~1808, NGC~7469), to study: \textit{a)} the spatial distributions of C~\textsc{i} and CO emission; \textit{b)} the observed line ratio \RCICO = I_{\mathrm{[C{\normalfont\textsc{i}}]}(1\textnormal{--}0)} / I_{\mathrm{CO}(2\textnormal{--}1)} as a function of various galactic properties; and \textit{c)} the abundance ratio of [C~\textsc{i}/CO].$
$We find excellent spatial correspondence between C~\textsc{i} and CO emission and nearly uniform \RCICO \sim 0.1 across the majority of the star-forming disks of NGC~3627 and NGC~4321.$
$However, \RCICO strongly varies from \sim 0.05 at the centre of NGC~4321 to > 0.2 \text{--} 0.5 in NGC~1808's starbursting centre and NGC~7469's centre with an X-ray-luminous active galactic nucleus (AGN).$
$Meanwhile, \RCICOdoes not obviously vary with \Umean, similar to the prediction of photodissociation dominated region (PDR) models.$
$We also find a mildly decreasing \RCICOwith an increasing metallicity over 0.7\text{--}0.85 \mathrm{Z_{\odot}}, consistent with the literature.$
$Assuming various typical ISM conditions representing giant molecular clouds, active star-forming regions and strong starbursting environments, we calculate the local-thermodynamic-equilibrium radiative transfer and estimate the [C~\textsc{i}/CO] abundance ratio to be \sim 0.1 across the disks of NGC~3627 and NGC~4321, similar to previous large-scale findings in Galactic studies.$
$However, this abundance ratio likely has a substantial increase to \sim 1 and \gtrsim 1--5 in NGC~1808's starburst and NGC~7469's strong AGN environments, respectively, in line with the expectations for cosmic-ray dominated region (CRDR) and X-ray dominated region (XDR) chemistry.$
$Finally, we do not find a robust evidence for a generally CO-dark, C~\textsc{i}-bright gas in the disk areas we probed.$
$}$
$\keywords{galaxies: ISM --- ISM: molecules --- ISM: atoms --- ISM: abundances --- galaxies: spiral --- galaxies: individual: NGC 3627, NGC 4321, NGC 1808, NGC 7469}$
$\maketitle$
$\section{Introduction}$
$\label{subsec: intro RCICO}$
$\subsection{Neutral atomic carbon in the interstellar medium}$
$Neutral atomic carbon (C^0 or \neutralcarbon) is an important phase of carbon in the interstellar medium (ISM) besides ionized carbon (C^+ or \ionizedcarbon) and carbon monoxide (CO).$
$Its ^{3}P state is split into three fine-structure levels. The^{3}P_{1} \to  ^{3}P_{0} transition line at 492.16065~GHz,$
$hereafter \Remark{CI10}$
$, has an upper energy level of E_u = 23.620~K and a critical density of n_{\mathrm{crit}} \sim 1.0 \times 10^{3} \;\mathrm{cm^{-3}} (at a temperature of 50~K) \footnote{n_{\mathrm{crit}} \equiv A / \gamma, where A is the Einstein A coefficient for the transition and \gamma the rate of collisional de-excitation with hydrogen molecules, with values from \cite{Schroder1991} as compiled in the Leiden Atomic and Molecular Database (\citealt{Schoier2005}), \url{https://home.strw.leidenuniv.nl/~moldata/}},$
$and the^{3}P_{2} \to  ^{3}P_{1} transition line at 809.34197~GHz has E_u = 62.462~K and n_{\mathrm{crit}} \sim 3.4 \times 10^{3} \;\mathrm{cm^{-3}}.$
$Therefore, they are easily excited in the cold ISM environment.$
$Given the high abundance of carbon in the ISM, \neutralcarbon, CO and \ionizedcarbon  are the most powerful cold ISM tracers, and are the most widely used tools to probe cold ISM properties and hence galaxy evolution at cosmological distances.$
$The interstellar \neutralcarbon  is produced from$
$the photodissociation of CO molecules by UV photons and the recombination of \ionizedcarbon , both happening in photodissociation regions (PDRs; \citealt{Langer1976,deJong1980,Tielens1985a,Tielens1985b,vanDishoeck1986,vanDishoeck1988,Sternberg1989,Sternberg1995,Hollenbach1991,Hollenbach1999,Kaufman1999,Wolfire2010,Madden2020,Bisbas2021}, also see reviews by \citealt{Genzel1989,Jaffe1985,Hollenbach1997,Wolfire2022}).$
$In the simple plane-parallel (i.e., 1-D) PDR model, \neutralcarbon  exists in a layer where UV photons can penetrate through the molecular gas but cannot maintain a high carbon ionizing rate.$
$Such a layer is suggested to have an intermediate gas surface density (N_{\mathrm{H_2}} \sim 1 \textnormal{--} 4 \times 10^{20} \;\mathrm{cm^{-2}}), fairly cold temperature (10 to a few tens K), and moderate visual extinction (A_V \sim 1 - 4).$
$CO molecules dominate the more shielded interior of this layer and \ionizedcarbon  dominates the exterior.$
$Hydrogen molecules are gradually photodissociated to atoms across this layer (A_V \sim 0.1 - 4).$
$This plane-parallel scenario qualitatively explains the layered structure of the Orion Bar PDR in our Galaxy on sub-pc scales (e.g.,  \citealt{Genzel1989,Tielens1993,Tauber1994,Hogerheijde1995b,Hollenbach1999,Goicoechea2016}), and the \rho~Ophiuchi~A PDR in a recent study with an unprecedented resolution of 360~AU (\citealt{Yamagishi2021}), where the interplay between individual young, massive (O, B) stars and the ISM can be directly observed.$
$However, such simple models cannot fully explain molecular clouds. Plenty of observations in Galactic molecular clouds, \ionizedhydrogen  regions and star-formation complexes have revealed a widespread \neutralcarbon  distribution over most areas of molecular clouds on scales of a few to ten pc, and a relatively high \neutralcarbon  fractional abundance even at a large A_V depth > 10~mag (\citealt{Phillips1980,Phillips1981,Wootten1982,Keene1985,Keene1987,Jaffe1985,Zmuidzinas1986,Zmuidzinas1988,Genzel1988,Frerking1989,Plume1994,Plume1999,Gerin1998a,Tatematsu1999,Ikeda1999,Ikeda2002,Yamamoto2001,Zhang2001,Kamegai2003,Oka2004,Izumi2021}).$
$This observational evidence actually requires PDRs to be clumpy  (\citealt{Stutzki1988,Genzel1988,Burton1990,Meixner1993,Spaans1997,Kramer2004,Kramer2008,Pineda2008,Sun2008}), with a volume filling factor much less than unity (e.g., \sim 0.1 - 0.3; \citealt{Stutzki1988}), so that$
$UV photons can penetrate through most of the cloud, except for the densest clumps.$
$Therefore, the spatial co-existence of \neutralcarbon  and low rotational transition (low-J) CO lines can be well explained by such an inter-clump medium inside the molecular clouds.$
$Alternatively, cosmic rays (CRs) have also been proposed to be the reason for the \neutralcarbon -CO association as they can penetrate much deeper than UV photons into clouds, and thus more uniformly dissociate CO molecules into \neutralcarbon  inside clouds (\citealt{Field1969,Padovani2009,Papadopoulos2010c,Ivlev2015,Papadopoulos2018}).$
$\subsection{Variations of the \CI/\CO line ratio (\RCICO) in the literature}$
$Although the spatial distributions of \CI  and low-J CO emission resemble each other surprisingly well from molecular clouds to Galactic scales (e.g., across the Galactic plane; \citealt{Wright1991,Bennett1994,Oka2005,Burton2015}) and in external galaxies with spatially resolved observations (e.g., \citealt{Schilke1993,White1994b,Harrison1995,Israel1995,Israel2001,Israel2003,Zhang2014,Krips2016,Cicone2018,Crocker2019,Jiao2019,Miyamoto2018,Miyamoto2021,Salak2019,Izumi2020,Saito2020,Michiyama2020,Michiyama2021}),$
$their line intensity ratio, \Remark{RCI10CO10} = \Remark{ICI10} / \Remark{ICO10} \footnote{The line intensity I has brightness temperature units \mathrm{K km s^{-1}}.}, does vary according to local ISM conditions --- \CI  and CO fractional abundance (i.e., metallicity), excitation condition (e.g., excitation temperature T_{\mathrm{ex}} and optical depth \tau), and UV and/or CR radiation field.$
$At individual molecular cloud scales, \Remark{RCI10CO10} (or column density ratio) seems to vary from \gtrsim 0.5 (or \gtrsim 0.2 for column density ratio) to <0.1 with an increasing N_{\mathrm{H2}} and A_V, e.g., from cloud surface or PDR front to the interior \citep[e.g.][]{Frerking1982,Genzel1988,Oka2004,Kramer2004,Kramer2008,Mookerjea2006,Sun2008,Yamagishi2021}.$
$At sub-galactic scales, when individual clouds are smoothed out, \Remark{RCI10CO10} is found to lie in a narrower range of \sim 0.1 \text{--} 0.3. For example, across the Galactic plane, the ratio is found to change only slightly within a galactocentric radius of 3--7~kpc (\sim 0.08 \text{--} 0.12, overall mean = 0.105 \pm 0.004; \citealt{Oka2005});$
$whereas in the starburst galaxies like M~82 and NGC~253, \Remark{RCI10CO10} or the [\CI]/CO abundance ratio is a factor of 3--5 higher than the Galactic value (\citealt{Schilke1993,White1994b,Krips2016}).$
$Recent high-resolution (hundred pc scales) \Remark{CI10} mapping studies in the nearby spiral galaxy M~83 (\citealt{Miyamoto2021}) and starburst galaxy IRAS~F18293-3413 (\citealt{Saito2020}) reported tightly-correlated \Remark{CI10} and \Remark{CO10} emission, with scaling relations at hundred parsecs equivalent to$
$\Remark{RCI10CO10} = 0.14 \times (L^{\prime}_{\Remark{CO10}}/10^{6.0} [\Remark{Kkmspc2}])^{-0.13}$
$and$
$\Remark{RCI10CO10} = 0.21 \times (L^{\prime}_{\Remark{CO10}}/10^{6.8} [\Remark{Kkmspc2}])^{+0.54},$
$respectively.$
$The discrepant power-law indices reflects different trends in individual galaxies that are not fully understood.$
$Meanwhile, in the central \lesssim 100~pc of NGC~7469 close to the X-ray-luminous active galactic nucleus (AGN), \RCICO  is found to be substantially enhanced by a factor of >10 (reaching \Remark{RCI10CO10} \approx 1; \citealt{Izumi2020}; see also the Circinus Galaxy's AGN with \Remark{RCI10CO32} \approx 0.9 in \citealt{Izumi2018}).$
$Furthermore, in the low-metallicity Large Magellanic Cloud (LMC) and Small Magellanic Cloud (SMC) environments, a factor of 1.5 - 3 \times higher \RCICO  values have been found (\citealt{Bolatto2000b,Bolatto2000c}; see also other low-metallicity galaxies: \citealt{Bolatto2000a,Hunt2017}).$
$These studies indicate that \RCICO  is sensitive to ISM conditions among and within galaxies.$
$However, previous extragalactic studies are mainly focused on starburst galaxies and bright galaxy centres. How \RCICO  behaves across the disks and spiral arms of typical star-forming galaxies is much less explored. Whether \CI  can trace the so-called CO-dark molecular gas at a low gas density or metallicity is also still in question.$
$Understanding \RCICO  variation and quantifying the \CI  and \CO  excitation in extragalactic environments is now particularly important given the rapidly growing numbers and diverse galactic environments of \CI  line observations at cosmological distances (e.g., \citealt{Weiss2003,Weiss2005,Pety2004,Wagg2006,Walter2011,Danielson2011,AlaghbandZadeh2013,Gullberg2016,Bothwell2017,Popping2017,Yang2017,Emonts2018,Valentino2018,Valentino2020,Bourne2019,Nesvadba2019,Dannerbauer2019,Jin2019,Brisbin2019,Boogaard2020,Cortzen2020,Harrington2021,Dunne2021,Lee2021}).$
$\subsection{What drives \RCICO? --- the need for high-resolution mapping across wide galactic environments}$
$High-resolution mapping of \CI  and CO from disks and spiral arms of nearby galaxies is indispensable for understanding and quantifying the ISM \CI  and CO physics. In particular, the following questions will be answered only with such observations:$
$Do \CI  and low-J CO have the same spatial distributions across galaxy disks?$
$Does the \RCICO  line ratio vary significantly across galaxy disks?$
$How does the \RCICO  line ratio vary among different local galaxies?$
$How can the observed \RCICO  line ratio be reliably converted to the \CI  and CO column density ratio (i.e., solving their radiation transfer equations and level population statistics and obtaining optical depths and excitation temperature)?$
$Currently, high-resolution (hundreds of parsec scale) imaging/mapping of \CI  in nearby galaxies relies on radio telescopes at high altitudes, e.g., the Atacama Large Millimeter/submillimeter Array (ALMA), Atacama Submillimeter Telescope Experiment (ASTE, 10-m single-dish), Atacama Pathfinder Experiment (APEX, 12-m single-dish), and is limited to the available weather conditions corresponding to the high frequency of \CI  lines. Therefore, only a handful of galaxies have been imaged/mapped.$
$These include NGC~6240 by \citet{Cicone2018}, Circinus Galaxy by \citet{Izumi2018}, NGC~613 by \citet{Miyamoto2018}, NGC~1808 by \citet{Salak2019}, NGC~7469 by \citet{Izumi2020}, IRAS~F18293-3413 by \citet{Saito2020}, NGC~6052 by \citet[][\CI  undetected]{Michiyama2020}, 36 (U)LIRGs by \citet{Michiyama2021}, M~83 by \citet[][mapped the centre and northern part with ASTE]{Miyamoto2021}, Arp~220 by \citet{Ueda2022}, and NGC~1068 by \citet{Saito2022a,Saito2022b}.$
$Nevertheless, all of these observations, except for M~83, targeted the centres of starburst, IR-luminous and/or AGN host galaxies. M~83 is the only typical star-forming main sequence galaxy having a high-quality \CI  mapping. However, its map is still limited to a quarter of its inner \sim3~kpc disk.$
$In this work, we present two new \CI  mapping observations with the ALMA Atacama Compact Array (or Morita Array; 7-m dish, hereafter ACA or 7m) in nearby spiral galaxies NGC~3627 and NGC~4321.$
$Our observations aimed at detecting \CI  in the disks, and thus are much larger-area and deeper than previous \CI  maps.$
$By further adding a starburst and an AGN-host galaxy from the literature into our analysis, we present a comprehensive study of the \Remark{RCI10CO21} line ratio (hereafter \RCICO; ratios of other \CI /CO transitions will be explicitly written otherwise) in nearby star-forming disk and starburst and AGN environments.$
$In order to quantify the \CI  and CO abundance ratio,$
$we have further calculated the local thermodynamic equilibrium (LTE) and non-LTE radiative transfer of \CI  and CO under several \textit{representative ISM conditions}.$
$The results from this study could thus be one of the most comprehensive local benchmarks for understanding the \CI  and CO line ratio and abundance variations.$
$The paper, as the first one of our series of papers, is organized as follows.$
$Sect.~\ref{sec: Observations} describes the sample selection, observations and data reduction.$
$Sect.~\ref{sec: Results} presents the spatial distribution and variation of \RCICO.$
$Sect.~\ref{sec: Excitation Analysis} presents the analysis of \CI  and CO excitation and radiative transfer and thus the abundance (species column density) ratio.$
$In Sect.~\ref{sec: Discussion} we discuss various topics relating to the [\CI/CO] abundance variation, CRDR, XDR, and CO-dark gas.$
$Finally we conclude in Sect.~\ref{sec: Conclusion}.$
$Our companion paper~II (D. Liu et al. in prep.) presents the detailed radiative transfer calculation and \CI  and CO conversion factors, and is related to the abundance calculation in this paper.$
$\vspace{2ex}$
$\section{Targets and observations}$
$\label{sec: Observations}$
$\subsection{Target selection}$
$\label{subsection: Target Selection}$
$Our targets, NGC~3627 and NGC~4321, are selected from a joint sample of the PHANGS-ALMA CO(2--1) (\citealt{Leroy2021phangs}), PHANGS-HST (\citealt{PHANGSHST}), and PHANGS-MUSE (\citealt{PHANGSMUSE}) surveys, with CO line intensities and systemic velocities best suitable for ALMA Band 8 \CI  mapping.$
$The PHANGS-ALMA sample consists of 90 nearby star-forming galaxies initially selected at distances of 2 \lesssim d / \mathrm{Mpc} \lesssim 23, with a stellar mass \log (M_{\star}/\mathrm{M_{\odot}}) > 9.75, and which are not too inclined and visible to ALMA. They represent the typical star-forming main sequence galaxies in the local Universe.$
$The PHANGS-HST sample \citep{PHANGSHST} is a subsample of 38 PHANGS-ALMA galaxies, providing high-resolution stellar properties.$
$The PHANGS-MUSE large program \citep{PHANGSMUSE} targeted a subsample of 19 PHANGS-ALMA galaxies suitable for VLT/MUSE optical integral field unit (IFU) observations, achieving similar spatial coverage as in PHANGS-ALMA and providing rich nebular emission lines, attenuation, stellar age and metallicity and H~\textsc{ii} region information (e.g., \citealt{Kreckel2019, Kreckel2020, Pessa2021, Santoro2021, Williams2022}).$
$For the legacy value, we selected our sample from the joint ALMA+MUSE+HST sample pool, considered CO surface brightness, mapping area, \CI  line frequency and the transmission in the ALMA Band 8.$
$In this work, we present the new \CI  data of NGC~3627 and NGC~4321.$
$Because of the ALMA Band 8 sensitivity and the expected fainter \CI  line intensity than CO, the \CI  mapping areas are smaller than their CO(2--1) maps, but they are still the largest (\sim1'-2.35') and deepest (rms~\sim0.04~K per \sim5~km~s^{-1}) \CI  mapping with ALMA.$
$Furthermore, in order to study the \CI  and CO in different galactic environments, we selected two starbursting and/or AGN-host galaxies from the literature whose ALMA \Remark{CI10} and CO(2--1) data are available in the archive: NGC~1808 \citep{Salak2019} and NGC~7469 \citep{Izumi2020}.$
$NGC~1808 has ALMA mosaic observations covering its \sim 1~kpc starburst ring, and NGC~7469 has ALMA single-pointing observations covering its inner \sim 3~kpc area.$
$\subsection{New ALMA ACA Band 8 observations}$
$\label{subsection: ALMA Band 8 Observation}$
$Our \CI  observations for NGC~3627 were taken under the ALMA project 2018.1.01290.S (PI: D. Liu) during June 3rd to Sept. 29th, 2019 with ACA-only at Band~8. The total on-source integration time was 30.8~hours. A mosaic with 149 pointings was used to map an area of 94" \times 136" at a position angle of -27^\circ, which covers the galaxy centre, bar and the majority of the spiral arms, and is about 1/3 of the full PHANGS-ALMA CO(2-1) map area.$
$The achieved line sensitivity per 5 \kms is 45--55~mK (107--130~\mathrm{mJy/beam}) across the \Remark{CI10} line frequencies with a beam of 3.46".$
$The \CI  mapping for NGC~4321 were taken under the ALMA project 2019.1.01635.S (PI: D. Liu; NGC~1365 is also partially observed for this project) during Dec. 3rd, 2019 to July 1st, 2021, also with ACA-only at Band~8. The on-source integration time reached 16.1~hours for the mapping of an area of 55" \times 42" with 23 pointings using the 7m array. The achieved line sensitivity is similar to that of NGC~3627.$
$More detailed information is newd in Table~\ref{table: targets}.$
$ACA (7m) was chosen to new a matched and slightly coarser synthesized beam (\sim 3"-4") than the PHANGS-ALMA CO(2-1) data. In comparison, the main 12m ALMA array in compact C43-1/2 configurations will result in a \sim 0.4"-0.7" beam and significantly miss large scale (>4-6") emission without ACA.$
$No total power observations were requested in our presented observation, therefore we matched the uv ranges of the raw CO(2--1) and \Remark{CI10} visibility data during our data reduction.$
$We verified that missing flux will not substantially (\sim 30\%) bias our analysis using three methods as detailed in Appendix~\ref{appendix: missing flux}. This includes a CASA simulation of visibilities mimicking our ACA mosaic observations in NGC~3627, from which we find a missing flux as small as 7\% in \CI-bright pixels and up to \sim 50\% in the faintest pixels (and mostly < 30\%).$
$Our data reduction follows the PHANGS-ALMA imaging and post-processing pipeline \citep[][]{Leroy2021pipeline}, including imaging and deconvolution, mosaicking, broad and strict/signal mask generation, and moment map creation.$
$The additional steps in this study are: (a) uv-clipping in the CO(2--1) data to match the uv sampling range of \Remark{CI10}; and (b) making a joint mask of the individual CO(2--1) and \Remark{CI10} masks then extracting the moment maps (in the joint broad mask) and line ratios maps (in the joint strict mask).$
$More technical details are newd in Appendix~\ref{appendix: data reduction}.$
$Our \Remark{CI10} and CO(2--1) line intensity maps, the line ratio maps and \textit{HST} images for the two PHANGS galaxies, NGC~3627 and NGC~4321, are presented in Figs.~\ref{fig: NGC3627 line ratio map} and \ref{fig: NGC4321 line ratio map}, respectively.$
$Our data cubes and moment maps are also made publicly available at$
$the PHANGS-ALMA CADC public repository \footnote{\url{https://www.canfar.net/storage/list/phangs/RELEASES/DZLIU_etal_2022}}.$
$\begin{table*}[htb]$
$    \centering$
$    \caption{\label{table: targets}$
$        Physical and Data Properties of the Sample$
$    }$
$    \begin{tabularx}{\linewidth}{l c c c c}$
$        \setlength{\mymincolwidth}{\dimexpr 0.85\linewidth/5 \relax}$
$        \global\mymincolwidth=\mymincolwidth$
$        \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt}$
$            \\[-\arraystretch\normalbaselineskip]$
$        \hline$
$        \hline$
$         --- &$
$            NGC 3627 &$
$            NGC 4321 &$
$            NGC 1808 &$
$            NGC 7469 \\hline$
$         redshift &$
$            0.00243 &$
$            0.00524 &$
$            0.00332 &$
$            0.01627 \distance (Mpc) &$
$            11.32 &$
$            15.21 &$
$            10.8 &$
$            71.2\linear scale (pc/") &$
$            54.9 &$
$            73.7 &$
$            52.4 &$
$            345.2 \angular resolution &$
$            3.46" (190 pc) &$
$            3.65" (269 pc) &$
$            2.67" (140 pc) &$
$            0.46" (159 pc) \channel width (\mathrm{km s^{-1}}) &$
$            4.77 &$
$            4.76 &$
$            9.52 &$
$            4.76 \\textnormal{[}C \textsc{i}\textnormal{]}(1--0) flux scale (\mathrm{K Jy^{-1}}) &$
$            0.42 &$
$            0.38 &$
$            0.71 &$
$            23.8 \CO(2--1) flux scale (\mathrm{K Jy^{-1}}) &$
$            1.92 &$
$            3.70 &$
$            3.24 &$
$            108.4 \\textnormal{[}C \textsc{i}\textnormal{]}(1--0) channel rms (K) &$
$            0.044 &$
$            0.043 &$
$            0.056 &$
$            0.11 \CO(2--1) channel rms (K) &$
$            0.022 &$
$            0.015 &$
$            0.083 &$
$            0.14\imaging area &2.35' \times 1.60' &$
$            1.28' \times 1.00' &$
$            40" \times 40" &$
$            \textnormal{r}\sim9"$
$            \imaging type &$
$            7m, mosaic &$
$            7m, mosaic &$
$            12m+7m+tp, mosaic &$
$            12m+7m, single\M_{\star} (10^{10}   \mathrm{M_{\odot}}) &$
$            6.8 &$
$            5.6 &$
$            2.0 &$
$            10.7$
$            \L_{\mathrm{FIR}} (10^{10}   \mathrm{L_{\odot}}) &$
$            1.7 &$
$            1.9 &$
$            3.5 &$
$            25$
$            \L_{\mathrm{TIR}} (10^{10}   \mathrm{L_{\odot}}) &$
$            2.4 &$
$            2.5 &$
$            5.1 &$
$            39$
$            \L_{X} (10^{40}   \mathrm{erg s^{-1}}) &$
$            0.032 &$
$            0.12 &$
$            1.6 &$
$            1500$
$            \SFR (\mathrm{M_{\odot} yr^{-1}}) &$
$            3.84 &$
$            3.56 &$
$            7.9 (SB ring: 4.5) &$
$            76$
$            \M_{\mathrm{H_2}} (10^{9}   \mathrm{M_{\odot}}) &$
$            6.0 &$
$            7.8 &$
$            (SB ring: 0.52) &$
$            15$
$            \M_{\mathrm{H}{\textnormal{\textsc{i}}}} (10^{9}   \mathrm{M_{\odot}}) &$
$            1.1 &$
$            2.7 &$
$            1.7 &$
$            5.7$
$            \mean\RCICO  &$
$            0.11 &$
$            0.10 &$
$            0.16 &$
$            0.21$
$            \mean\log_{10} \RCICO&$
$            -0.96 &$
$            -1.02 &$
$            -0.83 &$
$            -0.69$
$            \\pm 2 \sigma range of \log_{10} \RCICO&$
$            -1.26, -0.66 &$
$            -1.24, -0.80 &$
$            -1.03, -0.63 &$
$            -0.95, -0.43$
$            \\hline$
$        \hline$
$    \end{tabularx}$
$    \tablefoot{$
$        Redshifts are taken from the NASA Extragalactic Database (NED).$
$        The distances of NGC~3627, NGC~4321, NGC~1808 and NGC~7469 are taken from \cite{2009AJ....138..332J}, \cite{2001ApJ...553...47F}, \cite{Salak2019} and \cite{Izumi2020},$
$        respectively.$
$        The stellar masses, SFRs, M_{\mathrm{H_2}} and M_{\mathrm{H}{\textnormal{\textsc{i}}}} of NGC~3627 and NGC~4321 are taken from the PHANGS sample table v1.6 of \citet[][PHANGS survey]{Leroy2021phangs}.$
$        The stellar masses and SFRs of NGC~1808 and NGC~7469 are taken from the z0MGS survey (\citealt{Leroy2019}; \url{https://irsa.ipac.caltech.edu/data/WISE/z0MGS/}).$
$        The M_{\mathrm{H_2}} and M_{\mathrm{H}{\textnormal{\textsc{i}}}} of NGC~7469 are adopted from \cite{Papadopoulos2000}.$
$        The global M_{\mathrm{H}{\textnormal{\textsc{i}}}} mass in NGC~1808 is measured by \citet{Koribalski1993}.$
$        L_{\mathrm{FIR}} and L_{\mathrm{TIR}} are IR 40--400\mum and 8--1000\mum luminosities all from \citet{Sanders2003}.$
$        The X-ray luminosities for NGC~3627 and NGC~4321 are both L_{0.3\textnormal{--}8\mathrm{keV}} from \citet{2011ApJ...731...60G}, and those for NGC~1808 and NGC~7469 are L_{2\textnormal{--}10\mathrm{keV}} from \citet{2005A&A...442..861J} and \cite{Izumi2020}, respectively.$
$    }$
$\end{table*}$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=\textwidth, trim=0 0 0 0]{Plot_two_line_ratio_four_panel_image_NGC3627_CI10_CO21_with_regions_with_HST}$
$\caption{\textit{Top panels}: NGC~3627 \Remark{CI10} (\textit{left}) and \Remark{CO21} (\textit{right}) integrated line intensity (moment-0) maps within our broad mask (representing high-completeness signals; see Appendix~\ref{appendix: data reduction}).$
$\textit{Bottom left panel}: \Remark{CI10}/\Remark{CO21} line ratio map within the combined signal mask.$
$\textit{Bottom right panel}: HST F814W-F555W-F438W RGB composite image (PHANGS-HST; \citealt{PHANGSHST}).$
$Regions marked with the white boxes and labeled with numbers are mainly for illustration purpose and for discussion in the text. Their zoom-in images and extracted spectra are newd in our online data release. Examples for Regions~1, 4 and 22 are presented in Appendix~\ref{appendix: all regions}.$
$All images have the same field of view of 179" \times 179", or 9.83 \mathrm{kpc} \times 9.83 \mathrm{kpc}.$
$}$
$\label{fig: NGC3627 line ratio map}$
$\end{figure*}$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=\textwidth, trim=0 0 0 0]{Plot_two_line_ratio_four_panel_image_NGC4321_CI10_CO21_with_regions_with_HST}$
$\caption{$
$Similar to Fig.~\ref{fig: NGC3627 line ratio map}, the \Remark{CI10} and CO(2--1) moment-0 intensity maps and the line ratio map and the HST composite RGB image for NGC~4321. The field of view is 78" \times 78", or 5.75 \mathrm{kpc} \times 5.75 \mathrm{kpc}.$
$}$
$\label{fig: NGC4321 line ratio map}$
$\end{figure*}$
$\subsection{ALMA observations of the additional targets from the literature}$
$\label{subsection: additional targets}$
$The NGC~1808 ALMA \Remark{CI10}, \Remark{CO10} and \Remark{CO21} observations are presented in \citet{Salak2019} and all are interferometry plus total power, i.e., 12m+7m+tp, thus having no missing flux issue.$
$To study the \CI /\CO  line ratio consistently with our main targets, we only used the \Remark{CI10} and \Remark{CO21} data.$
$We re-reduced the raw data under ALMA project 2017.1.00984.S (PI: D. Salak) with the observatory calibration pipeline, then imaged, primary-beam-corrected and short-spacing-corrected using the PHANGS-ALMA pipeline in the same way as for our main targets (Appendix~\ref{appendix: data reduction}).$
$Note that in this step we re-reduced and imaged the total power data with the PHANGS-ALMA singledish pipeline (\citealt{Herrera2020,Leroy2021pipeline}) and did the short-spacing correction with the PHANGS-ALMA pipeline right after the primary beam correction.$
$The properties of the final \CO -\CI  beam-matched data cubes are listed in Table~\ref{table: targets}.$
$The NGC~7469 ALMA data are presented in \citet{Izumi2020} and we used the archival raw data of \Remark{CI10} and \Remark{CO21} under ALMA project 2017.1.00078.S (PI: T. Izumi). These are single pointings toward the centre. \Remark{CI10} is observed with ACA 7m and \Remark{CO21} with the 12m array, both without total power. We re-reduced the data following the same procedure as for NGC~3627 and NGC~4321 with the PHANGS-ALMA pipeline.$
$Our re-reduced \Remark{CI10} and CO(2--1) line intensity and ratio maps for these additional galaxies are shown in Fig.~\ref{fig: other line ratio maps}.$
$\vspace{2ex}$
$\section{Results of the observed line ratios}$
$\label{sec: Results}$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.9\textwidth, trim=0 7.7mm 0 2mm, clip]{Plot_two_line_ratio_two_panel_image_NGC1808_CI10_CO21}$
$\includegraphics[width=0.9\textwidth, trim=0 0 0 2mm, clip]{Plot_two_line_ratio_two_panel_image_NGC7469_CI10_CO21}$
$\caption{$
$\Remark{CI10} (\textit{left}) and \RCICO  (\textit{right}) maps of NGC~1808 in the upper panels and NGC~7469 in the lower panels.$
$Similar to Figs.~\ref{fig: NGC3627 line ratio map} and \ref{fig: NGC4321 line ratio map}, the line intensity maps are newcommandined in the broad mask and the \RCICO  maps are restricted to the combined signal mask.$
$Boxes in the left panels indicate the manually selected central regions where the pixels are highlighted in Fig.~\ref{fig: line ratio vs CO}.$
$Contours in the right panels indicate the 5, 10, 20 and 50-\sigma levels of the left-panel \CI  intensity maps.$
$\label{fig: other line ratio maps}$
$}$
$\end{figure*}$
$\subsection{Spatial distributions}$
$In Figs.~\ref{fig: NGC3627 line ratio map}, \ref{fig: NGC4321 line ratio map} and \ref{fig: other line ratio maps}, we present the beam-matched \Remark{CI10} and \Remark{CO21} integrated line intensity maps and their ratio maps.$
$The \CI/\CO  line ratio map is computed only for pixels within the joint strict mask area of the two lines, as described in Appendix~\ref{appendix: data reduction}, representing high-confidence signals.$
$In NGC~3627 (Fig.~\ref{fig: NGC3627 line ratio map}), we visually marked 23 regions based on the \CI  and CO commonly-detected (\mathrm{S/N}>3), strict-mask areas for later analysis. Region IDs are sorted by their \Remark{CI10} brightness.$
$Region 1 corresponds to the galaxy centre, Regions 2 and 3 are the southern and northern bar ends, Region 4 is an offset peak next to the galaxy centre, and other regions are mostly on the spiral arms. Most of these regions show a similar \RCICO \sim 0.1, except for, e.g., Regions 20 and 22 which exhibit a somewhat higher ratio of \RCICO \sim 0.5. The edge of Region 4 also shows an enhancement of \CI  yet it is hard to tell whether this is physical or simply caused by edge effects, e.g., as seen in our simulations (Appendix~\ref{appendix: simulation}).$
$We also examined the integrated spectra of each region in Appendix~\ref{appendix: all regions}.$
$The \CI  and CO line profiles are not always well-matched in width and shape. In the galaxy centre (Region 1) and at bright bar-ends (Regions 2, 3, 5), \CI  and CO have similar line widths but \CI  has a lower intensity than CO, whereas in some fainter regions (9, 10, 13, 14), \CI  is narrower than CO and relatively weaker. In Regions 4, 20 and 22 where \RCICO is high, \CI  and CO have similar line widths but the \CI  intensity is enhanced.$
$In NGC~4321 (Fig.~\ref{fig: NGC4321 line ratio map}), we mapped a smaller area due to its fainter CO brightness than NGC~3627. There are no regions with \Remark{CO21} integrated intensities >100  \Kkms outside our selected area in NGC~4321, whereas in comparison the NGC~3627 bar ends have \Remark{CO21} integrated intensities \gtrsim 300  \Kkms. The \CI  mapping in NGC~4321 has a similar depth as in NGC~3627.$
$We marked 7 regions in NGC~4321 to guide the visual inspection. A clear \RCICO  newcommandicit can be seen in the galaxy centre (Region 1), with \RCICO \sim 0.06, nearly a factor of two smaller than its inner disk which has similar \RCICO \sim 0.1 as seen in the NGC~3627 disk. Some \CI  enhancement can be found at the edges or slightly outside these regions, where CO and \CI  are both faint (\Remark{ICO21} \sim 30-100  \Kkms) and the edge effect of interferometric missing flux might play a role (see Appendix~\ref{appendix: simulation}).$
$The \RCICO  spatial distributions of the other two starburst/AGN galaxies (Fig.~\ref{fig: other line ratio maps}) show in general more enhanced \RCICO  than in the NGC~3627 and NGC~4321 disks.$
$Large variations from the centres to outer areas can be seen, especially at the the S/N threshold of \sim 3 \text{--} 5 (see contours).$
$To quantify the spatial co-localization of the \CI  and CO emission, we use the \incode{JACoP} tool \citep{JACoP} \footnote{\url{https://imagej.net/plugins/jacop}}$
$in the \incode{ImageJ} software \citep{ImageJ} to measure the commonly used co-localization indicators, namely the Pearson's coefficient (PC; equal to 0 and 1 for null and 100\% localization, respectively) and Manders' coefficients (M1 and M2; which measure the overlap between two images and are equal to one for perfect overlap). We turn on the option of using Costes' automatic threshold \citep{Costes2004} which automatically set a pixel value threshold for each image to minimize the contribution of noise for the comparison \citep{JACoP}.$
$For the \CI  and CO emission in our galaxies, we find \mathrm{PC/M1/M2} values all very close to 1, that is, nearly 100\% co-localization (\mathrm{PC/M1/M2} = 0.934/0.985/0.990 for NGC~3627, 0.816/0.986/0.964 for NGC~4321, 0.978/0.890/0.906 for NGC~1808, and 0.935/0.999/0.998 for NGC~7469).$
$Overall, the spatial comparison of the \CI  and CO intensities demonstrates that the two species are well-correlated when observed at hundred parsec scales.$
$Meanwhile, we do see certain spectral differences in the line profiles as shown in Appendix~\ref{appendix: all regions}. This may suggest that variations exist at smaller scales than our resolution, which could be the sign of different evolutionary stages \citep[e.g.][]{Kruijssen2014} and would be interesting for higher-resolution follow-ups, e.g., with the main ALMA 12m array.$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=\textwidth]{Plot_Line_Ratio_CI10_CO21_New_surface_density}$
$    \caption{$
$        \textit{Left}: \Remark{ICI10} versus \Remark{ICO21} in the four galaxies we studied: NGC~3627, NGC~4321, NGC~1808 and NGC~7469. \Remark{ICI10} and \Remark{ICO21} integrated fluxes are moment-0 intensities extracted from their beam and pixel size matched data cubes. Only data points with propagated moment-0 \SNR \ge 5 are shown.$
$        The blue (with crossed hatch), green (with horizontal line hatch), gray (with dotted hatch) and light gray shadings indicate the 5 \sigma thresholds for the four galaxies, respectively. Typical line widths of 10~km~s^{-1} (NGC~3627 and NGC~4321) and 30~km~s^{-1} (NGC~1808 and NGC~7469) in sigma are used to calculate the thresholds together with the channel rms in Table~\ref{table: targets}.$
$        For NGC~3627 and NGC~4321, their (uv-matched) CO(2--1) 5 \sigma thresholds are out of the plotting range.$
$        Top axis shows the simply-converted H_2 surface density using a constant \alphaCO = 4.3 and CO excitation (i.e., CO(2--1) to CO(1--0) line ratio) R_{21} = 0.65.$
$        \textit{Right}: Same data but shown as the line ratio \RCICO = \Remark{ICI10}/\Remark{ICO21} versus \Remark{ICO21}.$
$        The blue horizontal line indicates a star-forming (SF) disk-like \RCICO \sim 0.1, i.e., the spiral arms of NGC~3627 and NGC~4321. The gray horizontal line indicates a starburst (SB) disk-like \RCICO \sim 0.2, i.e., the \sim 1~kpc SB disk/ring in NGC~1808 and NGC~7469.$
$        The red line with a slope of 0.8 in logarithm (0.035   (I_{\Remark{CO21}}/[100 \Kkms])^{0.8}) visually guides the trend of increasing \RCICO in the central regions of these galaxies (see boxes in Figs.~\ref{fig: NGC3627 line ratio map} to \ref{fig: other line ratio maps}, sizes \sim 300--500~pc).$
$    }$
$    \label{fig: line ratio vs CO}$
$\end{figure*}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=0.493\textwidth]{Plot_Histogram_of_Line_Ratio_CI10_CO21}$
$    \includegraphics[width=0.493\textwidth]{Plot_Percentiles_of_Line_Ratio_CI10_CO21}$
$    \caption{$
$    \textit{Left panel} shows the histograms of the sum of CO(2--1) surface brightness over pixels in each \CI/\CO  line ratio interval in the four galaxies. The bold curves are the cumulative distributions of the corresponding histograms.$
$    \textit{Right panel} shows the mean \CI/\CO  line ratio for pixels in each bin of CO(2--1) surface brightness (or molecular gas surface density as indicated by the top-axis assuming a constant \alphaCO = 4.3 and R_{21} = 0.65). Shaded area enveloping each mean trend indicates the 16- to 84-th percentiles in each bin, and dashed line indicates the 50-th percentile.$
$    This panel thus more clearly illustrates the ridge and scatter of the trends in the right panel of Fig.~\ref{fig: line ratio vs CO}.$
$    Detection limits are shown consistently with Fig.~\ref{fig: line ratio vs CO}.$
$    }$
$    \label{fig: line ratio histogram}$
$\end{figure*}$
$\subsection{\RCICO versus CO surface brightness and gas surface density}$
$In Fig.~\ref{fig: line ratio vs CO} we show the scatter points of \CI  and CO line brightnesses and the \RCICO  line ratios.$
$Systematic differences in \RCICO can be found among these galaxies and within their galactic environments. We discuss these environments separately as follows.$
$\subsubsection{Star-forming and starburst disk environments}$
$The majority of molecular gas in the 3--7~kpc star-forming disks of NGC~3627 and NGC~4321 exhibits a tight distribution of \RCICO around 0.1 (within about \pm50\%). The distribution is fairly flat over an order of magnitude in \Remark{ICO21}. We denote this as the "SF disk" regime.$
$The ratio rises to about 0.2 within a galactocentric radius of few hundred parsec to \sim 1~kpc starburst areas in NGC~1808 and NGC~7469, which we newcommandine as the "SB disk" regime.$
$The nearly flat "SB disk" distribution is consistent with earlier studies. \citet{Salak2019} found that in NGC~1808 \Remark{ICI10} is proportional to \Remark{ICO21}^{ +1.05}. They also studied CO(1--0), finding that \Remark{ICI10} \propto \Remark{ICO10}^{ +1.46}, i.e., with a steeper slope.$
$\citet{Saito2020} found a similar slope between \Remark{ICI10} and \Remark{ICO10}: \Remark{ICI10} \propto \Remark{ICO10}^{ +1.54} in IRAS~F18293-3413 (whose CO(2--1) data is insufficient for such a study as previously mentioned).$
$This correlation slope is likely to change with different CO transitions due to the CO excitation's dependence on the CO luminosity or star formation rate themselves (e.g., \citealt{Daddi2015,Liudz2021a}).$
$For higher-J CO transitions, \citet{Michiyama2021} studied \CI  and CO(4--3) lines in 36 (U)LIRGs, finding \Remark{ICI10} \propto \Remark{ICO43}^{ +0.97}.$
$The upward tails seen at the CO-faint-end in the right panel of Fig.~\ref{fig: line ratio vs CO} are mostly due to the detection limits in each galaxy. The color shading indicates the 5-\sigma limit for each galaxy, assuming a typical line width of 10~km~s^{-1}. Region~22 of NGC~3627 and the outer pixels in the two starburst/AGN galaxies make the tips of the blue and gray faint-end tails, respectively. But most of these features should still be spurious. We performed CASA simulations following the real observations of NGC~3627 to verify what we would get if \RCICO  were constant across the galaxies (Appendix~\ref{appendix: simulation})$
$We find that noise can indeed boost a simulated \RCICO  of 0.2 to a measurement as high as 0.5 in a NGC~3627-like large mosaic, or even higher to >1 in single-pointing ALMA data without total power or sufficient short spacing. However, the faint-end tail from the simulation is not very similar to that of NGC~3627, raising some questions about the nature of the high-\RCICO  in Region~22.$
$Furthermore, we also did stacking experiments using the CO line profile as the prior to measure the \CI  line fluxes in bins of CO line brightnesses or galactocentric radii. We find no systematically higher \RCICO  at fainter CO pixels or at the outer radii.$
$\subsubsection{Galaxy centre environments}$
$Most interestingly, the \lesssim 250~pc centres of all four galaxies show dramatically different \RCICO. It varies by a factor of ten from \sim 0.05 in the centre of NGC~4321  to \sim 0.1 in the centre of NGC~3627, then to \sim 0.15--0.2 at the centre of NGC~1808, and to \sim 0.2--0.5 in the nucleus of NGC~7469.$
$This "nuclei" trend has a slope of about 0.8, i.e., \Remark{ICI10} \approx 0.035 \; \Remark{ICO21}^{ +0.8}, and corresponds to a \Sigmol range of about 10^{3} to 10^{4} \; \Mspc2 if adopting an \alphaCO = 4.3  \mathrm{M_{\odot} \; (K   km   s^{-1}   pc^2)^{-1}} (\citealt{Bolatto2013}) and a CO excitation R_{21} = 0.65 (\citealt{Leroy2021c}).$
$The reasons for the nuclei showing this trend likely include both the effect of varying [\CI/CO] abundance ratio and the excitation and radiative transfer. They are determined by the actual ISM conditions in each environment, i.e., the gas kinetic temperature, \CI  and CO column densities (\CIcolumndensity and \COcolumndensity), line widths (\Delta{v}), and \mathrm{H_2} volume density (\nHtwo), etc.$
$They will be analyzed in Sect.~\ref{sec: Excitation Analysis}.$
$\subsubsection{Histogram and curve-of-growth of the line ratio distributions}$
$Fig.~\ref{fig: line ratio histogram} shows the histogram and cumulative distribution functions of \RCICO (left panel) and the mean \RCICO and the 16-th to 84-th percentiles in bins of CO line intensity (right panel) in our galaxies.$
$The histogram is computed as the sum of CO line intensity in each bin of \RCICO, representing the amount of gas at each \RCICO.$
$The NGC~4321 histogram is lower than others, showing that it has the least total molecular gas mass among the four galaxies within mapped areas. The NGC~3627 histogram has a peak about 0.5~dex higher than that of NGC~4321, but their peak locations are consistently at \log \RCICO \sim -1, same conclusion as our previous scatter plots show.$
$The two SB galaxies NGC~1808 and NGC~7469, despite having much smaller observed areas (\sim 1--2~kpc), have more than an order of magnitude higher peaks hence total molecular gas masses.$
$The locations of their histogram peaks also shift to a systematically higher \RCICO.$
$It is clearly seen that both SF and SB disks lack a high-\RCICO (\log \RCICO > -0.35) gas component which dominates the gas in the strong AGN host galaxy NGC~7469.$
$The curve-of-growth in the left panel of Fig.~\ref{fig: line ratio histogram} further shows the cumulative CO brightness in bins of \RCICO, and reflects the fact that the majority of gas in our galactic disks has a narrow distribution of \RCICO (see also last few rows in Table~\ref{table: targets}).$
$In NGC~3627, NGC~4321 and NGC~1808, the higher- and lower-\RCICO gas outside \pm 0.1~dex around the mean \RCICO only sums up to <5--10\% of their total gas masses.$
$In the right panel of Fig.~\ref{fig: line ratio histogram}, we illustrate how the mean \RCICO changes with different gas surface density, as well as the scatter of the data point distribution. The 2D pixels are binned by CO brightness, and the mean and 16\%/50\%/84\%-th percentiles of the \RCICO are computed for each bin.$
$It sketches out the mean trends in Fig.~\ref{fig: line ratio vs CO} (right panel), and also reveals that$
$a) the scatter of \RCICO is the largest (\pm 0.20~dex) in the starburst disk of NGC~7469 compared to other galaxies;$
$b) the scatter is the smallest (\pm 0.06~dex) in the star-forming disks of NGC~4321 and NGC~3627 where \Sigmol \sim 100 \text{--} 500 \; \Mspc2;$
$c) the scatters are very similarly (\pm 0.10~dex) at a relatively high \Sigmol \sim 1000 \; \Mspc2 in all our galaxies except for NGC~4321's centre.$
$\subsection{\RCICO versus ISRF}$
$\label{subsection: ISRF}$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.9\textwidth]{Plot_CI_CO_line_ratio_vs_IRSF_intensity_maps}$
$\caption{$
$    Line ratio versus mean ISRF intensity \Umean measured from SED fitting to the near- to far-IR (\textit{Spitzer}, \textit{Herschel}) data.$
$    \textit{Left four panels} are the \Umean maps of NGC~3627, NGC~4321, NGC~1808 and NGC~7469, and as described in Sect.~\ref{subsection: ISRF} are obtained from the works done by J. Chastenet et al. (in prep.) as a continued effort of \citet{Chastenet2021}.$
$    \textit{Right panel} shows the scatter plot of \RCICO versus \Umean at a spatial resolution of \sim 18" in the manually-selected apertures shown in the left panels.$
$    We computed a beam-weighted mean \RCICO and a centre pixel \RCICO for each aperture, displayed as the solid and open symbols, respectively.$
$    Error bars are the corresponding beam-weighted mean or central pixel \RCICO  uncertainties in each region.$
$    The region diameter (\sim 18") corresponds to the coarsest resolution of the \textit{Herschel} data used in the SED fitting.$
$    The gray dotted line shows \RCICO = 0.07   U^{0.25} as a guiding line.$
$    The colored dotted contours (labeled 0.1--0.5) are the model contours of \RCICO computed from the PDR Toolbox (using the latest \incode{wk2020} model and solar metallicity; \citealt{Kaufman2006}). We convert G_0 from the model contour to the x-axis U by multiplying with a factor of 0.88 (\citealt{Draine2007}).$
$    The right y-axis indicates the PDR model's \nHtwo, whose range is manually adjusted so that the model contours roughly match the left x-axis.$
$    The PDR model contours show little dependence of \RCICO on U in the horizontal direction but a strong correlation with \nHtwo as seen by the vertical gradient.$
$\label{fig: line ratio vs ISRF}$
$}\vspace{1ex}$
$\end{figure*}$
$Here we study how \RCICO  correlates with the interstellar radiation field (ISRF) intensity (U; in units of Milky Way mean ISRF; \citealt{Habing1968}), which is a measure of the UV radiation field strength impacting the ISM. Since \CI  originates from PDRs, its abundance should be highly related to the PDR properties, thus whether or how \RCICO  correlates with the ISRF that illuminates the PDR is a key question.$
$The ISRF intensity U is usually measured by the re-radiated dust emission, as the original UV photons are highly attenuated by dust in the ISM. \citet[][hereafter DL07]{DL07} developed a series of dust grain models and synthesized SED templates that can describe nearby galaxies' observed dust SEDs. A number of studies (e.g., \citealt{Draine2007,Galliano2011,Dale2012,Aniano2012,Aniano2020,Daddi2015,Chastenet2017,Liudz2021a,Chastenet2021}) have used the DL07 models to fit the dust SEDs of galaxies near and far.$
$For our study, we adopt the mass-averaged ISRF \Umean maps from an ongoing effort by J. Chastenet et al. (in prep.) which is a continuation of the work published for other galaxies by \citet{Chastenet2017,Chastenet2021} with the same method.$
$In this method, the \textit{Herschel} far-infrared, \textit{Spitzer} and/or \textit{WISE} near-infrared images \footnote{For the two (U)LIRGs, the WISE maps are saturated, so their inferred ISRF intensity U should be considered with a large uncertainty.} are first PSF-matched to a common resolution \sim 18" (\textit{Herschel} SPIRE 250\mum), then SED fitting is performed using the DL07 models at each pixel.$
$Each dust SED is fitted by two dust components, one in the ambient ISM with a constant ISRF intensity U_{\mathrm{min}}, and the other in regions with higher U with a power law distributed ISRF from U_{\mathrm{min}} up to U_{\mathrm{max}} = 10^{7}.$
$The fitting then gives the best-fit U_{\mathrm{min}} and the power-law index \gamma of the U distribution.$
$The mean ISRF intensity \Umean is calculated as the dust mass-weighted mean of the ambient and power-law ISRF intensities (in DL07, U_{\rm PDR} \ge 10^2 is considered to be associated with the PDR), and better represents the overall ISRF than U_{\mathrm{min}}.$
$Fig.~\ref{fig: line ratio vs ISRF} presents the \Umean maps and the \RCICO  versus \Umean scatter plot. Since the spatial resolution of the \Umean maps is 18", we calculate a weighted \RCICO in each circular aperture shown in Fig.~\ref{fig: line ratio vs ISRF} whose diameter equals the \Umean map resolution, with weighting following the 2D Gaussian beam.$
$In addition, we also show the central pixel value for each region in Fig.~\ref{fig: line ratio vs ISRF}.$
$We restrict the analysis to regions where both \Umean and \RCICO  are measured.$
$A weak correlation between \RCICO  and \Umean is seen from the SF to SB disks, with a large scatter. For reference, we plot a \RCICO = 0.07   U^{0.25} line to indicate this weak trend. The lowest-\Umean data point in NGC~3627 (mainly Region~22) and the centre of NGC~7469 do not follow this trend, possibly indicating other \CI  enhancement mechanisms than \Umean.$
$We use the popular PDR modelling software, PDR Toolbox (version 2.2.9; \citealt{Kaufman2006,Pound2008,pdrtoolbox}) \footnote{\url{https://dustem.astro.umd.edu/}; \url{https://github.com/mpound/pdrtpy}.}$
$to compute the theoretical \RCICO in uniform grids of ISRF intensity in the \citet{Habing1968} G_0 unit and H nucleus density n_{\mathrm{H}} (both in log space). We adopt the latest \incode{wk2020} model set and a solar metallicity (\mathrm{[O/H_2]} = 3.2 \times 10^{-4}, \mathrm{[C/H_2]} = 1.6 \times 10^{-4}).$
$The ISRF intensity grid can be directly scaled to U by multiplying a factor 0.88 (\citealt{Draine2007}), corresponding to the x-axis of Fig.~\ref{fig: line ratio vs ISRF}.$
$The n_{\mathrm{H}} grid is not directly linked to the left y-axis of Fig.~\ref{fig: line ratio vs ISRF}, but because the model contours show fairly flat \RCICO versus U trends, we can match the model contours and indicate the n_{\mathrm{H}} grid in the right y-axis of Fig.~\ref{fig: line ratio vs ISRF}.$
$The PDR modelling predicts generally flat \RCICO versus U trends similar to the distribution of the observed data points, although our U are diluted values at a much larger physical scale than in the PDR models.$
$However, in PDR models, \RCICO  changes moderately with n_{\mathrm{H}}, and a relatively high n_{\mathrm{H}} \sim 10^{3.5\text{--}3.7}   \mathrm{cm^{-3}} is needed to reproduce an \RCICO \sim 0.1--0.2, or equivalently \sim 1.0--1.9 for the ratio of \Remark{CI10} and \Remark{CO21} fluxes in \mathrm{erg s^{-1} cm^{-2} sr^{-1}} units used by the PDR Toolbox \footnote{See \url{https://github.com/mpound/pdrtpy-nb/blob/master/notebooks/PDRT_Example_Model_Plotting.ipynb}.$
$We scale the ratio of intensities in \mathrm{erg s^{-1} cm^{-2} sr^{-1}} to the ratio of intensities in \mathrm{K km s^{-1}} by a factor of (\nu_{\mathrm{CI10}}/\nu_{\mathrm{CO21}})^{-3} = 9.73, where \nu is the rest-frame frequency of the lines. We caution that the old version of \incode{PDRToolbox} \incode{v2.1.1} does not include the \incode{CI_609/CO_21} line ratio, but has \incode{CI_609/CO_43} and \incode{CO_43/CO_21} ratios. However, we tested that the multiplication of \incode{CI_609/CO_43} and \incode{CO_43/CO_21} does not produce the same result as the direct \incode{CI_609/CO_21} ratio in the new \incode{PDRToolbox} \incode{v2.2.9}. We use the new version for this work.$
$}.$
$Furthermore, the PDR model needs a \sim0.5~dex lower n_{\mathrm{H}} to explain the 2\times elevated \RCICO  in SB disks, which seems somewhat counterintuitive.$
$This apparent tension is likely caused by the observations probing a much coarser resolution that consists of a large number of PDRs with different n_{\mathrm{H}} and U (e.g., increasing U from 10 to 400 can increase \RCICO from 0.2 to 0.5 based on our PDR Toolbox modelling). It may also relate to an abundance variation driven by non-PDR mechanisms ---$
$X-ray dominated region (XDR; \citealt{Maloney1996,Meijerink2005,Meijerink2007}; see also review by \citealt{Wolfire2022}) and Cosmic Ray Dominated Region (CRDR; \citealt{Papadopoulos2010c,Papadopoulos2011,Papadopoulos2018,Bisbas2015,Bisbas2017}) are well-known to play an important role in starbursting and AGN environments, respectively.$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=0.9\textwidth]{Plot_CI_CO_line_ratio_vs_metallicity_maps}$
$\caption{$
$    \textit{Left two panels} show the metallicity maps of NGC~3627 and NGC~4321 from \citet{Williams2022} based on PHANGS-MUSE (\citealt{PHANGSMUSE}) data. Boxes are regions as newcommandined in Figs.~\ref{fig: NGC3627 line ratio map} and \ref{fig: NGC4321 line ratio map}.$
$    \textit{Right panel} shows the \RCICO  versus metallicity scatter plot for regions in NGC~3627 and NGC~4321. The bottom-axis is metallicity in solar units, top-axis is metallicity in 12+\log(\mathrm{O/H}), left-axis is the line ratio of \Remark{CI10} and \Remark{CO21} in \Kkms units, and right-axis is the ratio of fluxes in \mathrm{erg s^{-1} cm^{-2} sr^{-1}} units.$
$    The left- and right-axes are matched by a factor of (\nu_{\mathrm{C{\textsc{i}}1\textnormal{--}0}}/\nu_{\mathrm{CO1\textnormal{--}0}})^3 \times R_{21}, where we adopt a R_{21}=0.65 (\citealt{Leroy2021c}).$
$    Data points are the mean metallicity and \RCICO  in each region shown in left panels, with error bars indicating the typical uncertainty in \RCICO , and vertical color shading indicating the 16- to 84-th percentiles of pixels' \RCICO  distribution inside each region.$
$    The \citet{Bolatto2000b} empirical fitting and \citet{Bolatto1999} model prediction lines are overlaid.$
$    The three outliers in NGC~3627 with \RCICO \sim 0.19, 0.28 and 0.35 are Regions 20, 21 and 22, respectively. The outlier point in NGC~4321 with \RCICO \sim 0.06 is its centre Region 1.$
$    \label{fig: line ratio vs metallicity}$
$}\vspace{1ex}$
$\end{figure*}$
$As pointed out by \citet{Papadopoulos2011} and later works, ultra-luminous infrared galaxies (ULIRGs) with IR luminosities L_{\mathrm{IR, 8-1000\mu\mathrm{m}}} \ge 10^{12}   \Lsun and corresponding \mathrm{SFRs} > 1000   \Msyr have cosmic-ray (CR) energy densities \sim 10^{3} \text{--} 10^{4}\times larger than the Milky Way. In our sample, the two SB galaxies are not as intensively star-forming as ULIRGs, but their SFRs are still a factor of 2 to 20 higher than our SF galaxies. More importantly, the SB disks have considerably higher (e.g., one to two orders of magnitude) SFR surface densities than the SF disks, plausibly leading to a factor of a few tens to hundreds higher CR intensities. The CRs emitted from OB star clusters and supernova remnants can penetrate deeply into the ISM, and thus nearly uniformly illuminate the ISM and effectively dissociate CO into \CI .$
$A detailed simulation by \citet{Bisbas2017} showed that 10\times and 100\times higher CR ionization rates (\xi_{\mathrm{CR}}) than the Milky Way level results into a factor of about 2 and 3 mildly-increased \CI  abundance and about 1/2 and 1/30 strongly-decreased CO abundance, respectively (see their Fig.~3), in a 10~pc GMC with a mean gas density \left<n_{\mathrm{H}} \right>= 760   \mathrm{cm^{-3}}, resembling Milky Way and typical SF galaxies' ISM conditions.$
$Our SB galaxies probably have a \xi_{\mathrm{CR}} around or slightly above 10, but not reaching 100 given their mild \Umean \sim 20-30 compared to the \Umean \gtrsim 50 in extreme SB galaxies, e.g., local ULIRGs and high-redshift hyperluminous IR-bright (HyLIRG) and submm galaxies (\citealt{Daddi2015,Silverman2018,Liudz2021a}).$
$In Sect.~\ref{sec: Excitation Analysis}, we show that the mildly-increased [\CI/CO] abundance ratio from a CRDR and temperature/density-driven excitation can lead to the factor of 2 increase in \RCICO  when going from SF to SB disks.$
$\subsection{\RCICO versus Metallicity}$
$\label{subsec: metallicity}$
$Our current sample covers a relatively small range in metallicity for a metallicity--line ratio study. Given their stellar masses and metallicity maps (e.g., \citealt{Kreckel2019,Kreckel2020,Williams2022}), our targets probe a near-solar metallicity. NGC~3627 has a well-measured$
$metallicity gradient of Z = 8.53 + 0.06 \times[r/R_{25}] from \citet{Kreckel2019} \footnote{\citet{Kreckel2019} used a distance of 10.6 \pm 0.9 \; \mathrm{Mpc} for NGC~3627. We do not directly use this metallicity gradient, but use the metallicity maps from \citet{Williams2022} where the same new distance as in this work is adopted.}, but the metallicity of individual \ionizedhydrogen  regions scatter between \sim 8.45--8.65.$
$\citet{Williams2022} produced the 2D spatial distributions of gas-phase metallicity in PHANGS-MUSE galaxies, including NGC~3627 and NGC~4321, by applying the Gaussian Process Regression to map the smooth, higher-order metallicity variation from the individual, sparse \ionizedhydrogen  regions detected in PHANGS-MUSE (\citealt{PHANGSMUSE, Santoro2021}).$
$This method is significantly superior to a na\"{i}ve nearest neighbor interpolation method. We refer the reader to \citet{Williams2022} for the technical details and validation demonstration.$
$In Fig.~\ref{fig: line ratio vs metallicity} we present the trend between \RCICO  and metallicity in NGC~3627 and NGC~4321.$
$Due to our limited metallicity range, only a very weak decreasing trend can be seen.$
$A similar trend has been reported by \citet{Bolatto2000b}, who observed CO(1--0) and \Remark{CI10} in the Region N27 of the Small Magellanic Cloud and combined with literature data to present a correlation between \RCICO  and metallicity over \sim 0.2 \text{--} 2   Z_{\odot}.$
$We overlay the \citet{Bolatto2000b} empirical fitting \footnote{$
$We adopt the equation from \citep[][Fig.~2 caption]{Bolatto2000b}, with a typo-correction in the brackets so that the equations are:$
$\log ( F_{\Remark{CI10}} / F_{\Remark{CO10}} ) = -0.47   \left[12 + \log \mathrm{(O/H)} \right]+ 5.0 and \log ( F_{\Remark{CI10}} / F_{\Remark{CO10}} ) = -0.82   \left[12 + \log \mathrm{(O/H)} \right]+ 8.2, respectively, where F is the line flux in \mathrm{erg   s^{-1}   cm^{-2}   sr^{-1}}.$
$}$
$as well as a theoretical PDR model prediction line from \citet{Bolatto1999} in Fig.~\ref{fig: line ratio vs metallicity}.$
$Although our data covers only a small metallicity range, it is encouraging to see the overall good agreement with the observationally-driven trend in \citet{Bolatto2000b}.$
$More detailed understanding of the deviation from PDR models will likely need more \CI  mapping in lower-metallicity galaxies.$
$\vspace{2ex}$
$\section{Estimation of abundance ratios under representative ISM conditions}$
$\label{sec: Excitation Analysis}$
$The observed \RCICO  depends on not only the \CI  and CO abundances (\COabundance  and \CIabundance ), but also the ISM conditions that determine the excitation (level population) and radiative transfer of the C atoms and CO molecules --- i.e., the gas kinetic temperature \Tkin, volume density of H_2 (\nHtwo; as the primary collision partners of C and CO), turbulent line width \dv, and the species' column densities \COcolumndensity and \CIcolumndensity.$
$The column density divided by the line width (N/\dv) further determines the optical depth \tau of each line.$
$In this work, having only a CO(2--1) and a \Remark{CI10} line is insufficient to numerically solve all the \Tkin, \nHtwo, \COcolumndensity/\dv, \COabundance  and \CIabundance . Therefore, we assume several \textit{representative ISM conditions} where \Tkin, \dv and \COabundance  are fixed to reasonable values, then solve the \COcolumndensity and \CIcolumndensity. We assume that \CI  and CO are spatially mixed at our resolution, i.e., they have the same filling factor. This is a relatively strong assumption and is likely not the real case, because although we see spatial co-localization at our \sim 200~pc resolutions, the spectral profiles of CO and \CI show discrepancies (Appendix~\ref{appendix: all regions}), indicating different underlying distributions at smaller scales. With the assumption of the same filling factor (and 3D distribution), the column density ratio \CIcolumndensity/\COcolumndensity equals the [\CI/CO] abundance ratio.$
$We perform both LTE and non-LTE calculations.$
$LTE assumes that all transitions are thermalized locally, and the level population follows Boltzmann statistics, so that each transition's excitation temperature T_{\mathrm{ex}} as newcommandined by the level population equals \Tkin (e.g., \citealt{Spitzer1998Book,Tielens2010Book,Draine2011Book}). The LTE assumption facilitates the computation as we do not need to solve the level population and thus \nHtwo is not used.$
$However, LTE is not easily reached in typical ISM conditions. Subthermalized lines need non-LTE calculations. The commonly adopted non-LTE approach is the Large Velocity Gradient (LVG; \citealt{Scoville1974,Goldreich1974}), where an escape fraction is used to facilitate the local radiation intensity calculation hence the detailed balance of radiative and collisional (de-)excitations.$
$We use RADEX (\citealt{RADEX}) to compute the non-LTE \CI  and CO line fluxes for each fixed parameter set (\Tkin, \nHtwo, \COcolumndensity, \CIcolumndensity, \dv), then study how [\CI/CO] determines the line flux ratio \RCICO.$
$We assume the following 7 representative ISM conditions:$
$\textit{(1)} inner Galactic GMCs,$
$\textit{(2)} NGC~3627/4321 disk GMCs,$
$\textit{(3)} NGC~3627 bar-end environments,$
$\textit{(4)} NGC~1808/7469 SB disk,$
$\textit{(5)} NGC~4321 centre,$
$\textit{(6)} NGC~3627 centre,$
$and \textit{(7)} NGC~7469 centre.$
$Conditions \textit{(1)}--\textit{(3)} correspond to the "SF disk" in Fig.~\ref{fig: line ratio vs CO}, and condition \textit{(4)} represents the "SB disk".$
$The last three conditions are for the "nuclei".$
$Under each ISM condition, we set a fixed \Tkin (in units of K), \dv (in units of km~s^{-1}), \COabundance , and \Sigmol (in units of \Mspc2).$
$The adopted values are shown at the top of each panel in Figs.~\ref{fig: LTE RCICO} and \ref{fig: NonLTE RCICO}.$
$Our choice of \Tkin considered multi-line measurements in the literature, e.g., \citet{Heyer2015} for Galactic GMC ISM condition, and \citet{Teng2022} for NGC~3627 nucleus, as well as typical values in simulations (e.g., \citealt{Offner2014,Glover2015,Glover2016,Clark2019,HuChiaYu2021a}).$
$Our \dv is based on the observed line width in the highest-resolution ALMA data, i.e., from the original PHANGS-ALMA 1" data (\citealt{Leroy2021phangs}) or from \citet{Salak2019} and \citet{Izumi2020} for NGC~1808 and NGC~7469, respectively. The observed line width should at least new an upper limit, if not directly probing the turbulent \dv.$
$The \COabundance  is more like a fiducial value and does not obviously affect later [\CI/CO] results.$
$We also take a fiducial \Sigmol which matches the observed CO(1--0) line brightness temperature and a CO-to-H_2 conversion factor \alphaCO \sim 4 \text{--} 0.8 \; \mathrm{M_{\odot}   (K   km   s^{-1})^{-1}} depending on the assumed conditions (e.g., \citealt{Bolatto2013}; and our paper~II).$
$This \Sigmol directly converts to an N_{\mathrm{H_2}} if assuming that 36\% of the mass is helium. Then N_{\mathrm{H_2}} and \COabundance  together indicate a \COcolumndensity . Only with these assumptions can we unambiguously link an abundance ratio [\CI/CO] (i.e., \CIcolumndensity/\COcolumndensity) to a line flux ratio \RCICO .$
$Additionally, with the same filling factor assumption, \RCICOis also equivalent to the ratio of the CO-to-H_2 and \CI -to-H_2 conversion factors, i.e., \RCICO = \alphaCO/\alphaCI, because they trace the same molecular gas at our resolution.$
$In a companion paper (paper II; Liu et al. 2022 in prep.), we study the behaviors of \alphaCO and \alphaCI in more details, also under both LTE and non-LTE conditions.$
$\subsection{LTE results}$
$\label{subsec: LTE}$
$\begin{figure*}[htb]$
$    \centering$
$    \includegraphics[width=0.99\textwidth]{Plot_LTE_RCI10CO21_vs_XCICO_varying_N_CO_T_kin}$
$    \caption{$
$        Observed/calculated \RCICO as a function of CO column density \COcolumndensity  and [\CI/CO] abundance (column density) ratio \RNCINCO for 7 representative ISM conditions, based on LTE calculations.$
$        \textit{Upper panels}: Curved lines are \RCICO versus \RNCINCO for different \COcolumndensity  as indicated by the color. Dashed means CO(2--1) optically thick and \Remark{CI10} optically thin, dotted means both are both optically thin, and solid line means both optically thick.$
$        Horizontal thick line is the observed \RCICO value.$
$        \textit{Lower panels}: Black contours are \RCICO = 0.05,   0.10,   0.20 and 0.50 in the CO column density versus \CI/CO abundance ratio plane.$
$        Vertical color shading indicates the final \RNCINCO that can match both the observed \RCICOand the assumed \Sigmol value noted at the top of each panel.$
$    }$
$    \label{fig: LTE RCICO}$
$\end{figure*}$
$We determine the LTE solution of [\CI/CO] in Fig.~\ref{fig: LTE RCICO}.$
$To illustrate the complexity of this problem and the degeneracy of the \Tkin, \COcolumndensity/\dv and [\CI/CO] parameters, we show how \RCICO (y-axis) varies with \RNCINCO (x-axis) and \COcolumndensity  (color bar) in the upper panels, and how \RCICO (contour) can be pinpointed by determining the \Sigmol (right y-axis).$
$At the top of each column the key parameters of the aforementioned representative ISM conditions are shown as text. The assumed \Tkin and \dv generally increase from panels (1) to (7) mimicking real conditions. Observational-driven \Remark{ICO10} ranges and the assumed \Sigmol for these ISM environments are also listed, and can be converted one into the other via \alphaCO. We also list the chosen \COabundance  for clarity.$
$As shown in the upper panels, \RCICO increases with both \RNCINCO and \COcolumndensity  under low-\Tkin conditions. However, a maximum \RCICO is reached in the high \CI  column density regime, where an increasing abundance ratio will no longer increase \RCICO. The line ratio saturates at about 0.5 when \Tkin \sim 15~K, or at about 1 for \Tkin \sim 100~K. The observed line ratios (horizontal dashed lines) are all lower than the saturation limit of \RCICO _{\mathrm{max}} \sim 0.7 \text{--} 0.9, and corresponds to a range of \COcolumndensity or \Sigmol. This range is very wide in the low-\Tkin conditions, with \log \COcolumndensity / \mathrm{cm^{-2}} \sim 17 - 19, but is very narrow at \Tkin \sim 100~K, that is, \RCICO can be determined from [\CI/CO] without much dependence on \COcolumndensity.$
$In the lower panels, the contours of \RCICO = 0.05, 0.1, 0.2 and 0.5 are shown in the plane of CO column density \COcolumndensity versus \CI/CO abundance ratio.$
$For a fixed \RCICO , a higher \RNCINCO corresponds to a lower \COcolumndensity in the low-\Tkin regime, but \RCICO  becomes insensitive to \COcolumndensity at a high \Tkin. In the latter case, \RNCINCO alone determines \RCICO .$
$By matching to the expected \Sigmol for each ISM condition, we can obtain an unambiguous \RNCINCO from the observed \RCICO  as shown in Fig.~\ref{fig: LTE RCICO}.$
$In this way, we determine the \RNCINCO to be around 0.1--0.2 for conditions \textit{(1)} and \textit{(2)}, and \sim 0.4 for conditions \textit{(3)} and \textit{(5)}.$
$The SB disk (condition \textit{4}) and AGN environments (conditions \textit{6}/\textit{7}) have \RNCINCO \sim 2 and \sim 1 \text{--} 10, respectively.$
$The strongly enhanced \CI/CO abundance ratios in SB disk and AGN environments are clear signatures of CRDR and XDR, respectively.$
$The NGC~4321 centre is a low-ionization nuclear emission-line region (LINER, e.g., \citealt{GarciaBurillo2005}), and given our assumptions it requires \RNCINCO \sim 0.4 in order to explain the observed low line ratio.$
$The gradually increasing \CI/CO abundance ratios in the nuclei of NGC~4321, NGC~3627, NGC~1808 and NGC~7469 agree well with the increasing power of AGN activities, e.g., their X-ray luminosities (Table~\ref{table: targets}).$
$Our assumptions for the Galactic cloud ISM condition are mainly based on \citet{Heyer2009}, \citet{Heyer2015} and \cite{RomanDuval2010}. The obtained [\CI/CO]~\sim 0.1 also agrees well with previous studies using \CI  and CO isotopologues (see Sect.~\ref{subsec: intro RCICO}.$
$For example, \citet{Ikeda1999} reported \RNCINCO \sim 0.05 \textnormal{--} 0.2 for the Orion cloud;$
$\citet{Oka2001} found \RNCINCO = 0.064 \pm 0.035 but a peak of \RNCINCO \sim 0.15 in the DR 15 \HII  region and two IR dark clouds;$
$\citet{Kramer2004} found \RNCINCO \sim 0.11 \textnormal{--} 0.12 in the massive SF region W3;$
$\citet{Genzel1988} found \RNCINCO \sim 0.2 for the massive SF region W51;$
$\citet{Zmuidzinas1988} reported \RNCINCO \sim 0.05 \textnormal{--} 0.15 in several dense clouds.$
$Recently, \citet{Izumi2021} derived an overall \RNCINCO \sim 0.1 in the massive SF region RCW38 at A_V \sim 10 \textnormal{--} 100, consistent with the Orion cloud, but also saw high [\CI/CO] \sim 0.2 \textnormal{--} 0.6 in some even lower column density regions.$
$Accurate determination of all the ISM properties instead of simple assumptions as in this work requires comprehensive, multi-J CO isotopologue observations (e.g., \citealt{Teng2022}).$
$Our representative ISM conditions only new a qualitative diagnostic to understand how to infer the \CI/CO abundance ratio from the observed \RCICO  quantity.$
$Their properties may not be very accurate, and are subject to change with future multi-tracer/CO isotopologue studies.$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=\textwidth]{Plot_RADEX_R_CI10CO21_sim_XCICO_NonLTE_varying_N_CO_T_kin_n_H2}$
$\caption{$
$Similar to Fig.~\ref{fig: LTE RCICO} but with non-LTE calculations.$
$See Fig.~\ref{fig: LTE RCICO} caption for the details.$
$}$
$\label{fig: NonLTE RCICO}$
$\end{figure*}$
$\subsection{Non-LTE results}$
$\label{subsec: non-LTE}$
$We perform the non-LTE calculations using the RADEX code (\citealt{vanderTak2007})$
$with the Large Velocity Gradient (LVG; i.e., expanding sphere) escape probability (\citealt{Scoville1974,Goldreich1974}).$
$Comparing to LTE, the volume density of H_2 is now an additional property that plays a role. The excitation temperature of each line no longer equals \Tkin.$
$We compute the non-LTE line ratios in a grid of ( \Tkin,   \dv,   \COabundance,   \RNCINCO,   \COcolumndensity,   \nHtwo ) parameter sets, then study the line and abundance ratios similarly as in the previous section.$
$In Fig.~\ref{fig: NonLTE RCICO} we show the non-LTE version of Fig.~\ref{fig: LTE RCICO}.$
$The non-LTE result does not qualitatively differ from that of LTE.$
$The most obvious difference is that at high temperature (\Tkin > 50~K, given our other assumed conditions), the required [\CI/CO] is lower by \times 2 for the same observed \RCICO, whereas at low temperature (\Tkin \sim 15~K) a \lesssim 2 \times [\CI/CO] is expected for the observed line ratio.$
$As listed in the title of each panel, we assume \log \nHtwo / \mathrm{cm^{-3}} = 2.5 to 3.0 for our conditions (1) to (7).$
$This is based on the fact that low-J CO and \CI  trace the bulk of molecular gas with \log \nHtwo / \mathrm{cm^{-3}} \sim 2--3 (e.g., \citealt{Leroy2017,Liudz2021a}), with a typical value \nHtwo \sim 300   \mathrm{cm^{-3}} widely adopted/found for Galactic and nearby galaxy GMCs (e.g., \citealt{Koda2012,Pety2017}, see also simulations of \citealt{Glover2012b}), whereas a smaller portion of their emission is from the dense gas with \log \nHtwo / \mathrm{cm^{-3}} \sim 4--6 (e.g., \citealt{Gao2004a}).$
$The fraction of the dense gas is expected to increase in more starbursting environment (\citealt{Gao2004b}), so does the mean gas density (\citealt{Liudz2021a}). Therefore, a \log \nHtwo / \mathrm{cm^{-3}} of 3.0 is perhaps a reasonable assumption, or at least a lower limit for our galaxy centres. A 10 \times higher density does not qualitatively change Fig.~\ref{fig: NonLTE RCICO} except for shifting the expected \RNCINCO to be 50\% higher in the \Tkin > 50~K conditions (making Fig.~\ref{fig: NonLTE RCICO} more like Fig.~\ref{fig: LTE RCICO}).$
$Therefore, our non-LTE result strongly confirms the LTE result in the previous section, that the intrinsic \CI/CO abundance has to vary significantly from about 0.1--0.2 to about 1--3, in order to explain the observed line ratios from Galactic GMCs (hundred pc scales) and NGC~3627/4321 SF disk GMCs to NGC~1808/7469 SB disk/AGN (r \lesssim 250~pc) environments.$
$\vspace{2ex}$
$\section{Discussion}$
$\label{sec: Discussion}$
$From this study we see that the line ratio \RCICO increases from 0.1 to 0.2 and > 0.5 from the SF disk to the SB disk and the vicinity (few hundred pc) of X-ray luminous AGN. Although we may be limited by our sample,$
$we suggest that this trend can be extended to general cases at both local and high redshift, because the \CI/CO abundance ratio enhancement is well-predicted by the CRDR and XDR theories.$
$\citet{Papadopoulos2010c} and \citet{Papadopoulos2011} pointed out the key role of CRDR in local (U)LIRGs. With UV photons alone, cold gas in starbursting environments can still be well shielded, hence as cold as \sim 10~K. However, with CRs produced from O, B star clusters and supernova remnants, the gas can be uniformly illuminated and heated up.$
$The extremely starbursty local ULIRGs have a CR density (\xi_{\mathrm{CR}}) 10^{3}--10^{4} times that of the Milky Way (\citealt{Papadopoulos2010c}), and thus will naturally dissociate more CO into \CI .$
$\citet{Bisbas2017} and \citet{Papadopoulos2018} conducted simulations of molecular gas at tens of pc scale with varying CR strength, and found that when \xi_{\mathrm{CR}} is \sim 1 \textnormal{--} 100   \times Galactic value, the gas temperature in the dense gas remains largely unaffected (\sim 10~K).$
$However, when \xi_{\mathrm{CR}} reaches \sim 1000   \times Galactic value as in (U)LIRGs, the gas temperature increases to 30--50~K. The \citet{Bisbas2017} simulation also shows that the \CI  and \ionizedcarbon  fractional abundances at \log A_V \sim 0.5 increase by one and three orders of magnitude, respectively, and the CO fractional abundance decreases correspondingly by over two orders of magnitude, with an increasing \xi_{\mathrm{CR}} from the Galactic value to 100   \times of that.$
$These results are in line with our calculations in Sect.~\ref{subsec: LTE}.$
$Meanwhile, the AGN-driven XDR chemistry and molecule/atom excitation have been firstly modeled in great detail by \citet{Maloney1996}, \citet{Meijerink2005} and \citet{Meijerink2007}. In XDRs, photo-ionization becomes the dominant heating source instead of photo-electric emission from dust grains in PDRs, thus more molecules are dissociated and/or ionized. \citet{Meijerink2005}'s Model~1 shows that CO molecules are fully dissociated when N_{\mathrm{H}} < 10^{23}   \mathrm{cm^{-2}}, then the fractional abundance is gradually increased by six orders of magnitudes from N_{\mathrm{H}} = 10^{23}   \mathrm{cm^{-2}} to 10^{24}   \mathrm{cm^{-2}}. By contrast, the \CI  and \ionizedcarbon  abundances are about constant until reaching the densest part (N_{\mathrm{H}} > 10^{24.5}   \mathrm{cm^{-2}}). Unlike in PDRs, there is no clear \ionizedcarbon /\CI /CO stratification in XDRs (\citealt{Meijerink2007,Wolfire2022}). Their Model~2 with a 100 \times higher energy deposition rate shows that the CO distribution contracts to the highest gas density spots, and the total cumulative [\CI/CO] is increased by about 0.2~dex, reaching \RNCINCO \sim 1.6.$
$This value agrees well with our calculations shown in Figs.~\ref{fig: LTE RCICO} and \ref{fig: NonLTE RCICO} (the ISM condition 7).$
$Therefore, both in theoretical and observational views, it is clear that the \RCICO line ratio is a good indicator of the starburst's CRDR and AGN's XDR.$
$If a \RCICO line ratio (to be specific, \Remark{ICI10}/\Remark{ICO21}) reaches 0.2--0.5, then the CRDR plays a crucial role and the abundance ratio [\CI/CO] could be \sim 1.$
$If it exceeds 0.5--1.0, then it may only be enhanced by an XDR, e.g., caused by an X-ray luminous AGN, which boosts the abundance ratio [\CI/CO] to \gtrsim 1--2.$
$\vspace{2ex}$
$\section{Conclusion}$
$\label{sec: Conclusion}$
$We presented new beam- and uv-coverage-matched \Remark{CI10} and CO(2--1) maps in the galactocentric radius r \lesssim 7~kpc SF disk of NGC~3627 at \sim 190~pc resolution, and in the r \lesssim 3~kpc disk of NGC~4321 at \sim 270~pc resolution.$
$They are among the largest \CI  mosaic mappings available in nearby galaxies.$
$We combined the imaging of \Remark{CI10} and CO(2--1) in two nearby, more starbursty and AGN-hosting galaxies, i.e., NGC~1808 central \sim1~kpc, and NGC~7469 central \sim1.5~kpc at 140--160~pc resolutions.$
$Together, we studied the spatial distributions of \CI  and CO, and their line ratio \RCICO as functions of various galaxy resolved properties, including CO brightness (Fig.~\ref{fig: line ratio vs CO}), ISRF \Umean (Fig.~\ref{fig: line ratio vs ISRF}), and metallicity (Fig.~\ref{fig: line ratio vs metallicity}).$
$In order to obtain the underlying intrinsic [\CI/CO] abundance ratio from the observed line ratio \RCICO, we assumed 7 representative ISM conditions and did both LTE and non-LTE calculations.$
$\vspace{1ex}$
$We summarize our findings as below:$
$\begin{itemize}[itemsep=0ex,topsep=1ex,leftmargin=4ex]$
$    \item The majority of the molecular gas in the SF disks of NGC~3627 and NGC~4321 has a uniform \Remark{RCI10CO21} = 0.10 \pm 0.05.$
$    \Remark{CI10} and \Remark{CO21} exhibit very similar spatial distributions at our 140--270~pc resolution.$
$    \item The majority of the molecular gas in the SB disk of NGC~1808 has \Remark{RCI10CO21} = 0.20 \pm 0.05.$
$    This elevated line ratio compared to the SF disk is likely caused by a \sim 3 \text{--} 5 \times increased [\CI/CO] abundance ratio, based on our LTE and non-LTE calculations and parameter assumptions (Figs.~\ref{fig: LTE RCICO} and \ref{fig: NonLTE RCICO}).$
$    \item The centres (r \lesssim 250~pc) of NGC~4321, NGC~3627, NGC~1808 and NGC~7469 show a strongly increasing trend in \Remark{RCI10CO21}, from 0.05 to 0.5, as a function of I_{\Remark{CO21}} --- i.e., \Remark{RCI10CO21} \approx 0.035   (I_{\Remark{CO21}}/[100\;\Kkms])^{0.8}.$
$    The inferred [\CI/CO] abundance ratio from our non-LTE calculation changes from \sim 0.2 to \sim 3, in line with the increasing X-ray luminosity of the centres/AGNs.$
$    \item The observed \RCICO versus ISRF trend is fairly flat, i.e., \RCICO  is insensitive to ISRF intensity changes, similar to the PDR model prediction calculated with PDRToolbox (although our resolution is much coarser than the physical scale of PDR models; Sect.~\ref{subsection: ISRF}).$
$    \item We find a \RCICO versus metallicity trend consistent with previous observations that covered lower-metallicity environments (\sim 0.2   \mathrm{Z_{\odot}}; \citealt{Bolatto2000b}), within the metallicity range (\sim 0.7 \text{--} 1   \mathrm{Z_{\odot}}) of our sample (Sect.~\ref{subsec: metallicity}).$
$    \item Our inferred [\CI/CO] abundance ratios agree well with the CRDR and XDR's theoretical studies.$
$    Given the drastically changed \RCICO , we propose that the \CI  to CO line ratio can be a promising indicator of CRDR (starbursting) and XDR (AGN). For instance, an observed line flux ratio of$
$    \Remark{RCI10CO21} \sim 0.2 \textnormal{--} 0.5 and \gtrsim 0.5 effectively indicates CRDR and XDR, respectively.$
$    \item We do not find ubiquitous "CO-dark", \CI -bright gas at the outer r\sim 1 \text{--} 3~kpc disks of NGC~3627 and NGC4321, nor systematic enhancement in \RCICO  at CO-faintest sightlines via stacking, mainly due to the incompleteness of our data.$
$    Nevertheless, a few sightlines, e.g., Region~22 in NGC~3627 show high \RCICO ~\sim 0.5 which do not fully comply with the noise behavior seen in our CASA simulations.$
$\end{itemize}$
$\vspace{2ex}$
$\begin{acknowledgements}$
$We thank the anonymous referee for very helpful comments.$
$ES, TS and TGW acknowledge funding from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme (grant agreement No. 694343).$
$ER acknowledges the support of the Natural Sciences and Engineering Research Council of Canada (NSERC), funding reference number RGPIN-2017-03987.$
$The work of AKL is partially supported by the National Science Foundation under Grants No. 1615105, 1615109, and 1653300.$
$AU acknowledges support from the Spanish grants PGC2018-094671-B-I00, funded by MCIN/AEI/10.13039/501100011033 and by "ERDF A way of making Europe", and PID2019-108765GB-I00, funded by MCIN/AEI/10.13039/501100011033.$
$RSK and SCOG acknowledge financial support from the German Research Foundation (DFG) via the collaborative research centre (SFB 881, Project-ID 138713538) “The Milky Way System” (subprojects A1, B1, B2, and B8). They also acknowledge funding from the Heidelberg Cluster of Excellence "STRUCTURES" in the framework of Germany’s Excellence Strategy (grant EXC-2181/1, Project-ID 390900948) and from the European Research Council via the ERC Synergy Grant "ECOGAL" (grant 855130).$
$FB and IB acknowledge funding from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme (grant agreement No.726384/Empire).$
$JC acknowledges funding from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme DustOrigin (ERC-2019-StG-851622).$
$MC gratefully acknowledges funding from the Deutsche Forschungsgemeinschaft (DFG) in the form of an Emmy Noether Research Group (grant number CH2137/1-1).$
$Y.G.’s work is partially supported by National Key Basic Research and Development Program of China (grant No. 2017YFA0402704), National Natural Science Foundation of China (NSFC, Nos.$
$12033004, and 11861131007), and Chinese Academy of Sciences Key Research$
$Program of Frontier Sciences (grant No. QYZDJ-SSW-SLH008).$
$AH was supported by the Programme National Cosmology et Galaxies (PNCG) of CNRS/INSU with INP and IN2P3, co-funded by CEA and CNES, and by the Programme National “Physique et Chimie du Milieu Interstellaire” (PCMI) of CNRS/INSU with INC/INP co-funded by CEA and CNES.$
$K.K. gratefully acknowledges funding from the German Research Foundation (DFG) in the form of an Emmy Noether Research Group (grant number KR4598/2-1, PI Kreckel)$
$JMDK and MC gratefully acknowledge funding from the Deutsche Forschungsgemeinschaft (DFG) in the form of an Emmy Noether Research Group (grant number KR4801/1-1) and the DFG Sachbeihilfe (grant number KR4801/2-1), and from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme via the ERC Starting Grant MUSTANG (grant agreement number 714907).$
$JP acknowledges support from the Programme National “Physique et Chimie du Milieu Interstellaire” (PCMI) of CNRS/INSU with INC/INP co-funded by CEA and CNES.$
$HAP acknowledges support by the Ministry of Science and Technology of Taiwan under grant 110-2112-M-032-020-MY3.$
$The work of JS is partially supported by the Natural Sciences and Engineering Research Council of Canada (NSERC) through the Canadian Institute for Theoretical Astrophysics (CITA) National Fellowship.$
$This paper makes use of the following ALMA data:$
$\incode{ADS/JAO.ALMA#2015.1.00902.S},$
$\incode{ADS/JAO.ALMA#2015.1.00956.S},$
$\incode{ADS/JAO.ALMA#2015.1.01191.S},$
$\incode{ADS/JAO.ALMA#2017.1.00078.S},$
$\incode{ADS/JAO.ALMA#2017.1.00984.S},$
$\incode{ADS/JAO.ALMA#2018.1.00994.S},$
$\incode{ADS/JAO.ALMA#2018.1.01290.S},$
$\incode{ADS/JAO.ALMA#2019.1.01635.S}.$
$ALMA is a partnership of ESO (representing its member states), NSF (USA) and NINS (Japan), together with NRC (Canada), MOST and ASIAA (Taiwan), and KASI (Republic of Korea), in cooperation with the Republic of Chile. The Joint ALMA Observatory is operated by ESO, AUI/NRAO and NAOJ.$
$\end{acknowledgements}$
$\bibliographystyle{aa_url}$
$\documentclass[twocolumn,longauth]{aa}$
$\usepackage{graphicx}$
$\usepackage{txfonts}$
$\input{Header}$
$\begin{document}$
$\title{C~\textsc{i} and CO in Nearby Spiral Galaxies - I. Line Ratio and Abundance Variations at \sim 200~pc Scales}$
$\titlerunning{C~\textsc{i} \& CO - I. Line Ratio and Abundance Ratio}$
$\authorrunning{D. Liu et al.}$
$\author{$
$    Daizhong Liu \inst{\ref{MPE}}$
$\and$
$    Eva Schinnerer \inst{\ref{MPIA}}$
$\and$
$    Toshiki Saito \inst{\ref{NAOJ}}$
$\and$
$    Erik Rosolowsky \inst{\ref{UAlberta}}$
$\and$
$    Adam Leroy \inst{\ref{OSU}}$
$\and$
$    Antonio Usero \inst{\ref{Madrid}}$
$\and$
$    Karin Sandstrom \inst{\ref{UCSD}}$
$\and$
$    Ralf S. Klessen \inst{\ref{HeidelbergZfA},\ref{HeidelbergZfW}}$
$\and$
$    Simon C. O. Glover \inst{\ref{HeidelbergZfA}}$
$\and$
$    Yiping Ao \inst{\ref{PMO},\ref{USTC}}$
$\and$
$    Ivana Be\v{s}li\'c \inst{\ref{BonnAIfA}}$
$\and$
$    Frank Bigiel \inst{\ref{BonnAIfA}}$
$\and$
$    Yixian Cao \inst{\ref{MPE}}$
$\and$
$    Jérémy Chastenet \inst{\ref{UCSD},\ref{Gent}}$
$\and$
$    Mélanie Chevance \inst{\ref{HeidelbergARI},\ref{HeidelbergZfA}}$
$\and$
$    Daniel A. Dale \inst{\ref{Wyoming}}$
$\and$
$    Yu Gao \inst{\ref{XiamenUniv}}\thanks{Deceased.}$
$\and$
$    Annie Hughes \inst{\ref{Toulouse}}$
$\and$
$    Kathryn Kreckel \inst{\ref{HeidelbergARI}}$
$\and$
$    J.~M.~Diederik Kruijssen \inst{\ref{HeidelbergARI}}$
$\and$
$    Hsi-An Pan \inst{\ref{TamkangUniv}}$
$\and$
$    J\'er\^ome Pety \inst{\ref{IRAM},\ref{SorbonneUniv}}$
$\and$
$    Dragan Salak \inst{\ref{HokkaidoUniv},\ref{HokkaidoUnivGrad}}$
$\and$
$    Francesco Santoro \inst{\ref{MPIA}}$
$\and$
$    Andreas Schruba \inst{\ref{MPE}}$
$\and$
$    Jiayi Sun \inst{\ref{McMasterUniv},\ref{TorontoUniv}}$
$\and$
$    Yu-Hsuan Teng \inst{\ref{UCSD}}$
$\and$
$    Thomas Williams \inst{\ref{MPIA}}$
$}$
$\institute{$
$    Max-Planck-Institut f\"ur extraterrestrische Physik (MPE), Giessenbachstrasse 1, D-85748 Garching, Germany\\email{dzliu@mpe.mpg.de, astro.dzliu@gmail.com}$
$    \label{MPE}$
$\and$
$    Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany$
$    \label{MPIA}$
$\and$
$National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo, 181-8588, Japan$
$    \label{NAOJ}$
$\and$
$Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada$
$    \label{UAlberta}$
$\and$
$Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA$
$    \label{OSU}$
$\and$
$    Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain$
$    \label{Madrid}$
$\and$
$    Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA$
$    \label{UCSD}$
$\and$
$    Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany$
$    \label{HeidelbergZfA}$
$\and$
$    Universit\"at Heidelberg, Interdisziplin\"ares Zentrum f\"ur Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany$
$    \label{HeidelbergZfW}$
$\and$
$     Purple Mountain Observatory and Key Laboratory for Radio Astronomy, Chinese Academy of Sciences, Nanjing, China$
$     \label{PMO}$
$\and$
$    School of Astronomy and Space Science, University of Science and Technology of China, Hefei, China$
$    \label{USTC}$
$\and$
$    Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany$
$    \label{BonnAIfA}$
$\and$
$    Sterrenkundig Observatorium, Ghent University, Krijgslaan 281-S9, 9000 Gent, Belgium$
$    \label{Gent}$
$\and$
$    Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany$
$    \label{HeidelbergARI}$
$\and$
$    Department of Physics \& Astronomy, University of Wyoming, Laramie, WY 82071, USA$
$    \label{Wyoming}$
$\and$
$    Department of Astronomy, Xiamen University, Xiamen, Fujian 361005, China$
$    \label{XiamenUniv}$
$\and$
$    CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France; Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France$
$    \label{Toulouse}$
$\and$
$    Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan$
$    \label{TamkangUniv}$
$\and$
$    Institut de Radioastronomie Millimétrique (IRAM), 300 rue de la Piscine, F-38406 Saint-Martin-d'Hères, France$
$    \label{IRAM}$
$\and$
$    Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France$
$    \label{SorbonneUniv}$
$\and$
$    Institute for the Advancement of Higher Education, Hokkaido University, Kita 17 Nishi 8, Kita-ku, Sapporo, Hokkaido 060-0817, Japan$
$    \label{HokkaidoUniv}$
$\and$
$    Department of Cosmosciences, Graduate School of Science, Hokkaido University, Kita 10 Nishi 8, Kita-ku, Sapporo, Hokkaido 060-0817, Japan$
$    \label{HokkaidoUnivGrad}$
$\and$
$    Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada$
$    \label{McMasterUniv}$
$\and$
$    Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada$
$    \label{TorontoUniv}$
$}$
$\date{Accepted: December 13, 2022}$
$\abstract{$
$We present new neutral atomic carbon [C~\textsc{i}]~( ^3P_1~\to~ ^3P_0) mapping observations within the inner \sim 7~kpc and \sim 4~kpc of the disks of NGC~3627 and NGC~4321 at a spatial resolution of 190~pc and 270~pc, respectively, using the Atacama Large Millimeter/Submillimeter Array (ALMA) Atacama Compact Array (ACA).$
$We combine these with the CO(2--1) data from PHANGS-ALMA, and literature [C~\textsc{i}] and CO data for two other starburst and/or active galactic nucleus (AGN) galaxies (NGC~1808, NGC~7469), to study: \textit{a)} the spatial distributions of C~\textsc{i} and CO emission; \textit{b)} the observed line ratio \RCICO = I_{\mathrm{[C{\normalfont\textsc{i}}]}(1\textnormal{--}0)} / I_{\mathrm{CO}(2\textnormal{--}1)} as a function of various galactic properties; and \textit{c)} the abundance ratio of [C~\textsc{i}/CO].$
$We find excellent spatial correspondence between C~\textsc{i} and CO emission and nearly uniform \RCICO \sim 0.1 across the majority of the star-forming disks of NGC~3627 and NGC~4321.$
$However, \RCICO strongly varies from \sim 0.05 at the centre of NGC~4321 to > 0.2 \text{--} 0.5 in NGC~1808's starbursting centre and NGC~7469's centre with an X-ray-luminous active galactic nucleus (AGN).$
$Meanwhile, \RCICOdoes not obviously vary with \Umean, similar to the prediction of photodissociation dominated region (PDR) models.$
$We also find a mildly decreasing \RCICOwith an increasing metallicity over 0.7\text{--}0.85 \mathrm{Z_{\odot}}, consistent with the literature.$
$Assuming various typical ISM conditions representing giant molecular clouds, active star-forming regions and strong starbursting environments, we calculate the local-thermodynamic-equilibrium radiative transfer and estimate the [C~\textsc{i}/CO] abundance ratio to be \sim 0.1 across the disks of NGC~3627 and NGC~4321, similar to previous large-scale findings in Galactic studies.$
$However, this abundance ratio likely has a substantial increase to \sim 1 and \gtrsim 1--5 in NGC~1808's starburst and NGC~7469's strong AGN environments, respectively, in line with the expectations for cosmic-ray dominated region (CRDR) and X-ray dominated region (XDR) chemistry.$
$Finally, we do not find a robust evidence for a generally CO-dark, C~\textsc{i}-bright gas in the disk areas we probed.$
$}$
$\keywords{galaxies: ISM --- ISM: molecules --- ISM: atoms --- ISM: abundances --- galaxies: spiral --- galaxies: individual: NGC 3627, NGC 4321, NGC 1808, NGC 7469}$
$\maketitle$
$\section{Introduction}$
$\label{subsec: intro RCICO}$
$\subsection{Neutral atomic carbon in the interstellar medium}$
$Neutral atomic carbon (C^0 or \neutralcarbon) is an important phase of carbon in the interstellar medium (ISM) besides ionized carbon (C^+ or \ionizedcarbon) and carbon monoxide (CO).$
$Its ^{3}P state is split into three fine-structure levels. The^{3}P_{1} \to  ^{3}P_{0} transition line at 492.16065~GHz,$
$hereafter \Remark{CI10}$
$, has an upper energy level of E_u = 23.620~K and a critical density of n_{\mathrm{crit}} \sim 1.0 \times 10^{3} \;\mathrm{cm^{-3}} (at a temperature of 50~K) \footnote{n_{\mathrm{crit}} \equiv A / \gamma, where A is the Einstein A coefficient for the transition and \gamma the rate of collisional de-excitation with hydrogen molecules, with values from \cite{Schroder1991} as compiled in the Leiden Atomic and Molecular Database (\citealt{Schoier2005}), \url{https://home.strw.leidenuniv.nl/~moldata/}},$
$and the^{3}P_{2} \to  ^{3}P_{1} transition line at 809.34197~GHz has E_u = 62.462~K and n_{\mathrm{crit}} \sim 3.4 \times 10^{3} \;\mathrm{cm^{-3}}.$
$Therefore, they are easily excited in the cold ISM environment.$
$Given the high abundance of carbon in the ISM, \neutralcarbon, CO and \ionizedcarbon  are the most powerful cold ISM tracers, and are the most widely used tools to probe cold ISM properties and hence galaxy evolution at cosmological distances.$
$The interstellar \neutralcarbon  is produced from$
$the photodissociation of CO molecules by UV photons and the recombination of \ionizedcarbon , both happening in photodissociation regions (PDRs; \citealt{Langer1976,deJong1980,Tielens1985a,Tielens1985b,vanDishoeck1986,vanDishoeck1988,Sternberg1989,Sternberg1995,Hollenbach1991,Hollenbach1999,Kaufman1999,Wolfire2010,Madden2020,Bisbas2021}, also see reviews by \citealt{Genzel1989,Jaffe1985,Hollenbach1997,Wolfire2022}).$
$In the simple plane-parallel (i.e., 1-D) PDR model, \neutralcarbon  exists in a layer where UV photons can penetrate through the molecular gas but cannot maintain a high carbon ionizing rate.$
$Such a layer is suggested to have an intermediate gas surface density (N_{\mathrm{H_2}} \sim 1 \textnormal{--} 4 \times 10^{20} \;\mathrm{cm^{-2}}), fairly cold temperature (10 to a few tens K), and moderate visual extinction (A_V \sim 1 - 4).$
$CO molecules dominate the more shielded interior of this layer and \ionizedcarbon  dominates the exterior.$
$Hydrogen molecules are gradually photodissociated to atoms across this layer (A_V \sim 0.1 - 4).$
$This plane-parallel scenario qualitatively explains the layered structure of the Orion Bar PDR in our Galaxy on sub-pc scales (e.g.,  \citealt{Genzel1989,Tielens1993,Tauber1994,Hogerheijde1995b,Hollenbach1999,Goicoechea2016}), and the \rho~Ophiuchi~A PDR in a recent study with an unprecedented resolution of 360~AU (\citealt{Yamagishi2021}), where the interplay between individual young, massive (O, B) stars and the ISM can be directly observed.$
$However, such simple models cannot fully explain molecular clouds. Plenty of observations in Galactic molecular clouds, \ionizedhydrogen  regions and star-formation complexes have revealed a widespread \neutralcarbon  distribution over most areas of molecular clouds on scales of a few to ten pc, and a relatively high \neutralcarbon  fractional abundance even at a large A_V depth > 10~mag (\citealt{Phillips1980,Phillips1981,Wootten1982,Keene1985,Keene1987,Jaffe1985,Zmuidzinas1986,Zmuidzinas1988,Genzel1988,Frerking1989,Plume1994,Plume1999,Gerin1998a,Tatematsu1999,Ikeda1999,Ikeda2002,Yamamoto2001,Zhang2001,Kamegai2003,Oka2004,Izumi2021}).$
$This observational evidence actually requires PDRs to be clumpy  (\citealt{Stutzki1988,Genzel1988,Burton1990,Meixner1993,Spaans1997,Kramer2004,Kramer2008,Pineda2008,Sun2008}), with a volume filling factor much less than unity (e.g., \sim 0.1 - 0.3; \citealt{Stutzki1988}), so that$
$UV photons can penetrate through most of the cloud, except for the densest clumps.$
$Therefore, the spatial co-existence of \neutralcarbon  and low rotational transition (low-J) CO lines can be well explained by such an inter-clump medium inside the molecular clouds.$
$Alternatively, cosmic rays (CRs) have also been proposed to be the reason for the \neutralcarbon -CO association as they can penetrate much deeper than UV photons into clouds, and thus more uniformly dissociate CO molecules into \neutralcarbon  inside clouds (\citealt{Field1969,Padovani2009,Papadopoulos2010c,Ivlev2015,Papadopoulos2018}).$
$\subsection{Variations of the \CI/\CO line ratio (\RCICO) in the literature}$
$Although the spatial distributions of \CI  and low-J CO emission resemble each other surprisingly well from molecular clouds to Galactic scales (e.g., across the Galactic plane; \citealt{Wright1991,Bennett1994,Oka2005,Burton2015}) and in external galaxies with spatially resolved observations (e.g., \citealt{Schilke1993,White1994b,Harrison1995,Israel1995,Israel2001,Israel2003,Zhang2014,Krips2016,Cicone2018,Crocker2019,Jiao2019,Miyamoto2018,Miyamoto2021,Salak2019,Izumi2020,Saito2020,Michiyama2020,Michiyama2021}),$
$their line intensity ratio, \Remark{RCI10CO10} = \Remark{ICI10} / \Remark{ICO10} \footnote{The line intensity I has brightness temperature units \mathrm{K km s^{-1}}.}, does vary according to local ISM conditions --- \CI  and CO fractional abundance (i.e., metallicity), excitation condition (e.g., excitation temperature T_{\mathrm{ex}} and optical depth \tau), and UV and/or CR radiation field.$
$At individual molecular cloud scales, \Remark{RCI10CO10} (or column density ratio) seems to vary from \gtrsim 0.5 (or \gtrsim 0.2 for column density ratio) to <0.1 with an increasing N_{\mathrm{H2}} and A_V, e.g., from cloud surface or PDR front to the interior \citep[e.g.][]{Frerking1982,Genzel1988,Oka2004,Kramer2004,Kramer2008,Mookerjea2006,Sun2008,Yamagishi2021}.$
$At sub-galactic scales, when individual clouds are smoothed out, \Remark{RCI10CO10} is found to lie in a narrower range of \sim 0.1 \text{--} 0.3. For example, across the Galactic plane, the ratio is found to change only slightly within a galactocentric radius of 3--7~kpc (\sim 0.08 \text{--} 0.12, overall mean = 0.105 \pm 0.004; \citealt{Oka2005});$
$whereas in the starburst galaxies like M~82 and NGC~253, \Remark{RCI10CO10} or the [\CI]/CO abundance ratio is a factor of 3--5 higher than the Galactic value (\citealt{Schilke1993,White1994b,Krips2016}).$
$Recent high-resolution (hundred pc scales) \Remark{CI10} mapping studies in the nearby spiral galaxy M~83 (\citealt{Miyamoto2021}) and starburst galaxy IRAS~F18293-3413 (\citealt{Saito2020}) reported tightly-correlated \Remark{CI10} and \Remark{CO10} emission, with scaling relations at hundred parsecs equivalent to$
$\Remark{RCI10CO10} = 0.14 \times (L^{\prime}_{\Remark{CO10}}/10^{6.0} [\Remark{Kkmspc2}])^{-0.13}$
$and$
$\Remark{RCI10CO10} = 0.21 \times (L^{\prime}_{\Remark{CO10}}/10^{6.8} [\Remark{Kkmspc2}])^{+0.54},$
$respectively.$
$The discrepant power-law indices reflects different trends in individual galaxies that are not fully understood.$
$Meanwhile, in the central \lesssim 100~pc of NGC~7469 close to the X-ray-luminous active galactic nucleus (AGN), \RCICO  is found to be substantially enhanced by a factor of >10 (reaching \Remark{RCI10CO10} \approx 1; \citealt{Izumi2020}; see also the Circinus Galaxy's AGN with \Remark{RCI10CO32} \approx 0.9 in \citealt{Izumi2018}).$
$Furthermore, in the low-metallicity Large Magellanic Cloud (LMC) and Small Magellanic Cloud (SMC) environments, a factor of 1.5 - 3 \times higher \RCICO  values have been found (\citealt{Bolatto2000b,Bolatto2000c}; see also other low-metallicity galaxies: \citealt{Bolatto2000a,Hunt2017}).$
$These studies indicate that \RCICO  is sensitive to ISM conditions among and within galaxies.$
$However, previous extragalactic studies are mainly focused on starburst galaxies and bright galaxy centres. How \RCICO  behaves across the disks and spiral arms of typical star-forming galaxies is much less explored. Whether \CI  can trace the so-called CO-dark molecular gas at a low gas density or metallicity is also still in question.$
$Understanding \RCICO  variation and quantifying the \CI  and \CO  excitation in extragalactic environments is now particularly important given the rapidly growing numbers and diverse galactic environments of \CI  line observations at cosmological distances (e.g., \citealt{Weiss2003,Weiss2005,Pety2004,Wagg2006,Walter2011,Danielson2011,AlaghbandZadeh2013,Gullberg2016,Bothwell2017,Popping2017,Yang2017,Emonts2018,Valentino2018,Valentino2020,Bourne2019,Nesvadba2019,Dannerbauer2019,Jin2019,Brisbin2019,Boogaard2020,Cortzen2020,Harrington2021,Dunne2021,Lee2021}).$
$\subsection{What drives \RCICO? --- the need for high-resolution mapping across wide galactic environments}$
$High-resolution mapping of \CI  and CO from disks and spiral arms of nearby galaxies is indispensable for understanding and quantifying the ISM \CI  and CO physics. In particular, the following questions will be answered only with such observations:$
$Do \CI  and low-J CO have the same spatial distributions across galaxy disks?$
$Does the \RCICO  line ratio vary significantly across galaxy disks?$
$How does the \RCICO  line ratio vary among different local galaxies?$
$How can the observed \RCICO  line ratio be reliably converted to the \CI  and CO column density ratio (i.e., solving their radiation transfer equations and level population statistics and obtaining optical depths and excitation temperature)?$
$Currently, high-resolution (hundreds of parsec scale) imaging/mapping of \CI  in nearby galaxies relies on radio telescopes at high altitudes, e.g., the Atacama Large Millimeter/submillimeter Array (ALMA), Atacama Submillimeter Telescope Experiment (ASTE, 10-m single-dish), Atacama Pathfinder Experiment (APEX, 12-m single-dish), and is limited to the available weather conditions corresponding to the high frequency of \CI  lines. Therefore, only a handful of galaxies have been imaged/mapped.$
$These include NGC~6240 by \citet{Cicone2018}, Circinus Galaxy by \citet{Izumi2018}, NGC~613 by \citet{Miyamoto2018}, NGC~1808 by \citet{Salak2019}, NGC~7469 by \citet{Izumi2020}, IRAS~F18293-3413 by \citet{Saito2020}, NGC~6052 by \citet[][\CI  undetected]{Michiyama2020}, 36 (U)LIRGs by \citet{Michiyama2021}, M~83 by \citet[][mapped the centre and northern part with ASTE]{Miyamoto2021}, Arp~220 by \citet{Ueda2022}, and NGC~1068 by \citet{Saito2022a,Saito2022b}.$
$Nevertheless, all of these observations, except for M~83, targeted the centres of starburst, IR-luminous and/or AGN host galaxies. M~83 is the only typical star-forming main sequence galaxy having a high-quality \CI  mapping. However, its map is still limited to a quarter of its inner \sim3~kpc disk.$
$In this work, we present two new \CI  mapping observations with the ALMA Atacama Compact Array (or Morita Array; 7-m dish, hereafter ACA or 7m) in nearby spiral galaxies NGC~3627 and NGC~4321.$
$Our observations aimed at detecting \CI  in the disks, and thus are much larger-area and deeper than previous \CI  maps.$
$By further adding a starburst and an AGN-host galaxy from the literature into our analysis, we present a comprehensive study of the \Remark{RCI10CO21} line ratio (hereafter \RCICO; ratios of other \CI /CO transitions will be explicitly written otherwise) in nearby star-forming disk and starburst and AGN environments.$
$In order to quantify the \CI  and CO abundance ratio,$
$we have further calculated the local thermodynamic equilibrium (LTE) and non-LTE radiative transfer of \CI  and CO under several \textit{representative ISM conditions}.$
$The results from this study could thus be one of the most comprehensive local benchmarks for understanding the \CI  and CO line ratio and abundance variations.$
$The paper, as the first one of our series of papers, is organized as follows.$
$Sect.~\ref{sec: Observations} describes the sample selection, observations and data reduction.$
$Sect.~\ref{sec: Results} presents the spatial distribution and variation of \RCICO.$
$Sect.~\ref{sec: Excitation Analysis} presents the analysis of \CI  and CO excitation and radiative transfer and thus the abundance (species column density) ratio.$
$In Sect.~\ref{sec: Discussion} we discuss various topics relating to the [\CI/CO] abundance variation, CRDR, XDR, and CO-dark gas.$
$Finally we conclude in Sect.~\ref{sec: Conclusion}.$
$Our companion paper~II (D. Liu et al. in prep.) presents the detailed radiative transfer calculation and \CI  and CO conversion factors, and is related to the abundance calculation in this paper.$
$\vspace{2ex}$
$\section{Targets and observations}$
$\label{sec: Observations}$
$\subsection{Target selection}$
$\label{subsection: Target Selection}$
$Our targets, NGC~3627 and NGC~4321, are selected from a joint sample of the PHANGS-ALMA CO(2--1) (\citealt{Leroy2021phangs}), PHANGS-HST (\citealt{PHANGSHST}), and PHANGS-MUSE (\citealt{PHANGSMUSE}) surveys, with CO line intensities and systemic velocities best suitable for ALMA Band 8 \CI  mapping.$
$The PHANGS-ALMA sample consists of 90 nearby star-forming galaxies initially selected at distances of 2 \lesssim d / \mathrm{Mpc} \lesssim 23, with a stellar mass \log (M_{\star}/\mathrm{M_{\odot}}) > 9.75, and which are not too inclined and visible to ALMA. They represent the typical star-forming main sequence galaxies in the local Universe.$
$The PHANGS-HST sample \citep{PHANGSHST} is a subsample of 38 PHANGS-ALMA galaxies, providing high-resolution stellar properties.$
$The PHANGS-MUSE large program \citep{PHANGSMUSE} targeted a subsample of 19 PHANGS-ALMA galaxies suitable for VLT/MUSE optical integral field unit (IFU) observations, achieving similar spatial coverage as in PHANGS-ALMA and providing rich nebular emission lines, attenuation, stellar age and metallicity and H~\textsc{ii} region information (e.g., \citealt{Kreckel2019, Kreckel2020, Pessa2021, Santoro2021, Williams2022}).$
$For the legacy value, we selected our sample from the joint ALMA+MUSE+HST sample pool, considered CO surface brightness, mapping area, \CI  line frequency and the transmission in the ALMA Band 8.$
$In this work, we present the new \CI  data of NGC~3627 and NGC~4321.$
$Because of the ALMA Band 8 sensitivity and the expected fainter \CI  line intensity than CO, the \CI  mapping areas are smaller than their CO(2--1) maps, but they are still the largest (\sim1'-2.35') and deepest (rms~\sim0.04~K per \sim5~km~s^{-1}) \CI  mapping with ALMA.$
$Furthermore, in order to study the \CI  and CO in different galactic environments, we selected two starbursting and/or AGN-host galaxies from the literature whose ALMA \Remark{CI10} and CO(2--1) data are available in the archive: NGC~1808 \citep{Salak2019} and NGC~7469 \citep{Izumi2020}.$
$NGC~1808 has ALMA mosaic observations covering its \sim 1~kpc starburst ring, and NGC~7469 has ALMA single-pointing observations covering its inner \sim 3~kpc area.$
$\subsection{New ALMA ACA Band 8 observations}$
$\label{subsection: ALMA Band 8 Observation}$
$Our \CI  observations for NGC~3627 were taken under the ALMA project 2018.1.01290.S (PI: D. Liu) during June 3rd to Sept. 29th, 2019 with ACA-only at Band~8. The total on-source integration time was 30.8~hours. A mosaic with 149 pointings was used to map an area of 94" \times 136" at a position angle of -27^\circ, which covers the galaxy centre, bar and the majority of the spiral arms, and is about 1/3 of the full PHANGS-ALMA CO(2-1) map area.$
$The achieved line sensitivity per 5 \kms is 45--55~mK (107--130~\mathrm{mJy/beam}) across the \Remark{CI10} line frequencies with a beam of 3.46".$
$The \CI  mapping for NGC~4321 were taken under the ALMA project 2019.1.01635.S (PI: D. Liu; NGC~1365 is also partially observed for this project) during Dec. 3rd, 2019 to July 1st, 2021, also with ACA-only at Band~8. The on-source integration time reached 16.1~hours for the mapping of an area of 55" \times 42" with 23 pointings using the 7m array. The achieved line sensitivity is similar to that of NGC~3627.$
$More detailed information is newd in Table~\ref{table: targets}.$
$ACA (7m) was chosen to new a matched and slightly coarser synthesized beam (\sim 3"-4") than the PHANGS-ALMA CO(2-1) data. In comparison, the main 12m ALMA array in compact C43-1/2 configurations will result in a \sim 0.4"-0.7" beam and significantly miss large scale (>4-6") emission without ACA.$
$No total power observations were requested in our presented observation, therefore we matched the uv ranges of the raw CO(2--1) and \Remark{CI10} visibility data during our data reduction.$
$We verified that missing flux will not substantially (\sim 30\%) bias our analysis using three methods as detailed in Appendix~\ref{appendix: missing flux}. This includes a CASA simulation of visibilities mimicking our ACA mosaic observations in NGC~3627, from which we find a missing flux as small as 7\% in \CI-bright pixels and up to \sim 50\% in the faintest pixels (and mostly < 30\%).$
$Our data reduction follows the PHANGS-ALMA imaging and post-processing pipeline \citep[][]{Leroy2021pipeline}, including imaging and deconvolution, mosaicking, broad and strict/signal mask generation, and moment map creation.$
$The additional steps in this study are: (a) uv-clipping in the CO(2--1) data to match the uv sampling range of \Remark{CI10}; and (b) making a joint mask of the individual CO(2--1) and \Remark{CI10} masks then extracting the moment maps (in the joint broad mask) and line ratios maps (in the joint strict mask).$
$More technical details are newd in Appendix~\ref{appendix: data reduction}.$
$Our \Remark{CI10} and CO(2--1) line intensity maps, the line ratio maps and \textit{HST} images for the two PHANGS galaxies, NGC~3627 and NGC~4321, are presented in Figs.~\ref{fig: NGC3627 line ratio map} and \ref{fig: NGC4321 line ratio map}, respectively.$
$Our data cubes and moment maps are also made publicly available at$
$the PHANGS-ALMA CADC public repository \footnote{\url{https://www.canfar.net/storage/list/phangs/RELEASES/DZLIU_etal_2022}}.$
$\begin{table*}[htb]$
$    \centering$
$    \caption{\label{table: targets}$
$        Physical and Data Properties of the Sample$
$    }$
$    \begin{tabularx}{\linewidth}{l c c c c}$
$        \setlength{\mymincolwidth}{\dimexpr 0.85\linewidth/5 \relax}$
$        \global\mymincolwidth=\mymincolwidth$
$        \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt}$
$            \\[-\arraystretch\normalbaselineskip]$
$        \hline$
$        \hline$
$         --- &$
$            NGC 3627 &$
$            NGC 4321 &$
$            NGC 1808 &$
$            NGC 7469 \\hline$
$         redshift &$
$            0.00243 &$
$            0.00524 &$
$            0.00332 &$
$            0.01627 \distance (Mpc) &$
$            11.32 &$
$            15.21 &$
$            10.8 &$
$            71.2\linear scale (pc/") &$
$            54.9 &$
$            73.7 &$
$            52.4 &$
$            345.2 \angular resolution &$
$            3.46" (190 pc) &$
$            3.65" (269 pc) &$
$            2.67" (140 pc) &$
$            0.46" (159 pc) \channel width (\mathrm{km s^{-1}}) &$
$            4.77 &$
$            4.76 &$
$            9.52 &$
$            4.76 \\textnormal{[}C \textsc{i}\textnormal{]}(1--0) flux scale (\mathrm{K Jy^{-1}}) &$
$            0.42 &$
$            0.38 &$
$            0.71 &$
$            23.8 \CO(2--1) flux scale (\mathrm{K Jy^{-1}}) &$
$            1.92 &$
$            3.70 &$
$            3.24 &$
$            108.4 \\textnormal{[}C \textsc{i}\textnormal{]}(1--0) channel rms (K) &$
$            0.044 &$
$            0.043 &$
$            0.056 &$
$            0.11 \CO(2--1) channel rms (K) &$
$            0.022 &$
$            0.015 &$
$            0.083 &$
$            0.14\imaging area &2.35' \times 1.60' &$
$            1.28' \times 1.00' &$
$            40" \times 40" &$
$            \textnormal{r}\sim9"$
$            \imaging type &$
$            7m, mosaic &$
$            7m, mosaic &$
$            12m+7m+tp, mosaic &$
$            12m+7m, single\M_{\star} (10^{10}   \mathrm{M_{\odot}}) &$
$            6.8 &$
$            5.6 &$
$            2.0 &$
$            10.7$
$            \L_{\mathrm{FIR}} (10^{10}   \mathrm{L_{\odot}}) &$
$            1.7 &$
$            1.9 &$
$            3.5 &$
$            25$
$            \L_{\mathrm{TIR}} (10^{10}   \mathrm{L_{\odot}}) &$
$            2.4 &$
$            2.5 &$
$            5.1 &$
$            39$
$            \L_{X} (10^{40}   \mathrm{erg s^{-1}}) &$
$            0.032 &$
$            0.12 &$
$            1.6 &$
$            1500$
$            \SFR (\mathrm{M_{\odot} yr^{-1}}) &$
$            3.84 &$
$            3.56 &$
$            7.9 (SB ring: 4.5) &$
$            76$
$            \M_{\mathrm{H_2}} (10^{9}   \mathrm{M_{\odot}}) &$
$            6.0 &$
$            7.8 &$
$            (SB ring: 0.52) &$
$            15$
$            \M_{\mathrm{H}{\textnormal{\textsc{i}}}} (10^{9}   \mathrm{M_{\odot}}) &$
$            1.1 &$
$            2.7 &$
$            1.7 &$
$            5.7$
$            \mean\RCICO  &$
$            0.11 &$
$            0.10 &$
$            0.16 &$
$            0.21$
$            \mean\log_{10} \RCICO&$
$            -0.96 &$
$            -1.02 &$
$            -0.83 &$
$            -0.69$
$            \\pm 2 \sigma range of \log_{10} \RCICO&$
$            -1.26, -0.66 &$
$            -1.24, -0.80 &$
$            -1.03, -0.63 &$
$            -0.95, -0.43$
$            \\hline$
$        \hline$
$    \end{tabularx}$
$    \tablefoot{$
$        Redshifts are taken from the NASA Extragalactic Database (NED).$
$        The distances of NGC~3627, NGC~4321, NGC~1808 and NGC~7469 are taken from \cite{2009AJ....138..332J}, \cite{2001ApJ...553...47F}, \cite{Salak2019} and \cite{Izumi2020},$
$        respectively.$
$        The stellar masses, SFRs, M_{\mathrm{H_2}} and M_{\mathrm{H}{\textnormal{\textsc{i}}}} of NGC~3627 and NGC~4321 are taken from the PHANGS sample table v1.6 of \citet[][PHANGS survey]{Leroy2021phangs}.$
$        The stellar masses and SFRs of NGC~1808 and NGC~7469 are taken from the z0MGS survey (\citealt{Leroy2019}; \url{https://irsa.ipac.caltech.edu/data/WISE/z0MGS/}).$
$        The M_{\mathrm{H_2}} and M_{\mathrm{H}{\textnormal{\textsc{i}}}} of NGC~7469 are adopted from \cite{Papadopoulos2000}.$
$        The global M_{\mathrm{H}{\textnormal{\textsc{i}}}} mass in NGC~1808 is measured by \citet{Koribalski1993}.$
$        L_{\mathrm{FIR}} and L_{\mathrm{TIR}} are IR 40--400\mum and 8--1000\mum luminosities all from \citet{Sanders2003}.$
$        The X-ray luminosities for NGC~3627 and NGC~4321 are both L_{0.3\textnormal{--}8\mathrm{keV}} from \citet{2011ApJ...731...60G}, and those for NGC~1808 and NGC~7469 are L_{2\textnormal{--}10\mathrm{keV}} from \citet{2005A&A...442..861J} and \cite{Izumi2020}, respectively.$
$    }$
$\end{table*}$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=\textwidth, trim=0 0 0 0]{Plot_two_line_ratio_four_panel_image_NGC3627_CI10_CO21_with_regions_with_HST}$
$\caption{\textit{Top panels}: NGC~3627 \Remark{CI10} (\textit{left}) and \Remark{CO21} (\textit{right}) integrated line intensity (moment-0) maps within our broad mask (representing high-completeness signals; see Appendix~\ref{appendix: data reduction}).$
$\textit{Bottom left panel}: \Remark{CI10}/\Remark{CO21} line ratio map within the combined signal mask.$
$\textit{Bottom right panel}: HST F814W-F555W-F438W RGB composite image (PHANGS-HST; \citealt{PHANGSHST}).$
$Regions marked with the white boxes and labeled with numbers are mainly for illustration purpose and for discussion in the text. Their zoom-in images and extracted spectra are newd in our online data release. Examples for Regions~1, 4 and 22 are presented in Appendix~\ref{appendix: all regions}.$
$All images have the same field of view of 179" \times 179", or 9.83 \mathrm{kpc} \times 9.83 \mathrm{kpc}.$
$}$
$\label{fig: NGC3627 line ratio map}$
$\end{figure*}$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=\textwidth, trim=0 0 0 0]{Plot_two_line_ratio_four_panel_image_NGC4321_CI10_CO21_with_regions_with_HST}$
$\caption{$
$Similar to Fig.~\ref{fig: NGC3627 line ratio map}, the \Remark{CI10} and CO(2--1) moment-0 intensity maps and the line ratio map and the HST composite RGB image for NGC~4321. The field of view is 78" \times 78", or 5.75 \mathrm{kpc} \times 5.75 \mathrm{kpc}.$
$}$
$\label{fig: NGC4321 line ratio map}$
$\end{figure*}$
$\subsection{ALMA observations of the additional targets from the literature}$
$\label{subsection: additional targets}$
$The NGC~1808 ALMA \Remark{CI10}, \Remark{CO10} and \Remark{CO21} observations are presented in \citet{Salak2019} and all are interferometry plus total power, i.e., 12m+7m+tp, thus having no missing flux issue.$
$To study the \CI /\CO  line ratio consistently with our main targets, we only used the \Remark{CI10} and \Remark{CO21} data.$
$We re-reduced the raw data under ALMA project 2017.1.00984.S (PI: D. Salak) with the observatory calibration pipeline, then imaged, primary-beam-corrected and short-spacing-corrected using the PHANGS-ALMA pipeline in the same way as for our main targets (Appendix~\ref{appendix: data reduction}).$
$Note that in this step we re-reduced and imaged the total power data with the PHANGS-ALMA singledish pipeline (\citealt{Herrera2020,Leroy2021pipeline}) and did the short-spacing correction with the PHANGS-ALMA pipeline right after the primary beam correction.$
$The properties of the final \CO -\CI  beam-matched data cubes are listed in Table~\ref{table: targets}.$
$The NGC~7469 ALMA data are presented in \citet{Izumi2020} and we used the archival raw data of \Remark{CI10} and \Remark{CO21} under ALMA project 2017.1.00078.S (PI: T. Izumi). These are single pointings toward the centre. \Remark{CI10} is observed with ACA 7m and \Remark{CO21} with the 12m array, both without total power. We re-reduced the data following the same procedure as for NGC~3627 and NGC~4321 with the PHANGS-ALMA pipeline.$
$Our re-reduced \Remark{CI10} and CO(2--1) line intensity and ratio maps for these additional galaxies are shown in Fig.~\ref{fig: other line ratio maps}.$
$\vspace{2ex}$
$\section{Results of the observed line ratios}$
$\label{sec: Results}$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.9\textwidth, trim=0 7.7mm 0 2mm, clip]{Plot_two_line_ratio_two_panel_image_NGC1808_CI10_CO21}$
$\includegraphics[width=0.9\textwidth, trim=0 0 0 2mm, clip]{Plot_two_line_ratio_two_panel_image_NGC7469_CI10_CO21}$
$\caption{$
$\Remark{CI10} (\textit{left}) and \RCICO  (\textit{right}) maps of NGC~1808 in the upper panels and NGC~7469 in the lower panels.$
$Similar to Figs.~\ref{fig: NGC3627 line ratio map} and \ref{fig: NGC4321 line ratio map}, the line intensity maps are newcommandined in the broad mask and the \RCICO  maps are restricted to the combined signal mask.$
$Boxes in the left panels indicate the manually selected central regions where the pixels are highlighted in Fig.~\ref{fig: line ratio vs CO}.$
$Contours in the right panels indicate the 5, 10, 20 and 50-\sigma levels of the left-panel \CI  intensity maps.$
$\label{fig: other line ratio maps}$
$}$
$\end{figure*}$
$\subsection{Spatial distributions}$
$In Figs.~\ref{fig: NGC3627 line ratio map}, \ref{fig: NGC4321 line ratio map} and \ref{fig: other line ratio maps}, we present the beam-matched \Remark{CI10} and \Remark{CO21} integrated line intensity maps and their ratio maps.$
$The \CI/\CO  line ratio map is computed only for pixels within the joint strict mask area of the two lines, as described in Appendix~\ref{appendix: data reduction}, representing high-confidence signals.$
$In NGC~3627 (Fig.~\ref{fig: NGC3627 line ratio map}), we visually marked 23 regions based on the \CI  and CO commonly-detected (\mathrm{S/N}>3), strict-mask areas for later analysis. Region IDs are sorted by their \Remark{CI10} brightness.$
$Region 1 corresponds to the galaxy centre, Regions 2 and 3 are the southern and northern bar ends, Region 4 is an offset peak next to the galaxy centre, and other regions are mostly on the spiral arms. Most of these regions show a similar \RCICO \sim 0.1, except for, e.g., Regions 20 and 22 which exhibit a somewhat higher ratio of \RCICO \sim 0.5. The edge of Region 4 also shows an enhancement of \CI  yet it is hard to tell whether this is physical or simply caused by edge effects, e.g., as seen in our simulations (Appendix~\ref{appendix: simulation}).$
$We also examined the integrated spectra of each region in Appendix~\ref{appendix: all regions}.$
$The \CI  and CO line profiles are not always well-matched in width and shape. In the galaxy centre (Region 1) and at bright bar-ends (Regions 2, 3, 5), \CI  and CO have similar line widths but \CI  has a lower intensity than CO, whereas in some fainter regions (9, 10, 13, 14), \CI  is narrower than CO and relatively weaker. In Regions 4, 20 and 22 where \RCICO is high, \CI  and CO have similar line widths but the \CI  intensity is enhanced.$
$In NGC~4321 (Fig.~\ref{fig: NGC4321 line ratio map}), we mapped a smaller area due to its fainter CO brightness than NGC~3627. There are no regions with \Remark{CO21} integrated intensities >100  \Kkms outside our selected area in NGC~4321, whereas in comparison the NGC~3627 bar ends have \Remark{CO21} integrated intensities \gtrsim 300  \Kkms. The \CI  mapping in NGC~4321 has a similar depth as in NGC~3627.$
$We marked 7 regions in NGC~4321 to guide the visual inspection. A clear \RCICO  newcommandicit can be seen in the galaxy centre (Region 1), with \RCICO \sim 0.06, nearly a factor of two smaller than its inner disk which has similar \RCICO \sim 0.1 as seen in the NGC~3627 disk. Some \CI  enhancement can be found at the edges or slightly outside these regions, where CO and \CI  are both faint (\Remark{ICO21} \sim 30-100  \Kkms) and the edge effect of interferometric missing flux might play a role (see Appendix~\ref{appendix: simulation}).$
$The \RCICO  spatial distributions of the other two starburst/AGN galaxies (Fig.~\ref{fig: other line ratio maps}) show in general more enhanced \RCICO  than in the NGC~3627 and NGC~4321 disks.$
$Large variations from the centres to outer areas can be seen, especially at the the S/N threshold of \sim 3 \text{--} 5 (see contours).$
$To quantify the spatial co-localization of the \CI  and CO emission, we use the \incode{JACoP} tool \citep{JACoP} \footnote{\url{https://imagej.net/plugins/jacop}}$
$in the \incode{ImageJ} software \citep{ImageJ} to measure the commonly used co-localization indicators, namely the Pearson's coefficient (PC; equal to 0 and 1 for null and 100\% localization, respectively) and Manders' coefficients (M1 and M2; which measure the overlap between two images and are equal to one for perfect overlap). We turn on the option of using Costes' automatic threshold \citep{Costes2004} which automatically set a pixel value threshold for each image to minimize the contribution of noise for the comparison \citep{JACoP}.$
$For the \CI  and CO emission in our galaxies, we find \mathrm{PC/M1/M2} values all very close to 1, that is, nearly 100\% co-localization (\mathrm{PC/M1/M2} = 0.934/0.985/0.990 for NGC~3627, 0.816/0.986/0.964 for NGC~4321, 0.978/0.890/0.906 for NGC~1808, and 0.935/0.999/0.998 for NGC~7469).$
$Overall, the spatial comparison of the \CI  and CO intensities demonstrates that the two species are well-correlated when observed at hundred parsec scales.$
$Meanwhile, we do see certain spectral differences in the line profiles as shown in Appendix~\ref{appendix: all regions}. This may suggest that variations exist at smaller scales than our resolution, which could be the sign of different evolutionary stages \citep[e.g.][]{Kruijssen2014} and would be interesting for higher-resolution follow-ups, e.g., with the main ALMA 12m array.$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=\textwidth]{Plot_Line_Ratio_CI10_CO21_New_surface_density}$
$    \caption{$
$        \textit{Left}: \Remark{ICI10} versus \Remark{ICO21} in the four galaxies we studied: NGC~3627, NGC~4321, NGC~1808 and NGC~7469. \Remark{ICI10} and \Remark{ICO21} integrated fluxes are moment-0 intensities extracted from their beam and pixel size matched data cubes. Only data points with propagated moment-0 \SNR \ge 5 are shown.$
$        The blue (with crossed hatch), green (with horizontal line hatch), gray (with dotted hatch) and light gray shadings indicate the 5 \sigma thresholds for the four galaxies, respectively. Typical line widths of 10~km~s^{-1} (NGC~3627 and NGC~4321) and 30~km~s^{-1} (NGC~1808 and NGC~7469) in sigma are used to calculate the thresholds together with the channel rms in Table~\ref{table: targets}.$
$        For NGC~3627 and NGC~4321, their (uv-matched) CO(2--1) 5 \sigma thresholds are out of the plotting range.$
$        Top axis shows the simply-converted H_2 surface density using a constant \alphaCO = 4.3 and CO excitation (i.e., CO(2--1) to CO(1--0) line ratio) R_{21} = 0.65.$
$        \textit{Right}: Same data but shown as the line ratio \RCICO = \Remark{ICI10}/\Remark{ICO21} versus \Remark{ICO21}.$
$        The blue horizontal line indicates a star-forming (SF) disk-like \RCICO \sim 0.1, i.e., the spiral arms of NGC~3627 and NGC~4321. The gray horizontal line indicates a starburst (SB) disk-like \RCICO \sim 0.2, i.e., the \sim 1~kpc SB disk/ring in NGC~1808 and NGC~7469.$
$        The red line with a slope of 0.8 in logarithm (0.035   (I_{\Remark{CO21}}/[100 \Kkms])^{0.8}) visually guides the trend of increasing \RCICO in the central regions of these galaxies (see boxes in Figs.~\ref{fig: NGC3627 line ratio map} to \ref{fig: other line ratio maps}, sizes \sim 300--500~pc).$
$    }$
$    \label{fig: line ratio vs CO}$
$\end{figure*}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=0.493\textwidth]{Plot_Histogram_of_Line_Ratio_CI10_CO21}$
$    \includegraphics[width=0.493\textwidth]{Plot_Percentiles_of_Line_Ratio_CI10_CO21}$
$    \caption{$
$    \textit{Left panel} shows the histograms of the sum of CO(2--1) surface brightness over pixels in each \CI/\CO  line ratio interval in the four galaxies. The bold curves are the cumulative distributions of the corresponding histograms.$
$    \textit{Right panel} shows the mean \CI/\CO  line ratio for pixels in each bin of CO(2--1) surface brightness (or molecular gas surface density as indicated by the top-axis assuming a constant \alphaCO = 4.3 and R_{21} = 0.65). Shaded area enveloping each mean trend indicates the 16- to 84-th percentiles in each bin, and dashed line indicates the 50-th percentile.$
$    This panel thus more clearly illustrates the ridge and scatter of the trends in the right panel of Fig.~\ref{fig: line ratio vs CO}.$
$    Detection limits are shown consistently with Fig.~\ref{fig: line ratio vs CO}.$
$    }$
$    \label{fig: line ratio histogram}$
$\end{figure*}$
$\subsection{\RCICO versus CO surface brightness and gas surface density}$
$In Fig.~\ref{fig: line ratio vs CO} we show the scatter points of \CI  and CO line brightnesses and the \RCICO  line ratios.$
$Systematic differences in \RCICO can be found among these galaxies and within their galactic environments. We discuss these environments separately as follows.$
$\subsubsection{Star-forming and starburst disk environments}$
$The majority of molecular gas in the 3--7~kpc star-forming disks of NGC~3627 and NGC~4321 exhibits a tight distribution of \RCICO around 0.1 (within about \pm50\%). The distribution is fairly flat over an order of magnitude in \Remark{ICO21}. We denote this as the "SF disk" regime.$
$The ratio rises to about 0.2 within a galactocentric radius of few hundred parsec to \sim 1~kpc starburst areas in NGC~1808 and NGC~7469, which we newcommandine as the "SB disk" regime.$
$The nearly flat "SB disk" distribution is consistent with earlier studies. \citet{Salak2019} found that in NGC~1808 \Remark{ICI10} is proportional to \Remark{ICO21}^{ +1.05}. They also studied CO(1--0), finding that \Remark{ICI10} \propto \Remark{ICO10}^{ +1.46}, i.e., with a steeper slope.$
$\citet{Saito2020} found a similar slope between \Remark{ICI10} and \Remark{ICO10}: \Remark{ICI10} \propto \Remark{ICO10}^{ +1.54} in IRAS~F18293-3413 (whose CO(2--1) data is insufficient for such a study as previously mentioned).$
$This correlation slope is likely to change with different CO transitions due to the CO excitation's dependence on the CO luminosity or star formation rate themselves (e.g., \citealt{Daddi2015,Liudz2021a}).$
$For higher-J CO transitions, \citet{Michiyama2021} studied \CI  and CO(4--3) lines in 36 (U)LIRGs, finding \Remark{ICI10} \propto \Remark{ICO43}^{ +0.97}.$
$The upward tails seen at the CO-faint-end in the right panel of Fig.~\ref{fig: line ratio vs CO} are mostly due to the detection limits in each galaxy. The color shading indicates the 5-\sigma limit for each galaxy, assuming a typical line width of 10~km~s^{-1}. Region~22 of NGC~3627 and the outer pixels in the two starburst/AGN galaxies make the tips of the blue and gray faint-end tails, respectively. But most of these features should still be spurious. We performed CASA simulations following the real observations of NGC~3627 to verify what we would get if \RCICO  were constant across the galaxies (Appendix~\ref{appendix: simulation})$
$We find that noise can indeed boost a simulated \RCICO  of 0.2 to a measurement as high as 0.5 in a NGC~3627-like large mosaic, or even higher to >1 in single-pointing ALMA data without total power or sufficient short spacing. However, the faint-end tail from the simulation is not very similar to that of NGC~3627, raising some questions about the nature of the high-\RCICO  in Region~22.$
$Furthermore, we also did stacking experiments using the CO line profile as the prior to measure the \CI  line fluxes in bins of CO line brightnesses or galactocentric radii. We find no systematically higher \RCICO  at fainter CO pixels or at the outer radii.$
$\subsubsection{Galaxy centre environments}$
$Most interestingly, the \lesssim 250~pc centres of all four galaxies show dramatically different \RCICO. It varies by a factor of ten from \sim 0.05 in the centre of NGC~4321  to \sim 0.1 in the centre of NGC~3627, then to \sim 0.15--0.2 at the centre of NGC~1808, and to \sim 0.2--0.5 in the nucleus of NGC~7469.$
$This "nuclei" trend has a slope of about 0.8, i.e., \Remark{ICI10} \approx 0.035 \; \Remark{ICO21}^{ +0.8}, and corresponds to a \Sigmol range of about 10^{3} to 10^{4} \; \Mspc2 if adopting an \alphaCO = 4.3  \mathrm{M_{\odot} \; (K   km   s^{-1}   pc^2)^{-1}} (\citealt{Bolatto2013}) and a CO excitation R_{21} = 0.65 (\citealt{Leroy2021c}).$
$The reasons for the nuclei showing this trend likely include both the effect of varying [\CI/CO] abundance ratio and the excitation and radiative transfer. They are determined by the actual ISM conditions in each environment, i.e., the gas kinetic temperature, \CI  and CO column densities (\CIcolumndensity and \COcolumndensity), line widths (\Delta{v}), and \mathrm{H_2} volume density (\nHtwo), etc.$
$They will be analyzed in Sect.~\ref{sec: Excitation Analysis}.$
$\subsubsection{Histogram and curve-of-growth of the line ratio distributions}$
$Fig.~\ref{fig: line ratio histogram} shows the histogram and cumulative distribution functions of \RCICO (left panel) and the mean \RCICO and the 16-th to 84-th percentiles in bins of CO line intensity (right panel) in our galaxies.$
$The histogram is computed as the sum of CO line intensity in each bin of \RCICO, representing the amount of gas at each \RCICO.$
$The NGC~4321 histogram is lower than others, showing that it has the least total molecular gas mass among the four galaxies within mapped areas. The NGC~3627 histogram has a peak about 0.5~dex higher than that of NGC~4321, but their peak locations are consistently at \log \RCICO \sim -1, same conclusion as our previous scatter plots show.$
$The two SB galaxies NGC~1808 and NGC~7469, despite having much smaller observed areas (\sim 1--2~kpc), have more than an order of magnitude higher peaks hence total molecular gas masses.$
$The locations of their histogram peaks also shift to a systematically higher \RCICO.$
$It is clearly seen that both SF and SB disks lack a high-\RCICO (\log \RCICO > -0.35) gas component which dominates the gas in the strong AGN host galaxy NGC~7469.$
$The curve-of-growth in the left panel of Fig.~\ref{fig: line ratio histogram} further shows the cumulative CO brightness in bins of \RCICO, and reflects the fact that the majority of gas in our galactic disks has a narrow distribution of \RCICO (see also last few rows in Table~\ref{table: targets}).$
$In NGC~3627, NGC~4321 and NGC~1808, the higher- and lower-\RCICO gas outside \pm 0.1~dex around the mean \RCICO only sums up to <5--10\% of their total gas masses.$
$In the right panel of Fig.~\ref{fig: line ratio histogram}, we illustrate how the mean \RCICO changes with different gas surface density, as well as the scatter of the data point distribution. The 2D pixels are binned by CO brightness, and the mean and 16\%/50\%/84\%-th percentiles of the \RCICO are computed for each bin.$
$It sketches out the mean trends in Fig.~\ref{fig: line ratio vs CO} (right panel), and also reveals that$
$a) the scatter of \RCICO is the largest (\pm 0.20~dex) in the starburst disk of NGC~7469 compared to other galaxies;$
$b) the scatter is the smallest (\pm 0.06~dex) in the star-forming disks of NGC~4321 and NGC~3627 where \Sigmol \sim 100 \text{--} 500 \; \Mspc2;$
$c) the scatters are very similarly (\pm 0.10~dex) at a relatively high \Sigmol \sim 1000 \; \Mspc2 in all our galaxies except for NGC~4321's centre.$
$\subsection{\RCICO versus ISRF}$
$\label{subsection: ISRF}$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.9\textwidth]{Plot_CI_CO_line_ratio_vs_IRSF_intensity_maps}$
$\caption{$
$    Line ratio versus mean ISRF intensity \Umean measured from SED fitting to the near- to far-IR (\textit{Spitzer}, \textit{Herschel}) data.$
$    \textit{Left four panels} are the \Umean maps of NGC~3627, NGC~4321, NGC~1808 and NGC~7469, and as described in Sect.~\ref{subsection: ISRF} are obtained from the works done by J. Chastenet et al. (in prep.) as a continued effort of \citet{Chastenet2021}.$
$    \textit{Right panel} shows the scatter plot of \RCICO versus \Umean at a spatial resolution of \sim 18" in the manually-selected apertures shown in the left panels.$
$    We computed a beam-weighted mean \RCICO and a centre pixel \RCICO for each aperture, displayed as the solid and open symbols, respectively.$
$    Error bars are the corresponding beam-weighted mean or central pixel \RCICO  uncertainties in each region.$
$    The region diameter (\sim 18") corresponds to the coarsest resolution of the \textit{Herschel} data used in the SED fitting.$
$    The gray dotted line shows \RCICO = 0.07   U^{0.25} as a guiding line.$
$    The colored dotted contours (labeled 0.1--0.5) are the model contours of \RCICO computed from the PDR Toolbox (using the latest \incode{wk2020} model and solar metallicity; \citealt{Kaufman2006}). We convert G_0 from the model contour to the x-axis U by multiplying with a factor of 0.88 (\citealt{Draine2007}).$
$    The right y-axis indicates the PDR model's \nHtwo, whose range is manually adjusted so that the model contours roughly match the left x-axis.$
$    The PDR model contours show little dependence of \RCICO on U in the horizontal direction but a strong correlation with \nHtwo as seen by the vertical gradient.$
$\label{fig: line ratio vs ISRF}$
$}\vspace{1ex}$
$\end{figure*}$
$Here we study how \RCICO  correlates with the interstellar radiation field (ISRF) intensity (U; in units of Milky Way mean ISRF; \citealt{Habing1968}), which is a measure of the UV radiation field strength impacting the ISM. Since \CI  originates from PDRs, its abundance should be highly related to the PDR properties, thus whether or how \RCICO  correlates with the ISRF that illuminates the PDR is a key question.$
$The ISRF intensity U is usually measured by the re-radiated dust emission, as the original UV photons are highly attenuated by dust in the ISM. \citet[][hereafter DL07]{DL07} developed a series of dust grain models and synthesized SED templates that can describe nearby galaxies' observed dust SEDs. A number of studies (e.g., \citealt{Draine2007,Galliano2011,Dale2012,Aniano2012,Aniano2020,Daddi2015,Chastenet2017,Liudz2021a,Chastenet2021}) have used the DL07 models to fit the dust SEDs of galaxies near and far.$
$For our study, we adopt the mass-averaged ISRF \Umean maps from an ongoing effort by J. Chastenet et al. (in prep.) which is a continuation of the work published for other galaxies by \citet{Chastenet2017,Chastenet2021} with the same method.$
$In this method, the \textit{Herschel} far-infrared, \textit{Spitzer} and/or \textit{WISE} near-infrared images \footnote{For the two (U)LIRGs, the WISE maps are saturated, so their inferred ISRF intensity U should be considered with a large uncertainty.} are first PSF-matched to a common resolution \sim 18" (\textit{Herschel} SPIRE 250\mum), then SED fitting is performed using the DL07 models at each pixel.$
$Each dust SED is fitted by two dust components, one in the ambient ISM with a constant ISRF intensity U_{\mathrm{min}}, and the other in regions with higher U with a power law distributed ISRF from U_{\mathrm{min}} up to U_{\mathrm{max}} = 10^{7}.$
$The fitting then gives the best-fit U_{\mathrm{min}} and the power-law index \gamma of the U distribution.$
$The mean ISRF intensity \Umean is calculated as the dust mass-weighted mean of the ambient and power-law ISRF intensities (in DL07, U_{\rm PDR} \ge 10^2 is considered to be associated with the PDR), and better represents the overall ISRF than U_{\mathrm{min}}.$
$Fig.~\ref{fig: line ratio vs ISRF} presents the \Umean maps and the \RCICO  versus \Umean scatter plot. Since the spatial resolution of the \Umean maps is 18", we calculate a weighted \RCICO in each circular aperture shown in Fig.~\ref{fig: line ratio vs ISRF} whose diameter equals the \Umean map resolution, with weighting following the 2D Gaussian beam.$
$In addition, we also show the central pixel value for each region in Fig.~\ref{fig: line ratio vs ISRF}.$
$We restrict the analysis to regions where both \Umean and \RCICO  are measured.$
$A weak correlation between \RCICO  and \Umean is seen from the SF to SB disks, with a large scatter. For reference, we plot a \RCICO = 0.07   U^{0.25} line to indicate this weak trend. The lowest-\Umean data point in NGC~3627 (mainly Region~22) and the centre of NGC~7469 do not follow this trend, possibly indicating other \CI  enhancement mechanisms than \Umean.$
$We use the popular PDR modelling software, PDR Toolbox (version 2.2.9; \citealt{Kaufman2006,Pound2008,pdrtoolbox}) \footnote{\url{https://dustem.astro.umd.edu/}; \url{https://github.com/mpound/pdrtpy}.}$
$to compute the theoretical \RCICO in uniform grids of ISRF intensity in the \citet{Habing1968} G_0 unit and H nucleus density n_{\mathrm{H}} (both in log space). We adopt the latest \incode{wk2020} model set and a solar metallicity (\mathrm{[O/H_2]} = 3.2 \times 10^{-4}, \mathrm{[C/H_2]} = 1.6 \times 10^{-4}).$
$The ISRF intensity grid can be directly scaled to U by multiplying a factor 0.88 (\citealt{Draine2007}), corresponding to the x-axis of Fig.~\ref{fig: line ratio vs ISRF}.$
$The n_{\mathrm{H}} grid is not directly linked to the left y-axis of Fig.~\ref{fig: line ratio vs ISRF}, but because the model contours show fairly flat \RCICO versus U trends, we can match the model contours and indicate the n_{\mathrm{H}} grid in the right y-axis of Fig.~\ref{fig: line ratio vs ISRF}.$
$The PDR modelling predicts generally flat \RCICO versus U trends similar to the distribution of the observed data points, although our U are diluted values at a much larger physical scale than in the PDR models.$
$However, in PDR models, \RCICO  changes moderately with n_{\mathrm{H}}, and a relatively high n_{\mathrm{H}} \sim 10^{3.5\text{--}3.7}   \mathrm{cm^{-3}} is needed to reproduce an \RCICO \sim 0.1--0.2, or equivalently \sim 1.0--1.9 for the ratio of \Remark{CI10} and \Remark{CO21} fluxes in \mathrm{erg s^{-1} cm^{-2} sr^{-1}} units used by the PDR Toolbox \footnote{See \url{https://github.com/mpound/pdrtpy-nb/blob/master/notebooks/PDRT_Example_Model_Plotting.ipynb}.$
$We scale the ratio of intensities in \mathrm{erg s^{-1} cm^{-2} sr^{-1}} to the ratio of intensities in \mathrm{K km s^{-1}} by a factor of (\nu_{\mathrm{CI10}}/\nu_{\mathrm{CO21}})^{-3} = 9.73, where \nu is the rest-frame frequency of the lines. We caution that the old version of \incode{PDRToolbox} \incode{v2.1.1} does not include the \incode{CI_609/CO_21} line ratio, but has \incode{CI_609/CO_43} and \incode{CO_43/CO_21} ratios. However, we tested that the multiplication of \incode{CI_609/CO_43} and \incode{CO_43/CO_21} does not produce the same result as the direct \incode{CI_609/CO_21} ratio in the new \incode{PDRToolbox} \incode{v2.2.9}. We use the new version for this work.$
$}.$
$Furthermore, the PDR model needs a \sim0.5~dex lower n_{\mathrm{H}} to explain the 2\times elevated \RCICO  in SB disks, which seems somewhat counterintuitive.$
$This apparent tension is likely caused by the observations probing a much coarser resolution that consists of a large number of PDRs with different n_{\mathrm{H}} and U (e.g., increasing U from 10 to 400 can increase \RCICO from 0.2 to 0.5 based on our PDR Toolbox modelling). It may also relate to an abundance variation driven by non-PDR mechanisms ---$
$X-ray dominated region (XDR; \citealt{Maloney1996,Meijerink2005,Meijerink2007}; see also review by \citealt{Wolfire2022}) and Cosmic Ray Dominated Region (CRDR; \citealt{Papadopoulos2010c,Papadopoulos2011,Papadopoulos2018,Bisbas2015,Bisbas2017}) are well-known to play an important role in starbursting and AGN environments, respectively.$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=0.9\textwidth]{Plot_CI_CO_line_ratio_vs_metallicity_maps}$
$\caption{$
$    \textit{Left two panels} show the metallicity maps of NGC~3627 and NGC~4321 from \citet{Williams2022} based on PHANGS-MUSE (\citealt{PHANGSMUSE}) data. Boxes are regions as newcommandined in Figs.~\ref{fig: NGC3627 line ratio map} and \ref{fig: NGC4321 line ratio map}.$
$    \textit{Right panel} shows the \RCICO  versus metallicity scatter plot for regions in NGC~3627 and NGC~4321. The bottom-axis is metallicity in solar units, top-axis is metallicity in 12+\log(\mathrm{O/H}), left-axis is the line ratio of \Remark{CI10} and \Remark{CO21} in \Kkms units, and right-axis is the ratio of fluxes in \mathrm{erg s^{-1} cm^{-2} sr^{-1}} units.$
$    The left- and right-axes are matched by a factor of (\nu_{\mathrm{C{\textsc{i}}1\textnormal{--}0}}/\nu_{\mathrm{CO1\textnormal{--}0}})^3 \times R_{21}, where we adopt a R_{21}=0.65 (\citealt{Leroy2021c}).$
$    Data points are the mean metallicity and \RCICO  in each region shown in left panels, with error bars indicating the typical uncertainty in \RCICO , and vertical color shading indicating the 16- to 84-th percentiles of pixels' \RCICO  distribution inside each region.$
$    The \citet{Bolatto2000b} empirical fitting and \citet{Bolatto1999} model prediction lines are overlaid.$
$    The three outliers in NGC~3627 with \RCICO \sim 0.19, 0.28 and 0.35 are Regions 20, 21 and 22, respectively. The outlier point in NGC~4321 with \RCICO \sim 0.06 is its centre Region 1.$
$    \label{fig: line ratio vs metallicity}$
$}\vspace{1ex}$
$\end{figure*}$
$As pointed out by \citet{Papadopoulos2011} and later works, ultra-luminous infrared galaxies (ULIRGs) with IR luminosities L_{\mathrm{IR, 8-1000\mu\mathrm{m}}} \ge 10^{12}   \Lsun and corresponding \mathrm{SFRs} > 1000   \Msyr have cosmic-ray (CR) energy densities \sim 10^{3} \text{--} 10^{4}\times larger than the Milky Way. In our sample, the two SB galaxies are not as intensively star-forming as ULIRGs, but their SFRs are still a factor of 2 to 20 higher than our SF galaxies. More importantly, the SB disks have considerably higher (e.g., one to two orders of magnitude) SFR surface densities than the SF disks, plausibly leading to a factor of a few tens to hundreds higher CR intensities. The CRs emitted from OB star clusters and supernova remnants can penetrate deeply into the ISM, and thus nearly uniformly illuminate the ISM and effectively dissociate CO into \CI .$
$A detailed simulation by \citet{Bisbas2017} showed that 10\times and 100\times higher CR ionization rates (\xi_{\mathrm{CR}}) than the Milky Way level results into a factor of about 2 and 3 mildly-increased \CI  abundance and about 1/2 and 1/30 strongly-decreased CO abundance, respectively (see their Fig.~3), in a 10~pc GMC with a mean gas density \left<n_{\mathrm{H}} \right>= 760   \mathrm{cm^{-3}}, resembling Milky Way and typical SF galaxies' ISM conditions.$
$Our SB galaxies probably have a \xi_{\mathrm{CR}} around or slightly above 10, but not reaching 100 given their mild \Umean \sim 20-30 compared to the \Umean \gtrsim 50 in extreme SB galaxies, e.g., local ULIRGs and high-redshift hyperluminous IR-bright (HyLIRG) and submm galaxies (\citealt{Daddi2015,Silverman2018,Liudz2021a}).$
$In Sect.~\ref{sec: Excitation Analysis}, we show that the mildly-increased [\CI/CO] abundance ratio from a CRDR and temperature/density-driven excitation can lead to the factor of 2 increase in \RCICO  when going from SF to SB disks.$
$\subsection{\RCICO versus Metallicity}$
$\label{subsec: metallicity}$
$Our current sample covers a relatively small range in metallicity for a metallicity--line ratio study. Given their stellar masses and metallicity maps (e.g., \citealt{Kreckel2019,Kreckel2020,Williams2022}), our targets probe a near-solar metallicity. NGC~3627 has a well-measured$
$metallicity gradient of Z = 8.53 + 0.06 \times[r/R_{25}] from \citet{Kreckel2019} \footnote{\citet{Kreckel2019} used a distance of 10.6 \pm 0.9 \; \mathrm{Mpc} for NGC~3627. We do not directly use this metallicity gradient, but use the metallicity maps from \citet{Williams2022} where the same new distance as in this work is adopted.}, but the metallicity of individual \ionizedhydrogen  regions scatter between \sim 8.45--8.65.$
$\citet{Williams2022} produced the 2D spatial distributions of gas-phase metallicity in PHANGS-MUSE galaxies, including NGC~3627 and NGC~4321, by applying the Gaussian Process Regression to map the smooth, higher-order metallicity variation from the individual, sparse \ionizedhydrogen  regions detected in PHANGS-MUSE (\citealt{PHANGSMUSE, Santoro2021}).$
$This method is significantly superior to a na\"{i}ve nearest neighbor interpolation method. We refer the reader to \citet{Williams2022} for the technical details and validation demonstration.$
$In Fig.~\ref{fig: line ratio vs metallicity} we present the trend between \RCICO  and metallicity in NGC~3627 and NGC~4321.$
$Due to our limited metallicity range, only a very weak decreasing trend can be seen.$
$A similar trend has been reported by \citet{Bolatto2000b}, who observed CO(1--0) and \Remark{CI10} in the Region N27 of the Small Magellanic Cloud and combined with literature data to present a correlation between \RCICO  and metallicity over \sim 0.2 \text{--} 2   Z_{\odot}.$
$We overlay the \citet{Bolatto2000b} empirical fitting \footnote{$
$We adopt the equation from \citep[][Fig.~2 caption]{Bolatto2000b}, with a typo-correction in the brackets so that the equations are:$
$\log ( F_{\Remark{CI10}} / F_{\Remark{CO10}} ) = -0.47   \left[12 + \log \mathrm{(O/H)} \right]+ 5.0 and \log ( F_{\Remark{CI10}} / F_{\Remark{CO10}} ) = -0.82   \left[12 + \log \mathrm{(O/H)} \right]+ 8.2, respectively, where F is the line flux in \mathrm{erg   s^{-1}   cm^{-2}   sr^{-1}}.$
$}$
$as well as a theoretical PDR model prediction line from \citet{Bolatto1999} in Fig.~\ref{fig: line ratio vs metallicity}.$
$Although our data covers only a small metallicity range, it is encouraging to see the overall good agreement with the observationally-driven trend in \citet{Bolatto2000b}.$
$More detailed understanding of the deviation from PDR models will likely need more \CI  mapping in lower-metallicity galaxies.$
$\vspace{2ex}$
$\section{Estimation of abundance ratios under representative ISM conditions}$
$\label{sec: Excitation Analysis}$
$The observed \RCICO  depends on not only the \CI  and CO abundances (\COabundance  and \CIabundance ), but also the ISM conditions that determine the excitation (level population) and radiative transfer of the C atoms and CO molecules --- i.e., the gas kinetic temperature \Tkin, volume density of H_2 (\nHtwo; as the primary collision partners of C and CO), turbulent line width \dv, and the species' column densities \COcolumndensity and \CIcolumndensity.$
$The column density divided by the line width (N/\dv) further determines the optical depth \tau of each line.$
$In this work, having only a CO(2--1) and a \Remark{CI10} line is insufficient to numerically solve all the \Tkin, \nHtwo, \COcolumndensity/\dv, \COabundance  and \CIabundance . Therefore, we assume several \textit{representative ISM conditions} where \Tkin, \dv and \COabundance  are fixed to reasonable values, then solve the \COcolumndensity and \CIcolumndensity. We assume that \CI  and CO are spatially mixed at our resolution, i.e., they have the same filling factor. This is a relatively strong assumption and is likely not the real case, because although we see spatial co-localization at our \sim 200~pc resolutions, the spectral profiles of CO and \CI show discrepancies (Appendix~\ref{appendix: all regions}), indicating different underlying distributions at smaller scales. With the assumption of the same filling factor (and 3D distribution), the column density ratio \CIcolumndensity/\COcolumndensity equals the [\CI/CO] abundance ratio.$
$We perform both LTE and non-LTE calculations.$
$LTE assumes that all transitions are thermalized locally, and the level population follows Boltzmann statistics, so that each transition's excitation temperature T_{\mathrm{ex}} as newcommandined by the level population equals \Tkin (e.g., \citealt{Spitzer1998Book,Tielens2010Book,Draine2011Book}). The LTE assumption facilitates the computation as we do not need to solve the level population and thus \nHtwo is not used.$
$However, LTE is not easily reached in typical ISM conditions. Subthermalized lines need non-LTE calculations. The commonly adopted non-LTE approach is the Large Velocity Gradient (LVG; \citealt{Scoville1974,Goldreich1974}), where an escape fraction is used to facilitate the local radiation intensity calculation hence the detailed balance of radiative and collisional (de-)excitations.$
$We use RADEX (\citealt{RADEX}) to compute the non-LTE \CI  and CO line fluxes for each fixed parameter set (\Tkin, \nHtwo, \COcolumndensity, \CIcolumndensity, \dv), then study how [\CI/CO] determines the line flux ratio \RCICO.$
$We assume the following 7 representative ISM conditions:$
$\textit{(1)} inner Galactic GMCs,$
$\textit{(2)} NGC~3627/4321 disk GMCs,$
$\textit{(3)} NGC~3627 bar-end environments,$
$\textit{(4)} NGC~1808/7469 SB disk,$
$\textit{(5)} NGC~4321 centre,$
$\textit{(6)} NGC~3627 centre,$
$and \textit{(7)} NGC~7469 centre.$
$Conditions \textit{(1)}--\textit{(3)} correspond to the "SF disk" in Fig.~\ref{fig: line ratio vs CO}, and condition \textit{(4)} represents the "SB disk".$
$The last three conditions are for the "nuclei".$
$Under each ISM condition, we set a fixed \Tkin (in units of K), \dv (in units of km~s^{-1}), \COabundance , and \Sigmol (in units of \Mspc2).$
$The adopted values are shown at the top of each panel in Figs.~\ref{fig: LTE RCICO} and \ref{fig: NonLTE RCICO}.$
$Our choice of \Tkin considered multi-line measurements in the literature, e.g., \citet{Heyer2015} for Galactic GMC ISM condition, and \citet{Teng2022} for NGC~3627 nucleus, as well as typical values in simulations (e.g., \citealt{Offner2014,Glover2015,Glover2016,Clark2019,HuChiaYu2021a}).$
$Our \dv is based on the observed line width in the highest-resolution ALMA data, i.e., from the original PHANGS-ALMA 1" data (\citealt{Leroy2021phangs}) or from \citet{Salak2019} and \citet{Izumi2020} for NGC~1808 and NGC~7469, respectively. The observed line width should at least new an upper limit, if not directly probing the turbulent \dv.$
$The \COabundance  is more like a fiducial value and does not obviously affect later [\CI/CO] results.$
$We also take a fiducial \Sigmol which matches the observed CO(1--0) line brightness temperature and a CO-to-H_2 conversion factor \alphaCO \sim 4 \text{--} 0.8 \; \mathrm{M_{\odot}   (K   km   s^{-1})^{-1}} depending on the assumed conditions (e.g., \citealt{Bolatto2013}; and our paper~II).$
$This \Sigmol directly converts to an N_{\mathrm{H_2}} if assuming that 36\% of the mass is helium. Then N_{\mathrm{H_2}} and \COabundance  together indicate a \COcolumndensity . Only with these assumptions can we unambiguously link an abundance ratio [\CI/CO] (i.e., \CIcolumndensity/\COcolumndensity) to a line flux ratio \RCICO .$
$Additionally, with the same filling factor assumption, \RCICOis also equivalent to the ratio of the CO-to-H_2 and \CI -to-H_2 conversion factors, i.e., \RCICO = \alphaCO/\alphaCI, because they trace the same molecular gas at our resolution.$
$In a companion paper (paper II; Liu et al. 2022 in prep.), we study the behaviors of \alphaCO and \alphaCI in more details, also under both LTE and non-LTE conditions.$
$\subsection{LTE results}$
$\label{subsec: LTE}$
$\begin{figure*}[htb]$
$    \centering$
$    \includegraphics[width=0.99\textwidth]{Plot_LTE_RCI10CO21_vs_XCICO_varying_N_CO_T_kin}$
$    \caption{$
$        Observed/calculated \RCICO as a function of CO column density \COcolumndensity  and [\CI/CO] abundance (column density) ratio \RNCINCO for 7 representative ISM conditions, based on LTE calculations.$
$        \textit{Upper panels}: Curved lines are \RCICO versus \RNCINCO for different \COcolumndensity  as indicated by the color. Dashed means CO(2--1) optically thick and \Remark{CI10} optically thin, dotted means both are both optically thin, and solid line means both optically thick.$
$        Horizontal thick line is the observed \RCICO value.$
$        \textit{Lower panels}: Black contours are \RCICO = 0.05,   0.10,   0.20 and 0.50 in the CO column density versus \CI/CO abundance ratio plane.$
$        Vertical color shading indicates the final \RNCINCO that can match both the observed \RCICOand the assumed \Sigmol value noted at the top of each panel.$
$    }$
$    \label{fig: LTE RCICO}$
$\end{figure*}$
$We determine the LTE solution of [\CI/CO] in Fig.~\ref{fig: LTE RCICO}.$
$To illustrate the complexity of this problem and the degeneracy of the \Tkin, \COcolumndensity/\dv and [\CI/CO] parameters, we show how \RCICO (y-axis) varies with \RNCINCO (x-axis) and \COcolumndensity  (color bar) in the upper panels, and how \RCICO (contour) can be pinpointed by determining the \Sigmol (right y-axis).$
$At the top of each column the key parameters of the aforementioned representative ISM conditions are shown as text. The assumed \Tkin and \dv generally increase from panels (1) to (7) mimicking real conditions. Observational-driven \Remark{ICO10} ranges and the assumed \Sigmol for these ISM environments are also listed, and can be converted one into the other via \alphaCO. We also list the chosen \COabundance  for clarity.$
$As shown in the upper panels, \RCICO increases with both \RNCINCO and \COcolumndensity  under low-\Tkin conditions. However, a maximum \RCICO is reached in the high \CI  column density regime, where an increasing abundance ratio will no longer increase \RCICO. The line ratio saturates at about 0.5 when \Tkin \sim 15~K, or at about 1 for \Tkin \sim 100~K. The observed line ratios (horizontal dashed lines) are all lower than the saturation limit of \RCICO _{\mathrm{max}} \sim 0.7 \text{--} 0.9, and corresponds to a range of \COcolumndensity or \Sigmol. This range is very wide in the low-\Tkin conditions, with \log \COcolumndensity / \mathrm{cm^{-2}} \sim 17 - 19, but is very narrow at \Tkin \sim 100~K, that is, \RCICO can be determined from [\CI/CO] without much dependence on \COcolumndensity.$
$In the lower panels, the contours of \RCICO = 0.05, 0.1, 0.2 and 0.5 are shown in the plane of CO column density \COcolumndensity versus \CI/CO abundance ratio.$
$For a fixed \RCICO , a higher \RNCINCO corresponds to a lower \COcolumndensity in the low-\Tkin regime, but \RCICO  becomes insensitive to \COcolumndensity at a high \Tkin. In the latter case, \RNCINCO alone determines \RCICO .$
$By matching to the expected \Sigmol for each ISM condition, we can obtain an unambiguous \RNCINCO from the observed \RCICO  as shown in Fig.~\ref{fig: LTE RCICO}.$
$In this way, we determine the \RNCINCO to be around 0.1--0.2 for conditions \textit{(1)} and \textit{(2)}, and \sim 0.4 for conditions \textit{(3)} and \textit{(5)}.$
$The SB disk (condition \textit{4}) and AGN environments (conditions \textit{6}/\textit{7}) have \RNCINCO \sim 2 and \sim 1 \text{--} 10, respectively.$
$The strongly enhanced \CI/CO abundance ratios in SB disk and AGN environments are clear signatures of CRDR and XDR, respectively.$
$The NGC~4321 centre is a low-ionization nuclear emission-line region (LINER, e.g., \citealt{GarciaBurillo2005}), and given our assumptions it requires \RNCINCO \sim 0.4 in order to explain the observed low line ratio.$
$The gradually increasing \CI/CO abundance ratios in the nuclei of NGC~4321, NGC~3627, NGC~1808 and NGC~7469 agree well with the increasing power of AGN activities, e.g., their X-ray luminosities (Table~\ref{table: targets}).$
$Our assumptions for the Galactic cloud ISM condition are mainly based on \citet{Heyer2009}, \citet{Heyer2015} and \cite{RomanDuval2010}. The obtained [\CI/CO]~\sim 0.1 also agrees well with previous studies using \CI  and CO isotopologues (see Sect.~\ref{subsec: intro RCICO}.$
$For example, \citet{Ikeda1999} reported \RNCINCO \sim 0.05 \textnormal{--} 0.2 for the Orion cloud;$
$\citet{Oka2001} found \RNCINCO = 0.064 \pm 0.035 but a peak of \RNCINCO \sim 0.15 in the DR 15 \HII  region and two IR dark clouds;$
$\citet{Kramer2004} found \RNCINCO \sim 0.11 \textnormal{--} 0.12 in the massive SF region W3;$
$\citet{Genzel1988} found \RNCINCO \sim 0.2 for the massive SF region W51;$
$\citet{Zmuidzinas1988} reported \RNCINCO \sim 0.05 \textnormal{--} 0.15 in several dense clouds.$
$Recently, \citet{Izumi2021} derived an overall \RNCINCO \sim 0.1 in the massive SF region RCW38 at A_V \sim 10 \textnormal{--} 100, consistent with the Orion cloud, but also saw high [\CI/CO] \sim 0.2 \textnormal{--} 0.6 in some even lower column density regions.$
$Accurate determination of all the ISM properties instead of simple assumptions as in this work requires comprehensive, multi-J CO isotopologue observations (e.g., \citealt{Teng2022}).$
$Our representative ISM conditions only new a qualitative diagnostic to understand how to infer the \CI/CO abundance ratio from the observed \RCICO  quantity.$
$Their properties may not be very accurate, and are subject to change with future multi-tracer/CO isotopologue studies.$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=\textwidth]{Plot_RADEX_R_CI10CO21_sim_XCICO_NonLTE_varying_N_CO_T_kin_n_H2}$
$\caption{$
$Similar to Fig.~\ref{fig: LTE RCICO} but with non-LTE calculations.$
$See Fig.~\ref{fig: LTE RCICO} caption for the details.$
$}$
$\label{fig: NonLTE RCICO}$
$\end{figure*}$
$\subsection{Non-LTE results}$
$\label{subsec: non-LTE}$
$We perform the non-LTE calculations using the RADEX code (\citealt{vanderTak2007})$
$with the Large Velocity Gradient (LVG; i.e., expanding sphere) escape probability (\citealt{Scoville1974,Goldreich1974}).$
$Comparing to LTE, the volume density of H_2 is now an additional property that plays a role. The excitation temperature of each line no longer equals \Tkin.$
$We compute the non-LTE line ratios in a grid of ( \Tkin,   \dv,   \COabundance,   \RNCINCO,   \COcolumndensity,   \nHtwo ) parameter sets, then study the line and abundance ratios similarly as in the previous section.$
$In Fig.~\ref{fig: NonLTE RCICO} we show the non-LTE version of Fig.~\ref{fig: LTE RCICO}.$
$The non-LTE result does not qualitatively differ from that of LTE.$
$The most obvious difference is that at high temperature (\Tkin > 50~K, given our other assumed conditions), the required [\CI/CO] is lower by \times 2 for the same observed \RCICO, whereas at low temperature (\Tkin \sim 15~K) a \lesssim 2 \times [\CI/CO] is expected for the observed line ratio.$
$As listed in the title of each panel, we assume \log \nHtwo / \mathrm{cm^{-3}} = 2.5 to 3.0 for our conditions (1) to (7).$
$This is based on the fact that low-J CO and \CI  trace the bulk of molecular gas with \log \nHtwo / \mathrm{cm^{-3}} \sim 2--3 (e.g., \citealt{Leroy2017,Liudz2021a}), with a typical value \nHtwo \sim 300   \mathrm{cm^{-3}} widely adopted/found for Galactic and nearby galaxy GMCs (e.g., \citealt{Koda2012,Pety2017}, see also simulations of \citealt{Glover2012b}), whereas a smaller portion of their emission is from the dense gas with \log \nHtwo / \mathrm{cm^{-3}} \sim 4--6 (e.g., \citealt{Gao2004a}).$
$The fraction of the dense gas is expected to increase in more starbursting environment (\citealt{Gao2004b}), so does the mean gas density (\citealt{Liudz2021a}). Therefore, a \log \nHtwo / \mathrm{cm^{-3}} of 3.0 is perhaps a reasonable assumption, or at least a lower limit for our galaxy centres. A 10 \times higher density does not qualitatively change Fig.~\ref{fig: NonLTE RCICO} except for shifting the expected \RNCINCO to be 50\% higher in the \Tkin > 50~K conditions (making Fig.~\ref{fig: NonLTE RCICO} more like Fig.~\ref{fig: LTE RCICO}).$
$Therefore, our non-LTE result strongly confirms the LTE result in the previous section, that the intrinsic \CI/CO abundance has to vary significantly from about 0.1--0.2 to about 1--3, in order to explain the observed line ratios from Galactic GMCs (hundred pc scales) and NGC~3627/4321 SF disk GMCs to NGC~1808/7469 SB disk/AGN (r \lesssim 250~pc) environments.$
$\vspace{2ex}$
$\section{Discussion}$
$\label{sec: Discussion}$
$From this study we see that the line ratio \RCICO increases from 0.1 to 0.2 and > 0.5 from the SF disk to the SB disk and the vicinity (few hundred pc) of X-ray luminous AGN. Although we may be limited by our sample,$
$we suggest that this trend can be extended to general cases at both local and high redshift, because the \CI/CO abundance ratio enhancement is well-predicted by the CRDR and XDR theories.$
$\citet{Papadopoulos2010c} and \citet{Papadopoulos2011} pointed out the key role of CRDR in local (U)LIRGs. With UV photons alone, cold gas in starbursting environments can still be well shielded, hence as cold as \sim 10~K. However, with CRs produced from O, B star clusters and supernova remnants, the gas can be uniformly illuminated and heated up.$
$The extremely starbursty local ULIRGs have a CR density (\xi_{\mathrm{CR}}) 10^{3}--10^{4} times that of the Milky Way (\citealt{Papadopoulos2010c}), and thus will naturally dissociate more CO into \CI .$
$\citet{Bisbas2017} and \citet{Papadopoulos2018} conducted simulations of molecular gas at tens of pc scale with varying CR strength, and found that when \xi_{\mathrm{CR}} is \sim 1 \textnormal{--} 100   \times Galactic value, the gas temperature in the dense gas remains largely unaffected (\sim 10~K).$
$However, when \xi_{\mathrm{CR}} reaches \sim 1000   \times Galactic value as in (U)LIRGs, the gas temperature increases to 30--50~K. The \citet{Bisbas2017} simulation also shows that the \CI  and \ionizedcarbon  fractional abundances at \log A_V \sim 0.5 increase by one and three orders of magnitude, respectively, and the CO fractional abundance decreases correspondingly by over two orders of magnitude, with an increasing \xi_{\mathrm{CR}} from the Galactic value to 100   \times of that.$
$These results are in line with our calculations in Sect.~\ref{subsec: LTE}.$
$Meanwhile, the AGN-driven XDR chemistry and molecule/atom excitation have been firstly modeled in great detail by \citet{Maloney1996}, \citet{Meijerink2005} and \citet{Meijerink2007}. In XDRs, photo-ionization becomes the dominant heating source instead of photo-electric emission from dust grains in PDRs, thus more molecules are dissociated and/or ionized. \citet{Meijerink2005}'s Model~1 shows that CO molecules are fully dissociated when N_{\mathrm{H}} < 10^{23}   \mathrm{cm^{-2}}, then the fractional abundance is gradually increased by six orders of magnitudes from N_{\mathrm{H}} = 10^{23}   \mathrm{cm^{-2}} to 10^{24}   \mathrm{cm^{-2}}. By contrast, the \CI  and \ionizedcarbon  abundances are about constant until reaching the densest part (N_{\mathrm{H}} > 10^{24.5}   \mathrm{cm^{-2}}). Unlike in PDRs, there is no clear \ionizedcarbon /\CI /CO stratification in XDRs (\citealt{Meijerink2007,Wolfire2022}). Their Model~2 with a 100 \times higher energy deposition rate shows that the CO distribution contracts to the highest gas density spots, and the total cumulative [\CI/CO] is increased by about 0.2~dex, reaching \RNCINCO \sim 1.6.$
$This value agrees well with our calculations shown in Figs.~\ref{fig: LTE RCICO} and \ref{fig: NonLTE RCICO} (the ISM condition 7).$
$Therefore, both in theoretical and observational views, it is clear that the \RCICO line ratio is a good indicator of the starburst's CRDR and AGN's XDR.$
$If a \RCICO line ratio (to be specific, \Remark{ICI10}/\Remark{ICO21}) reaches 0.2--0.5, then the CRDR plays a crucial role and the abundance ratio [\CI/CO] could be \sim 1.$
$If it exceeds 0.5--1.0, then it may only be enhanced by an XDR, e.g., caused by an X-ray luminous AGN, which boosts the abundance ratio [\CI/CO] to \gtrsim 1--2.$
$\vspace{2ex}$
$\section{Conclusion}$
$\label{sec: Conclusion}$
$We presented new beam- and uv-coverage-matched \Remark{CI10} and CO(2--1) maps in the galactocentric radius r \lesssim 7~kpc SF disk of NGC~3627 at \sim 190~pc resolution, and in the r \lesssim 3~kpc disk of NGC~4321 at \sim 270~pc resolution.$
$They are among the largest \CI  mosaic mappings available in nearby galaxies.$
$We combined the imaging of \Remark{CI10} and CO(2--1) in two nearby, more starbursty and AGN-hosting galaxies, i.e., NGC~1808 central \sim1~kpc, and NGC~7469 central \sim1.5~kpc at 140--160~pc resolutions.$
$Together, we studied the spatial distributions of \CI  and CO, and their line ratio \RCICO as functions of various galaxy resolved properties, including CO brightness (Fig.~\ref{fig: line ratio vs CO}), ISRF \Umean (Fig.~\ref{fig: line ratio vs ISRF}), and metallicity (Fig.~\ref{fig: line ratio vs metallicity}).$
$In order to obtain the underlying intrinsic [\CI/CO] abundance ratio from the observed line ratio \RCICO, we assumed 7 representative ISM conditions and did both LTE and non-LTE calculations.$
$\vspace{1ex}$
$We summarize our findings as below:$
$\begin{itemize}[itemsep=0ex,topsep=1ex,leftmargin=4ex]$
$    \item The majority of the molecular gas in the SF disks of NGC~3627 and NGC~4321 has a uniform \Remark{RCI10CO21} = 0.10 \pm 0.05.$
$    \Remark{CI10} and \Remark{CO21} exhibit very similar spatial distributions at our 140--270~pc resolution.$
$    \item The majority of the molecular gas in the SB disk of NGC~1808 has \Remark{RCI10CO21} = 0.20 \pm 0.05.$
$    This elevated line ratio compared to the SF disk is likely caused by a \sim 3 \text{--} 5 \times increased [\CI/CO] abundance ratio, based on our LTE and non-LTE calculations and parameter assumptions (Figs.~\ref{fig: LTE RCICO} and \ref{fig: NonLTE RCICO}).$
$    \item The centres (r \lesssim 250~pc) of NGC~4321, NGC~3627, NGC~1808 and NGC~7469 show a strongly increasing trend in \Remark{RCI10CO21}, from 0.05 to 0.5, as a function of I_{\Remark{CO21}} --- i.e., \Remark{RCI10CO21} \approx 0.035   (I_{\Remark{CO21}}/[100\;\Kkms])^{0.8}.$
$    The inferred [\CI/CO] abundance ratio from our non-LTE calculation changes from \sim 0.2 to \sim 3, in line with the increasing X-ray luminosity of the centres/AGNs.$
$    \item The observed \RCICO versus ISRF trend is fairly flat, i.e., \RCICO  is insensitive to ISRF intensity changes, similar to the PDR model prediction calculated with PDRToolbox (although our resolution is much coarser than the physical scale of PDR models; Sect.~\ref{subsection: ISRF}).$
$    \item We find a \RCICO versus metallicity trend consistent with previous observations that covered lower-metallicity environments (\sim 0.2   \mathrm{Z_{\odot}}; \citealt{Bolatto2000b}), within the metallicity range (\sim 0.7 \text{--} 1   \mathrm{Z_{\odot}}) of our sample (Sect.~\ref{subsec: metallicity}).$
$    \item Our inferred [\CI/CO] abundance ratios agree well with the CRDR and XDR's theoretical studies.$
$    Given the drastically changed \RCICO , we propose that the \CI  to CO line ratio can be a promising indicator of CRDR (starbursting) and XDR (AGN). For instance, an observed line flux ratio of$
$    \Remark{RCI10CO21} \sim 0.2 \textnormal{--} 0.5 and \gtrsim 0.5 effectively indicates CRDR and XDR, respectively.$
$    \item We do not find ubiquitous "CO-dark", \CI -bright gas at the outer r\sim 1 \text{--} 3~kpc disks of NGC~3627 and NGC4321, nor systematic enhancement in \RCICO  at CO-faintest sightlines via stacking, mainly due to the incompleteness of our data.$
$    Nevertheless, a few sightlines, e.g., Region~22 in NGC~3627 show high \RCICO ~\sim 0.5 which do not fully comply with the noise behavior seen in our CASA simulations.$
$\end{itemize}$
$\vspace{2ex}$
$\begin{acknowledgements}$
$We thank the anonymous referee for very helpful comments.$
$ES, TS and TGW acknowledge funding from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme (grant agreement No. 694343).$
$ER acknowledges the support of the Natural Sciences and Engineering Research Council of Canada (NSERC), funding reference number RGPIN-2017-03987.$
$The work of AKL is partially supported by the National Science Foundation under Grants No. 1615105, 1615109, and 1653300.$
$AU acknowledges support from the Spanish grants PGC2018-094671-B-I00, funded by MCIN/AEI/10.13039/501100011033 and by "ERDF A way of making Europe", and PID2019-108765GB-I00, funded by MCIN/AEI/10.13039/501100011033.$
$RSK and SCOG acknowledge financial support from the German Research Foundation (DFG) via the collaborative research centre (SFB 881, Project-ID 138713538) “The Milky Way System” (subprojects A1, B1, B2, and B8). They also acknowledge funding from the Heidelberg Cluster of Excellence "STRUCTURES" in the framework of Germany’s Excellence Strategy (grant EXC-2181/1, Project-ID 390900948) and from the European Research Council via the ERC Synergy Grant "ECOGAL" (grant 855130).$
$FB and IB acknowledge funding from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme (grant agreement No.726384/Empire).$
$JC acknowledges funding from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme DustOrigin (ERC-2019-StG-851622).$
$MC gratefully acknowledges funding from the Deutsche Forschungsgemeinschaft (DFG) in the form of an Emmy Noether Research Group (grant number CH2137/1-1).$
$Y.G.’s work is partially supported by National Key Basic Research and Development Program of China (grant No. 2017YFA0402704), National Natural Science Foundation of China (NSFC, Nos.$
$12033004, and 11861131007), and Chinese Academy of Sciences Key Research$
$Program of Frontier Sciences (grant No. QYZDJ-SSW-SLH008).$
$AH was supported by the Programme National Cosmology et Galaxies (PNCG) of CNRS/INSU with INP and IN2P3, co-funded by CEA and CNES, and by the Programme National “Physique et Chimie du Milieu Interstellaire” (PCMI) of CNRS/INSU with INC/INP co-funded by CEA and CNES.$
$K.K. gratefully acknowledges funding from the German Research Foundation (DFG) in the form of an Emmy Noether Research Group (grant number KR4598/2-1, PI Kreckel)$
$JMDK and MC gratefully acknowledge funding from the Deutsche Forschungsgemeinschaft (DFG) in the form of an Emmy Noether Research Group (grant number KR4801/1-1) and the DFG Sachbeihilfe (grant number KR4801/2-1), and from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme via the ERC Starting Grant MUSTANG (grant agreement number 714907).$
$JP acknowledges support from the Programme National “Physique et Chimie du Milieu Interstellaire” (PCMI) of CNRS/INSU with INC/INP co-funded by CEA and CNES.$
$HAP acknowledges support by the Ministry of Science and Technology of Taiwan under grant 110-2112-M-032-020-MY3.$
$The work of JS is partially supported by the Natural Sciences and Engineering Research Council of Canada (NSERC) through the Canadian Institute for Theoretical Astrophysics (CITA) National Fellowship.$
$This paper makes use of the following ALMA data:$
$\incode{ADS/JAO.ALMA#2015.1.00902.S},$
$\incode{ADS/JAO.ALMA#2015.1.00956.S},$
$\incode{ADS/JAO.ALMA#2015.1.01191.S},$
$\incode{ADS/JAO.ALMA#2017.1.00078.S},$
$\incode{ADS/JAO.ALMA#2017.1.00984.S},$
$\incode{ADS/JAO.ALMA#2018.1.00994.S},$
$\incode{ADS/JAO.ALMA#2018.1.01290.S},$
$\incode{ADS/JAO.ALMA#2019.1.01635.S}.$
$ALMA is a partnership of ESO (representing its member states), NSF (USA) and NINS (Japan), together with NRC (Canada), MOST and ASIAA (Taiwan), and KASI (Republic of Korea), in cooperation with the Republic of Chile. The Joint ALMA Observatory is operated by ESO, AUI/NRAO and NAOJ.$
$\end{acknowledgements}$
$\bibliographystyle{aa_url}$
$\input{Article_CI_AA.bbl}$
$\begin{appendix}$
$\section{Data reduction, imaging and moment map generation}$
$\label{appendix: data reduction}$
$After downloading the raw data, i.e., 12m+7m for \Remark{CO21} and 7m for \Remark{CI10}, respectively, we ran the observatory calibration pipeline ("scriptForPI") to reduce the data.$
$As shown in \citet{Leroy2021pipeline}, the observatory calibration produces well-calibrated visibility data without further manual flagging.$
$We then split out the visibility data and proceeded to the imaging step.$
$In order to match the spatial resolution of \Remark{CO21} and \Remark{CI10}, we carefully matched their uv ranges.$
$We flagged the \Remark{CO21} visibility data outside the uv range of 19--68~m (\sim15--52k\lambda) so as to match the uv range of the \Remark{CI10} data (minimum to 80\% percentile: 8.9--32~m, or \sim15--52k\lambda).$
$Given that the \Remark{CI10} uv plane distribution is elliptic, while the uv range selection in \incode{CASA} is circular, our chosen thresholds are adjusted to make the final \Remark{CO21} synthesized beam similar to that of \Remark{CI10}.$
$Note that in a later step we also convolved both image cubes to the exact same rounded beam (3.46") before generating the moment maps.$
$The \textit{PHANGS-ALMA imaging \& post-processing pipeline} \footnote{\url{https://github.com/akleroy/phangs_imaging_scripts}; version 2.0.} developed by \citet{Leroy2021pipeline} is used for creating line cubes and moment maps. The code is suitable for general nearby galaxy strong line mapping interferometric and total power mosaic data. In brief, the visibility data are carefully transformed into the user-given channel width and frequency range corresponding to the line plus line-free channels, then continuum is subtracted by fitting an order of zero polynomial. Then, two stages of imaging ("cleaning") are performed: first a multi-scale cleaning down to \sim4\sigma, then a single-scale cleaning down to \sim1\sigma. At each stage, \incode{CASA}'s \incode{tclean} task is iteratively called so as to force many major cycles, which has proven to improve the accuracy of the deconvolution. Before and during the second stage cleaning, a clean mask is computed at each iteration according to the outcome of previous iteration by finding all >2\sigma pixels associated with >4\sigma peaks in a watershed algorithm (\citealt{Rosolowsky2006,Rosolowsky2021}). In this way the cleaning can include as much signal as possible and ensure a signal-free residual.$
$Then the image cubes are corrected for the primary beam attenuation, converted to Kelvin units, and convolved to a common beam, which is the coarser beam of \Remark{CO21} and \Remark{CI10}. Because of our uv range flagging before imaging, the directly \incode{tclean}-imaged beam sizes are already close. This convolution further precisely matches \Remark{CO21} and \Remark{CI10} image cube beams.$
$Next, moment maps are generated with modules in the PHANGS-ALMA pipeline, consistently for CO and \CI . This process includes firstly constructing noise and mask cubes from the image cubes, then combining the masks (i.e., where science signal should present), then computing moment 0 -- line integrated intensity, moment 1 -- line peak velocity, and moment 2 -- line width. The mask cubes are the key in producing moment maps. Here two masks are built: a "broad mask" which contains as much plausible science signal as possible, and a "strict mask" which is optimized for high-confidence signal. The strict mask is made by finding peaks that have >4\sigma over two consecutive channels then expanding down to 2\sigma also with the watershed algorithm. Then the broad mask is obtained by aggregating strict masks made from image cubes convolved to coarser resolutions (\sim30" in this work as well as in \citealt{Leroy2021pipeline}).$
$To make consistent moment maps for CO and \CI , we aggregate their masks together as a joint mask, for broad and strict masks, respectively, then their final moment maps are computed within the joint mask.$
$The resulting \Remark{CO21} and \Remark{CI10} line intensity maps are then used for our next line ratio analysis. The line velocity and line width maps are not further analyzed, but are consistent with the products from the PHANGS-ALMA survey's data release as well as in \citet{Lang2020}. Given that the PHANGS-ALMA 12m+7m+tp \Remark{CO21} has the full spatial information and are deeper, their line velocity and line width maps are more useful for kinematic studies than ours produced from this work.$
$\section{Assessing the missing flux in the NGC~3627 \CI  data}$
$\label{appendix: missing flux}$
$We assess the level of missing flux in the NGC~3627 \CI  data in three ways:$
$(a) comparing with the \textit{Herschel} SPIRE FTS \Remark{CI10} observation in this galaxy,$
$(b) comparing the uv-trimmed CO(2-1) moment-0 map with the PHANGS-ALMA short-spacing corrected (12m+7m+tp) moment-0 map,$
$and (c) performing CASA simulation and comparing with the "true" moment-0 from the skymodel and the derived product from the simulated visibilities.$
$Method (a):$
$NGC~3627 has been observed with the \textit{Herschel} SPIRE (\citealt{Griffin2010}) Fourier Transform Spectrometer (FTS; \citealt{Naylor2010}) in mapping mode with nearly Nyquist sampling.$
$\cite{Liudz2015} reduced the archival data and extracted the line fluxes for each groups of pointings. \Remark{CI10} is detected with \SNR\sim4-5 at three sightlines from the galaxy centre to the southern bar end, with R.A. and Dec. coordinates (J2000) 170.061905, +12.991336, 170.066361 +12.987091, and 170.071243 +12.981615, respectively. The \Remark{CI10} luminosities are 1.44 \pm 0.31, 2.01 \pm 0.48, and 1.71 \pm 0.40 \times 10^{7} \; \mathrm{K km s^{-1} pc^{2}}, respectively, where the errors are purely from the RMS of line-free FTS spectra. By convolving our \Remark{CI10} line cube to the FTS beam of \sim39" then extracting the moment maps, we measure a \Remark{CI10} line luminosity of 0.52, 0.33 and 0.29 \times 10^{7} \; \mathrm{K km s^{-1} pc^{2}} at the above three positions, respectively, which means a missing flux of 36-75\% at the galaxy centre and 68-88\% at the southern bar end (at a 95\% confidence level).$
$Method (b):$
$We made ratio maps of our uv-trimmed CO(2-1) moment-0 map, and that of the original PHANGS-ALMA short-spacing-corrected (12m+7m+tp) CO(2-1) data but convolved to the corresponding resolution before making the moment maps. The ratio maps are made at two resolutions: the FTS 39" beam, and our working resolution of 3.46". The former leads to a missing flux of 26\%, 31\% and 36\% from the galaxy centre to the southern bar end. Note that the above \textit{Herschel} FTS analysis indicates in general a higher missing flux, but the FTS measurements are of low \SNR and errors purely from the RMS of the spectra can not fully account for other errors like pointing, spectral response, off-axis spaxel calibration, etc.$
$The latter ratio map shows that the missing flux at our working resolution and within the strict mask varies from \sim60\% to as low as 6\% at the brightest pixels. This is consistent with our next method based on simulation.$
$Method (c):$
$We performed detailed simulations of 7m-only \Remark{CI10} and 12m+7m \Remark{CO21} observations with the \incode{CASA} \incode{simobserve} task. We use the PHANGS-ALMA 12m+7m CO(2-1) clean model cube (one of the output products of \incode{CASA} \incode{tclean} runs) as the input \incode{skymodel} for \Remark{CO21}, and that scaled by a factor of 0.2 in Kelvin units for \Remark{CI10} (representing a constant \RCICO=0.2).$
$Note that we do not use the short-spacing-corrected PHANGS-ALMA CO(2-1) cube because it has a finite beam size of \sim1", and simulation based on that may result in more extended emissions. On the contrary, the clean model cube has an infinite resolution, and because both "multiscale" and "singlescale" cleaning are used in the PHANGS-ALMA imaging, the clean model contains point as well as extended sources.$
$We also set the actual observing dates and integration times as the real observations.$
$Then we run \incode{CASA} \incode{simobserve} to simulate the visibilities of each observing block, concatenate them together, and process them into image cubes and moment maps in the same way as described in Appendix~\ref{appendix: data reduction}. Meanwhile, by convolving the input \incode{skymodel} with a 2D Gaussian beam corresponding to our working resolution, and integrating over the spectral axis, we obtain a "true" \incode{skymodel} moment-0 map which can be directly compared to the imaged moment-0 maps from the simulation. Their ratio map shows a missing flux of \sim50\% to \sim7\% in the strict mask, and more specifically, 24\%, 58\%, 85\% and 96\% pixels selected in our method as for real data have a missing flux < 20\%, < 30\%, < 40\% and < 50\%, respectively.$
$Fig.~\ref{fig: missing flux} illustrates the results from methods (b) (upper panels) and (c) (lower panels). Left panels show the \Remark{CO21} moment-0 maps, analyzed at the same working resolution as our main analysis in Appendix~\ref{appendix: data reduction}. The simulated data show a fainter emission because we used the clean mode cube as the \incode{skymodel}, which was made by cleaning the PHANGS-ALMA 12m+7m data down to 1~\sigma within the clean mask voxels. This will indeed lose some fluxes. However, the right panels show that the two methods lead to consistent missing flux results.$
$Note that in the last panel of Fig.~\ref{fig: missing flux}, some regions appear black in the simulation-based analysis because we require line ratios to be analyzed in the strict mask, while these regions are not in it anymore. This is because the \incode{skymodel} loses some fluxes as mentioned above, thus the strict mask is smaller, and also because in the simulation we assumed a constant \RCICO which may lead to fainter \CI  than real data along faint sightlines, e.g., Region 22,  thus further shrinking the strict mask where both lines need to have sufficient \SNR.$
$Bearing these in mind, we conclude that methods (b) and (c) have consistent results on the missing flux, and method (a) is less accurate due to the unconsidered uncertainties.$
$These checks show that the overall missing flux in NGC~3627 is 31\% based on the full- and trimmed-uv \Remark{CO21} analysis, or \sim36-88\% based on the more-uncertain \textit{Herschel} data. At the pixel level, it varies from 7\% to \sim50\% from bright to faint sightlines at our working resolution of 3.46".$
$These values are as expected given our ACA-only observation setup.$
$We emphasize again that by applying the uv-matching in our data reduction, our \CI /\CO  line ratio study is not obviously affected by the missing flux issue, except perhaps in the faintest pixels or "cloud edges".$
$\begin{figure*}$
$\centering$
$\includegraphics[width=\textwidth, trim=0 7mm 0 0, clip]{Plot_missing_flux_NGC3627_CO21_uvtrimmed_to_CO21_full_uvrange}$
$\includegraphics[width=\textwidth, trim=0 0 0 5mm, clip]{Plot_missing_flux_NGC3627_CO21_uvtrimmed_to_CO21_skymodel_from_simulation}$
$\caption{$
$Upper left panel: short-spacing corrected (12m+7m+tp) PHANGS-ALMA CO(2-1) derived moment-0 map.$
$Upper right panel: ratio map of the uv-trimmed CO(2-1) moment-0 map used in this work to that in the left panel.$
$Lower left panel: the CO(2-1) moment-0 map derived from our simulation with uv trimming as for real data.$
$Lower right panel: ratio map of the uv-trimmed CO(2-1) moment-0 map based on the simulation to the "true" \incode{skymodel} moment-0 map as shown left.$
$All moment maps are derived by first convolving cubes to our working resolution of 3.46" then performing masking analysis as described in Appendix~\ref{appendix: data reduction}, except for the "true" \incode{skymodel} moment-0 map which is computed without any masking.$
$Note that in the simulation based analysis, some regions have too low \SNR due to their faint CO brightness or insufficient cleaning, thus they are out of the mask and show blank line ratios (instead of very low missing flux) in the lower right panel.$
$\label{fig: missing flux}$
$}$
$\end{figure*}$
$\section{Assessing noise-induced \CI  enhancement with simulation}$
$\label{appendix: simulation}$
$In Appendix~\ref{appendix: missing flux} we described our simulation. We also use it to assess how the noise can induce artificial [\CI/CO] line ratio variation.$
$In Fig.~\ref{fig: sim line ratio image} we show the line ratio map from the simulation with a constant \RCICO=0.2, and in Fig.~\ref{fig: sim line ratio scatter} we show the scatter plots with simulation results similar to Fig.~\ref{fig: line ratio vs CO}. Interestingly, we see certain \CI/CO variations, especially \CI  "enhancement" at the edge of galaxy centre's gas clump (region~1), and the edge of the northern spiral arm (Region 14). Nevertheless, the trend is very different from the real data, i.e., Fig.~\ref{fig: line ratio vs CO}.$
$First, galaxy centre has a much weaker [\CI/CO] in the real data, whereas in the simulation it is inverted.$
$Second, the scatter is much larger in real data, with a tail of \CI  enhanced regions which is not seen in the simulation.$
$Third, from the bottom panels of Fig.~\ref{fig: sim line ratio scatter}, where \SNR is shown, we see that \RCICO is biased to be lower in low-\SNR situation. Thus the high \CI  in real data is unlikely to be a bias of low \SNR.$
$\begin{figure*}$
$\centering$
$\includegraphics[width=\textwidth]{Plot_two_line_ratio_two_panel_image_NGC3627_CI10_CO21_with_regions_from_simulation}$
$\caption{$
$\CI  line flux and \RCICO line ratio image from the simulation mimicking real observation but with a constant input line ratio \RCICO=0.2.$
$}$
$\label{fig: sim line ratio image}$
$\end{figure*}$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.9\textwidth]{Plot_Simulation_Line_Ratio_CI10_CO21_vs_Line_Flux_CO21}$
$\includegraphics[width=0.9\textwidth, trim=0 0 0 13mm, clip]{Plot_Simulation_Line_Ratio_CI10_CO21_vs_Line_SNR_CI10}$
$\caption{$
$\textit{Upper panels}: \Remark{CI10} and \Remark{CO21} line brightness and ratio similar to Fig.\ref{fig: line ratio vs CO} but from our simulation (with a constant input \RCICO=0.2 as indicated by the horizontal shading in the right panels).$
$\textit{Lower panels}: Similar to the upper panels but showing line \SNR instead of brightness. This illustrates how \RCICO can be biased in low-\SNR situation.$
$}$
$\label{fig: sim line ratio scatter}$
$\end{figure*}$
$\section{\CI and \CO spectra of selected regions in NGC~3627}$
$\label{appendix: all regions}$
$We show the examples of \Remark{CI10} and \Remark{CO21} moment-0 maps and spectra for Regions 1, 4 \& 22 in Fig.~\ref{fig: NGC3627 regions}. They represent the NGC~3627 centre, off-centre location and the location at the outer disk of the gaseous spiral arm as discussed in Sect.~\ref{sec: Results}.$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.7\textwidth]{Plot_zoom_in_image_region_1}$
$\includegraphics[width=0.7\textwidth]{Plot_zoom_in_image_region_4}$
$\includegraphics[width=0.7\textwidth]{Plot_zoom_in_image_region_22}$
$\caption{\Remark{CI10} and \Remark{CO21} moment-0 maps and spectra of Regions 1, 4 and 22 as newcommandined in Fig.~\ref{fig: NGC3627 line ratio map}. All regions' spectra are available in our online data release (see Sect.~\ref{subsection: ALMA Band 8 Observation}).$
$\label{fig: NGC3627 regions}$
$}$
$\end{figure*}$
$\end{appendix}$
$\end{document}$
$\begin{appendix}$
$\section{Data reduction, imaging and moment map generation}$
$\label{appendix: data reduction}$
$After downloading the raw data, i.e., 12m+7m for \Remark{CO21} and 7m for \Remark{CI10}, respectively, we ran the observatory calibration pipeline ("scriptForPI") to reduce the data.$
$As shown in \citet{Leroy2021pipeline}, the observatory calibration produces well-calibrated visibility data without further manual flagging.$
$We then split out the visibility data and proceeded to the imaging step.$
$In order to match the spatial resolution of \Remark{CO21} and \Remark{CI10}, we carefully matched their uv ranges.$
$We flagged the \Remark{CO21} visibility data outside the uv range of 19--68~m (\sim15--52k\lambda) so as to match the uv range of the \Remark{CI10} data (minimum to 80\% percentile: 8.9--32~m, or \sim15--52k\lambda).$
$Given that the \Remark{CI10} uv plane distribution is elliptic, while the uv range selection in \incode{CASA} is circular, our chosen thresholds are adjusted to make the final \Remark{CO21} synthesized beam similar to that of \Remark{CI10}.$
$Note that in a later step we also convolved both image cubes to the exact same rounded beam (3.46") before generating the moment maps.$
$The \textit{PHANGS-ALMA imaging \& post-processing pipeline} \footnote{\url{https://github.com/akleroy/phangs_imaging_scripts}; version 2.0.} developed by \citet{Leroy2021pipeline} is used for creating line cubes and moment maps. The code is suitable for general nearby galaxy strong line mapping interferometric and total power mosaic data. In brief, the visibility data are carefully transformed into the user-given channel width and frequency range corresponding to the line plus line-free channels, then continuum is subtracted by fitting an order of zero polynomial. Then, two stages of imaging ("cleaning") are performed: first a multi-scale cleaning down to \sim4\sigma, then a single-scale cleaning down to \sim1\sigma. At each stage, \incode{CASA}'s \incode{tclean} task is iteratively called so as to force many major cycles, which has proven to improve the accuracy of the deconvolution. Before and during the second stage cleaning, a clean mask is computed at each iteration according to the outcome of previous iteration by finding all >2\sigma pixels associated with >4\sigma peaks in a watershed algorithm (\citealt{Rosolowsky2006,Rosolowsky2021}). In this way the cleaning can include as much signal as possible and ensure a signal-free residual.$
$Then the image cubes are corrected for the primary beam attenuation, converted to Kelvin units, and convolved to a common beam, which is the coarser beam of \Remark{CO21} and \Remark{CI10}. Because of our uv range flagging before imaging, the directly \incode{tclean}-imaged beam sizes are already close. This convolution further precisely matches \Remark{CO21} and \Remark{CI10} image cube beams.$
$Next, moment maps are generated with modules in the PHANGS-ALMA pipeline, consistently for CO and \CI . This process includes firstly constructing noise and mask cubes from the image cubes, then combining the masks (i.e., where science signal should present), then computing moment 0 -- line integrated intensity, moment 1 -- line peak velocity, and moment 2 -- line width. The mask cubes are the key in producing moment maps. Here two masks are built: a "broad mask" which contains as much plausible science signal as possible, and a "strict mask" which is optimized for high-confidence signal. The strict mask is made by finding peaks that have >4\sigma over two consecutive channels then expanding down to 2\sigma also with the watershed algorithm. Then the broad mask is obtained by aggregating strict masks made from image cubes convolved to coarser resolutions (\sim30" in this work as well as in \citealt{Leroy2021pipeline}).$
$To make consistent moment maps for CO and \CI , we aggregate their masks together as a joint mask, for broad and strict masks, respectively, then their final moment maps are computed within the joint mask.$
$The resulting \Remark{CO21} and \Remark{CI10} line intensity maps are then used for our next line ratio analysis. The line velocity and line width maps are not further analyzed, but are consistent with the products from the PHANGS-ALMA survey's data release as well as in \citet{Lang2020}. Given that the PHANGS-ALMA 12m+7m+tp \Remark{CO21} has the full spatial information and are deeper, their line velocity and line width maps are more useful for kinematic studies than ours produced from this work.$
$\section{Assessing the missing flux in the NGC~3627 \CI  data}$
$\label{appendix: missing flux}$
$We assess the level of missing flux in the NGC~3627 \CI  data in three ways:$
$(a) comparing with the \textit{Herschel} SPIRE FTS \Remark{CI10} observation in this galaxy,$
$(b) comparing the uv-trimmed CO(2-1) moment-0 map with the PHANGS-ALMA short-spacing corrected (12m+7m+tp) moment-0 map,$
$and (c) performing CASA simulation and comparing with the "true" moment-0 from the skymodel and the derived product from the simulated visibilities.$
$Method (a):$
$NGC~3627 has been observed with the \textit{Herschel} SPIRE (\citealt{Griffin2010}) Fourier Transform Spectrometer (FTS; \citealt{Naylor2010}) in mapping mode with nearly Nyquist sampling.$
$\cite{Liudz2015} reduced the archival data and extracted the line fluxes for each groups of pointings. \Remark{CI10} is detected with \SNR\sim4-5 at three sightlines from the galaxy centre to the southern bar end, with R.A. and Dec. coordinates (J2000) 170.061905, +12.991336, 170.066361 +12.987091, and 170.071243 +12.981615, respectively. The \Remark{CI10} luminosities are 1.44 \pm 0.31, 2.01 \pm 0.48, and 1.71 \pm 0.40 \times 10^{7} \; \mathrm{K km s^{-1} pc^{2}}, respectively, where the errors are purely from the RMS of line-free FTS spectra. By convolving our \Remark{CI10} line cube to the FTS beam of \sim39" then extracting the moment maps, we measure a \Remark{CI10} line luminosity of 0.52, 0.33 and 0.29 \times 10^{7} \; \mathrm{K km s^{-1} pc^{2}} at the above three positions, respectively, which means a missing flux of 36-75\% at the galaxy centre and 68-88\% at the southern bar end (at a 95\% confidence level).$
$Method (b):$
$We made ratio maps of our uv-trimmed CO(2-1) moment-0 map, and that of the original PHANGS-ALMA short-spacing-corrected (12m+7m+tp) CO(2-1) data but convolved to the corresponding resolution before making the moment maps. The ratio maps are made at two resolutions: the FTS 39" beam, and our working resolution of 3.46". The former leads to a missing flux of 26\%, 31\% and 36\% from the galaxy centre to the southern bar end. Note that the above \textit{Herschel} FTS analysis indicates in general a higher missing flux, but the FTS measurements are of low \SNR and errors purely from the RMS of the spectra can not fully account for other errors like pointing, spectral response, off-axis spaxel calibration, etc.$
$The latter ratio map shows that the missing flux at our working resolution and within the strict mask varies from \sim60\% to as low as 6\% at the brightest pixels. This is consistent with our next method based on simulation.$
$Method (c):$
$We performed detailed simulations of 7m-only \Remark{CI10} and 12m+7m \Remark{CO21} observations with the \incode{CASA} \incode{simobserve} task. We use the PHANGS-ALMA 12m+7m CO(2-1) clean model cube (one of the output products of \incode{CASA} \incode{tclean} runs) as the input \incode{skymodel} for \Remark{CO21}, and that scaled by a factor of 0.2 in Kelvin units for \Remark{CI10} (representing a constant \RCICO=0.2).$
$Note that we do not use the short-spacing-corrected PHANGS-ALMA CO(2-1) cube because it has a finite beam size of \sim1", and simulation based on that may result in more extended emissions. On the contrary, the clean model cube has an infinite resolution, and because both "multiscale" and "singlescale" cleaning are used in the PHANGS-ALMA imaging, the clean model contains point as well as extended sources.$
$We also set the actual observing dates and integration times as the real observations.$
$Then we run \incode{CASA} \incode{simobserve} to simulate the visibilities of each observing block, concatenate them together, and process them into image cubes and moment maps in the same way as described in Appendix~\ref{appendix: data reduction}. Meanwhile, by convolving the input \incode{skymodel} with a 2D Gaussian beam corresponding to our working resolution, and integrating over the spectral axis, we obtain a "true" \incode{skymodel} moment-0 map which can be directly compared to the imaged moment-0 maps from the simulation. Their ratio map shows a missing flux of \sim50\% to \sim7\% in the strict mask, and more specifically, 24\%, 58\%, 85\% and 96\% pixels selected in our method as for real data have a missing flux < 20\%, < 30\%, < 40\% and < 50\%, respectively.$
$Fig.~\ref{fig: missing flux} illustrates the results from methods (b) (upper panels) and (c) (lower panels). Left panels show the \Remark{CO21} moment-0 maps, analyzed at the same working resolution as our main analysis in Appendix~\ref{appendix: data reduction}. The simulated data show a fainter emission because we used the clean mode cube as the \incode{skymodel}, which was made by cleaning the PHANGS-ALMA 12m+7m data down to 1~\sigma within the clean mask voxels. This will indeed lose some fluxes. However, the right panels show that the two methods lead to consistent missing flux results.$
$Note that in the last panel of Fig.~\ref{fig: missing flux}, some regions appear black in the simulation-based analysis because we require line ratios to be analyzed in the strict mask, while these regions are not in it anymore. This is because the \incode{skymodel} loses some fluxes as mentioned above, thus the strict mask is smaller, and also because in the simulation we assumed a constant \RCICO which may lead to fainter \CI  than real data along faint sightlines, e.g., Region 22,  thus further shrinking the strict mask where both lines need to have sufficient \SNR.$
$Bearing these in mind, we conclude that methods (b) and (c) have consistent results on the missing flux, and method (a) is less accurate due to the unconsidered uncertainties.$
$These checks show that the overall missing flux in NGC~3627 is 31\% based on the full- and trimmed-uv \Remark{CO21} analysis, or \sim36-88\% based on the more-uncertain \textit{Herschel} data. At the pixel level, it varies from 7\% to \sim50\% from bright to faint sightlines at our working resolution of 3.46".$
$These values are as expected given our ACA-only observation setup.$
$We emphasize again that by applying the uv-matching in our data reduction, our \CI /\CO  line ratio study is not obviously affected by the missing flux issue, except perhaps in the faintest pixels or "cloud edges".$
$\begin{figure*}$
$\centering$
$\includegraphics[width=\textwidth, trim=0 7mm 0 0, clip]{Plot_missing_flux_NGC3627_CO21_uvtrimmed_to_CO21_full_uvrange}$
$\includegraphics[width=\textwidth, trim=0 0 0 5mm, clip]{Plot_missing_flux_NGC3627_CO21_uvtrimmed_to_CO21_skymodel_from_simulation}$
$\caption{$
$Upper left panel: short-spacing corrected (12m+7m+tp) PHANGS-ALMA CO(2-1) derived moment-0 map.$
$Upper right panel: ratio map of the uv-trimmed CO(2-1) moment-0 map used in this work to that in the left panel.$
$Lower left panel: the CO(2-1) moment-0 map derived from our simulation with uv trimming as for real data.$
$Lower right panel: ratio map of the uv-trimmed CO(2-1) moment-0 map based on the simulation to the "true" \incode{skymodel} moment-0 map as shown left.$
$All moment maps are derived by first convolving cubes to our working resolution of 3.46" then performing masking analysis as described in Appendix~\ref{appendix: data reduction}, except for the "true" \incode{skymodel} moment-0 map which is computed without any masking.$
$Note that in the simulation based analysis, some regions have too low \SNR due to their faint CO brightness or insufficient cleaning, thus they are out of the mask and show blank line ratios (instead of very low missing flux) in the lower right panel.$
$\label{fig: missing flux}$
$}$
$\end{figure*}$
$\section{Assessing noise-induced \CI  enhancement with simulation}$
$\label{appendix: simulation}$
$In Appendix~\ref{appendix: missing flux} we described our simulation. We also use it to assess how the noise can induce artificial [\CI/CO] line ratio variation.$
$In Fig.~\ref{fig: sim line ratio image} we show the line ratio map from the simulation with a constant \RCICO=0.2, and in Fig.~\ref{fig: sim line ratio scatter} we show the scatter plots with simulation results similar to Fig.~\ref{fig: line ratio vs CO}. Interestingly, we see certain \CI/CO variations, especially \CI  "enhancement" at the edge of galaxy centre's gas clump (region~1), and the edge of the northern spiral arm (Region 14). Nevertheless, the trend is very different from the real data, i.e., Fig.~\ref{fig: line ratio vs CO}.$
$First, galaxy centre has a much weaker [\CI/CO] in the real data, whereas in the simulation it is inverted.$
$Second, the scatter is much larger in real data, with a tail of \CI  enhanced regions which is not seen in the simulation.$
$Third, from the bottom panels of Fig.~\ref{fig: sim line ratio scatter}, where \SNR is shown, we see that \RCICO is biased to be lower in low-\SNR situation. Thus the high \CI  in real data is unlikely to be a bias of low \SNR.$
$\begin{figure*}$
$\centering$
$\includegraphics[width=\textwidth]{Plot_two_line_ratio_two_panel_image_NGC3627_CI10_CO21_with_regions_from_simulation}$
$\caption{$
$\CI  line flux and \RCICO line ratio image from the simulation mimicking real observation but with a constant input line ratio \RCICO=0.2.$
$}$
$\label{fig: sim line ratio image}$
$\end{figure*}$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.9\textwidth]{Plot_Simulation_Line_Ratio_CI10_CO21_vs_Line_Flux_CO21}$
$\includegraphics[width=0.9\textwidth, trim=0 0 0 13mm, clip]{Plot_Simulation_Line_Ratio_CI10_CO21_vs_Line_SNR_CI10}$
$\caption{$
$\textit{Upper panels}: \Remark{CI10} and \Remark{CO21} line brightness and ratio similar to Fig.\ref{fig: line ratio vs CO} but from our simulation (with a constant input \RCICO=0.2 as indicated by the horizontal shading in the right panels).$
$\textit{Lower panels}: Similar to the upper panels but showing line \SNR instead of brightness. This illustrates how \RCICO can be biased in low-\SNR situation.$
$}$
$\label{fig: sim line ratio scatter}$
$\end{figure*}$
$\section{\CI and \CO spectra of selected regions in NGC~3627}$
$\label{appendix: all regions}$
$We show the examples of \Remark{CI10} and \Remark{CO21} moment-0 maps and spectra for Regions 1, 4 \& 22 in Fig.~\ref{fig: NGC3627 regions}. They represent the NGC~3627 centre, off-centre location and the location at the outer disk of the gaseous spiral arm as discussed in Sect.~\ref{sec: Results}.$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.7\textwidth]{Plot_zoom_in_image_region_1}$
$\includegraphics[width=0.7\textwidth]{Plot_zoom_in_image_region_4}$
$\includegraphics[width=0.7\textwidth]{Plot_zoom_in_image_region_22}$
$\caption{\Remark{CI10} and \Remark{CO21} moment-0 maps and spectra of Regions 1, 4 and 22 as newcommandined in Fig.~\ref{fig: NGC3627 line ratio map}. All regions' spectra are available in our online data release (see Sect.~\ref{subsection: ALMA Band 8 Observation}).$
$\label{fig: NGC3627 regions}$
$}$
$\end{figure*}$
$\end{appendix}$
$\end{document}}$
$\newcommand{\}{hyper}$
$\newcommand{\}{hyper}$
$\newcommand{\}{hyper}$
$\newcommand{\}{hyper}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\rom$}[1]{{{\uppercase\expandafter{$\rom$annumeral #1}}}}$
$\newcommand{$\rom$up}[1]{{\textup{\uppercase\expandafter{$\rom$annumeral #1}}}}$
$\newcommand{$\incode$}[1]{{\raggedright\lstinline|#1|}}$
$\newcommand{$\incode$p}[1]{{\raggedright\lstinline|"#1"|}}$
$\newcommand{$\neutralcarbon$}{\ifmmode \text{C\text{\sc{i}}} \else{\sc C i}\fi}$
$\newcommand{$\ion$izedcarbon}{\ifmmode \text{C\text{\sc{ii}}} \else{\sc C ii}\fi}$
$\newcommand{$\neutralhydrogen$}{\ifmmode \text{H\text{\sc{i}}} \else{\sc H i}\fi}$
$\newcommand{$\ion$izedhydrogen}{\ifmmode \text{H\text{\sc{ii}}} \else{\sc H ii}\fi}$
$\newcommand{$\carbonmonoxide$}{\ifmmode \text{$\textsc${CO}} \else{\sc CO}\fi}$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{$\Remark$}[1]{\csname rmk-#1\endcsname}$
$\newcommand$
$\newcommand{$\HII$}{$\ion$izedhydrogen}$
$\newcommand{$\CI$}{$\neutralcarbon$}$
$\newcommand{$\CO$}{$\carbonmonoxide$}$
$\newcommand{\}{alphaCO}$
$\newcommand{\}{alphaCI}$
$\newcommand{\}{XCO}$
$\newcommand{\}{XCI}$
$\newcommand{\}{SNR}$
$\newcommand{\}{RCICO}$
$\newcommand{\}{tauCO}$
$\newcommand{\}{tauCI}$
$\newcommand{\}{XCICO}$
$\newcommand{\}{RNCINCO}$
$\newcommand{\}{Tkin}$
$\newcommand{\}{Tdust}$
$\newcommand{\}{CIabundance}$
$\newcommand{\}{CIcolumndensity}$
$\newcommand{\}{COabundance}$
$\newcommand{\}{CIabundance}$
$\newcommand{\}{COcolumndensity}$
$\newcommand{\}{CObrightness}$
$\newcommand{\}{HydrogenSurfaceDensity}$
$\newcommand{\}{Sigmol}$
$\newcommand{\}{nHtwo}$
$\newcommand{\}{dv}$
$\newcommand{$\NoHe$}{^{\neg\mathrm{He}}}$
$\newcommand{$\Tprime$}{T^{\prime}}$
$\newcommand{$\Kkms$}{\mathrm{K km s^{-1}}}$
$\newcommand{$\Mspc$2}{\mathrm{M_{\odot} pc^{-2}}}$
$\newcommand{$\Mskpc$2}{\mathrm{M_{\odot} kpc^{-2}}}$
$\newcommand{$\kms$}{\mathrm{km s^{-1}}}$
$\newcommand{$\Msyr$}{\mathrm{M_{\odot} yr^{-1}}}$
$\newcommand{$\Lsun$}{\mathrm{L_{\odot}}}$
$\newcommand{$\Msun$}{\mathrm{M_{\odot}}}$
$\newcommand{\Umean}{\left<U\right>}$
$\newcommand{\DefineRemark}[2]{$
$\expandafter\newcommand\csname rmk-#1\endcsname{#2}$
$}$
$\newcommand{$\Remark$}[1]{\csname rmk-#1\endcsname}$
$\DefineRemark{CI3P13P0}{[\mathrm{C}{\text{\sc{i}}}]\;(^{3}P_{1}\textnormal{--}^{3}P_{0})}$
$\DefineRemark{CI}{\mathrm{C}{\text{\sc{i}}}}$
$\DefineRemark{CI10}{[\mathrm{C}{\text{\sc{i}}}] (1\textnormal{--}0)}$
$\DefineRemark{CI21}{[\mathrm{C}{\text{\sc{i}}}] (2\textnormal{--}1)}$
$\DefineRemark{CO}{\mathrm{CO}}$
$\DefineRemark{CO10}{\mathrm{CO} (1\textnormal{--}0)}$
$\DefineRemark{CO21}{\mathrm{CO} (2\textnormal{--}1)}$
$\DefineRemark{CO32}{\mathrm{CO} (3\textnormal{--}2)}$
$\DefineRemark{CO43}{\mathrm{CO} (4\textnormal{--}3)}$
$\DefineRemark{COJ10}{\mathrm{CO} (J=1 \to 0)}$
$\DefineRemark{COJ21}{\mathrm{CO} (J=2 \to 1)}$
$\DefineRemark{COJ32}{\mathrm{CO} (J=3 \to 2)}$
$\DefineRemark{COJ43}{\mathrm{CO} (J=4 \to 3)}$
$\DefineRemark{RCI10CO10}{$
${R}_{\mathrm{C{\text{\sc{i}}}{10}}/\mathrm{CO{10}}}$
$}$
$\DefineRemark{RCI10CO21}{$
${R}_{\mathrm{C{\text{\sc{i}}}{10}}/\mathrm{CO{21}}}$
$}$
$\DefineRemark{RCI10CO32}{$
${R}_{\mathrm{C{\text{\sc{i}}}{10}}/\mathrm{CO{32}}}$
$}$
$\DefineRemark{RCI10CO43}{$
${R}_{\mathrm{C{\text{\sc{i}}}{10}}/\mathrm{CO{43}}}$
$}$
$\DefineRemark{LPrimeCO21}{L^{\prime}_{$\Remark${CO21}}}$
$\DefineRemark{LPrimeCI10}{L^{\prime}_{$\Remark${CI10}}}$
$\DefineRemark{ICO10}{I_{$\Remark${CO10}}}$
$\DefineRemark{ICO21}{I_{$\Remark${CO21}}}$
$\DefineRemark{ICO32}{I_{$\Remark${CO32}}}$
$\DefineRemark{ICO43}{I_{$\Remark${CO43}}}$
$\DefineRemark{ICI10}{I_{$\Remark${CI10}}}$
$\DefineRemark{Kkmspc2}{\mathrm{K km s^{-1} pc^{2}}}$
$\begin{document}$
$\title{C~$\textsc${i} and CO in Nearby Spiral Galaxies - I. Line Ratio and Abundance Variations at \sim 200~pc Scales}$
$\titlerunning{C~$\textsc${i} \& CO - I. Line Ratio and Abundance Ratio}$
$\authorrunning{D. Liu et al.}$
$\author{$
$    Daizhong Liu \inst{\ref{MPE}}$
$\and$
$    Eva Schinnerer \inst{\ref{MPIA}}$
$\and$
$    Toshiki Saito \inst{\ref{NAOJ}}$
$\and$
$    Erik Rosolowsky \inst{\ref{UAlberta}}$
$\and$
$    Adam Leroy \inst{\ref{OSU}}$
$\and$
$    Antonio Usero \inst{\ref{Madrid}}$
$\and$
$    Karin Sandstrom \inst{\ref{UCSD}}$
$\and$
$    Ralf S. Klessen \inst{\ref{HeidelbergZfA},\ref{HeidelbergZfW}}$
$\and$
$    Simon C. O. Glover \inst{\ref{HeidelbergZfA}}$
$\and$
$    Yiping Ao \inst{\ref{PMO},\ref{USTC}}$
$\and$
$    Ivana Be\v{s}li\'c \inst{\ref{BonnAIfA}}$
$\and$
$    Frank Bigiel \inst{\ref{BonnAIfA}}$
$\and$
$    Yixian Cao \inst{\ref{MPE}}$
$\and$
$    Jérémy Chastenet \inst{\ref{UCSD},\ref{Gent}}$
$\and$
$    Mélanie Chevance \inst{\ref{HeidelbergARI},\ref{HeidelbergZfA}}$
$\and$
$    Daniel A. Dale \inst{\ref{Wyoming}}$
$\and$
$    Yu Gao \inst{\ref{XiamenUniv}}\thanks{Deceased.}$
$\and$
$    Annie Hughes \inst{\ref{Toulouse}}$
$\and$
$    Kathryn Kreckel \inst{\ref{HeidelbergARI}}$
$\and$
$    J.~M.~Diederik Kruijssen \inst{\ref{HeidelbergARI}}$
$\and$
$    Hsi-An Pan \inst{\ref{TamkangUniv}}$
$\and$
$    J\'er\^ome Pety \inst{\ref{IRAM},\ref{SorbonneUniv}}$
$\and$
$    Dragan Salak \inst{\ref{HokkaidoUniv},\ref{HokkaidoUnivGrad}}$
$\and$
$    Francesco Santoro \inst{\ref{MPIA}}$
$\and$
$    Andreas Schruba \inst{\ref{MPE}}$
$\and$
$    Jiayi Sun \inst{\ref{McMasterUniv},\ref{TorontoUniv}}$
$\and$
$    Yu-Hsuan Teng \inst{\ref{UCSD}}$
$\and$
$    Thomas Williams \inst{\ref{MPIA}}$
$}$
$\institute{$
$    Max-Planck-Institut f\"ur extraterrestrische Physik (MPE), Giessenbachstrasse 1, D-85748 Garching, Germany\\email{dzliu@mpe.mpg.de, astro.dzliu@gmail.com}$
$    \label{MPE}$
$\and$
$    Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany$
$    \label{MPIA}$
$\and$
$National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo, 181-8588, Japan$
$    \label{NAOJ}$
$\and$
$Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada$
$    \label{UAlberta}$
$\and$
$Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA$
$    \label{OSU}$
$\and$
$    Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain$
$    \label{Madrid}$
$\and$
$    Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA$
$    \label{UCSD}$
$\and$
$    Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany$
$    \label{HeidelbergZfA}$
$\and$
$    Universit\"at Heidelberg, Interdisziplin\"ares Zentrum f\"ur Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany$
$    \label{HeidelbergZfW}$
$\and$
$     Purple Mountain Observatory and Key Laboratory for Radio Astronomy, Chinese Academy of Sciences, Nanjing, China$
$     \label{PMO}$
$\and$
$    School of Astronomy and Space Science, University of Science and Technology of China, Hefei, China$
$    \label{USTC}$
$\and$
$    Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany$
$    \label{BonnAIfA}$
$\and$
$    Sterrenkundig Observatorium, Ghent University, Krijgslaan 281-S9, 9000 Gent, Belgium$
$    \label{Gent}$
$\and$
$    Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany$
$    \label{HeidelbergARI}$
$\and$
$    Department of Physics \& Astronomy, University of Wyoming, Laramie, WY 82071, USA$
$    \label{Wyoming}$
$\and$
$    Department of Astronomy, Xiamen University, Xiamen, Fujian 361005, China$
$    \label{XiamenUniv}$
$\and$
$    CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France; Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France$
$    \label{Toulouse}$
$\and$
$    Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan$
$    \label{TamkangUniv}$
$\and$
$    Institut de Radioastronomie Millimétrique (IRAM), 300 rue de la Piscine, F-38406 Saint-Martin-d'Hères, France$
$    \label{IRAM}$
$\and$
$    Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France$
$    \label{SorbonneUniv}$
$\and$
$    Institute for the Advancement of Higher Education, Hokkaido University, Kita 17 Nishi 8, Kita-ku, Sapporo, Hokkaido 060-0817, Japan$
$    \label{HokkaidoUniv}$
$\and$
$    Department of Cosmosciences, Graduate School of Science, Hokkaido University, Kita 10 Nishi 8, Kita-ku, Sapporo, Hokkaido 060-0817, Japan$
$    \label{HokkaidoUnivGrad}$
$\and$
$    Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada$
$    \label{McMasterUniv}$
$\and$
$    Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada$
$    \label{TorontoUniv}$
$}$
$\date{Accepted: December 13, 2022}$
$\abstract{$
$We present new neutral atomic carbon [C~$\textsc${i}]~( ^3P_1~\to~ ^3P_0) mapping observations within the inner \sim 7~kpc and \sim 4~kpc of the disks of NGC~3627 and NGC~4321 at a spatial resolution of 190~pc and 270~pc, respectively, using the Atacama Large Millimeter/Submillimeter Array (ALMA) Atacama Compact Array (ACA).$
$We combine these with the CO(2--1) data from PHANGS-ALMA, and literature [C~$\textsc${i}] and CO data for two other starburst and/or active galactic nucleus (AGN) galaxies (NGC~1808, NGC~7469), to study: \textit{a)} the spatial distributions of C~$\textsc${i} and CO emission; \textit{b)} the observed line ratio \RCICO = I_{\mathrm{[C{\normalfont$\textsc${i}}]}(1\textnormal{--}0)} / I_{\mathrm{CO}(2\textnormal{--}1)} as a function of various galactic properties; and \textit{c)} the abundance ratio of [C~$\textsc${i}/CO].$
$We find excellent spatial correspondence between C~$\textsc${i} and CO emission and nearly uniform \RCICO \sim 0.1 across the majority of the star-forming disks of NGC~3627 and NGC~4321.$
$However, \RCICO strongly varies from \sim 0.05 at the centre of NGC~4321 to > 0.2 \text{--} 0.5 in NGC~1808's starbursting centre and NGC~7469's centre with an X-ray-luminous active galactic nucleus (AGN).$
$Meanwhile, \RCICOdoes not obviously vary with \Umean, similar to the prediction of photodissociation dominated region (PDR) models.$
$We also find a mildly decreasing \RCICOwith an increasing metallicity over 0.7\text{--}0.85 \mathrm{Z_{\odot}}, consistent with the literature.$
$Assuming various typical ISM conditions representing giant molecular clouds, active star-forming regions and strong starbursting environments, we calculate the local-thermodynamic-equilibrium radiative transfer and estimate the [C~$\textsc${i}/CO] abundance ratio to be \sim 0.1 across the disks of NGC~3627 and NGC~4321, similar to previous large-scale findings in Galactic studies.$
$However, this abundance ratio likely has a substantial increase to \sim 1 and \gtrsim 1--5 in NGC~1808's starburst and NGC~7469's strong AGN environments, respectively, in line with the expectations for cosmic-ray dominated region (CRDR) and X-ray dominated region (XDR) chemistry.$
$Finally, we do not find a robust evidence for a generally CO-dark, C~$\textsc${i}-bright gas in the disk areas we probed.$
$}$
$\keywords{galaxies: ISM --- ISM: molecules --- ISM: atoms --- ISM: abundances --- galaxies: spiral --- galaxies: individual: NGC 3627, NGC 4321, NGC 1808, NGC 7469}$
$\maketitle$
$\section{Introduction}$
$\label{subsec: intro RCICO}$
$\subsection{Neutral atomic carbon in the interstellar medium}$
$Neutral atomic carbon (C^0 or $\neutralcarbon$) is an important phase of carbon in the interstellar medium (ISM) besides ionized carbon (C^+ or $\ion$izedcarbon) and carbon monoxide (CO).$
$Its ^{3}P state is split into three fine-structure levels. The^{3}P_{1} \to  ^{3}P_{0} transition line at 492.16065~GHz,$
$hereafter $\Remark${CI10}$
$, has an upper energy level of E_u = 23.620~K and a critical density of n_{\mathrm{crit}} \sim 1.0 \times 10^{3} \;\mathrm{cm^{-3}} (at a temperature of 50~K) \footnote{n_{\mathrm{crit}} \equiv A / \gamma, where A is the Einstein A coefficient for the transition and \gamma the rate of collisional de-excitation with hydrogen molecules, with values from \cite{Schroder1991} as compiled in the Leiden Atomic and Molecular Database (\citealt{Schoier2005}), \url{https://home.strw.leidenuniv.nl/~moldata/}},$
$and the^{3}P_{2} \to  ^{3}P_{1} transition line at 809.34197~GHz has E_u = 62.462~K and n_{\mathrm{crit}} \sim 3.4 \times 10^{3} \;\mathrm{cm^{-3}}.$
$Therefore, they are easily excited in the cold ISM environment.$
$Given the high abundance of carbon in the ISM, $\neutralcarbon$, CO and $\ion$izedcarbon  are the most powerful cold ISM tracers, and are the most widely used tools to probe cold ISM properties and hence galaxy evolution at cosmological distances.$
$The interstellar $\neutralcarbon$  is produced from$
$the photodissociation of CO molecules by UV photons and the recombination of $\ion$izedcarbon , both happening in photodissociation regions (PDRs; \citealt{Langer1976,deJong1980,Tielens1985a,Tielens1985b,vanDishoeck1986,vanDishoeck1988,Sternberg1989,Sternberg1995,Hollenbach1991,Hollenbach1999,Kaufman1999,Wolfire2010,Madden2020,Bisbas2021}, also see reviews by \citealt{Genzel1989,Jaffe1985,Hollenbach1997,Wolfire2022}).$
$In the simple plane-parallel (i.e., 1-D) PDR model, $\neutralcarbon$  exists in a layer where UV photons can penetrate through the molecular gas but cannot maintain a high carbon ionizing rate.$
$Such a layer is suggested to have an intermediate gas surface density (N_{\mathrm{H_2}} \sim 1 \textnormal{--} 4 \times 10^{20} \;\mathrm{cm^{-2}}), fairly cold temperature (10 to a few tens K), and moderate visual extinction (A_V \sim 1 - 4).$
$CO molecules dominate the more shielded interior of this layer and $\ion$izedcarbon  dominates the exterior.$
$Hydrogen molecules are gradually photodissociated to atoms across this layer (A_V \sim 0.1 - 4).$
$This plane-parallel scenario qualitatively explains the layered structure of the Orion Bar PDR in our Galaxy on sub-pc scales (e.g.,  \citealt{Genzel1989,Tielens1993,Tauber1994,Hogerheijde1995b,Hollenbach1999,Goicoechea2016}), and the \rho~Ophiuchi~A PDR in a recent study with an unprecedented resolution of 360~AU (\citealt{Yamagishi2021}), where the interplay between individual young, massive (O, B) stars and the ISM can be directly observed.$
$However, such simple models cannot fully explain molecular clouds. Plenty of observations in Galactic molecular clouds, $\ion$izedhydrogen  regions and star-formation complexes have revealed a widespread $\neutralcarbon$  distribution over most areas of molecular clouds on scales of a few to ten pc, and a relatively high $\neutralcarbon$  fractional abundance even at a large A_V depth > 10~mag (\citealt{Phillips1980,Phillips1981,Wootten1982,Keene1985,Keene1987,Jaffe1985,Zmuidzinas1986,Zmuidzinas1988,Genzel1988,Frerking1989,Plume1994,Plume1999,Gerin1998a,Tatematsu1999,Ikeda1999,Ikeda2002,Yamamoto2001,Zhang2001,Kamegai2003,Oka2004,Izumi2021}).$
$This observational evidence actually requires PDRs to be clumpy  (\citealt{Stutzki1988,Genzel1988,Burton1990,Meixner1993,Spaans1997,Kramer2004,Kramer2008,Pineda2008,Sun2008}), with a volume filling factor much less than unity (e.g., \sim 0.1 - 0.3; \citealt{Stutzki1988}), so that$
$UV photons can penetrate through most of the cloud, except for the densest clumps.$
$Therefore, the spatial co-existence of $\neutralcarbon$  and low rotational transition (low-J) CO lines can be well explained by such an inter-clump medium inside the molecular clouds.$
$Alternatively, cosmic rays (CRs) have also been proposed to be the reason for the $\neutralcarbon$ -CO association as they can penetrate much deeper than UV photons into clouds, and thus more uniformly dissociate CO molecules into $\neutralcarbon$  inside clouds (\citealt{Field1969,Padovani2009,Papadopoulos2010c,Ivlev2015,Papadopoulos2018}).$
$\subsection{Variations of the $\CI$/$\CO$ line ratio (\RCICO) in the literature}$
$Although the spatial distributions of $\CI$  and low-J CO emission resemble each other surprisingly well from molecular clouds to Galactic scales (e.g., across the Galactic plane; \citealt{Wright1991,Bennett1994,Oka2005,Burton2015}) and in external galaxies with spatially resolved observations (e.g., \citealt{Schilke1993,White1994b,Harrison1995,Israel1995,Israel2001,Israel2003,Zhang2014,Krips2016,Cicone2018,Crocker2019,Jiao2019,Miyamoto2018,Miyamoto2021,Salak2019,Izumi2020,Saito2020,Michiyama2020,Michiyama2021}),$
$their line intensity ratio, $\Remark${RCI10CO10} = $\Remark${ICI10} / $\Remark${ICO10} \footnote{The line intensity I has brightness temperature units \mathrm{K km s^{-1}}.}, does vary according to local ISM conditions --- $\CI$  and CO fractional abundance (i.e., metallicity), excitation condition (e.g., excitation temperature T_{\mathrm{ex}} and optical depth \tau), and UV and/or CR radiation field.$
$At individual molecular cloud scales, $\Remark${RCI10CO10} (or column density ratio) seems to vary from \gtrsim 0.5 (or \gtrsim 0.2 for column density ratio) to <0.1 with an increasing N_{\mathrm{H2}} and A_V, e.g., from cloud surface or PDR front to the interior \citep[e.g.][]{Frerking1982,Genzel1988,Oka2004,Kramer2004,Kramer2008,Mookerjea2006,Sun2008,Yamagishi2021}.$
$At sub-galactic scales, when individual clouds are smoothed out, $\Remark${RCI10CO10} is found to lie in a narrower range of \sim 0.1 \text{--} 0.3. For example, across the Galactic plane, the ratio is found to change only slightly within a galactocentric radius of 3--7~kpc (\sim 0.08 \text{--} 0.12, overall mean = 0.105 \pm 0.004; \citealt{Oka2005});$
$whereas in the starburst galaxies like M~82 and NGC~253, $\Remark${RCI10CO10} or the [$\CI$]/CO abundance ratio is a factor of 3--5 higher than the Galactic value (\citealt{Schilke1993,White1994b,Krips2016}).$
$Recent high-resolution (hundred pc scales) $\Remark${CI10} mapping studies in the nearby spiral galaxy M~83 (\citealt{Miyamoto2021}) and starburst galaxy IRAS~F18293-3413 (\citealt{Saito2020}) reported tightly-correlated $\Remark${CI10} and $\Remark${CO10} emission, with scaling relations at hundred parsecs equivalent to$
$\Remark{RCI10CO10} = 0.14 \times (L^{\prime}_{$\Remark${CO10}}/10^{6.0} [$\Remark${Kkmspc2}])^{-0.13}$
$and$
$\Remark{RCI10CO10} = 0.21 \times (L^{\prime}_{$\Remark${CO10}}/10^{6.8} [$\Remark${Kkmspc2}])^{+0.54},$
$respectively.$
$The discrepant power-law indices reflects different trends in individual galaxies that are not fully understood.$
$Meanwhile, in the central \lesssim 100~pc of NGC~7469 close to the X-ray-luminous active galactic nucleus (AGN), \RCICO  is found to be substantially enhanced by a factor of >10 (reaching $\Remark${RCI10CO10} \approx 1; \citealt{Izumi2020}; see also the Circinus Galaxy's AGN with $\Remark${RCI10CO32} \approx 0.9 in \citealt{Izumi2018}).$
$Furthermore, in the low-metallicity Large Magellanic Cloud (LMC) and Small Magellanic Cloud (SMC) environments, a factor of 1.5 - 3 \times higher \RCICO  values have been found (\citealt{Bolatto2000b,Bolatto2000c}; see also other low-metallicity galaxies: \citealt{Bolatto2000a,Hunt2017}).$
$These studies indicate that \RCICO  is sensitive to ISM conditions among and within galaxies.$
$However, previous extragalactic studies are mainly focused on starburst galaxies and bright galaxy centres. How \RCICO  behaves across the disks and spiral arms of typical star-forming galaxies is much less explored. Whether $\CI$  can trace the so-called CO-dark molecular gas at a low gas density or metallicity is also still in question.$
$Understanding \RCICO  variation and quantifying the $\CI$  and $\CO$  excitation in extragalactic environments is now particularly important given the rapidly growing numbers and diverse galactic environments of $\CI$  line observations at cosmological distances (e.g., \citealt{Weiss2003,Weiss2005,Pety2004,Wagg2006,Walter2011,Danielson2011,AlaghbandZadeh2013,Gullberg2016,Bothwell2017,Popping2017,Yang2017,Emonts2018,Valentino2018,Valentino2020,Bourne2019,Nesvadba2019,Dannerbauer2019,Jin2019,Brisbin2019,Boogaard2020,Cortzen2020,Harrington2021,Dunne2021,Lee2021}).$
$\subsection{What drives \RCICO? --- the need for high-resolution mapping across wide galactic environments}$
$High-resolution mapping of $\CI$  and CO from disks and spiral arms of nearby galaxies is indispensable for understanding and quantifying the ISM $\CI$  and CO physics. In particular, the following questions will be answered only with such observations:$
$Do $\CI$  and low-J CO have the same spatial distributions across galaxy disks?$
$Does the \RCICO  line ratio vary significantly across galaxy disks?$
$How does the \RCICO  line ratio vary among different local galaxies?$
$How can the observed \RCICO  line ratio be reliably converted to the $\CI$  and CO column density ratio (i.e., solving their radiation transfer equations and level population statistics and obtaining optical depths and excitation temperature)?$
$Currently, high-resolution (hundreds of parsec scale) imaging/mapping of $\CI$  in nearby galaxies relies on radio telescopes at high altitudes, e.g., the Atacama Large Millimeter/submillimeter Array (ALMA), Atacama Submillimeter Telescope Experiment (ASTE, 10-m single-dish), Atacama Pathfinder Experiment (APEX, 12-m single-dish), and is limited to the available weather conditions corresponding to the high frequency of $\CI$  lines. Therefore, only a handful of galaxies have been imaged/mapped.$
$These include NGC~6240 by \citet{Cicone2018}, Circinus Galaxy by \citet{Izumi2018}, NGC~613 by \citet{Miyamoto2018}, NGC~1808 by \citet{Salak2019}, NGC~7469 by \citet{Izumi2020}, IRAS~F18293-3413 by \citet{Saito2020}, NGC~6052 by \citet[][$\CI$  undetected]{Michiyama2020}, 36 (U)LIRGs by \citet{Michiyama2021}, M~83 by \citet[][mapped the centre and northern part with ASTE]{Miyamoto2021}, Arp~220 by \citet{Ueda2022}, and NGC~1068 by \citet{Saito2022a,Saito2022b}.$
$Nevertheless, all of these observations, except for M~83, targeted the centres of starburst, IR-luminous and/or AGN host galaxies. M~83 is the only typical star-forming main sequence galaxy having a high-quality $\CI$  mapping. However, its map is still limited to a quarter of its inner \sim3~kpc disk.$
$In this work, we present two new $\CI$  mapping observations with the ALMA Atacama Compact Array (or Morita Array; 7-m dish, hereafter ACA or 7m) in nearby spiral galaxies NGC~3627 and NGC~4321.$
$Our observations aimed at detecting $\CI$  in the disks, and thus are much larger-area and deeper than previous $\CI$  maps.$
$By further adding a starburst and an AGN-host galaxy from the literature into our analysis, we present a comprehensive study of the $\Remark${RCI10CO21} line ratio (hereafter \RCICO; ratios of other $\CI$ /CO transitions will be explicitly written otherwise) in nearby star-forming disk and starburst and AGN environments.$
$In order to quantify the $\CI$  and CO abundance ratio,$
$we have further calculated the local thermodynamic equilibrium (LTE) and non-LTE radiative transfer of $\CI$  and CO under several \textit{representative ISM conditions}.$
$The results from this study could thus be one of the most comprehensive local benchmarks for understanding the $\CI$  and CO line ratio and abundance variations.$
$The paper, as the first one of our series of papers, is organized as follows.$
$Sect.~\ref{sec: Observations} describes the sample selection, observations and data reduction.$
$Sect.~\ref{sec: Results} presents the spatial distribution and variation of \RCICO.$
$Sect.~\ref{sec: Excitation Analysis} presents the analysis of $\CI$  and CO excitation and radiative transfer and thus the abundance (species column density) ratio.$
$In Sect.~\ref{sec: Discussion} we discuss various topics relating to the [$\CI$/CO] abundance variation, CRDR, XDR, and CO-dark gas.$
$Finally we conclude in Sect.~\ref{sec: Conclusion}.$
$Our companion paper~II (D. Liu et al. in prep.) presents the detailed radiative transfer calculation and $\CI$  and CO conversion factors, and is related to the abundance calculation in this paper.$
$\vspace{2ex}$
$\section{Targets and observations}$
$\label{sec: Observations}$
$\subsection{Target selection}$
$\label{subsection: Target Selection}$
$Our targets, NGC~3627 and NGC~4321, are selected from a joint sample of the PHANGS-ALMA CO(2--1) (\citealt{Leroy2021phangs}), PHANGS-HST (\citealt{PHANGSHST}), and PHANGS-MUSE (\citealt{PHANGSMUSE}) surveys, with CO line intensities and systemic velocities best suitable for ALMA Band 8 $\CI$  mapping.$
$The PHANGS-ALMA sample consists of 90 nearby star-forming galaxies initially selected at distances of 2 \lesssim d / \mathrm{Mpc} \lesssim 23, with a stellar mass \log (M_{\star}/\mathrm{M_{\odot}}) > 9.75, and which are not too inclined and visible to ALMA. They represent the typical star-forming main sequence galaxies in the local Universe.$
$The PHANGS-HST sample \citep{PHANGSHST} is a subsample of 38 PHANGS-ALMA galaxies, providing high-resolution stellar properties.$
$The PHANGS-MUSE large program \citep{PHANGSMUSE} targeted a subsample of 19 PHANGS-ALMA galaxies suitable for VLT/MUSE optical integral field unit (IFU) observations, achieving similar spatial coverage as in PHANGS-ALMA and providing rich nebular emission lines, attenuation, stellar age and metallicity and H~$\textsc${ii} region information (e.g., \citealt{Kreckel2019, Kreckel2020, Pessa2021, Santoro2021, Williams2022}).$
$For the legacy value, we selected our sample from the joint ALMA+MUSE+HST sample pool, considered CO surface brightness, mapping area, $\CI$  line frequency and the transmission in the ALMA Band 8.$
$In this work, we present the new $\CI$  data of NGC~3627 and NGC~4321.$
$Because of the ALMA Band 8 sensitivity and the expected fainter $\CI$  line intensity than CO, the $\CI$  mapping areas are smaller than their CO(2--1) maps, but they are still the largest (\sim1'-2.35') and deepest (rms~\sim0.04~K per \sim5~km~s^{-1}) $\CI$  mapping with ALMA.$
$Furthermore, in order to study the $\CI$  and CO in different galactic environments, we selected two starbursting and/or AGN-host galaxies from the literature whose ALMA $\Remark${CI10} and CO(2--1) data are available in the archive: NGC~1808 \citep{Salak2019} and NGC~7469 \citep{Izumi2020}.$
$NGC~1808 has ALMA mosaic observations covering its \sim 1~kpc starburst ring, and NGC~7469 has ALMA single-pointing observations covering its inner \sim 3~kpc area.$
$\subsection{New ALMA ACA Band 8 observations}$
$\label{subsection: ALMA Band 8 Observation}$
$Our $\CI$  observations for NGC~3627 were taken under the ALMA project 2018.1.01290.S (PI: D. Liu) during June 3rd to Sept. 29th, 2019 with ACA-only at Band~8. The total on-source integration time was 30.8~hours. A mosaic with 149 pointings was used to map an area of 94" \times 136" at a position angle of -27^\circ, which covers the galaxy centre, bar and the majority of the spiral arms, and is about 1/3 of the full PHANGS-ALMA CO(2-1) map area.$
$The achieved line sensitivity per 5 $\kms$ is 45--55~mK (107--130~\mathrm{mJy/beam}) across the $\Remark${CI10} line frequencies with a beam of 3.46".$
$The $\CI$  mapping for NGC~4321 were taken under the ALMA project 2019.1.01635.S (PI: D. Liu; NGC~1365 is also partially observed for this project) during Dec. 3rd, 2019 to July 1st, 2021, also with ACA-only at Band~8. The on-source integration time reached 16.1~hours for the mapping of an area of 55" \times 42" with 23 pointings using the 7m array. The achieved line sensitivity is similar to that of NGC~3627.$
$More detailed information is newd in Table~\ref{table: targets}.$
$ACA (7m) was chosen to new a matched and slightly coarser synthesized beam (\sim 3"-4") than the PHANGS-ALMA CO(2-1) data. In comparison, the main 12m ALMA array in compact C43-1/2 configurations will result in a \sim 0.4"-0.7" beam and significantly miss large scale (>4-6") emission without ACA.$
$No total power observations were requested in our presented observation, therefore we matched the uv ranges of the raw CO(2--1) and $\Remark${CI10} visibility data during our data reduction.$
$We verified that missing flux will not substantially (\sim 30\%) bias our analysis using three methods as detailed in Appendix~\ref{appendix: missing flux}. This includes a CASA simulation of visibilities mimicking our ACA mosaic observations in NGC~3627, from which we find a missing flux as small as 7\% in $\CI$-bright pixels and up to \sim 50\% in the faintest pixels (and mostly < 30\%).$
$Our data reduction follows the PHANGS-ALMA imaging and post-processing pipeline \citep[][]{Leroy2021pipeline}, including imaging and deconvolution, mosaicking, broad and strict/signal mask generation, and moment map creation.$
$The additional steps in this study are: (a) uv-clipping in the CO(2--1) data to match the uv sampling range of $\Remark${CI10}; and (b) making a joint mask of the individual CO(2--1) and $\Remark${CI10} masks then extracting the moment maps (in the joint broad mask) and line ratios maps (in the joint strict mask).$
$More technical details are newd in Appendix~\ref{appendix: data reduction}.$
$Our $\Remark${CI10} and CO(2--1) line intensity maps, the line ratio maps and \textit{HST} images for the two PHANGS galaxies, NGC~3627 and NGC~4321, are presented in Figs.~\ref{fig: NGC3627 line ratio map} and \ref{fig: NGC4321 line ratio map}, respectively.$
$Our data cubes and moment maps are also made publicly available at$
$the PHANGS-ALMA CADC public repository \footnote{\url{https://www.canfar.net/storage/list/phangs/RELEASES/DZLIU_etal_2022}}.$
$\begin{table*}[htb]$
$    \centering$
$    \caption{\label{table: targets}$
$        Physical and Data Properties of the Sample$
$    }$
$    \begin{tabularx}{\linewidth}{l c c c c}$
$        \setlength{\mymincolwidth}{\dimexpr 0.85\linewidth/5 \relax}$
$        \global\mymincolwidth=\mymincolwidth$
$        \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt}$
$            \\[-\arraystretch\normalbaselineskip]$
$        $\hl$ine$
$        $\hl$ine$
$         --- &$
$            NGC 3627 &$
$            NGC 4321 &$
$            NGC 1808 &$
$            NGC 7469 \$\hl$ine$
$         redshift &$
$            0.00243 &$
$            0.00524 &$
$            0.00332 &$
$            0.01627 \distance (Mpc) &$
$            11.32 &$
$            15.21 &$
$            10.8 &$
$            71.2\linear scale (pc/") &$
$            54.9 &$
$            73.7 &$
$            52.4 &$
$            345.2 \angular resolution &$
$            3.46" (190 pc) &$
$            3.65" (269 pc) &$
$            2.67" (140 pc) &$
$            0.46" (159 pc) \channel width (\mathrm{km s^{-1}}) &$
$            4.77 &$
$            4.76 &$
$            9.52 &$
$            4.76 \\textnormal{[}C $\textsc${i}\textnormal{]}(1--0) flux scale (\mathrm{K Jy^{-1}}) &$
$            0.42 &$
$            0.38 &$
$            0.71 &$
$            23.8 $\CO$(2--1) flux scale (\mathrm{K Jy^{-1}}) &$
$            1.92 &$
$            3.70 &$
$            3.24 &$
$            108.4 \\textnormal{[}C $\textsc${i}\textnormal{]}(1--0) channel rms (K) &$
$            0.044 &$
$            0.043 &$
$            0.056 &$
$            0.11 $\CO$(2--1) channel rms (K) &$
$            0.022 &$
$            0.015 &$
$            0.083 &$
$            0.14\imaging area &2.35' \times 1.60' &$
$            1.28' \times 1.00' &$
$            40" \times 40" &$
$            \textnormal{r}\sim9"$
$            \imaging type &$
$            7m, mosaic &$
$            7m, mosaic &$
$            12m+7m+tp, mosaic &$
$            12m+7m, single\M_{\star} (10^{10}   \mathrm{M_{\odot}}) &$
$            6.8 &$
$            5.6 &$
$            2.0 &$
$            10.7$
$            \L_{\mathrm{FIR}} (10^{10}   \mathrm{L_{\odot}}) &$
$            1.7 &$
$            1.9 &$
$            3.5 &$
$            25$
$            \L_{\mathrm{TIR}} (10^{10}   \mathrm{L_{\odot}}) &$
$            2.4 &$
$            2.5 &$
$            5.1 &$
$            39$
$            \L_{X} (10^{40}   \mathrm{erg s^{-1}}) &$
$            0.032 &$
$            0.12 &$
$            1.6 &$
$            1500$
$            \SFR (\mathrm{M_{\odot} yr^{-1}}) &$
$            3.84 &$
$            3.56 &$
$            7.9 (SB ring: 4.5) &$
$            76$
$            \M_{\mathrm{H_2}} (10^{9}   \mathrm{M_{\odot}}) &$
$            6.0 &$
$            7.8 &$
$            (SB ring: 0.52) &$
$            15$
$            \M_{\mathrm{H}{\textnormal{$\textsc${i}}}} (10^{9}   \mathrm{M_{\odot}}) &$
$            1.1 &$
$            2.7 &$
$            1.7 &$
$            5.7$
$            \mean\RCICO  &$
$            0.11 &$
$            0.10 &$
$            0.16 &$
$            0.21$
$            \mean\log_{10} \RCICO&$
$            -0.96 &$
$            -1.02 &$
$            -0.83 &$
$            -0.69$
$            \\pm 2 \sigma range of \log_{10} \RCICO&$
$            -1.26, -0.66 &$
$            -1.24, -0.80 &$
$            -1.03, -0.63 &$
$            -0.95, -0.43$
$            \$\hl$ine$
$        $\hl$ine$
$    \end{tabularx}$
$    \tablefoot{$
$        Redshifts are taken from the NASA Extragalactic Database (NED).$
$        The distances of NGC~3627, NGC~4321, NGC~1808 and NGC~7469 are taken from \cite{2009AJ....138..332J}, \cite{2001ApJ...553...47F}, \cite{Salak2019} and \cite{Izumi2020},$
$        respectively.$
$        The stellar masses, SFRs, M_{\mathrm{H_2}} and M_{\mathrm{H}{\textnormal{$\textsc${i}}}} of NGC~3627 and NGC~4321 are taken from the PHANGS sample table v1.6 of \citet[][PHANGS survey]{Leroy2021phangs}.$
$        The stellar masses and SFRs of NGC~1808 and NGC~7469 are taken from the z0MGS survey (\citealt{Leroy2019}; \url{https://irsa.ipac.caltech.edu/data/WISE/z0MGS/}).$
$        The M_{\mathrm{H_2}} and M_{\mathrm{H}{\textnormal{$\textsc${i}}}} of NGC~7469 are adopted from \cite{Papadopoulos2000}.$
$        The global M_{\mathrm{H}{\textnormal{$\textsc${i}}}} mass in NGC~1808 is measured by \citet{Koribalski1993}.$
$        L_{\mathrm{FIR}} and L_{\mathrm{TIR}} are IR 40--400\mum and 8--1000\mum luminosities all from \citet{Sanders2003}.$
$        The X-ray luminosities for NGC~3627 and NGC~4321 are both L_{0.3\textnormal{--}8\mathrm{keV}} from \citet{2011ApJ...731...60G}, and those for NGC~1808 and NGC~7469 are L_{2\textnormal{--}10\mathrm{keV}} from \citet{2005A&A...442..861J} and \cite{Izumi2020}, respectively.$
$    }$
$\end{table*}$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=\textwidth, trim=0 0 0 0]{Plot_two_line_ratio_four_panel_image_NGC3627_CI10_CO21_with_regions_with_HST}$
$\caption{\textit{Top panels}: NGC~3627 $\Remark${CI10} (\textit{left}) and $\Remark${CO21} (\textit{right}) integrated line intensity (moment-0) maps within our broad mask (representing high-completeness signals; see Appendix~\ref{appendix: data reduction}).$
$\textit{Bottom left panel}: $\Remark${CI10}/$\Remark${CO21} line ratio map within the combined signal mask.$
$\textit{Bottom right panel}: HST F814W-F555W-F438W RGB composite image (PHANGS-HST; \citealt{PHANGSHST}).$
$Regions marked with the white boxes and labeled with numbers are mainly for illustration purpose and for discussion in the text. Their zoom-in images and extracted spectra are newd in our online data release. Examples for Regions~1, 4 and 22 are presented in Appendix~\ref{appendix: all regions}.$
$All images have the same field of view of 179" \times 179", or 9.83 \mathrm{kpc} \times 9.83 \mathrm{kpc}.$
$}$
$\label{fig: NGC3627 line ratio map}$
$\end{figure*}$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=\textwidth, trim=0 0 0 0]{Plot_two_line_ratio_four_panel_image_NGC4321_CI10_CO21_with_regions_with_HST}$
$\caption{$
$Similar to Fig.~\ref{fig: NGC3627 line ratio map}, the $\Remark${CI10} and CO(2--1) moment-0 intensity maps and the line ratio map and the HST composite RGB image for NGC~4321. The field of view is 78" \times 78", or 5.75 \mathrm{kpc} \times 5.75 \mathrm{kpc}.$
$}$
$\label{fig: NGC4321 line ratio map}$
$\end{figure*}$
$\subsection{ALMA observations of the additional targets from the literature}$
$\label{subsection: additional targets}$
$The NGC~1808 ALMA $\Remark${CI10}, $\Remark${CO10} and $\Remark${CO21} observations are presented in \citet{Salak2019} and all are interferometry plus total power, i.e., 12m+7m+tp, thus having no missing flux issue.$
$To study the $\CI$ /$\CO$  line ratio consistently with our main targets, we only used the $\Remark${CI10} and $\Remark${CO21} data.$
$We re-reduced the raw data under ALMA project 2017.1.00984.S (PI: D. Salak) with the observatory calibration pipeline, then imaged, primary-beam-corrected and short-spacing-corrected using the PHANGS-ALMA pipeline in the same way as for our main targets (Appendix~\ref{appendix: data reduction}).$
$Note that in this step we re-reduced and imaged the total power data with the PHANGS-ALMA singledish pipeline (\citealt{Herrera2020,Leroy2021pipeline}) and did the short-spacing correction with the PHANGS-ALMA pipeline right after the primary beam correction.$
$The properties of the final $\CO$ -$\CI$  beam-matched data cubes are listed in Table~\ref{table: targets}.$
$The NGC~7469 ALMA data are presented in \citet{Izumi2020} and we used the archival raw data of $\Remark${CI10} and $\Remark${CO21} under ALMA project 2017.1.00078.S (PI: T. Izumi). These are single pointings toward the centre. $\Remark${CI10} is observed with ACA 7m and $\Remark${CO21} with the 12m array, both without total power. We re-reduced the data following the same procedure as for NGC~3627 and NGC~4321 with the PHANGS-ALMA pipeline.$
$Our re-reduced $\Remark${CI10} and CO(2--1) line intensity and ratio maps for these additional galaxies are shown in Fig.~\ref{fig: other line ratio maps}.$
$\vspace{2ex}$
$\section{Results of the observed line ratios}$
$\label{sec: Results}$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.9\textwidth, trim=0 7.7mm 0 2mm, clip]{Plot_two_line_ratio_two_panel_image_NGC1808_CI10_CO21}$
$\includegraphics[width=0.9\textwidth, trim=0 0 0 2mm, clip]{Plot_two_line_ratio_two_panel_image_NGC7469_CI10_CO21}$
$\caption{$
$\Remark{CI10} (\textit{left}) and \RCICO  (\textit{right}) maps of NGC~1808 in the upper panels and NGC~7469 in the lower panels.$
$Similar to Figs.~\ref{fig: NGC3627 line ratio map} and \ref{fig: NGC4321 line ratio map}, the line intensity maps are newcommandined in the broad mask and the \RCICO  maps are restricted to the combined signal mask.$
$Boxes in the left panels indicate the manually selected central regions where the pixels are highlighted in Fig.~\ref{fig: line ratio vs CO}.$
$Contours in the right panels indicate the 5, 10, 20 and 50-\sigma levels of the left-panel $\CI$  intensity maps.$
$\label{fig: other line ratio maps}$
$}$
$\end{figure*}$
$\subsection{Spatial distributions}$
$In Figs.~\ref{fig: NGC3627 line ratio map}, \ref{fig: NGC4321 line ratio map} and \ref{fig: other line ratio maps}, we present the beam-matched $\Remark${CI10} and $\Remark${CO21} integrated line intensity maps and their ratio maps.$
$The $\CI$/$\CO$  line ratio map is computed only for pixels within the joint strict mask area of the two lines, as described in Appendix~\ref{appendix: data reduction}, representing high-confidence signals.$
$In NGC~3627 (Fig.~\ref{fig: NGC3627 line ratio map}), we visually marked 23 regions based on the $\CI$  and CO commonly-detected (\mathrm{S/N}>3), strict-mask areas for later analysis. Region IDs are sorted by their $\Remark${CI10} brightness.$
$Region 1 corresponds to the galaxy centre, Regions 2 and 3 are the southern and northern bar ends, Region 4 is an offset peak next to the galaxy centre, and other regions are mostly on the spiral arms. Most of these regions show a similar \RCICO \sim 0.1, except for, e.g., Regions 20 and 22 which exhibit a somewhat higher ratio of \RCICO \sim 0.5. The edge of Region 4 also shows an enhancement of $\CI$  yet it is hard to tell whether this is physical or simply caused by edge effects, e.g., as seen in our simulations (Appendix~\ref{appendix: simulation}).$
$We also examined the integrated spectra of each region in Appendix~\ref{appendix: all regions}.$
$The $\CI$  and CO line profiles are not always well-matched in width and shape. In the galaxy centre (Region 1) and at bright bar-ends (Regions 2, 3, 5), $\CI$  and CO have similar line widths but $\CI$  has a lower intensity than CO, whereas in some fainter regions (9, 10, 13, 14), $\CI$  is narrower than CO and relatively weaker. In Regions 4, 20 and 22 where \RCICO is high, $\CI$  and CO have similar line widths but the $\CI$  intensity is enhanced.$
$In NGC~4321 (Fig.~\ref{fig: NGC4321 line ratio map}), we mapped a smaller area due to its fainter CO brightness than NGC~3627. There are no regions with $\Remark${CO21} integrated intensities >100  $\Kkms$ outside our selected area in NGC~4321, whereas in comparison the NGC~3627 bar ends have $\Remark${CO21} integrated intensities \gtrsim 300  $\Kkms$. The $\CI$  mapping in NGC~4321 has a similar depth as in NGC~3627.$
$We marked 7 regions in NGC~4321 to guide the visual inspection. A clear \RCICO  newcommandicit can be seen in the galaxy centre (Region 1), with \RCICO \sim 0.06, nearly a factor of two smaller than its inner disk which has similar \RCICO \sim 0.1 as seen in the NGC~3627 disk. Some $\CI$  enhancement can be found at the edges or slightly outside these regions, where CO and $\CI$  are both faint ($\Remark${ICO21} \sim 30-100  $\Kkms$) and the edge effect of interferometric missing flux might play a role (see Appendix~\ref{appendix: simulation}).$
$The \RCICO  spatial distributions of the other two starburst/AGN galaxies (Fig.~\ref{fig: other line ratio maps}) show in general more enhanced \RCICO  than in the NGC~3627 and NGC~4321 disks.$
$Large variations from the centres to outer areas can be seen, especially at the the S/N threshold of \sim 3 \text{--} 5 (see contours).$
$To quantify the spatial co-localization of the $\CI$  and CO emission, we use the $\incode${JACoP} tool \citep{JACoP} \footnote{\url{https://imagej.net/plugins/jacop}}$
$in the $\incode${ImageJ} software \citep{ImageJ} to measure the commonly used co-localization indicators, namely the Pearson's coefficient (PC; equal to 0 and 1 for null and 100\% localization, respectively) and Manders' coefficients (M1 and M2; which measure the overlap between two images and are equal to one for perfect overlap). We turn on the option of using Costes' automatic threshold \citep{Costes2004} which automatically set a pixel value threshold for each image to minimize the contribution of noise for the comparison \citep{JACoP}.$
$For the $\CI$  and CO emission in our galaxies, we find \mathrm{PC/M1/M2} values all very close to 1, that is, nearly 100\% co-localization (\mathrm{PC/M1/M2} = 0.934/0.985/0.990 for NGC~3627, 0.816/0.986/0.964 for NGC~4321, 0.978/0.890/0.906 for NGC~1808, and 0.935/0.999/0.998 for NGC~7469).$
$Overall, the spatial comparison of the $\CI$  and CO intensities demonstrates that the two species are well-correlated when observed at hundred parsec scales.$
$Meanwhile, we do see certain spectral differences in the line profiles as shown in Appendix~\ref{appendix: all regions}. This may suggest that variations exist at smaller scales than our resolution, which could be the sign of different evolutionary stages \citep[e.g.][]{Kruijssen2014} and would be interesting for higher-resolution follow-ups, e.g., with the main ALMA 12m array.$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=\textwidth]{Plot_Line_Ratio_CI10_CO21_New_surface_density}$
$    \caption{$
$        \textit{Left}: $\Remark${ICI10} versus $\Remark${ICO21} in the four galaxies we studied: NGC~3627, NGC~4321, NGC~1808 and NGC~7469. $\Remark${ICI10} and $\Remark${ICO21} integrated fluxes are moment-0 intensities extracted from their beam and pixel size matched data cubes. Only data points with propagated moment-0 \SNR \ge 5 are shown.$
$        The blue (with crossed hatch), green (with horizontal line hatch), gray (with dotted hatch) and light gray shadings indicate the 5 \sigma thresholds for the four galaxies, respectively. Typical line widths of 10~km~s^{-1} (NGC~3627 and NGC~4321) and 30~km~s^{-1} (NGC~1808 and NGC~7469) in sigma are used to calculate the thresholds together with the channel rms in Table~\ref{table: targets}.$
$        For NGC~3627 and NGC~4321, their (uv-matched) CO(2--1) 5 \sigma thresholds are out of the plotting range.$
$        Top axis shows the simply-converted H_2 surface density using a constant \alphaCO = 4.3 and CO excitation (i.e., CO(2--1) to CO(1--0) line ratio) R_{21} = 0.65.$
$        \textit{Right}: Same data but shown as the line ratio \RCICO = $\Remark${ICI10}/$\Remark${ICO21} versus $\Remark${ICO21}.$
$        The blue horizontal line indicates a star-forming (SF) disk-like \RCICO \sim 0.1, i.e., the spiral arms of NGC~3627 and NGC~4321. The gray horizontal line indicates a starburst (SB) disk-like \RCICO \sim 0.2, i.e., the \sim 1~kpc SB disk/ring in NGC~1808 and NGC~7469.$
$        The red line with a slope of 0.8 in logarithm (0.035   (I_{$\Remark${CO21}}/[100 $\Kkms$])^{0.8}) visually guides the trend of increasing \RCICO in the central regions of these galaxies (see boxes in Figs.~\ref{fig: NGC3627 line ratio map} to \ref{fig: other line ratio maps}, sizes \sim 300--500~pc).$
$    }$
$    \label{fig: line ratio vs CO}$
$\end{figure*}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=0.493\textwidth]{Plot_Histogram_of_Line_Ratio_CI10_CO21}$
$    \includegraphics[width=0.493\textwidth]{Plot_Percentiles_of_Line_Ratio_CI10_CO21}$
$    \caption{$
$    \textit{Left panel} shows the histograms of the sum of CO(2--1) surface brightness over pixels in each $\CI$/$\CO$  line ratio interval in the four galaxies. The bold curves are the cumulative distributions of the corresponding histograms.$
$    \textit{Right panel} shows the mean $\CI$/$\CO$  line ratio for pixels in each bin of CO(2--1) surface brightness (or molecular gas surface density as indicated by the top-axis assuming a constant \alphaCO = 4.3 and R_{21} = 0.65). Shaded area enveloping each mean trend indicates the 16- to 84-th percentiles in each bin, and dashed line indicates the 50-th percentile.$
$    This panel thus more clearly illustrates the ridge and scatter of the trends in the right panel of Fig.~\ref{fig: line ratio vs CO}.$
$    Detection limits are shown consistently with Fig.~\ref{fig: line ratio vs CO}.$
$    }$
$    \label{fig: line ratio histogram}$
$\end{figure*}$
$\subsection{\RCICO versus CO surface brightness and gas surface density}$
$In Fig.~\ref{fig: line ratio vs CO} we show the scatter points of $\CI$  and CO line brightnesses and the \RCICO  line ratios.$
$Systematic differences in \RCICO can be found among these galaxies and within their galactic environments. We discuss these environments separately as follows.$
$\subsubsection{Star-forming and starburst disk environments}$
$The majority of molecular gas in the 3--7~kpc star-forming disks of NGC~3627 and NGC~4321 exhibits a tight distribution of \RCICO around 0.1 (within about \pm50\%). The distribution is fairly flat over an order of magnitude in $\Remark${ICO21}. We denote this as the "SF disk" regime.$
$The ratio rises to about 0.2 within a galactocentric radius of few hundred parsec to \sim 1~kpc starburst areas in NGC~1808 and NGC~7469, which we newcommandine as the "SB disk" regime.$
$The nearly flat "SB disk" distribution is consistent with earlier studies. \citet{Salak2019} found that in NGC~1808 $\Remark${ICI10} is proportional to $\Remark${ICO21}^{ +1.05}. They also studied CO(1--0), finding that $\Remark${ICI10} \propto $\Remark${ICO10}^{ +1.46}, i.e., with a steeper slope.$
$\citet{Saito2020} found a similar slope between $\Remark${ICI10} and $\Remark${ICO10}: $\Remark${ICI10} \propto $\Remark${ICO10}^{ +1.54} in IRAS~F18293-3413 (whose CO(2--1) data is insufficient for such a study as previously mentioned).$
$This correlation slope is likely to change with different CO transitions due to the CO excitation's dependence on the CO luminosity or star formation rate themselves (e.g., \citealt{Daddi2015,Liudz2021a}).$
$For higher-J CO transitions, \citet{Michiyama2021} studied $\CI$  and CO(4--3) lines in 36 (U)LIRGs, finding $\Remark${ICI10} \propto $\Remark${ICO43}^{ +0.97}.$
$The upward tails seen at the CO-faint-end in the right panel of Fig.~\ref{fig: line ratio vs CO} are mostly due to the detection limits in each galaxy. The color shading indicates the 5-\sigma limit for each galaxy, assuming a typical line width of 10~km~s^{-1}. Region~22 of NGC~3627 and the outer pixels in the two starburst/AGN galaxies make the tips of the blue and gray faint-end tails, respectively. But most of these features should still be spurious. We performed CASA simulations following the real observations of NGC~3627 to verify what we would get if \RCICO  were constant across the galaxies (Appendix~\ref{appendix: simulation})$
$We find that noise can indeed boost a simulated \RCICO  of 0.2 to a measurement as high as 0.5 in a NGC~3627-like large mosaic, or even higher to >1 in single-pointing ALMA data without total power or sufficient short spacing. However, the faint-end tail from the simulation is not very similar to that of NGC~3627, raising some questions about the nature of the high-\RCICO  in Region~22.$
$Furthermore, we also did stacking experiments using the CO line profile as the prior to measure the $\CI$  line fluxes in bins of CO line brightnesses or galactocentric radii. We find no systematically higher \RCICO  at fainter CO pixels or at the outer radii.$
$\subsubsection{Galaxy centre environments}$
$Most interestingly, the \lesssim 250~pc centres of all four galaxies show dramatically different \RCICO. It varies by a factor of ten from \sim 0.05 in the centre of NGC~4321  to \sim 0.1 in the centre of NGC~3627, then to \sim 0.15--0.2 at the centre of NGC~1808, and to \sim 0.2--0.5 in the nucleus of NGC~7469.$
$This "nuclei" trend has a slope of about 0.8, i.e., $\Remark${ICI10} \approx 0.035 \; $\Remark${ICO21}^{ +0.8}, and corresponds to a \Sigmol range of about 10^{3} to 10^{4} \; $\Mspc$2 if adopting an \alphaCO = 4.3  \mathrm{M_{\odot} \; (K   km   s^{-1}   pc^2)^{-1}} (\citealt{Bolatto2013}) and a CO excitation R_{21} = 0.65 (\citealt{Leroy2021c}).$
$The reasons for the nuclei showing this trend likely include both the effect of varying [$\CI$/CO] abundance ratio and the excitation and radiative transfer. They are determined by the actual ISM conditions in each environment, i.e., the gas kinetic temperature, $\CI$  and CO column densities ($\CI$columndensity and $\CO$columndensity), line widths (\Delta{v}), and \mathrm{H_2} volume density (\nHtwo), etc.$
$They will be analyzed in Sect.~\ref{sec: Excitation Analysis}.$
$\subsubsection{Histogram and curve-of-growth of the line ratio distributions}$
$Fig.~\ref{fig: line ratio histogram} shows the histogram and cumulative distribution functions of \RCICO (left panel) and the mean \RCICO and the 16-th to 84-th percentiles in bins of CO line intensity (right panel) in our galaxies.$
$The histogram is computed as the sum of CO line intensity in each bin of \RCICO, representing the amount of gas at each \RCICO.$
$The NGC~4321 histogram is lower than others, showing that it has the least total molecular gas mass among the four galaxies within mapped areas. The NGC~3627 histogram has a peak about 0.5~dex higher than that of NGC~4321, but their peak locations are consistently at \log \RCICO \sim -1, same conclusion as our previous scatter plots show.$
$The two SB galaxies NGC~1808 and NGC~7469, despite having much smaller observed areas (\sim 1--2~kpc), have more than an order of magnitude higher peaks hence total molecular gas masses.$
$The locations of their histogram peaks also shift to a systematically higher \RCICO.$
$It is clearly seen that both SF and SB disks lack a high-\RCICO (\log \RCICO > -0.35) gas component which dominates the gas in the strong AGN host galaxy NGC~7469.$
$The curve-of-growth in the left panel of Fig.~\ref{fig: line ratio histogram} further shows the cumulative CO brightness in bins of \RCICO, and reflects the fact that the majority of gas in our galactic disks has a narrow distribution of \RCICO (see also last few rows in Table~\ref{table: targets}).$
$In NGC~3627, NGC~4321 and NGC~1808, the higher- and lower-\RCICO gas outside \pm 0.1~dex around the mean \RCICO only sums up to <5--10\% of their total gas masses.$
$In the right panel of Fig.~\ref{fig: line ratio histogram}, we illustrate how the mean \RCICO changes with different gas surface density, as well as the scatter of the data point distribution. The 2D pixels are binned by CO brightness, and the mean and 16\%/50\%/84\%-th percentiles of the \RCICO are computed for each bin.$
$It sketches out the mean trends in Fig.~\ref{fig: line ratio vs CO} (right panel), and also reveals that$
$a) the scatter of \RCICO is the largest (\pm 0.20~dex) in the starburst disk of NGC~7469 compared to other galaxies;$
$b) the scatter is the smallest (\pm 0.06~dex) in the star-forming disks of NGC~4321 and NGC~3627 where \Sigmol \sim 100 \text{--} 500 \; $\Mspc$2;$
$c) the scatters are very similarly (\pm 0.10~dex) at a relatively high \Sigmol \sim 1000 \; $\Mspc$2 in all our galaxies except for NGC~4321's centre.$
$\subsection{\RCICO versus ISRF}$
$\label{subsection: ISRF}$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.9\textwidth]{Plot_CI_CO_line_ratio_vs_IRSF_intensity_maps}$
$\caption{$
$    Line ratio versus mean ISRF intensity \Umean measured from SED fitting to the near- to far-IR (\textit{Spitzer}, \textit{Herschel}) data.$
$    \textit{Left four panels} are the \Umean maps of NGC~3627, NGC~4321, NGC~1808 and NGC~7469, and as described in Sect.~\ref{subsection: ISRF} are obtained from the works done by J. Chastenet et al. (in prep.) as a continued effort of \citet{Chastenet2021}.$
$    \textit{Right panel} shows the scatter plot of \RCICO versus \Umean at a spatial resolution of \sim 18" in the manually-selected apertures shown in the left panels.$
$    We computed a beam-weighted mean \RCICO and a centre pixel \RCICO for each aperture, displayed as the solid and open symbols, respectively.$
$    Error bars are the corresponding beam-weighted mean or central pixel \RCICO  uncertainties in each region.$
$    The region diameter (\sim 18") corresponds to the coarsest resolution of the \textit{Herschel} data used in the SED fitting.$
$    The gray dotted line shows \RCICO = 0.07   U^{0.25} as a guiding line.$
$    The colored dotted contours (labeled 0.1--0.5) are the model contours of \RCICO computed from the PDR Toolbox (using the latest $\incode${wk2020} model and solar metallicity; \citealt{Kaufman2006}). We convert G_0 from the model contour to the x-axis U by multiplying with a factor of 0.88 (\citealt{Draine2007}).$
$    The right y-axis indicates the PDR model's \nHtwo, whose range is manually adjusted so that the model contours roughly match the left x-axis.$
$    The PDR model contours show little dependence of \RCICO on U in the horizontal direction but a strong correlation with \nHtwo as seen by the vertical gradient.$
$\label{fig: line ratio vs ISRF}$
$}\vspace{1ex}$
$\end{figure*}$
$Here we study how \RCICO  correlates with the interstellar radiation field (ISRF) intensity (U; in units of Milky Way mean ISRF; \citealt{Habing1968}), which is a measure of the UV radiation field strength impacting the ISM. Since $\CI$  originates from PDRs, its abundance should be highly related to the PDR properties, thus whether or how \RCICO  correlates with the ISRF that illuminates the PDR is a key question.$
$The ISRF intensity U is usually measured by the re-radiated dust emission, as the original UV photons are highly attenuated by dust in the ISM. \citet[][hereafter DL07]{DL07} developed a series of dust grain models and synthesized SED templates that can describe nearby galaxies' observed dust SEDs. A number of studies (e.g., \citealt{Draine2007,Galliano2011,Dale2012,Aniano2012,Aniano2020,Daddi2015,Chastenet2017,Liudz2021a,Chastenet2021}) have used the DL07 models to fit the dust SEDs of galaxies near and far.$
$For our study, we adopt the mass-averaged ISRF \Umean maps from an ongoing effort by J. Chastenet et al. (in prep.) which is a continuation of the work published for other galaxies by \citet{Chastenet2017,Chastenet2021} with the same method.$
$In this method, the \textit{Herschel} far-infrared, \textit{Spitzer} and/or \textit{WISE} near-infrared images \footnote{For the two (U)LIRGs, the WISE maps are saturated, so their inferred ISRF intensity U should be considered with a large uncertainty.} are first PSF-matched to a common resolution \sim 18" (\textit{Herschel} SPIRE 250\mum), then SED fitting is performed using the DL07 models at each pixel.$
$Each dust SED is fitted by two dust components, one in the ambient ISM with a constant ISRF intensity U_{\mathrm{min}}, and the other in regions with higher U with a power law distributed ISRF from U_{\mathrm{min}} up to U_{\mathrm{max}} = 10^{7}.$
$The fitting then gives the best-fit U_{\mathrm{min}} and the power-law index \gamma of the U distribution.$
$The mean ISRF intensity \Umean is calculated as the dust mass-weighted mean of the ambient and power-law ISRF intensities (in DL07, U_{\rm PDR} \ge 10^2 is considered to be associated with the PDR), and better represents the overall ISRF than U_{\mathrm{min}}.$
$Fig.~\ref{fig: line ratio vs ISRF} presents the \Umean maps and the \RCICO  versus \Umean scatter plot. Since the spatial resolution of the \Umean maps is 18", we calculate a weighted \RCICO in each circular aperture shown in Fig.~\ref{fig: line ratio vs ISRF} whose diameter equals the \Umean map resolution, with weighting following the 2D Gaussian beam.$
$In addition, we also show the central pixel value for each region in Fig.~\ref{fig: line ratio vs ISRF}.$
$We restrict the analysis to regions where both \Umean and \RCICO  are measured.$
$A weak correlation between \RCICO  and \Umean is seen from the SF to SB disks, with a large scatter. For reference, we plot a \RCICO = 0.07   U^{0.25} line to indicate this weak trend. The lowest-\Umean data point in NGC~3627 (mainly Region~22) and the centre of NGC~7469 do not follow this trend, possibly indicating other $\CI$  enhancement mechanisms than \Umean.$
$We use the popular PDR modelling software, PDR Toolbox (version 2.2.9; \citealt{Kaufman2006,Pound2008,pdrtoolbox}) \footnote{\url{https://dustem.astro.umd.edu/}; \url{https://github.com/mpound/pdrtpy}.}$
$to compute the theoretical \RCICO in uniform grids of ISRF intensity in the \citet{Habing1968} G_0 unit and H nucleus density n_{\mathrm{H}} (both in log space). We adopt the latest $\incode${wk2020} model set and a solar metallicity (\mathrm{[O/H_2]} = 3.2 \times 10^{-4}, \mathrm{[C/H_2]} = 1.6 \times 10^{-4}).$
$The ISRF intensity grid can be directly scaled to U by multiplying a factor 0.88 (\citealt{Draine2007}), corresponding to the x-axis of Fig.~\ref{fig: line ratio vs ISRF}.$
$The n_{\mathrm{H}} grid is not directly linked to the left y-axis of Fig.~\ref{fig: line ratio vs ISRF}, but because the model contours show fairly flat \RCICO versus U trends, we can match the model contours and indicate the n_{\mathrm{H}} grid in the right y-axis of Fig.~\ref{fig: line ratio vs ISRF}.$
$The PDR modelling predicts generally flat \RCICO versus U trends similar to the distribution of the observed data points, although our U are diluted values at a much larger physical scale than in the PDR models.$
$However, in PDR models, \RCICO  changes moderately with n_{\mathrm{H}}, and a relatively high n_{\mathrm{H}} \sim 10^{3.5\text{--}3.7}   \mathrm{cm^{-3}} is needed to reproduce an \RCICO \sim 0.1--0.2, or equivalently \sim 1.0--1.9 for the ratio of $\Remark${CI10} and $\Remark${CO21} fluxes in \mathrm{erg s^{-1} cm^{-2} sr^{-1}} units used by the PDR Toolbox \footnote{See \url{https://github.com/mpound/pdrtpy-nb/blob/master/notebooks/PDRT_Example_Model_Plotting.ipynb}.$
$We scale the ratio of intensities in \mathrm{erg s^{-1} cm^{-2} sr^{-1}} to the ratio of intensities in \mathrm{K km s^{-1}} by a factor of (\nu_{\mathrm{CI10}}/\nu_{\mathrm{CO21}})^{-3} = 9.73, where \nu is the rest-frame frequency of the lines. We caution that the old version of $\incode${PDRToolbox} $\incode${v2.1.1} does not include the $\incode${CI_609/CO_21} line ratio, but has $\incode${CI_609/CO_43} and $\incode${CO_43/CO_21} ratios. However, we tested that the multiplication of $\incode${CI_609/CO_43} and $\incode${CO_43/CO_21} does not produce the same result as the direct $\incode${CI_609/CO_21} ratio in the new $\incode${PDRToolbox} $\incode${v2.2.9}. We use the new version for this work.$
$}.$
$Furthermore, the PDR model needs a \sim0.5~dex lower n_{\mathrm{H}} to explain the 2\times elevated \RCICO  in SB disks, which seems somewhat counterintuitive.$
$This apparent tension is likely caused by the observations probing a much coarser resolution that consists of a large number of PDRs with different n_{\mathrm{H}} and U (e.g., increasing U from 10 to 400 can increase \RCICO from 0.2 to 0.5 based on our PDR Toolbox modelling). It may also relate to an abundance variation driven by non-PDR mechanisms ---$
$X-ray dominated region (XDR; \citealt{Maloney1996,Meijerink2005,Meijerink2007}; see also review by \citealt{Wolfire2022}) and Cosmic Ray Dominated Region (CRDR; \citealt{Papadopoulos2010c,Papadopoulos2011,Papadopoulos2018,Bisbas2015,Bisbas2017}) are well-known to play an important role in starbursting and AGN environments, respectively.$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=0.9\textwidth]{Plot_CI_CO_line_ratio_vs_metallicity_maps}$
$\caption{$
$    \textit{Left two panels} show the metallicity maps of NGC~3627 and NGC~4321 from \citet{Williams2022} based on PHANGS-MUSE (\citealt{PHANGSMUSE}) data. Boxes are regions as newcommandined in Figs.~\ref{fig: NGC3627 line ratio map} and \ref{fig: NGC4321 line ratio map}.$
$    \textit{Right panel} shows the \RCICO  versus metallicity scatter plot for regions in NGC~3627 and NGC~4321. The bottom-axis is metallicity in solar units, top-axis is metallicity in 12+\log(\mathrm{O/H}), left-axis is the line ratio of $\Remark${CI10} and $\Remark${CO21} in $\Kkms$ units, and right-axis is the ratio of fluxes in \mathrm{erg s^{-1} cm^{-2} sr^{-1}} units.$
$    The left- and right-axes are matched by a factor of (\nu_{\mathrm{C{$\textsc${i}}1\textnormal{--}0}}/\nu_{\mathrm{CO1\textnormal{--}0}})^3 \times R_{21}, where we adopt a R_{21}=0.65 (\citealt{Leroy2021c}).$
$    Data points are the mean metallicity and \RCICO  in each region shown in left panels, with error bars indicating the typical uncertainty in \RCICO , and vertical color shading indicating the 16- to 84-th percentiles of pixels' \RCICO  distribution inside each region.$
$    The \citet{Bolatto2000b} empirical fitting and \citet{Bolatto1999} model prediction lines are overlaid.$
$    The three outliers in NGC~3627 with \RCICO \sim 0.19, 0.28 and 0.35 are Regions 20, 21 and 22, respectively. The outlier point in NGC~4321 with \RCICO \sim 0.06 is its centre Region 1.$
$    \label{fig: line ratio vs metallicity}$
$}\vspace{1ex}$
$\end{figure*}$
$As pointed out by \citet{Papadopoulos2011} and later works, ultra-luminous infrared galaxies (ULIRGs) with IR luminosities L_{\mathrm{IR, 8-1000\mu\mathrm{m}}} \ge 10^{12}   $\Lsun$ and corresponding \mathrm{SFRs} > 1000   $\Msyr$ have cosmic-ray (CR) energy densities \sim 10^{3} \text{--} 10^{4}\times larger than the Milky Way. In our sample, the two SB galaxies are not as intensively star-forming as ULIRGs, but their SFRs are still a factor of 2 to 20 higher than our SF galaxies. More importantly, the SB disks have considerably higher (e.g., one to two orders of magnitude) SFR surface densities than the SF disks, plausibly leading to a factor of a few tens to hundreds higher CR intensities. The CRs emitted from OB star clusters and supernova remnants can penetrate deeply into the ISM, and thus nearly uniformly illuminate the ISM and effectively dissociate CO into $\CI$ .$
$A detailed simulation by \citet{Bisbas2017} showed that 10\times and 100\times higher CR ionization rates (\xi_{\mathrm{CR}}) than the Milky Way level results into a factor of about 2 and 3 mildly-increased $\CI$  abundance and about 1/2 and 1/30 strongly-decreased CO abundance, respectively (see their Fig.~3), in a 10~pc GMC with a mean gas density \left<n_{\mathrm{H}} \right>= 760   \mathrm{cm^{-3}}, resembling Milky Way and typical SF galaxies' ISM conditions.$
$Our SB galaxies probably have a \xi_{\mathrm{CR}} around or slightly above 10, but not reaching 100 given their mild \Umean \sim 20-30 compared to the \Umean \gtrsim 50 in extreme SB galaxies, e.g., local ULIRGs and high-redshift hyperluminous IR-bright (HyLIRG) and submm galaxies (\citealt{Daddi2015,Silverman2018,Liudz2021a}).$
$In Sect.~\ref{sec: Excitation Analysis}, we show that the mildly-increased [$\CI$/CO] abundance ratio from a CRDR and temperature/density-driven excitation can lead to the factor of 2 increase in \RCICO  when going from SF to SB disks.$
$\subsection{\RCICO versus Metallicity}$
$\label{subsec: metallicity}$
$Our current sample covers a relatively small range in metallicity for a metallicity--line ratio study. Given their stellar masses and metallicity maps (e.g., \citealt{Kreckel2019,Kreckel2020,Williams2022}), our targets probe a near-solar metallicity. NGC~3627 has a well-measured$
$metallicity gradient of Z = 8.53 + 0.06 \times[r/R_{25}] from \citet{Kreckel2019} \footnote{\citet{Kreckel2019} used a distance of 10.6 \pm 0.9 \; \mathrm{Mpc} for NGC~3627. We do not directly use this metallicity gradient, but use the metallicity maps from \citet{Williams2022} where the same new distance as in this work is adopted.}, but the metallicity of individual $\ion$izedhydrogen  regions scatter between \sim 8.45--8.65.$
$\citet{Williams2022} produced the 2D spatial distributions of gas-phase metallicity in PHANGS-MUSE galaxies, including NGC~3627 and NGC~4321, by applying the Gaussian Process Regression to map the smooth, higher-order metallicity variation from the individual, sparse $\ion$izedhydrogen  regions detected in PHANGS-MUSE (\citealt{PHANGSMUSE, Santoro2021}).$
$This method is significantly superior to a na\"{i}ve nearest neighbor interpolation method. We refer the reader to \citet{Williams2022} for the technical details and validation demonstration.$
$In Fig.~\ref{fig: line ratio vs metallicity} we present the trend between \RCICO  and metallicity in NGC~3627 and NGC~4321.$
$Due to our limited metallicity range, only a very weak decreasing trend can be seen.$
$A similar trend has been reported by \citet{Bolatto2000b}, who observed CO(1--0) and $\Remark${CI10} in the Region N27 of the Small Magellanic Cloud and combined with literature data to present a correlation between \RCICO  and metallicity over \sim 0.2 \text{--} 2   Z_{\odot}.$
$We overlay the \citet{Bolatto2000b} empirical fitting \footnote{$
$We adopt the equation from \citep[][Fig.~2 caption]{Bolatto2000b}, with a typo-correction in the brackets so that the equations are:$
$\log ( F_{$\Remark${CI10}} / F_{$\Remark${CO10}} ) = -0.47   \left[12 + \log \mathrm{(O/H)} \right]+ 5.0 and \log ( F_{$\Remark${CI10}} / F_{$\Remark${CO10}} ) = -0.82   \left[12 + \log \mathrm{(O/H)} \right]+ 8.2, respectively, where F is the line flux in \mathrm{erg   s^{-1}   cm^{-2}   sr^{-1}}.$
$}$
$as well as a theoretical PDR model prediction line from \citet{Bolatto1999} in Fig.~\ref{fig: line ratio vs metallicity}.$
$Although our data covers only a small metallicity range, it is encouraging to see the overall good agreement with the observationally-driven trend in \citet{Bolatto2000b}.$
$More detailed understanding of the deviation from PDR models will likely need more $\CI$  mapping in lower-metallicity galaxies.$
$\vspace{2ex}$
$\section{Estimation of abundance ratios under representative ISM conditions}$
$\label{sec: Excitation Analysis}$
$The observed \RCICO  depends on not only the $\CI$  and CO abundances ($\CO$abundance  and $\CI$abundance ), but also the ISM conditions that determine the excitation (level population) and radiative transfer of the C atoms and CO molecules --- i.e., the gas kinetic temperature \Tkin, volume density of H_2 (\nHtwo; as the primary collision partners of C and CO), turbulent line width \dv, and the species' column densities $\CO$columndensity and $\CI$columndensity.$
$The column density divided by the line width (N/\dv) further determines the optical depth \tau of each line.$
$In this work, having only a CO(2--1) and a $\Remark${CI10} line is insufficient to numerically solve all the \Tkin, \nHtwo, $\CO$columndensity/\dv, $\CO$abundance  and $\CI$abundance . Therefore, we assume several \textit{representative ISM conditions} where \Tkin, \dv and $\CO$abundance  are fixed to reasonable values, then solve the $\CO$columndensity and $\CI$columndensity. We assume that $\CI$  and CO are spatially mixed at our resolution, i.e., they have the same filling factor. This is a relatively strong assumption and is likely not the real case, because although we see spatial co-localization at our \sim 200~pc resolutions, the spectral profiles of CO and $\CI$ show discrepancies (Appendix~\ref{appendix: all regions}), indicating different underlying distributions at smaller scales. With the assumption of the same filling factor (and 3D distribution), the column density ratio $\CI$columndensity/$\CO$columndensity equals the [$\CI$/CO] abundance ratio.$
$We perform both LTE and non-LTE calculations.$
$LTE assumes that all transitions are thermalized locally, and the level population follows Boltzmann statistics, so that each transition's excitation temperature T_{\mathrm{ex}} as newcommandined by the level population equals \Tkin (e.g., \citealt{Spitzer1998Book,Tielens2010Book,Draine2011Book}). The LTE assumption facilitates the computation as we do not need to solve the level population and thus \nHtwo is not used.$
$However, LTE is not easily reached in typical ISM conditions. Subthermalized lines need non-LTE calculations. The commonly adopted non-LTE approach is the Large Velocity Gradient (LVG; \citealt{Scoville1974,Goldreich1974}), where an escape fraction is used to facilitate the local radiation intensity calculation hence the detailed balance of radiative and collisional (de-)excitations.$
$We use RADEX (\citealt{RADEX}) to compute the non-LTE $\CI$  and CO line fluxes for each fixed parameter set (\Tkin, \nHtwo, $\CO$columndensity, $\CI$columndensity, \dv), then study how [$\CI$/CO] determines the line flux ratio \RCICO.$
$We assume the following 7 representative ISM conditions:$
$\textit{(1)} inner Galactic GMCs,$
$\textit{(2)} NGC~3627/4321 disk GMCs,$
$\textit{(3)} NGC~3627 bar-end environments,$
$\textit{(4)} NGC~1808/7469 SB disk,$
$\textit{(5)} NGC~4321 centre,$
$\textit{(6)} NGC~3627 centre,$
$and \textit{(7)} NGC~7469 centre.$
$Conditions \textit{(1)}--\textit{(3)} correspond to the "SF disk" in Fig.~\ref{fig: line ratio vs CO}, and condition \textit{(4)} represents the "SB disk".$
$The last three conditions are for the "nuclei".$
$Under each ISM condition, we set a fixed \Tkin (in units of K), \dv (in units of km~s^{-1}), $\CO$abundance , and \Sigmol (in units of $\Mspc$2).$
$The adopted values are shown at the top of each panel in Figs.~\ref{fig: LTE RCICO} and \ref{fig: NonLTE RCICO}.$
$Our choice of \Tkin considered multi-line measurements in the literature, e.g., \citet{Heyer2015} for Galactic GMC ISM condition, and \citet{Teng2022} for NGC~3627 nucleus, as well as typical values in simulations (e.g., \citealt{Offner2014,Glover2015,Glover2016,Clark2019,HuChiaYu2021a}).$
$Our \dv is based on the observed line width in the highest-resolution ALMA data, i.e., from the original PHANGS-ALMA 1" data (\citealt{Leroy2021phangs}) or from \citet{Salak2019} and \citet{Izumi2020} for NGC~1808 and NGC~7469, respectively. The observed line width should at least new an upper limit, if not directly probing the turbulent \dv.$
$The $\CO$abundance  is more like a fiducial value and does not obviously affect later [$\CI$/CO] results.$
$We also take a fiducial \Sigmol which matches the observed CO(1--0) line brightness temperature and a CO-to-H_2 conversion factor \alphaCO \sim 4 \text{--} 0.8 \; \mathrm{M_{\odot}   (K   km   s^{-1})^{-1}} depending on the assumed conditions (e.g., \citealt{Bolatto2013}; and our paper~II).$
$This \Sigmol directly converts to an N_{\mathrm{H_2}} if assuming that 36\% of the mass is helium. Then N_{\mathrm{H_2}} and $\CO$abundance  together indicate a $\CO$columndensity . Only with these assumptions can we unambiguously link an abundance ratio [$\CI$/CO] (i.e., $\CI$columndensity/$\CO$columndensity) to a line flux ratio \RCICO .$
$Additionally, with the same filling factor assumption, \RCICOis also equivalent to the ratio of the CO-to-H_2 and $\CI$ -to-H_2 conversion factors, i.e., \RCICO = \alphaCO/\alphaCI, because they trace the same molecular gas at our resolution.$
$In a companion paper (paper II; Liu et al. 2022 in prep.), we study the behaviors of \alphaCO and \alphaCI in more details, also under both LTE and non-LTE conditions.$
$\subsection{LTE results}$
$\label{subsec: LTE}$
$\begin{figure*}[htb]$
$    \centering$
$    \includegraphics[width=0.99\textwidth]{Plot_LTE_RCI10CO21_vs_XCICO_varying_N_CO_T_kin}$
$    \caption{$
$        Observed/calculated \RCICO as a function of CO column density $\CO$columndensity  and [$\CI$/CO] abundance (column density) ratio \RNCINCO for 7 representative ISM conditions, based on LTE calculations.$
$        \textit{Upper panels}: Curved lines are \RCICO versus \RNCINCO for different $\CO$columndensity  as indicated by the color. Dashed means CO(2--1) optically thick and $\Remark${CI10} optically thin, dotted means both are both optically thin, and solid line means both optically thick.$
$        Horizontal thick line is the observed \RCICO value.$
$        \textit{Lower panels}: Black contours are \RCICO = 0.05,   0.10,   0.20 and 0.50 in the CO column density versus $\CI$/CO abundance ratio plane.$
$        Vertical color shading indicates the final \RNCINCO that can match both the observed \RCICOand the assumed \Sigmol value noted at the top of each panel.$
$    }$
$    \label{fig: LTE RCICO}$
$\end{figure*}$
$We determine the LTE solution of [$\CI$/CO] in Fig.~\ref{fig: LTE RCICO}.$
$To illustrate the complexity of this problem and the degeneracy of the \Tkin, $\CO$columndensity/\dv and [$\CI$/CO] parameters, we show how \RCICO (y-axis) varies with \RNCINCO (x-axis) and $\CO$columndensity  (color bar) in the upper panels, and how \RCICO (contour) can be pinpointed by determining the \Sigmol (right y-axis).$
$At the top of each column the key parameters of the aforementioned representative ISM conditions are shown as text. The assumed \Tkin and \dv generally increase from panels (1) to (7) mimicking real conditions. Observational-driven $\Remark${ICO10} ranges and the assumed \Sigmol for these ISM environments are also listed, and can be converted one into the other via \alphaCO. We also list the chosen $\CO$abundance  for clarity.$
$As shown in the upper panels, \RCICO increases with both \RNCINCO and $\CO$columndensity  under low-\Tkin conditions. However, a maximum \RCICO is reached in the high $\CI$  column density regime, where an increasing abundance ratio will no longer increase \RCICO. The line ratio saturates at about 0.5 when \Tkin \sim 15~K, or at about 1 for \Tkin \sim 100~K. The observed line ratios (horizontal dashed lines) are all lower than the saturation limit of \RCICO _{\mathrm{max}} \sim 0.7 \text{--} 0.9, and corresponds to a range of $\CO$columndensity or \Sigmol. This range is very wide in the low-\Tkin conditions, with \log $\CO$columndensity / \mathrm{cm^{-2}} \sim 17 - 19, but is very narrow at \Tkin \sim 100~K, that is, \RCICO can be determined from [$\CI$/CO] without much dependence on $\CO$columndensity.$
$In the lower panels, the contours of \RCICO = 0.05, 0.1, 0.2 and 0.5 are shown in the plane of CO column density $\CO$columndensity versus $\CI$/CO abundance ratio.$
$For a fixed \RCICO , a higher \RNCINCO corresponds to a lower $\CO$columndensity in the low-\Tkin regime, but \RCICO  becomes insensitive to $\CO$columndensity at a high \Tkin. In the latter case, \RNCINCO alone determines \RCICO .$
$By matching to the expected \Sigmol for each ISM condition, we can obtain an unambiguous \RNCINCO from the observed \RCICO  as shown in Fig.~\ref{fig: LTE RCICO}.$
$In this way, we determine the \RNCINCO to be around 0.1--0.2 for conditions \textit{(1)} and \textit{(2)}, and \sim 0.4 for conditions \textit{(3)} and \textit{(5)}.$
$The SB disk (condition \textit{4}) and AGN environments (conditions \textit{6}/\textit{7}) have \RNCINCO \sim 2 and \sim 1 \text{--} 10, respectively.$
$The strongly enhanced $\CI$/CO abundance ratios in SB disk and AGN environments are clear signatures of CRDR and XDR, respectively.$
$The NGC~4321 centre is a low-ionization nuclear emission-line region (LINER, e.g., \citealt{GarciaBurillo2005}), and given our assumptions it requires \RNCINCO \sim 0.4 in order to explain the observed low line ratio.$
$The gradually increasing $\CI$/CO abundance ratios in the nuclei of NGC~4321, NGC~3627, NGC~1808 and NGC~7469 agree well with the increasing power of AGN activities, e.g., their X-ray luminosities (Table~\ref{table: targets}).$
$Our assumptions for the Galactic cloud ISM condition are mainly based on \citet{Heyer2009}, \citet{Heyer2015} and \cite{RomanDuval2010}. The obtained [$\CI$/CO]~\sim 0.1 also agrees well with previous studies using $\CI$  and CO isotopologues (see Sect.~\ref{subsec: intro RCICO}.$
$For example, \citet{Ikeda1999} reported \RNCINCO \sim 0.05 \textnormal{--} 0.2 for the Orion cloud;$
$\citet{Oka2001} found \RNCINCO = 0.064 \pm 0.035 but a peak of \RNCINCO \sim 0.15 in the DR 15 $\HII$  region and two IR dark clouds;$
$\citet{Kramer2004} found \RNCINCO \sim 0.11 \textnormal{--} 0.12 in the massive SF region W3;$
$\citet{Genzel1988} found \RNCINCO \sim 0.2 for the massive SF region W51;$
$\citet{Zmuidzinas1988} reported \RNCINCO \sim 0.05 \textnormal{--} 0.15 in several dense clouds.$
$Recently, \citet{Izumi2021} derived an overall \RNCINCO \sim 0.1 in the massive SF region RCW38 at A_V \sim 10 \textnormal{--} 100, consistent with the Orion cloud, but also saw high [$\CI$/CO] \sim 0.2 \textnormal{--} 0.6 in some even lower column density regions.$
$Accurate determination of all the ISM properties instead of simple assumptions as in this work requires comprehensive, multi-J CO isotopologue observations (e.g., \citealt{Teng2022}).$
$Our representative ISM conditions only new a qualitative diagnostic to understand how to infer the $\CI$/CO abundance ratio from the observed \RCICO  quantity.$
$Their properties may not be very accurate, and are subject to change with future multi-tracer/CO isotopologue studies.$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=\textwidth]{Plot_RADEX_R_CI10CO21_sim_XCICO_NonLTE_varying_N_CO_T_kin_n_H2}$
$\caption{$
$Similar to Fig.~\ref{fig: LTE RCICO} but with non-LTE calculations.$
$See Fig.~\ref{fig: LTE RCICO} caption for the details.$
$}$
$\label{fig: NonLTE RCICO}$
$\end{figure*}$
$\subsection{Non-LTE results}$
$\label{subsec: non-LTE}$
$We perform the non-LTE calculations using the RADEX code (\citealt{vanderTak2007})$
$with the Large Velocity Gradient (LVG; i.e., expanding sphere) escape probability (\citealt{Scoville1974,Goldreich1974}).$
$Comparing to LTE, the volume density of H_2 is now an additional property that plays a role. The excitation temperature of each line no longer equals \Tkin.$
$We compute the non-LTE line ratios in a grid of ( \Tkin,   \dv,   $\CO$abundance,   \RNCINCO,   $\CO$columndensity,   \nHtwo ) parameter sets, then study the line and abundance ratios similarly as in the previous section.$
$In Fig.~\ref{fig: NonLTE RCICO} we show the non-LTE version of Fig.~\ref{fig: LTE RCICO}.$
$The non-LTE result does not qualitatively differ from that of LTE.$
$The most obvious difference is that at high temperature (\Tkin > 50~K, given our other assumed conditions), the required [$\CI$/CO] is lower by \times 2 for the same observed \RCICO, whereas at low temperature (\Tkin \sim 15~K) a \lesssim 2 \times [$\CI$/CO] is expected for the observed line ratio.$
$As listed in the title of each panel, we assume \log \nHtwo / \mathrm{cm^{-3}} = 2.5 to 3.0 for our conditions (1) to (7).$
$This is based on the fact that low-J CO and $\CI$  trace the bulk of molecular gas with \log \nHtwo / \mathrm{cm^{-3}} \sim 2--3 (e.g., \citealt{Leroy2017,Liudz2021a}), with a typical value \nHtwo \sim 300   \mathrm{cm^{-3}} widely adopted/found for Galactic and nearby galaxy GMCs (e.g., \citealt{Koda2012,Pety2017}, see also simulations of \citealt{Glover2012b}), whereas a smaller portion of their emission is from the dense gas with \log \nHtwo / \mathrm{cm^{-3}} \sim 4--6 (e.g., \citealt{Gao2004a}).$
$The fraction of the dense gas is expected to increase in more starbursting environment (\citealt{Gao2004b}), so does the mean gas density (\citealt{Liudz2021a}). Therefore, a \log \nHtwo / \mathrm{cm^{-3}} of 3.0 is perhaps a reasonable assumption, or at least a lower limit for our galaxy centres. A 10 \times higher density does not qualitatively change Fig.~\ref{fig: NonLTE RCICO} except for shifting the expected \RNCINCO to be 50\% higher in the \Tkin > 50~K conditions (making Fig.~\ref{fig: NonLTE RCICO} more like Fig.~\ref{fig: LTE RCICO}).$
$Therefore, our non-LTE result strongly confirms the LTE result in the previous section, that the intrinsic $\CI$/CO abundance has to vary significantly from about 0.1--0.2 to about 1--3, in order to explain the observed line ratios from Galactic GMCs (hundred pc scales) and NGC~3627/4321 SF disk GMCs to NGC~1808/7469 SB disk/AGN (r \lesssim 250~pc) environments.$
$\vspace{2ex}$
$\section{Discussion}$
$\label{sec: Discussion}$
$From this study we see that the line ratio \RCICO increases from 0.1 to 0.2 and > 0.5 from the SF disk to the SB disk and the vicinity (few hundred pc) of X-ray luminous AGN. Although we may be limited by our sample,$
$we suggest that this trend can be extended to general cases at both local and high redshift, because the $\CI$/CO abundance ratio enhancement is well-predicted by the CRDR and XDR theories.$
$\citet{Papadopoulos2010c} and \citet{Papadopoulos2011} pointed out the key role of CRDR in local (U)LIRGs. With UV photons alone, cold gas in starbursting environments can still be well shielded, hence as cold as \sim 10~K. However, with CRs produced from O, B star clusters and supernova remnants, the gas can be uniformly illuminated and heated up.$
$The extremely starbursty local ULIRGs have a CR density (\xi_{\mathrm{CR}}) 10^{3}--10^{4} times that of the Milky Way (\citealt{Papadopoulos2010c}), and thus will naturally dissociate more CO into $\CI$ .$
$\citet{Bisbas2017} and \citet{Papadopoulos2018} conducted simulations of molecular gas at tens of pc scale with varying CR strength, and found that when \xi_{\mathrm{CR}} is \sim 1 \textnormal{--} 100   \times Galactic value, the gas temperature in the dense gas remains largely unaffected (\sim 10~K).$
$However, when \xi_{\mathrm{CR}} reaches \sim 1000   \times Galactic value as in (U)LIRGs, the gas temperature increases to 30--50~K. The \citet{Bisbas2017} simulation also shows that the $\CI$  and $\ion$izedcarbon  fractional abundances at \log A_V \sim 0.5 increase by one and three orders of magnitude, respectively, and the CO fractional abundance decreases correspondingly by over two orders of magnitude, with an increasing \xi_{\mathrm{CR}} from the Galactic value to 100   \times of that.$
$These results are in line with our calculations in Sect.~\ref{subsec: LTE}.$
$Meanwhile, the AGN-driven XDR chemistry and molecule/atom excitation have been firstly modeled in great detail by \citet{Maloney1996}, \citet{Meijerink2005} and \citet{Meijerink2007}. In XDRs, photo-ionization becomes the dominant heating source instead of photo-electric emission from dust grains in PDRs, thus more molecules are dissociated and/or ionized. \citet{Meijerink2005}'s Model~1 shows that CO molecules are fully dissociated when N_{\mathrm{H}} < 10^{23}   \mathrm{cm^{-2}}, then the fractional abundance is gradually increased by six orders of magnitudes from N_{\mathrm{H}} = 10^{23}   \mathrm{cm^{-2}} to 10^{24}   \mathrm{cm^{-2}}. By contrast, the $\CI$  and $\ion$izedcarbon  abundances are about constant until reaching the densest part (N_{\mathrm{H}} > 10^{24.5}   \mathrm{cm^{-2}}). Unlike in PDRs, there is no clear $\ion$izedcarbon /$\CI$ /CO stratification in XDRs (\citealt{Meijerink2007,Wolfire2022}). Their Model~2 with a 100 \times higher energy deposition rate shows that the CO distribution contracts to the highest gas density spots, and the total cumulative [$\CI$/CO] is increased by about 0.2~dex, reaching \RNCINCO \sim 1.6.$
$This value agrees well with our calculations shown in Figs.~\ref{fig: LTE RCICO} and \ref{fig: NonLTE RCICO} (the ISM condition 7).$
$Therefore, both in theoretical and observational views, it is clear that the \RCICO line ratio is a good indicator of the starburst's CRDR and AGN's XDR.$
$If a \RCICO line ratio (to be specific, $\Remark${ICI10}/$\Remark${ICO21}) reaches 0.2--0.5, then the CRDR plays a crucial role and the abundance ratio [$\CI$/CO] could be \sim 1.$
$If it exceeds 0.5--1.0, then it may only be enhanced by an XDR, e.g., caused by an X-ray luminous AGN, which boosts the abundance ratio [$\CI$/CO] to \gtrsim 1--2.$
$\vspace{2ex}$
$\section{Conclusion}$
$\label{sec: Conclusion}$
$We presented new beam- and uv-coverage-matched $\Remark${CI10} and CO(2--1) maps in the galactocentric radius r \lesssim 7~kpc SF disk of NGC~3627 at \sim 190~pc resolution, and in the r \lesssim 3~kpc disk of NGC~4321 at \sim 270~pc resolution.$
$They are among the largest $\CI$  mosaic mappings available in nearby galaxies.$
$We combined the imaging of $\Remark${CI10} and CO(2--1) in two nearby, more starbursty and AGN-hosting galaxies, i.e., NGC~1808 central \sim1~kpc, and NGC~7469 central \sim1.5~kpc at 140--160~pc resolutions.$
$Together, we studied the spatial distributions of $\CI$  and CO, and their line ratio \RCICO as functions of various galaxy resolved properties, including CO brightness (Fig.~\ref{fig: line ratio vs CO}), ISRF \Umean (Fig.~\ref{fig: line ratio vs ISRF}), and metallicity (Fig.~\ref{fig: line ratio vs metallicity}).$
$In order to obtain the underlying intrinsic [$\CI$/CO] abundance ratio from the observed line ratio \RCICO, we assumed 7 representative ISM conditions and did both LTE and non-LTE calculations.$
$\vspace{1ex}$
$We summarize our findings as below:$
$\begin{itemize}[itemsep=0ex,topsep=1ex,leftmargin=4ex]$
$    \item The majority of the molecular gas in the SF disks of NGC~3627 and NGC~4321 has a uniform $\Remark${RCI10CO21} = 0.10 \pm 0.05.$
$    $\Remark${CI10} and $\Remark${CO21} exhibit very similar spatial distributions at our 140--270~pc resolution.$
$    \item The majority of the molecular gas in the SB disk of NGC~1808 has $\Remark${RCI10CO21} = 0.20 \pm 0.05.$
$    This elevated line ratio compared to the SF disk is likely caused by a \sim 3 \text{--} 5 \times increased [$\CI$/CO] abundance ratio, based on our LTE and non-LTE calculations and parameter assumptions (Figs.~\ref{fig: LTE RCICO} and \ref{fig: NonLTE RCICO}).$
$    \item The centres (r \lesssim 250~pc) of NGC~4321, NGC~3627, NGC~1808 and NGC~7469 show a strongly increasing trend in $\Remark${RCI10CO21}, from 0.05 to 0.5, as a function of I_{$\Remark${CO21}} --- i.e., $\Remark${RCI10CO21} \approx 0.035   (I_{$\Remark${CO21}}/[100\;$\Kkms$])^{0.8}.$
$    The inferred [$\CI$/CO] abundance ratio from our non-LTE calculation changes from \sim 0.2 to \sim 3, in line with the increasing X-ray luminosity of the centres/AGNs.$
$    \item The observed \RCICO versus ISRF trend is fairly flat, i.e., \RCICO  is insensitive to ISRF intensity changes, similar to the PDR model prediction calculated with PDRToolbox (although our resolution is much coarser than the physical scale of PDR models; Sect.~\ref{subsection: ISRF}).$
$    \item We find a \RCICO versus metallicity trend consistent with previous observations that covered lower-metallicity environments (\sim 0.2   \mathrm{Z_{\odot}}; \citealt{Bolatto2000b}), within the metallicity range (\sim 0.7 \text{--} 1   \mathrm{Z_{\odot}}) of our sample (Sect.~\ref{subsec: metallicity}).$
$    \item Our inferred [$\CI$/CO] abundance ratios agree well with the CRDR and XDR's theoretical studies.$
$    Given the drastically changed \RCICO , we propose that the $\CI$  to CO line ratio can be a promising indicator of CRDR (starbursting) and XDR (AGN). For instance, an observed line flux ratio of$
$    $\Remark${RCI10CO21} \sim 0.2 \textnormal{--} 0.5 and \gtrsim 0.5 effectively indicates CRDR and XDR, respectively.$
$    \item We do not find ubiquitous "CO-dark", $\CI$ -bright gas at the outer r\sim 1 \text{--} 3~kpc disks of NGC~3627 and NGC4321, nor systematic enhancement in \RCICO  at CO-faintest sightlines via stacking, mainly due to the incompleteness of our data.$
$    Nevertheless, a few sightlines, e.g., Region~22 in NGC~3627 show high \RCICO ~\sim 0.5 which do not fully comply with the noise behavior seen in our CASA simulations.$
$\end{itemize}$
$\vspace{2ex}$
$\begin{acknowledgements}$
$We thank the anonymous referee for very helpful comments.$
$ES, TS and TGW acknowledge funding from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme (grant agreement No. 694343).$
$ER acknowledges the support of the Natural Sciences and Engineering Research Council of Canada (NSERC), funding reference number RGPIN-2017-03987.$
$The work of AKL is partially supported by the National Science Foundation under Grants No. 1615105, 1615109, and 1653300.$
$AU acknowledges support from the Spanish grants PGC2018-094671-B-I00, funded by MCIN/AEI/10.13039/501100011033 and by "ERDF A way of making Europe", and PID2019-108765GB-I00, funded by MCIN/AEI/10.13039/501100011033.$
$RSK and SCOG acknowledge financial support from the German Research Foundation (DFG) via the collaborative research centre (SFB 881, Project-ID 138713538) “The Milky Way System” (subprojects A1, B1, B2, and B8). They also acknowledge funding from the Heidelberg Cluster of Excellence "STRUCTURES" in the framework of Germany’s Excellence Strategy (grant EXC-2181/1, Project-ID 390900948) and from the European Research Council via the ERC Synergy Grant "ECOGAL" (grant 855130).$
$FB and IB acknowledge funding from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme (grant agreement No.726384/Empire).$
$JC acknowledges funding from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme DustOrigin (ERC-2019-StG-851622).$
$MC gratefully acknowledges funding from the Deutsche Forschungsgemeinschaft (DFG) in the form of an Emmy Noether Research Group (grant number CH2137/1-1).$
$Y.G.’s work is partially supported by National Key Basic Research and Development Program of China (grant No. 2017YFA0402704), National Natural Science Foundation of China (NSFC, Nos.$
$12033004, and 11861131007), and Chinese Academy of Sciences Key Research$
$Program of Frontier Sciences (grant No. QYZDJ-SSW-SLH008).$
$AH was supported by the Programme National Cosmology et Galaxies (PNCG) of CNRS/INSU with INP and IN2P3, co-funded by CEA and CNES, and by the Programme National “Physique et Chimie du Milieu Interstellaire” (PCMI) of CNRS/INSU with INC/INP co-funded by CEA and CNES.$
$K.K. gratefully acknowledges funding from the German Research Foundation (DFG) in the form of an Emmy Noether Research Group (grant number KR4598/2-1, PI Kreckel)$
$JMDK and MC gratefully acknowledge funding from the Deutsche Forschungsgemeinschaft (DFG) in the form of an Emmy Noether Research Group (grant number KR4801/1-1) and the DFG Sachbeihilfe (grant number KR4801/2-1), and from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme via the ERC Starting Grant MUSTANG (grant agreement number 714907).$
$JP acknowledges support from the Programme National “Physique et Chimie du Milieu Interstellaire” (PCMI) of CNRS/INSU with INC/INP co-funded by CEA and CNES.$
$HAP acknowledges support by the Ministry of Science and Technology of Taiwan under grant 110-2112-M-032-020-MY3.$
$The work of JS is partially supported by the Natural Sciences and Engineering Research Council of Canada (NSERC) through the Canadian Institute for Theoretical Astrophysics (CITA) National Fellowship.$
$This paper makes use of the following ALMA data:$
$\incode{ADS/JAO.ALMA#2015.1.00902.S},$
$\incode{ADS/JAO.ALMA#2015.1.00956.S},$
$\incode{ADS/JAO.ALMA#2015.1.01191.S},$
$\incode{ADS/JAO.ALMA#2017.1.00078.S},$
$\incode{ADS/JAO.ALMA#2017.1.00984.S},$
$\incode{ADS/JAO.ALMA#2018.1.00994.S},$
$\incode{ADS/JAO.ALMA#2018.1.01290.S},$
$\incode{ADS/JAO.ALMA#2019.1.01635.S}.$
$ALMA is a partnership of ESO (representing its member states), NSF (USA) and NINS (Japan), together with NRC (Canada), MOST and ASIAA (Taiwan), and KASI (Republic of Korea), in cooperation with the Republic of Chile. The Joint ALMA Observatory is operated by ESO, AUI/NRAO and NAOJ.$
$\end{acknowledgements}$
$\bibliographystyle{aa_url}$
$\documentclass[twocolumn,longauth]{aa}$
$\usepackage{graphicx}$
$\usepackage{txfonts}$
$\input{Header}$
$\begin{document}$
$\title{C~$\textsc${i} and CO in Nearby Spiral Galaxies - I. Line Ratio and Abundance Variations at \sim 200~pc Scales}$
$\titlerunning{C~$\textsc${i} \& CO - I. Line Ratio and Abundance Ratio}$
$\authorrunning{D. Liu et al.}$
$\author{$
$    Daizhong Liu \inst{\ref{MPE}}$
$\and$
$    Eva Schinnerer \inst{\ref{MPIA}}$
$\and$
$    Toshiki Saito \inst{\ref{NAOJ}}$
$\and$
$    Erik Rosolowsky \inst{\ref{UAlberta}}$
$\and$
$    Adam Leroy \inst{\ref{OSU}}$
$\and$
$    Antonio Usero \inst{\ref{Madrid}}$
$\and$
$    Karin Sandstrom \inst{\ref{UCSD}}$
$\and$
$    Ralf S. Klessen \inst{\ref{HeidelbergZfA},\ref{HeidelbergZfW}}$
$\and$
$    Simon C. O. Glover \inst{\ref{HeidelbergZfA}}$
$\and$
$    Yiping Ao \inst{\ref{PMO},\ref{USTC}}$
$\and$
$    Ivana Be\v{s}li\'c \inst{\ref{BonnAIfA}}$
$\and$
$    Frank Bigiel \inst{\ref{BonnAIfA}}$
$\and$
$    Yixian Cao \inst{\ref{MPE}}$
$\and$
$    Jérémy Chastenet \inst{\ref{UCSD},\ref{Gent}}$
$\and$
$    Mélanie Chevance \inst{\ref{HeidelbergARI},\ref{HeidelbergZfA}}$
$\and$
$    Daniel A. Dale \inst{\ref{Wyoming}}$
$\and$
$    Yu Gao \inst{\ref{XiamenUniv}}\thanks{Deceased.}$
$\and$
$    Annie Hughes \inst{\ref{Toulouse}}$
$\and$
$    Kathryn Kreckel \inst{\ref{HeidelbergARI}}$
$\and$
$    J.~M.~Diederik Kruijssen \inst{\ref{HeidelbergARI}}$
$\and$
$    Hsi-An Pan \inst{\ref{TamkangUniv}}$
$\and$
$    J\'er\^ome Pety \inst{\ref{IRAM},\ref{SorbonneUniv}}$
$\and$
$    Dragan Salak \inst{\ref{HokkaidoUniv},\ref{HokkaidoUnivGrad}}$
$\and$
$    Francesco Santoro \inst{\ref{MPIA}}$
$\and$
$    Andreas Schruba \inst{\ref{MPE}}$
$\and$
$    Jiayi Sun \inst{\ref{McMasterUniv},\ref{TorontoUniv}}$
$\and$
$    Yu-Hsuan Teng \inst{\ref{UCSD}}$
$\and$
$    Thomas Williams \inst{\ref{MPIA}}$
$}$
$\institute{$
$    Max-Planck-Institut f\"ur extraterrestrische Physik (MPE), Giessenbachstrasse 1, D-85748 Garching, Germany\\email{dzliu@mpe.mpg.de, astro.dzliu@gmail.com}$
$    \label{MPE}$
$\and$
$    Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany$
$    \label{MPIA}$
$\and$
$National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo, 181-8588, Japan$
$    \label{NAOJ}$
$\and$
$Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada$
$    \label{UAlberta}$
$\and$
$Department of Astronomy, The Ohio State University, 140 West 18th Ave, Columbus, OH 43210, USA$
$    \label{OSU}$
$\and$
$    Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain$
$    \label{Madrid}$
$\and$
$    Center for Astrophysics and Space Sciences, Department of Physics, University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA$
$    \label{UCSD}$
$\and$
$    Universit\"at Heidelberg, Zentrum f\"ur Astronomie, Institut f\"ur Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany$
$    \label{HeidelbergZfA}$
$\and$
$    Universit\"at Heidelberg, Interdisziplin\"ares Zentrum f\"ur Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany$
$    \label{HeidelbergZfW}$
$\and$
$     Purple Mountain Observatory and Key Laboratory for Radio Astronomy, Chinese Academy of Sciences, Nanjing, China$
$     \label{PMO}$
$\and$
$    School of Astronomy and Space Science, University of Science and Technology of China, Hefei, China$
$    \label{USTC}$
$\and$
$    Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany$
$    \label{BonnAIfA}$
$\and$
$    Sterrenkundig Observatorium, Ghent University, Krijgslaan 281-S9, 9000 Gent, Belgium$
$    \label{Gent}$
$\and$
$    Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, 69120 Heidelberg, Germany$
$    \label{HeidelbergARI}$
$\and$
$    Department of Physics \& Astronomy, University of Wyoming, Laramie, WY 82071, USA$
$    \label{Wyoming}$
$\and$
$    Department of Astronomy, Xiamen University, Xiamen, Fujian 361005, China$
$    \label{XiamenUniv}$
$\and$
$    CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France; Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France$
$    \label{Toulouse}$
$\and$
$    Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan$
$    \label{TamkangUniv}$
$\and$
$    Institut de Radioastronomie Millimétrique (IRAM), 300 rue de la Piscine, F-38406 Saint-Martin-d'Hères, France$
$    \label{IRAM}$
$\and$
$    Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France$
$    \label{SorbonneUniv}$
$\and$
$    Institute for the Advancement of Higher Education, Hokkaido University, Kita 17 Nishi 8, Kita-ku, Sapporo, Hokkaido 060-0817, Japan$
$    \label{HokkaidoUniv}$
$\and$
$    Department of Cosmosciences, Graduate School of Science, Hokkaido University, Kita 10 Nishi 8, Kita-ku, Sapporo, Hokkaido 060-0817, Japan$
$    \label{HokkaidoUnivGrad}$
$\and$
$    Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada$
$    \label{McMasterUniv}$
$\and$
$    Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada$
$    \label{TorontoUniv}$
$}$
$\date{Accepted: December 13, 2022}$
$\abstract{$
$We present new neutral atomic carbon [C~$\textsc${i}]~( ^3P_1~\to~ ^3P_0) mapping observations within the inner \sim 7~kpc and \sim 4~kpc of the disks of NGC~3627 and NGC~4321 at a spatial resolution of 190~pc and 270~pc, respectively, using the Atacama Large Millimeter/Submillimeter Array (ALMA) Atacama Compact Array (ACA).$
$We combine these with the CO(2--1) data from PHANGS-ALMA, and literature [C~$\textsc${i}] and CO data for two other starburst and/or active galactic nucleus (AGN) galaxies (NGC~1808, NGC~7469), to study: \textit{a)} the spatial distributions of C~$\textsc${i} and CO emission; \textit{b)} the observed line ratio \RCICO = I_{\mathrm{[C{\normalfont$\textsc${i}}]}(1\textnormal{--}0)} / I_{\mathrm{CO}(2\textnormal{--}1)} as a function of various galactic properties; and \textit{c)} the abundance ratio of [C~$\textsc${i}/CO].$
$We find excellent spatial correspondence between C~$\textsc${i} and CO emission and nearly uniform \RCICO \sim 0.1 across the majority of the star-forming disks of NGC~3627 and NGC~4321.$
$However, \RCICO strongly varies from \sim 0.05 at the centre of NGC~4321 to > 0.2 \text{--} 0.5 in NGC~1808's starbursting centre and NGC~7469's centre with an X-ray-luminous active galactic nucleus (AGN).$
$Meanwhile, \RCICOdoes not obviously vary with \Umean, similar to the prediction of photodissociation dominated region (PDR) models.$
$We also find a mildly decreasing \RCICOwith an increasing metallicity over 0.7\text{--}0.85 \mathrm{Z_{\odot}}, consistent with the literature.$
$Assuming various typical ISM conditions representing giant molecular clouds, active star-forming regions and strong starbursting environments, we calculate the local-thermodynamic-equilibrium radiative transfer and estimate the [C~$\textsc${i}/CO] abundance ratio to be \sim 0.1 across the disks of NGC~3627 and NGC~4321, similar to previous large-scale findings in Galactic studies.$
$However, this abundance ratio likely has a substantial increase to \sim 1 and \gtrsim 1--5 in NGC~1808's starburst and NGC~7469's strong AGN environments, respectively, in line with the expectations for cosmic-ray dominated region (CRDR) and X-ray dominated region (XDR) chemistry.$
$Finally, we do not find a robust evidence for a generally CO-dark, C~$\textsc${i}-bright gas in the disk areas we probed.$
$}$
$\keywords{galaxies: ISM --- ISM: molecules --- ISM: atoms --- ISM: abundances --- galaxies: spiral --- galaxies: individual: NGC 3627, NGC 4321, NGC 1808, NGC 7469}$
$\maketitle$
$\section{Introduction}$
$\label{subsec: intro RCICO}$
$\subsection{Neutral atomic carbon in the interstellar medium}$
$Neutral atomic carbon (C^0 or $\neutralcarbon$) is an important phase of carbon in the interstellar medium (ISM) besides ionized carbon (C^+ or $\ion$izedcarbon) and carbon monoxide (CO).$
$Its ^{3}P state is split into three fine-structure levels. The^{3}P_{1} \to  ^{3}P_{0} transition line at 492.16065~GHz,$
$hereafter $\Remark${CI10}$
$, has an upper energy level of E_u = 23.620~K and a critical density of n_{\mathrm{crit}} \sim 1.0 \times 10^{3} \;\mathrm{cm^{-3}} (at a temperature of 50~K) \footnote{n_{\mathrm{crit}} \equiv A / \gamma, where A is the Einstein A coefficient for the transition and \gamma the rate of collisional de-excitation with hydrogen molecules, with values from \cite{Schroder1991} as compiled in the Leiden Atomic and Molecular Database (\citealt{Schoier2005}), \url{https://home.strw.leidenuniv.nl/~moldata/}},$
$and the^{3}P_{2} \to  ^{3}P_{1} transition line at 809.34197~GHz has E_u = 62.462~K and n_{\mathrm{crit}} \sim 3.4 \times 10^{3} \;\mathrm{cm^{-3}}.$
$Therefore, they are easily excited in the cold ISM environment.$
$Given the high abundance of carbon in the ISM, $\neutralcarbon$, CO and $\ion$izedcarbon  are the most powerful cold ISM tracers, and are the most widely used tools to probe cold ISM properties and hence galaxy evolution at cosmological distances.$
$The interstellar $\neutralcarbon$  is produced from$
$the photodissociation of CO molecules by UV photons and the recombination of $\ion$izedcarbon , both happening in photodissociation regions (PDRs; \citealt{Langer1976,deJong1980,Tielens1985a,Tielens1985b,vanDishoeck1986,vanDishoeck1988,Sternberg1989,Sternberg1995,Hollenbach1991,Hollenbach1999,Kaufman1999,Wolfire2010,Madden2020,Bisbas2021}, also see reviews by \citealt{Genzel1989,Jaffe1985,Hollenbach1997,Wolfire2022}).$
$In the simple plane-parallel (i.e., 1-D) PDR model, $\neutralcarbon$  exists in a layer where UV photons can penetrate through the molecular gas but cannot maintain a high carbon ionizing rate.$
$Such a layer is suggested to have an intermediate gas surface density (N_{\mathrm{H_2}} \sim 1 \textnormal{--} 4 \times 10^{20} \;\mathrm{cm^{-2}}), fairly cold temperature (10 to a few tens K), and moderate visual extinction (A_V \sim 1 - 4).$
$CO molecules dominate the more shielded interior of this layer and $\ion$izedcarbon  dominates the exterior.$
$Hydrogen molecules are gradually photodissociated to atoms across this layer (A_V \sim 0.1 - 4).$
$This plane-parallel scenario qualitatively explains the layered structure of the Orion Bar PDR in our Galaxy on sub-pc scales (e.g.,  \citealt{Genzel1989,Tielens1993,Tauber1994,Hogerheijde1995b,Hollenbach1999,Goicoechea2016}), and the \rho~Ophiuchi~A PDR in a recent study with an unprecedented resolution of 360~AU (\citealt{Yamagishi2021}), where the interplay between individual young, massive (O, B) stars and the ISM can be directly observed.$
$However, such simple models cannot fully explain molecular clouds. Plenty of observations in Galactic molecular clouds, $\ion$izedhydrogen  regions and star-formation complexes have revealed a widespread $\neutralcarbon$  distribution over most areas of molecular clouds on scales of a few to ten pc, and a relatively high $\neutralcarbon$  fractional abundance even at a large A_V depth > 10~mag (\citealt{Phillips1980,Phillips1981,Wootten1982,Keene1985,Keene1987,Jaffe1985,Zmuidzinas1986,Zmuidzinas1988,Genzel1988,Frerking1989,Plume1994,Plume1999,Gerin1998a,Tatematsu1999,Ikeda1999,Ikeda2002,Yamamoto2001,Zhang2001,Kamegai2003,Oka2004,Izumi2021}).$
$This observational evidence actually requires PDRs to be clumpy  (\citealt{Stutzki1988,Genzel1988,Burton1990,Meixner1993,Spaans1997,Kramer2004,Kramer2008,Pineda2008,Sun2008}), with a volume filling factor much less than unity (e.g., \sim 0.1 - 0.3; \citealt{Stutzki1988}), so that$
$UV photons can penetrate through most of the cloud, except for the densest clumps.$
$Therefore, the spatial co-existence of $\neutralcarbon$  and low rotational transition (low-J) CO lines can be well explained by such an inter-clump medium inside the molecular clouds.$
$Alternatively, cosmic rays (CRs) have also been proposed to be the reason for the $\neutralcarbon$ -CO association as they can penetrate much deeper than UV photons into clouds, and thus more uniformly dissociate CO molecules into $\neutralcarbon$  inside clouds (\citealt{Field1969,Padovani2009,Papadopoulos2010c,Ivlev2015,Papadopoulos2018}).$
$\subsection{Variations of the $\CI$/$\CO$ line ratio (\RCICO) in the literature}$
$Although the spatial distributions of $\CI$  and low-J CO emission resemble each other surprisingly well from molecular clouds to Galactic scales (e.g., across the Galactic plane; \citealt{Wright1991,Bennett1994,Oka2005,Burton2015}) and in external galaxies with spatially resolved observations (e.g., \citealt{Schilke1993,White1994b,Harrison1995,Israel1995,Israel2001,Israel2003,Zhang2014,Krips2016,Cicone2018,Crocker2019,Jiao2019,Miyamoto2018,Miyamoto2021,Salak2019,Izumi2020,Saito2020,Michiyama2020,Michiyama2021}),$
$their line intensity ratio, $\Remark${RCI10CO10} = $\Remark${ICI10} / $\Remark${ICO10} \footnote{The line intensity I has brightness temperature units \mathrm{K km s^{-1}}.}, does vary according to local ISM conditions --- $\CI$  and CO fractional abundance (i.e., metallicity), excitation condition (e.g., excitation temperature T_{\mathrm{ex}} and optical depth \tau), and UV and/or CR radiation field.$
$At individual molecular cloud scales, $\Remark${RCI10CO10} (or column density ratio) seems to vary from \gtrsim 0.5 (or \gtrsim 0.2 for column density ratio) to <0.1 with an increasing N_{\mathrm{H2}} and A_V, e.g., from cloud surface or PDR front to the interior \citep[e.g.][]{Frerking1982,Genzel1988,Oka2004,Kramer2004,Kramer2008,Mookerjea2006,Sun2008,Yamagishi2021}.$
$At sub-galactic scales, when individual clouds are smoothed out, $\Remark${RCI10CO10} is found to lie in a narrower range of \sim 0.1 \text{--} 0.3. For example, across the Galactic plane, the ratio is found to change only slightly within a galactocentric radius of 3--7~kpc (\sim 0.08 \text{--} 0.12, overall mean = 0.105 \pm 0.004; \citealt{Oka2005});$
$whereas in the starburst galaxies like M~82 and NGC~253, $\Remark${RCI10CO10} or the [$\CI$]/CO abundance ratio is a factor of 3--5 higher than the Galactic value (\citealt{Schilke1993,White1994b,Krips2016}).$
$Recent high-resolution (hundred pc scales) $\Remark${CI10} mapping studies in the nearby spiral galaxy M~83 (\citealt{Miyamoto2021}) and starburst galaxy IRAS~F18293-3413 (\citealt{Saito2020}) reported tightly-correlated $\Remark${CI10} and $\Remark${CO10} emission, with scaling relations at hundred parsecs equivalent to$
$\Remark{RCI10CO10} = 0.14 \times (L^{\prime}_{$\Remark${CO10}}/10^{6.0} [$\Remark${Kkmspc2}])^{-0.13}$
$and$
$\Remark{RCI10CO10} = 0.21 \times (L^{\prime}_{$\Remark${CO10}}/10^{6.8} [$\Remark${Kkmspc2}])^{+0.54},$
$respectively.$
$The discrepant power-law indices reflects different trends in individual galaxies that are not fully understood.$
$Meanwhile, in the central \lesssim 100~pc of NGC~7469 close to the X-ray-luminous active galactic nucleus (AGN), \RCICO  is found to be substantially enhanced by a factor of >10 (reaching $\Remark${RCI10CO10} \approx 1; \citealt{Izumi2020}; see also the Circinus Galaxy's AGN with $\Remark${RCI10CO32} \approx 0.9 in \citealt{Izumi2018}).$
$Furthermore, in the low-metallicity Large Magellanic Cloud (LMC) and Small Magellanic Cloud (SMC) environments, a factor of 1.5 - 3 \times higher \RCICO  values have been found (\citealt{Bolatto2000b,Bolatto2000c}; see also other low-metallicity galaxies: \citealt{Bolatto2000a,Hunt2017}).$
$These studies indicate that \RCICO  is sensitive to ISM conditions among and within galaxies.$
$However, previous extragalactic studies are mainly focused on starburst galaxies and bright galaxy centres. How \RCICO  behaves across the disks and spiral arms of typical star-forming galaxies is much less explored. Whether $\CI$  can trace the so-called CO-dark molecular gas at a low gas density or metallicity is also still in question.$
$Understanding \RCICO  variation and quantifying the $\CI$  and $\CO$  excitation in extragalactic environments is now particularly important given the rapidly growing numbers and diverse galactic environments of $\CI$  line observations at cosmological distances (e.g., \citealt{Weiss2003,Weiss2005,Pety2004,Wagg2006,Walter2011,Danielson2011,AlaghbandZadeh2013,Gullberg2016,Bothwell2017,Popping2017,Yang2017,Emonts2018,Valentino2018,Valentino2020,Bourne2019,Nesvadba2019,Dannerbauer2019,Jin2019,Brisbin2019,Boogaard2020,Cortzen2020,Harrington2021,Dunne2021,Lee2021}).$
$\subsection{What drives \RCICO? --- the need for high-resolution mapping across wide galactic environments}$
$High-resolution mapping of $\CI$  and CO from disks and spiral arms of nearby galaxies is indispensable for understanding and quantifying the ISM $\CI$  and CO physics. In particular, the following questions will be answered only with such observations:$
$Do $\CI$  and low-J CO have the same spatial distributions across galaxy disks?$
$Does the \RCICO  line ratio vary significantly across galaxy disks?$
$How does the \RCICO  line ratio vary among different local galaxies?$
$How can the observed \RCICO  line ratio be reliably converted to the $\CI$  and CO column density ratio (i.e., solving their radiation transfer equations and level population statistics and obtaining optical depths and excitation temperature)?$
$Currently, high-resolution (hundreds of parsec scale) imaging/mapping of $\CI$  in nearby galaxies relies on radio telescopes at high altitudes, e.g., the Atacama Large Millimeter/submillimeter Array (ALMA), Atacama Submillimeter Telescope Experiment (ASTE, 10-m single-dish), Atacama Pathfinder Experiment (APEX, 12-m single-dish), and is limited to the available weather conditions corresponding to the high frequency of $\CI$  lines. Therefore, only a handful of galaxies have been imaged/mapped.$
$These include NGC~6240 by \citet{Cicone2018}, Circinus Galaxy by \citet{Izumi2018}, NGC~613 by \citet{Miyamoto2018}, NGC~1808 by \citet{Salak2019}, NGC~7469 by \citet{Izumi2020}, IRAS~F18293-3413 by \citet{Saito2020}, NGC~6052 by \citet[][$\CI$  undetected]{Michiyama2020}, 36 (U)LIRGs by \citet{Michiyama2021}, M~83 by \citet[][mapped the centre and northern part with ASTE]{Miyamoto2021}, Arp~220 by \citet{Ueda2022}, and NGC~1068 by \citet{Saito2022a,Saito2022b}.$
$Nevertheless, all of these observations, except for M~83, targeted the centres of starburst, IR-luminous and/or AGN host galaxies. M~83 is the only typical star-forming main sequence galaxy having a high-quality $\CI$  mapping. However, its map is still limited to a quarter of its inner \sim3~kpc disk.$
$In this work, we present two new $\CI$  mapping observations with the ALMA Atacama Compact Array (or Morita Array; 7-m dish, hereafter ACA or 7m) in nearby spiral galaxies NGC~3627 and NGC~4321.$
$Our observations aimed at detecting $\CI$  in the disks, and thus are much larger-area and deeper than previous $\CI$  maps.$
$By further adding a starburst and an AGN-host galaxy from the literature into our analysis, we present a comprehensive study of the $\Remark${RCI10CO21} line ratio (hereafter \RCICO; ratios of other $\CI$ /CO transitions will be explicitly written otherwise) in nearby star-forming disk and starburst and AGN environments.$
$In order to quantify the $\CI$  and CO abundance ratio,$
$we have further calculated the local thermodynamic equilibrium (LTE) and non-LTE radiative transfer of $\CI$  and CO under several \textit{representative ISM conditions}.$
$The results from this study could thus be one of the most comprehensive local benchmarks for understanding the $\CI$  and CO line ratio and abundance variations.$
$The paper, as the first one of our series of papers, is organized as follows.$
$Sect.~\ref{sec: Observations} describes the sample selection, observations and data reduction.$
$Sect.~\ref{sec: Results} presents the spatial distribution and variation of \RCICO.$
$Sect.~\ref{sec: Excitation Analysis} presents the analysis of $\CI$  and CO excitation and radiative transfer and thus the abundance (species column density) ratio.$
$In Sect.~\ref{sec: Discussion} we discuss various topics relating to the [$\CI$/CO] abundance variation, CRDR, XDR, and CO-dark gas.$
$Finally we conclude in Sect.~\ref{sec: Conclusion}.$
$Our companion paper~II (D. Liu et al. in prep.) presents the detailed radiative transfer calculation and $\CI$  and CO conversion factors, and is related to the abundance calculation in this paper.$
$\vspace{2ex}$
$\section{Targets and observations}$
$\label{sec: Observations}$
$\subsection{Target selection}$
$\label{subsection: Target Selection}$
$Our targets, NGC~3627 and NGC~4321, are selected from a joint sample of the PHANGS-ALMA CO(2--1) (\citealt{Leroy2021phangs}), PHANGS-HST (\citealt{PHANGSHST}), and PHANGS-MUSE (\citealt{PHANGSMUSE}) surveys, with CO line intensities and systemic velocities best suitable for ALMA Band 8 $\CI$  mapping.$
$The PHANGS-ALMA sample consists of 90 nearby star-forming galaxies initially selected at distances of 2 \lesssim d / \mathrm{Mpc} \lesssim 23, with a stellar mass \log (M_{\star}/\mathrm{M_{\odot}}) > 9.75, and which are not too inclined and visible to ALMA. They represent the typical star-forming main sequence galaxies in the local Universe.$
$The PHANGS-HST sample \citep{PHANGSHST} is a subsample of 38 PHANGS-ALMA galaxies, providing high-resolution stellar properties.$
$The PHANGS-MUSE large program \citep{PHANGSMUSE} targeted a subsample of 19 PHANGS-ALMA galaxies suitable for VLT/MUSE optical integral field unit (IFU) observations, achieving similar spatial coverage as in PHANGS-ALMA and providing rich nebular emission lines, attenuation, stellar age and metallicity and H~$\textsc${ii} region information (e.g., \citealt{Kreckel2019, Kreckel2020, Pessa2021, Santoro2021, Williams2022}).$
$For the legacy value, we selected our sample from the joint ALMA+MUSE+HST sample pool, considered CO surface brightness, mapping area, $\CI$  line frequency and the transmission in the ALMA Band 8.$
$In this work, we present the new $\CI$  data of NGC~3627 and NGC~4321.$
$Because of the ALMA Band 8 sensitivity and the expected fainter $\CI$  line intensity than CO, the $\CI$  mapping areas are smaller than their CO(2--1) maps, but they are still the largest (\sim1'-2.35') and deepest (rms~\sim0.04~K per \sim5~km~s^{-1}) $\CI$  mapping with ALMA.$
$Furthermore, in order to study the $\CI$  and CO in different galactic environments, we selected two starbursting and/or AGN-host galaxies from the literature whose ALMA $\Remark${CI10} and CO(2--1) data are available in the archive: NGC~1808 \citep{Salak2019} and NGC~7469 \citep{Izumi2020}.$
$NGC~1808 has ALMA mosaic observations covering its \sim 1~kpc starburst ring, and NGC~7469 has ALMA single-pointing observations covering its inner \sim 3~kpc area.$
$\subsection{New ALMA ACA Band 8 observations}$
$\label{subsection: ALMA Band 8 Observation}$
$Our $\CI$  observations for NGC~3627 were taken under the ALMA project 2018.1.01290.S (PI: D. Liu) during June 3rd to Sept. 29th, 2019 with ACA-only at Band~8. The total on-source integration time was 30.8~hours. A mosaic with 149 pointings was used to map an area of 94" \times 136" at a position angle of -27^\circ, which covers the galaxy centre, bar and the majority of the spiral arms, and is about 1/3 of the full PHANGS-ALMA CO(2-1) map area.$
$The achieved line sensitivity per 5 $\kms$ is 45--55~mK (107--130~\mathrm{mJy/beam}) across the $\Remark${CI10} line frequencies with a beam of 3.46".$
$The $\CI$  mapping for NGC~4321 were taken under the ALMA project 2019.1.01635.S (PI: D. Liu; NGC~1365 is also partially observed for this project) during Dec. 3rd, 2019 to July 1st, 2021, also with ACA-only at Band~8. The on-source integration time reached 16.1~hours for the mapping of an area of 55" \times 42" with 23 pointings using the 7m array. The achieved line sensitivity is similar to that of NGC~3627.$
$More detailed information is newd in Table~\ref{table: targets}.$
$ACA (7m) was chosen to new a matched and slightly coarser synthesized beam (\sim 3"-4") than the PHANGS-ALMA CO(2-1) data. In comparison, the main 12m ALMA array in compact C43-1/2 configurations will result in a \sim 0.4"-0.7" beam and significantly miss large scale (>4-6") emission without ACA.$
$No total power observations were requested in our presented observation, therefore we matched the uv ranges of the raw CO(2--1) and $\Remark${CI10} visibility data during our data reduction.$
$We verified that missing flux will not substantially (\sim 30\%) bias our analysis using three methods as detailed in Appendix~\ref{appendix: missing flux}. This includes a CASA simulation of visibilities mimicking our ACA mosaic observations in NGC~3627, from which we find a missing flux as small as 7\% in $\CI$-bright pixels and up to \sim 50\% in the faintest pixels (and mostly < 30\%).$
$Our data reduction follows the PHANGS-ALMA imaging and post-processing pipeline \citep[][]{Leroy2021pipeline}, including imaging and deconvolution, mosaicking, broad and strict/signal mask generation, and moment map creation.$
$The additional steps in this study are: (a) uv-clipping in the CO(2--1) data to match the uv sampling range of $\Remark${CI10}; and (b) making a joint mask of the individual CO(2--1) and $\Remark${CI10} masks then extracting the moment maps (in the joint broad mask) and line ratios maps (in the joint strict mask).$
$More technical details are newd in Appendix~\ref{appendix: data reduction}.$
$Our $\Remark${CI10} and CO(2--1) line intensity maps, the line ratio maps and \textit{HST} images for the two PHANGS galaxies, NGC~3627 and NGC~4321, are presented in Figs.~\ref{fig: NGC3627 line ratio map} and \ref{fig: NGC4321 line ratio map}, respectively.$
$Our data cubes and moment maps are also made publicly available at$
$the PHANGS-ALMA CADC public repository \footnote{\url{https://www.canfar.net/storage/list/phangs/RELEASES/DZLIU_etal_2022}}.$
$\begin{table*}[htb]$
$    \centering$
$    \caption{\label{table: targets}$
$        Physical and Data Properties of the Sample$
$    }$
$    \begin{tabularx}{\linewidth}{l c c c c}$
$        \setlength{\mymincolwidth}{\dimexpr 0.85\linewidth/5 \relax}$
$        \global\mymincolwidth=\mymincolwidth$
$        \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt} &$
$            \rule{\mymincolwidth}{0pt}$
$            \\[-\arraystretch\normalbaselineskip]$
$        $\hl$ine$
$        $\hl$ine$
$         --- &$
$            NGC 3627 &$
$            NGC 4321 &$
$            NGC 1808 &$
$            NGC 7469 \$\hl$ine$
$         redshift &$
$            0.00243 &$
$            0.00524 &$
$            0.00332 &$
$            0.01627 \distance (Mpc) &$
$            11.32 &$
$            15.21 &$
$            10.8 &$
$            71.2\linear scale (pc/") &$
$            54.9 &$
$            73.7 &$
$            52.4 &$
$            345.2 \angular resolution &$
$            3.46" (190 pc) &$
$            3.65" (269 pc) &$
$            2.67" (140 pc) &$
$            0.46" (159 pc) \channel width (\mathrm{km s^{-1}}) &$
$            4.77 &$
$            4.76 &$
$            9.52 &$
$            4.76 \\textnormal{[}C $\textsc${i}\textnormal{]}(1--0) flux scale (\mathrm{K Jy^{-1}}) &$
$            0.42 &$
$            0.38 &$
$            0.71 &$
$            23.8 $\CO$(2--1) flux scale (\mathrm{K Jy^{-1}}) &$
$            1.92 &$
$            3.70 &$
$            3.24 &$
$            108.4 \\textnormal{[}C $\textsc${i}\textnormal{]}(1--0) channel rms (K) &$
$            0.044 &$
$            0.043 &$
$            0.056 &$
$            0.11 $\CO$(2--1) channel rms (K) &$
$            0.022 &$
$            0.015 &$
$            0.083 &$
$            0.14\imaging area &2.35' \times 1.60' &$
$            1.28' \times 1.00' &$
$            40" \times 40" &$
$            \textnormal{r}\sim9"$
$            \imaging type &$
$            7m, mosaic &$
$            7m, mosaic &$
$            12m+7m+tp, mosaic &$
$            12m+7m, single\M_{\star} (10^{10}   \mathrm{M_{\odot}}) &$
$            6.8 &$
$            5.6 &$
$            2.0 &$
$            10.7$
$            \L_{\mathrm{FIR}} (10^{10}   \mathrm{L_{\odot}}) &$
$            1.7 &$
$            1.9 &$
$            3.5 &$
$            25$
$            \L_{\mathrm{TIR}} (10^{10}   \mathrm{L_{\odot}}) &$
$            2.4 &$
$            2.5 &$
$            5.1 &$
$            39$
$            \L_{X} (10^{40}   \mathrm{erg s^{-1}}) &$
$            0.032 &$
$            0.12 &$
$            1.6 &$
$            1500$
$            \SFR (\mathrm{M_{\odot} yr^{-1}}) &$
$            3.84 &$
$            3.56 &$
$            7.9 (SB ring: 4.5) &$
$            76$
$            \M_{\mathrm{H_2}} (10^{9}   \mathrm{M_{\odot}}) &$
$            6.0 &$
$            7.8 &$
$            (SB ring: 0.52) &$
$            15$
$            \M_{\mathrm{H}{\textnormal{$\textsc${i}}}} (10^{9}   \mathrm{M_{\odot}}) &$
$            1.1 &$
$            2.7 &$
$            1.7 &$
$            5.7$
$            \mean\RCICO  &$
$            0.11 &$
$            0.10 &$
$            0.16 &$
$            0.21$
$            \mean\log_{10} \RCICO&$
$            -0.96 &$
$            -1.02 &$
$            -0.83 &$
$            -0.69$
$            \\pm 2 \sigma range of \log_{10} \RCICO&$
$            -1.26, -0.66 &$
$            -1.24, -0.80 &$
$            -1.03, -0.63 &$
$            -0.95, -0.43$
$            \$\hl$ine$
$        $\hl$ine$
$    \end{tabularx}$
$    \tablefoot{$
$        Redshifts are taken from the NASA Extragalactic Database (NED).$
$        The distances of NGC~3627, NGC~4321, NGC~1808 and NGC~7469 are taken from \cite{2009AJ....138..332J}, \cite{2001ApJ...553...47F}, \cite{Salak2019} and \cite{Izumi2020},$
$        respectively.$
$        The stellar masses, SFRs, M_{\mathrm{H_2}} and M_{\mathrm{H}{\textnormal{$\textsc${i}}}} of NGC~3627 and NGC~4321 are taken from the PHANGS sample table v1.6 of \citet[][PHANGS survey]{Leroy2021phangs}.$
$        The stellar masses and SFRs of NGC~1808 and NGC~7469 are taken from the z0MGS survey (\citealt{Leroy2019}; \url{https://irsa.ipac.caltech.edu/data/WISE/z0MGS/}).$
$        The M_{\mathrm{H_2}} and M_{\mathrm{H}{\textnormal{$\textsc${i}}}} of NGC~7469 are adopted from \cite{Papadopoulos2000}.$
$        The global M_{\mathrm{H}{\textnormal{$\textsc${i}}}} mass in NGC~1808 is measured by \citet{Koribalski1993}.$
$        L_{\mathrm{FIR}} and L_{\mathrm{TIR}} are IR 40--400\mum and 8--1000\mum luminosities all from \citet{Sanders2003}.$
$        The X-ray luminosities for NGC~3627 and NGC~4321 are both L_{0.3\textnormal{--}8\mathrm{keV}} from \citet{2011ApJ...731...60G}, and those for NGC~1808 and NGC~7469 are L_{2\textnormal{--}10\mathrm{keV}} from \citet{2005A&A...442..861J} and \cite{Izumi2020}, respectively.$
$    }$
$\end{table*}$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=\textwidth, trim=0 0 0 0]{Plot_two_line_ratio_four_panel_image_NGC3627_CI10_CO21_with_regions_with_HST}$
$\caption{\textit{Top panels}: NGC~3627 $\Remark${CI10} (\textit{left}) and $\Remark${CO21} (\textit{right}) integrated line intensity (moment-0) maps within our broad mask (representing high-completeness signals; see Appendix~\ref{appendix: data reduction}).$
$\textit{Bottom left panel}: $\Remark${CI10}/$\Remark${CO21} line ratio map within the combined signal mask.$
$\textit{Bottom right panel}: HST F814W-F555W-F438W RGB composite image (PHANGS-HST; \citealt{PHANGSHST}).$
$Regions marked with the white boxes and labeled with numbers are mainly for illustration purpose and for discussion in the text. Their zoom-in images and extracted spectra are newd in our online data release. Examples for Regions~1, 4 and 22 are presented in Appendix~\ref{appendix: all regions}.$
$All images have the same field of view of 179" \times 179", or 9.83 \mathrm{kpc} \times 9.83 \mathrm{kpc}.$
$}$
$\label{fig: NGC3627 line ratio map}$
$\end{figure*}$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=\textwidth, trim=0 0 0 0]{Plot_two_line_ratio_four_panel_image_NGC4321_CI10_CO21_with_regions_with_HST}$
$\caption{$
$Similar to Fig.~\ref{fig: NGC3627 line ratio map}, the $\Remark${CI10} and CO(2--1) moment-0 intensity maps and the line ratio map and the HST composite RGB image for NGC~4321. The field of view is 78" \times 78", or 5.75 \mathrm{kpc} \times 5.75 \mathrm{kpc}.$
$}$
$\label{fig: NGC4321 line ratio map}$
$\end{figure*}$
$\subsection{ALMA observations of the additional targets from the literature}$
$\label{subsection: additional targets}$
$The NGC~1808 ALMA $\Remark${CI10}, $\Remark${CO10} and $\Remark${CO21} observations are presented in \citet{Salak2019} and all are interferometry plus total power, i.e., 12m+7m+tp, thus having no missing flux issue.$
$To study the $\CI$ /$\CO$  line ratio consistently with our main targets, we only used the $\Remark${CI10} and $\Remark${CO21} data.$
$We re-reduced the raw data under ALMA project 2017.1.00984.S (PI: D. Salak) with the observatory calibration pipeline, then imaged, primary-beam-corrected and short-spacing-corrected using the PHANGS-ALMA pipeline in the same way as for our main targets (Appendix~\ref{appendix: data reduction}).$
$Note that in this step we re-reduced and imaged the total power data with the PHANGS-ALMA singledish pipeline (\citealt{Herrera2020,Leroy2021pipeline}) and did the short-spacing correction with the PHANGS-ALMA pipeline right after the primary beam correction.$
$The properties of the final $\CO$ -$\CI$  beam-matched data cubes are listed in Table~\ref{table: targets}.$
$The NGC~7469 ALMA data are presented in \citet{Izumi2020} and we used the archival raw data of $\Remark${CI10} and $\Remark${CO21} under ALMA project 2017.1.00078.S (PI: T. Izumi). These are single pointings toward the centre. $\Remark${CI10} is observed with ACA 7m and $\Remark${CO21} with the 12m array, both without total power. We re-reduced the data following the same procedure as for NGC~3627 and NGC~4321 with the PHANGS-ALMA pipeline.$
$Our re-reduced $\Remark${CI10} and CO(2--1) line intensity and ratio maps for these additional galaxies are shown in Fig.~\ref{fig: other line ratio maps}.$
$\vspace{2ex}$
$\section{Results of the observed line ratios}$
$\label{sec: Results}$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.9\textwidth, trim=0 7.7mm 0 2mm, clip]{Plot_two_line_ratio_two_panel_image_NGC1808_CI10_CO21}$
$\includegraphics[width=0.9\textwidth, trim=0 0 0 2mm, clip]{Plot_two_line_ratio_two_panel_image_NGC7469_CI10_CO21}$
$\caption{$
$\Remark{CI10} (\textit{left}) and \RCICO  (\textit{right}) maps of NGC~1808 in the upper panels and NGC~7469 in the lower panels.$
$Similar to Figs.~\ref{fig: NGC3627 line ratio map} and \ref{fig: NGC4321 line ratio map}, the line intensity maps are newcommandined in the broad mask and the \RCICO  maps are restricted to the combined signal mask.$
$Boxes in the left panels indicate the manually selected central regions where the pixels are highlighted in Fig.~\ref{fig: line ratio vs CO}.$
$Contours in the right panels indicate the 5, 10, 20 and 50-\sigma levels of the left-panel $\CI$  intensity maps.$
$\label{fig: other line ratio maps}$
$}$
$\end{figure*}$
$\subsection{Spatial distributions}$
$In Figs.~\ref{fig: NGC3627 line ratio map}, \ref{fig: NGC4321 line ratio map} and \ref{fig: other line ratio maps}, we present the beam-matched $\Remark${CI10} and $\Remark${CO21} integrated line intensity maps and their ratio maps.$
$The $\CI$/$\CO$  line ratio map is computed only for pixels within the joint strict mask area of the two lines, as described in Appendix~\ref{appendix: data reduction}, representing high-confidence signals.$
$In NGC~3627 (Fig.~\ref{fig: NGC3627 line ratio map}), we visually marked 23 regions based on the $\CI$  and CO commonly-detected (\mathrm{S/N}>3), strict-mask areas for later analysis. Region IDs are sorted by their $\Remark${CI10} brightness.$
$Region 1 corresponds to the galaxy centre, Regions 2 and 3 are the southern and northern bar ends, Region 4 is an offset peak next to the galaxy centre, and other regions are mostly on the spiral arms. Most of these regions show a similar \RCICO \sim 0.1, except for, e.g., Regions 20 and 22 which exhibit a somewhat higher ratio of \RCICO \sim 0.5. The edge of Region 4 also shows an enhancement of $\CI$  yet it is hard to tell whether this is physical or simply caused by edge effects, e.g., as seen in our simulations (Appendix~\ref{appendix: simulation}).$
$We also examined the integrated spectra of each region in Appendix~\ref{appendix: all regions}.$
$The $\CI$  and CO line profiles are not always well-matched in width and shape. In the galaxy centre (Region 1) and at bright bar-ends (Regions 2, 3, 5), $\CI$  and CO have similar line widths but $\CI$  has a lower intensity than CO, whereas in some fainter regions (9, 10, 13, 14), $\CI$  is narrower than CO and relatively weaker. In Regions 4, 20 and 22 where \RCICO is high, $\CI$  and CO have similar line widths but the $\CI$  intensity is enhanced.$
$In NGC~4321 (Fig.~\ref{fig: NGC4321 line ratio map}), we mapped a smaller area due to its fainter CO brightness than NGC~3627. There are no regions with $\Remark${CO21} integrated intensities >100  $\Kkms$ outside our selected area in NGC~4321, whereas in comparison the NGC~3627 bar ends have $\Remark${CO21} integrated intensities \gtrsim 300  $\Kkms$. The $\CI$  mapping in NGC~4321 has a similar depth as in NGC~3627.$
$We marked 7 regions in NGC~4321 to guide the visual inspection. A clear \RCICO  newcommandicit can be seen in the galaxy centre (Region 1), with \RCICO \sim 0.06, nearly a factor of two smaller than its inner disk which has similar \RCICO \sim 0.1 as seen in the NGC~3627 disk. Some $\CI$  enhancement can be found at the edges or slightly outside these regions, where CO and $\CI$  are both faint ($\Remark${ICO21} \sim 30-100  $\Kkms$) and the edge effect of interferometric missing flux might play a role (see Appendix~\ref{appendix: simulation}).$
$The \RCICO  spatial distributions of the other two starburst/AGN galaxies (Fig.~\ref{fig: other line ratio maps}) show in general more enhanced \RCICO  than in the NGC~3627 and NGC~4321 disks.$
$Large variations from the centres to outer areas can be seen, especially at the the S/N threshold of \sim 3 \text{--} 5 (see contours).$
$To quantify the spatial co-localization of the $\CI$  and CO emission, we use the $\incode${JACoP} tool \citep{JACoP} \footnote{\url{https://imagej.net/plugins/jacop}}$
$in the $\incode${ImageJ} software \citep{ImageJ} to measure the commonly used co-localization indicators, namely the Pearson's coefficient (PC; equal to 0 and 1 for null and 100\% localization, respectively) and Manders' coefficients (M1 and M2; which measure the overlap between two images and are equal to one for perfect overlap). We turn on the option of using Costes' automatic threshold \citep{Costes2004} which automatically set a pixel value threshold for each image to minimize the contribution of noise for the comparison \citep{JACoP}.$
$For the $\CI$  and CO emission in our galaxies, we find \mathrm{PC/M1/M2} values all very close to 1, that is, nearly 100\% co-localization (\mathrm{PC/M1/M2} = 0.934/0.985/0.990 for NGC~3627, 0.816/0.986/0.964 for NGC~4321, 0.978/0.890/0.906 for NGC~1808, and 0.935/0.999/0.998 for NGC~7469).$
$Overall, the spatial comparison of the $\CI$  and CO intensities demonstrates that the two species are well-correlated when observed at hundred parsec scales.$
$Meanwhile, we do see certain spectral differences in the line profiles as shown in Appendix~\ref{appendix: all regions}. This may suggest that variations exist at smaller scales than our resolution, which could be the sign of different evolutionary stages \citep[e.g.][]{Kruijssen2014} and would be interesting for higher-resolution follow-ups, e.g., with the main ALMA 12m array.$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=\textwidth]{Plot_Line_Ratio_CI10_CO21_New_surface_density}$
$    \caption{$
$        \textit{Left}: $\Remark${ICI10} versus $\Remark${ICO21} in the four galaxies we studied: NGC~3627, NGC~4321, NGC~1808 and NGC~7469. $\Remark${ICI10} and $\Remark${ICO21} integrated fluxes are moment-0 intensities extracted from their beam and pixel size matched data cubes. Only data points with propagated moment-0 \SNR \ge 5 are shown.$
$        The blue (with crossed hatch), green (with horizontal line hatch), gray (with dotted hatch) and light gray shadings indicate the 5 \sigma thresholds for the four galaxies, respectively. Typical line widths of 10~km~s^{-1} (NGC~3627 and NGC~4321) and 30~km~s^{-1} (NGC~1808 and NGC~7469) in sigma are used to calculate the thresholds together with the channel rms in Table~\ref{table: targets}.$
$        For NGC~3627 and NGC~4321, their (uv-matched) CO(2--1) 5 \sigma thresholds are out of the plotting range.$
$        Top axis shows the simply-converted H_2 surface density using a constant \alphaCO = 4.3 and CO excitation (i.e., CO(2--1) to CO(1--0) line ratio) R_{21} = 0.65.$
$        \textit{Right}: Same data but shown as the line ratio \RCICO = $\Remark${ICI10}/$\Remark${ICO21} versus $\Remark${ICO21}.$
$        The blue horizontal line indicates a star-forming (SF) disk-like \RCICO \sim 0.1, i.e., the spiral arms of NGC~3627 and NGC~4321. The gray horizontal line indicates a starburst (SB) disk-like \RCICO \sim 0.2, i.e., the \sim 1~kpc SB disk/ring in NGC~1808 and NGC~7469.$
$        The red line with a slope of 0.8 in logarithm (0.035   (I_{$\Remark${CO21}}/[100 $\Kkms$])^{0.8}) visually guides the trend of increasing \RCICO in the central regions of these galaxies (see boxes in Figs.~\ref{fig: NGC3627 line ratio map} to \ref{fig: other line ratio maps}, sizes \sim 300--500~pc).$
$    }$
$    \label{fig: line ratio vs CO}$
$\end{figure*}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=0.493\textwidth]{Plot_Histogram_of_Line_Ratio_CI10_CO21}$
$    \includegraphics[width=0.493\textwidth]{Plot_Percentiles_of_Line_Ratio_CI10_CO21}$
$    \caption{$
$    \textit{Left panel} shows the histograms of the sum of CO(2--1) surface brightness over pixels in each $\CI$/$\CO$  line ratio interval in the four galaxies. The bold curves are the cumulative distributions of the corresponding histograms.$
$    \textit{Right panel} shows the mean $\CI$/$\CO$  line ratio for pixels in each bin of CO(2--1) surface brightness (or molecular gas surface density as indicated by the top-axis assuming a constant \alphaCO = 4.3 and R_{21} = 0.65). Shaded area enveloping each mean trend indicates the 16- to 84-th percentiles in each bin, and dashed line indicates the 50-th percentile.$
$    This panel thus more clearly illustrates the ridge and scatter of the trends in the right panel of Fig.~\ref{fig: line ratio vs CO}.$
$    Detection limits are shown consistently with Fig.~\ref{fig: line ratio vs CO}.$
$    }$
$    \label{fig: line ratio histogram}$
$\end{figure*}$
$\subsection{\RCICO versus CO surface brightness and gas surface density}$
$In Fig.~\ref{fig: line ratio vs CO} we show the scatter points of $\CI$  and CO line brightnesses and the \RCICO  line ratios.$
$Systematic differences in \RCICO can be found among these galaxies and within their galactic environments. We discuss these environments separately as follows.$
$\subsubsection{Star-forming and starburst disk environments}$
$The majority of molecular gas in the 3--7~kpc star-forming disks of NGC~3627 and NGC~4321 exhibits a tight distribution of \RCICO around 0.1 (within about \pm50\%). The distribution is fairly flat over an order of magnitude in $\Remark${ICO21}. We denote this as the "SF disk" regime.$
$The ratio rises to about 0.2 within a galactocentric radius of few hundred parsec to \sim 1~kpc starburst areas in NGC~1808 and NGC~7469, which we newcommandine as the "SB disk" regime.$
$The nearly flat "SB disk" distribution is consistent with earlier studies. \citet{Salak2019} found that in NGC~1808 $\Remark${ICI10} is proportional to $\Remark${ICO21}^{ +1.05}. They also studied CO(1--0), finding that $\Remark${ICI10} \propto $\Remark${ICO10}^{ +1.46}, i.e., with a steeper slope.$
$\citet{Saito2020} found a similar slope between $\Remark${ICI10} and $\Remark${ICO10}: $\Remark${ICI10} \propto $\Remark${ICO10}^{ +1.54} in IRAS~F18293-3413 (whose CO(2--1) data is insufficient for such a study as previously mentioned).$
$This correlation slope is likely to change with different CO transitions due to the CO excitation's dependence on the CO luminosity or star formation rate themselves (e.g., \citealt{Daddi2015,Liudz2021a}).$
$For higher-J CO transitions, \citet{Michiyama2021} studied $\CI$  and CO(4--3) lines in 36 (U)LIRGs, finding $\Remark${ICI10} \propto $\Remark${ICO43}^{ +0.97}.$
$The upward tails seen at the CO-faint-end in the right panel of Fig.~\ref{fig: line ratio vs CO} are mostly due to the detection limits in each galaxy. The color shading indicates the 5-\sigma limit for each galaxy, assuming a typical line width of 10~km~s^{-1}. Region~22 of NGC~3627 and the outer pixels in the two starburst/AGN galaxies make the tips of the blue and gray faint-end tails, respectively. But most of these features should still be spurious. We performed CASA simulations following the real observations of NGC~3627 to verify what we would get if \RCICO  were constant across the galaxies (Appendix~\ref{appendix: simulation})$
$We find that noise can indeed boost a simulated \RCICO  of 0.2 to a measurement as high as 0.5 in a NGC~3627-like large mosaic, or even higher to >1 in single-pointing ALMA data without total power or sufficient short spacing. However, the faint-end tail from the simulation is not very similar to that of NGC~3627, raising some questions about the nature of the high-\RCICO  in Region~22.$
$Furthermore, we also did stacking experiments using the CO line profile as the prior to measure the $\CI$  line fluxes in bins of CO line brightnesses or galactocentric radii. We find no systematically higher \RCICO  at fainter CO pixels or at the outer radii.$
$\subsubsection{Galaxy centre environments}$
$Most interestingly, the \lesssim 250~pc centres of all four galaxies show dramatically different \RCICO. It varies by a factor of ten from \sim 0.05 in the centre of NGC~4321  to \sim 0.1 in the centre of NGC~3627, then to \sim 0.15--0.2 at the centre of NGC~1808, and to \sim 0.2--0.5 in the nucleus of NGC~7469.$
$This "nuclei" trend has a slope of about 0.8, i.e., $\Remark${ICI10} \approx 0.035 \; $\Remark${ICO21}^{ +0.8}, and corresponds to a \Sigmol range of about 10^{3} to 10^{4} \; $\Mspc$2 if adopting an \alphaCO = 4.3  \mathrm{M_{\odot} \; (K   km   s^{-1}   pc^2)^{-1}} (\citealt{Bolatto2013}) and a CO excitation R_{21} = 0.65 (\citealt{Leroy2021c}).$
$The reasons for the nuclei showing this trend likely include both the effect of varying [$\CI$/CO] abundance ratio and the excitation and radiative transfer. They are determined by the actual ISM conditions in each environment, i.e., the gas kinetic temperature, $\CI$  and CO column densities ($\CI$columndensity and $\CO$columndensity), line widths (\Delta{v}), and \mathrm{H_2} volume density (\nHtwo), etc.$
$They will be analyzed in Sect.~\ref{sec: Excitation Analysis}.$
$\subsubsection{Histogram and curve-of-growth of the line ratio distributions}$
$Fig.~\ref{fig: line ratio histogram} shows the histogram and cumulative distribution functions of \RCICO (left panel) and the mean \RCICO and the 16-th to 84-th percentiles in bins of CO line intensity (right panel) in our galaxies.$
$The histogram is computed as the sum of CO line intensity in each bin of \RCICO, representing the amount of gas at each \RCICO.$
$The NGC~4321 histogram is lower than others, showing that it has the least total molecular gas mass among the four galaxies within mapped areas. The NGC~3627 histogram has a peak about 0.5~dex higher than that of NGC~4321, but their peak locations are consistently at \log \RCICO \sim -1, same conclusion as our previous scatter plots show.$
$The two SB galaxies NGC~1808 and NGC~7469, despite having much smaller observed areas (\sim 1--2~kpc), have more than an order of magnitude higher peaks hence total molecular gas masses.$
$The locations of their histogram peaks also shift to a systematically higher \RCICO.$
$It is clearly seen that both SF and SB disks lack a high-\RCICO (\log \RCICO > -0.35) gas component which dominates the gas in the strong AGN host galaxy NGC~7469.$
$The curve-of-growth in the left panel of Fig.~\ref{fig: line ratio histogram} further shows the cumulative CO brightness in bins of \RCICO, and reflects the fact that the majority of gas in our galactic disks has a narrow distribution of \RCICO (see also last few rows in Table~\ref{table: targets}).$
$In NGC~3627, NGC~4321 and NGC~1808, the higher- and lower-\RCICO gas outside \pm 0.1~dex around the mean \RCICO only sums up to <5--10\% of their total gas masses.$
$In the right panel of Fig.~\ref{fig: line ratio histogram}, we illustrate how the mean \RCICO changes with different gas surface density, as well as the scatter of the data point distribution. The 2D pixels are binned by CO brightness, and the mean and 16\%/50\%/84\%-th percentiles of the \RCICO are computed for each bin.$
$It sketches out the mean trends in Fig.~\ref{fig: line ratio vs CO} (right panel), and also reveals that$
$a) the scatter of \RCICO is the largest (\pm 0.20~dex) in the starburst disk of NGC~7469 compared to other galaxies;$
$b) the scatter is the smallest (\pm 0.06~dex) in the star-forming disks of NGC~4321 and NGC~3627 where \Sigmol \sim 100 \text{--} 500 \; $\Mspc$2;$
$c) the scatters are very similarly (\pm 0.10~dex) at a relatively high \Sigmol \sim 1000 \; $\Mspc$2 in all our galaxies except for NGC~4321's centre.$
$\subsection{\RCICO versus ISRF}$
$\label{subsection: ISRF}$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.9\textwidth]{Plot_CI_CO_line_ratio_vs_IRSF_intensity_maps}$
$\caption{$
$    Line ratio versus mean ISRF intensity \Umean measured from SED fitting to the near- to far-IR (\textit{Spitzer}, \textit{Herschel}) data.$
$    \textit{Left four panels} are the \Umean maps of NGC~3627, NGC~4321, NGC~1808 and NGC~7469, and as described in Sect.~\ref{subsection: ISRF} are obtained from the works done by J. Chastenet et al. (in prep.) as a continued effort of \citet{Chastenet2021}.$
$    \textit{Right panel} shows the scatter plot of \RCICO versus \Umean at a spatial resolution of \sim 18" in the manually-selected apertures shown in the left panels.$
$    We computed a beam-weighted mean \RCICO and a centre pixel \RCICO for each aperture, displayed as the solid and open symbols, respectively.$
$    Error bars are the corresponding beam-weighted mean or central pixel \RCICO  uncertainties in each region.$
$    The region diameter (\sim 18") corresponds to the coarsest resolution of the \textit{Herschel} data used in the SED fitting.$
$    The gray dotted line shows \RCICO = 0.07   U^{0.25} as a guiding line.$
$    The colored dotted contours (labeled 0.1--0.5) are the model contours of \RCICO computed from the PDR Toolbox (using the latest $\incode${wk2020} model and solar metallicity; \citealt{Kaufman2006}). We convert G_0 from the model contour to the x-axis U by multiplying with a factor of 0.88 (\citealt{Draine2007}).$
$    The right y-axis indicates the PDR model's \nHtwo, whose range is manually adjusted so that the model contours roughly match the left x-axis.$
$    The PDR model contours show little dependence of \RCICO on U in the horizontal direction but a strong correlation with \nHtwo as seen by the vertical gradient.$
$\label{fig: line ratio vs ISRF}$
$}\vspace{1ex}$
$\end{figure*}$
$Here we study how \RCICO  correlates with the interstellar radiation field (ISRF) intensity (U; in units of Milky Way mean ISRF; \citealt{Habing1968}), which is a measure of the UV radiation field strength impacting the ISM. Since $\CI$  originates from PDRs, its abundance should be highly related to the PDR properties, thus whether or how \RCICO  correlates with the ISRF that illuminates the PDR is a key question.$
$The ISRF intensity U is usually measured by the re-radiated dust emission, as the original UV photons are highly attenuated by dust in the ISM. \citet[][hereafter DL07]{DL07} developed a series of dust grain models and synthesized SED templates that can describe nearby galaxies' observed dust SEDs. A number of studies (e.g., \citealt{Draine2007,Galliano2011,Dale2012,Aniano2012,Aniano2020,Daddi2015,Chastenet2017,Liudz2021a,Chastenet2021}) have used the DL07 models to fit the dust SEDs of galaxies near and far.$
$For our study, we adopt the mass-averaged ISRF \Umean maps from an ongoing effort by J. Chastenet et al. (in prep.) which is a continuation of the work published for other galaxies by \citet{Chastenet2017,Chastenet2021} with the same method.$
$In this method, the \textit{Herschel} far-infrared, \textit{Spitzer} and/or \textit{WISE} near-infrared images \footnote{For the two (U)LIRGs, the WISE maps are saturated, so their inferred ISRF intensity U should be considered with a large uncertainty.} are first PSF-matched to a common resolution \sim 18" (\textit{Herschel} SPIRE 250\mum), then SED fitting is performed using the DL07 models at each pixel.$
$Each dust SED is fitted by two dust components, one in the ambient ISM with a constant ISRF intensity U_{\mathrm{min}}, and the other in regions with higher U with a power law distributed ISRF from U_{\mathrm{min}} up to U_{\mathrm{max}} = 10^{7}.$
$The fitting then gives the best-fit U_{\mathrm{min}} and the power-law index \gamma of the U distribution.$
$The mean ISRF intensity \Umean is calculated as the dust mass-weighted mean of the ambient and power-law ISRF intensities (in DL07, U_{\rm PDR} \ge 10^2 is considered to be associated with the PDR), and better represents the overall ISRF than U_{\mathrm{min}}.$
$Fig.~\ref{fig: line ratio vs ISRF} presents the \Umean maps and the \RCICO  versus \Umean scatter plot. Since the spatial resolution of the \Umean maps is 18", we calculate a weighted \RCICO in each circular aperture shown in Fig.~\ref{fig: line ratio vs ISRF} whose diameter equals the \Umean map resolution, with weighting following the 2D Gaussian beam.$
$In addition, we also show the central pixel value for each region in Fig.~\ref{fig: line ratio vs ISRF}.$
$We restrict the analysis to regions where both \Umean and \RCICO  are measured.$
$A weak correlation between \RCICO  and \Umean is seen from the SF to SB disks, with a large scatter. For reference, we plot a \RCICO = 0.07   U^{0.25} line to indicate this weak trend. The lowest-\Umean data point in NGC~3627 (mainly Region~22) and the centre of NGC~7469 do not follow this trend, possibly indicating other $\CI$  enhancement mechanisms than \Umean.$
$We use the popular PDR modelling software, PDR Toolbox (version 2.2.9; \citealt{Kaufman2006,Pound2008,pdrtoolbox}) \footnote{\url{https://dustem.astro.umd.edu/}; \url{https://github.com/mpound/pdrtpy}.}$
$to compute the theoretical \RCICO in uniform grids of ISRF intensity in the \citet{Habing1968} G_0 unit and H nucleus density n_{\mathrm{H}} (both in log space). We adopt the latest $\incode${wk2020} model set and a solar metallicity (\mathrm{[O/H_2]} = 3.2 \times 10^{-4}, \mathrm{[C/H_2]} = 1.6 \times 10^{-4}).$
$The ISRF intensity grid can be directly scaled to U by multiplying a factor 0.88 (\citealt{Draine2007}), corresponding to the x-axis of Fig.~\ref{fig: line ratio vs ISRF}.$
$The n_{\mathrm{H}} grid is not directly linked to the left y-axis of Fig.~\ref{fig: line ratio vs ISRF}, but because the model contours show fairly flat \RCICO versus U trends, we can match the model contours and indicate the n_{\mathrm{H}} grid in the right y-axis of Fig.~\ref{fig: line ratio vs ISRF}.$
$The PDR modelling predicts generally flat \RCICO versus U trends similar to the distribution of the observed data points, although our U are diluted values at a much larger physical scale than in the PDR models.$
$However, in PDR models, \RCICO  changes moderately with n_{\mathrm{H}}, and a relatively high n_{\mathrm{H}} \sim 10^{3.5\text{--}3.7}   \mathrm{cm^{-3}} is needed to reproduce an \RCICO \sim 0.1--0.2, or equivalently \sim 1.0--1.9 for the ratio of $\Remark${CI10} and $\Remark${CO21} fluxes in \mathrm{erg s^{-1} cm^{-2} sr^{-1}} units used by the PDR Toolbox \footnote{See \url{https://github.com/mpound/pdrtpy-nb/blob/master/notebooks/PDRT_Example_Model_Plotting.ipynb}.$
$We scale the ratio of intensities in \mathrm{erg s^{-1} cm^{-2} sr^{-1}} to the ratio of intensities in \mathrm{K km s^{-1}} by a factor of (\nu_{\mathrm{CI10}}/\nu_{\mathrm{CO21}})^{-3} = 9.73, where \nu is the rest-frame frequency of the lines. We caution that the old version of $\incode${PDRToolbox} $\incode${v2.1.1} does not include the $\incode${CI_609/CO_21} line ratio, but has $\incode${CI_609/CO_43} and $\incode${CO_43/CO_21} ratios. However, we tested that the multiplication of $\incode${CI_609/CO_43} and $\incode${CO_43/CO_21} does not produce the same result as the direct $\incode${CI_609/CO_21} ratio in the new $\incode${PDRToolbox} $\incode${v2.2.9}. We use the new version for this work.$
$}.$
$Furthermore, the PDR model needs a \sim0.5~dex lower n_{\mathrm{H}} to explain the 2\times elevated \RCICO  in SB disks, which seems somewhat counterintuitive.$
$This apparent tension is likely caused by the observations probing a much coarser resolution that consists of a large number of PDRs with different n_{\mathrm{H}} and U (e.g., increasing U from 10 to 400 can increase \RCICO from 0.2 to 0.5 based on our PDR Toolbox modelling). It may also relate to an abundance variation driven by non-PDR mechanisms ---$
$X-ray dominated region (XDR; \citealt{Maloney1996,Meijerink2005,Meijerink2007}; see also review by \citealt{Wolfire2022}) and Cosmic Ray Dominated Region (CRDR; \citealt{Papadopoulos2010c,Papadopoulos2011,Papadopoulos2018,Bisbas2015,Bisbas2017}) are well-known to play an important role in starbursting and AGN environments, respectively.$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=0.9\textwidth]{Plot_CI_CO_line_ratio_vs_metallicity_maps}$
$\caption{$
$    \textit{Left two panels} show the metallicity maps of NGC~3627 and NGC~4321 from \citet{Williams2022} based on PHANGS-MUSE (\citealt{PHANGSMUSE}) data. Boxes are regions as newcommandined in Figs.~\ref{fig: NGC3627 line ratio map} and \ref{fig: NGC4321 line ratio map}.$
$    \textit{Right panel} shows the \RCICO  versus metallicity scatter plot for regions in NGC~3627 and NGC~4321. The bottom-axis is metallicity in solar units, top-axis is metallicity in 12+\log(\mathrm{O/H}), left-axis is the line ratio of $\Remark${CI10} and $\Remark${CO21} in $\Kkms$ units, and right-axis is the ratio of fluxes in \mathrm{erg s^{-1} cm^{-2} sr^{-1}} units.$
$    The left- and right-axes are matched by a factor of (\nu_{\mathrm{C{$\textsc${i}}1\textnormal{--}0}}/\nu_{\mathrm{CO1\textnormal{--}0}})^3 \times R_{21}, where we adopt a R_{21}=0.65 (\citealt{Leroy2021c}).$
$    Data points are the mean metallicity and \RCICO  in each region shown in left panels, with error bars indicating the typical uncertainty in \RCICO , and vertical color shading indicating the 16- to 84-th percentiles of pixels' \RCICO  distribution inside each region.$
$    The \citet{Bolatto2000b} empirical fitting and \citet{Bolatto1999} model prediction lines are overlaid.$
$    The three outliers in NGC~3627 with \RCICO \sim 0.19, 0.28 and 0.35 are Regions 20, 21 and 22, respectively. The outlier point in NGC~4321 with \RCICO \sim 0.06 is its centre Region 1.$
$    \label{fig: line ratio vs metallicity}$
$}\vspace{1ex}$
$\end{figure*}$
$As pointed out by \citet{Papadopoulos2011} and later works, ultra-luminous infrared galaxies (ULIRGs) with IR luminosities L_{\mathrm{IR, 8-1000\mu\mathrm{m}}} \ge 10^{12}   $\Lsun$ and corresponding \mathrm{SFRs} > 1000   $\Msyr$ have cosmic-ray (CR) energy densities \sim 10^{3} \text{--} 10^{4}\times larger than the Milky Way. In our sample, the two SB galaxies are not as intensively star-forming as ULIRGs, but their SFRs are still a factor of 2 to 20 higher than our SF galaxies. More importantly, the SB disks have considerably higher (e.g., one to two orders of magnitude) SFR surface densities than the SF disks, plausibly leading to a factor of a few tens to hundreds higher CR intensities. The CRs emitted from OB star clusters and supernova remnants can penetrate deeply into the ISM, and thus nearly uniformly illuminate the ISM and effectively dissociate CO into $\CI$ .$
$A detailed simulation by \citet{Bisbas2017} showed that 10\times and 100\times higher CR ionization rates (\xi_{\mathrm{CR}}) than the Milky Way level results into a factor of about 2 and 3 mildly-increased $\CI$  abundance and about 1/2 and 1/30 strongly-decreased CO abundance, respectively (see their Fig.~3), in a 10~pc GMC with a mean gas density \left<n_{\mathrm{H}} \right>= 760   \mathrm{cm^{-3}}, resembling Milky Way and typical SF galaxies' ISM conditions.$
$Our SB galaxies probably have a \xi_{\mathrm{CR}} around or slightly above 10, but not reaching 100 given their mild \Umean \sim 20-30 compared to the \Umean \gtrsim 50 in extreme SB galaxies, e.g., local ULIRGs and high-redshift hyperluminous IR-bright (HyLIRG) and submm galaxies (\citealt{Daddi2015,Silverman2018,Liudz2021a}).$
$In Sect.~\ref{sec: Excitation Analysis}, we show that the mildly-increased [$\CI$/CO] abundance ratio from a CRDR and temperature/density-driven excitation can lead to the factor of 2 increase in \RCICO  when going from SF to SB disks.$
$\subsection{\RCICO versus Metallicity}$
$\label{subsec: metallicity}$
$Our current sample covers a relatively small range in metallicity for a metallicity--line ratio study. Given their stellar masses and metallicity maps (e.g., \citealt{Kreckel2019,Kreckel2020,Williams2022}), our targets probe a near-solar metallicity. NGC~3627 has a well-measured$
$metallicity gradient of Z = 8.53 + 0.06 \times[r/R_{25}] from \citet{Kreckel2019} \footnote{\citet{Kreckel2019} used a distance of 10.6 \pm 0.9 \; \mathrm{Mpc} for NGC~3627. We do not directly use this metallicity gradient, but use the metallicity maps from \citet{Williams2022} where the same new distance as in this work is adopted.}, but the metallicity of individual $\ion$izedhydrogen  regions scatter between \sim 8.45--8.65.$
$\citet{Williams2022} produced the 2D spatial distributions of gas-phase metallicity in PHANGS-MUSE galaxies, including NGC~3627 and NGC~4321, by applying the Gaussian Process Regression to map the smooth, higher-order metallicity variation from the individual, sparse $\ion$izedhydrogen  regions detected in PHANGS-MUSE (\citealt{PHANGSMUSE, Santoro2021}).$
$This method is significantly superior to a na\"{i}ve nearest neighbor interpolation method. We refer the reader to \citet{Williams2022} for the technical details and validation demonstration.$
$In Fig.~\ref{fig: line ratio vs metallicity} we present the trend between \RCICO  and metallicity in NGC~3627 and NGC~4321.$
$Due to our limited metallicity range, only a very weak decreasing trend can be seen.$
$A similar trend has been reported by \citet{Bolatto2000b}, who observed CO(1--0) and $\Remark${CI10} in the Region N27 of the Small Magellanic Cloud and combined with literature data to present a correlation between \RCICO  and metallicity over \sim 0.2 \text{--} 2   Z_{\odot}.$
$We overlay the \citet{Bolatto2000b} empirical fitting \footnote{$
$We adopt the equation from \citep[][Fig.~2 caption]{Bolatto2000b}, with a typo-correction in the brackets so that the equations are:$
$\log ( F_{$\Remark${CI10}} / F_{$\Remark${CO10}} ) = -0.47   \left[12 + \log \mathrm{(O/H)} \right]+ 5.0 and \log ( F_{$\Remark${CI10}} / F_{$\Remark${CO10}} ) = -0.82   \left[12 + \log \mathrm{(O/H)} \right]+ 8.2, respectively, where F is the line flux in \mathrm{erg   s^{-1}   cm^{-2}   sr^{-1}}.$
$}$
$as well as a theoretical PDR model prediction line from \citet{Bolatto1999} in Fig.~\ref{fig: line ratio vs metallicity}.$
$Although our data covers only a small metallicity range, it is encouraging to see the overall good agreement with the observationally-driven trend in \citet{Bolatto2000b}.$
$More detailed understanding of the deviation from PDR models will likely need more $\CI$  mapping in lower-metallicity galaxies.$
$\vspace{2ex}$
$\section{Estimation of abundance ratios under representative ISM conditions}$
$\label{sec: Excitation Analysis}$
$The observed \RCICO  depends on not only the $\CI$  and CO abundances ($\CO$abundance  and $\CI$abundance ), but also the ISM conditions that determine the excitation (level population) and radiative transfer of the C atoms and CO molecules --- i.e., the gas kinetic temperature \Tkin, volume density of H_2 (\nHtwo; as the primary collision partners of C and CO), turbulent line width \dv, and the species' column densities $\CO$columndensity and $\CI$columndensity.$
$The column density divided by the line width (N/\dv) further determines the optical depth \tau of each line.$
$In this work, having only a CO(2--1) and a $\Remark${CI10} line is insufficient to numerically solve all the \Tkin, \nHtwo, $\CO$columndensity/\dv, $\CO$abundance  and $\CI$abundance . Therefore, we assume several \textit{representative ISM conditions} where \Tkin, \dv and $\CO$abundance  are fixed to reasonable values, then solve the $\CO$columndensity and $\CI$columndensity. We assume that $\CI$  and CO are spatially mixed at our resolution, i.e., they have the same filling factor. This is a relatively strong assumption and is likely not the real case, because although we see spatial co-localization at our \sim 200~pc resolutions, the spectral profiles of CO and $\CI$ show discrepancies (Appendix~\ref{appendix: all regions}), indicating different underlying distributions at smaller scales. With the assumption of the same filling factor (and 3D distribution), the column density ratio $\CI$columndensity/$\CO$columndensity equals the [$\CI$/CO] abundance ratio.$
$We perform both LTE and non-LTE calculations.$
$LTE assumes that all transitions are thermalized locally, and the level population follows Boltzmann statistics, so that each transition's excitation temperature T_{\mathrm{ex}} as newcommandined by the level population equals \Tkin (e.g., \citealt{Spitzer1998Book,Tielens2010Book,Draine2011Book}). The LTE assumption facilitates the computation as we do not need to solve the level population and thus \nHtwo is not used.$
$However, LTE is not easily reached in typical ISM conditions. Subthermalized lines need non-LTE calculations. The commonly adopted non-LTE approach is the Large Velocity Gradient (LVG; \citealt{Scoville1974,Goldreich1974}), where an escape fraction is used to facilitate the local radiation intensity calculation hence the detailed balance of radiative and collisional (de-)excitations.$
$We use RADEX (\citealt{RADEX}) to compute the non-LTE $\CI$  and CO line fluxes for each fixed parameter set (\Tkin, \nHtwo, $\CO$columndensity, $\CI$columndensity, \dv), then study how [$\CI$/CO] determines the line flux ratio \RCICO.$
$We assume the following 7 representative ISM conditions:$
$\textit{(1)} inner Galactic GMCs,$
$\textit{(2)} NGC~3627/4321 disk GMCs,$
$\textit{(3)} NGC~3627 bar-end environments,$
$\textit{(4)} NGC~1808/7469 SB disk,$
$\textit{(5)} NGC~4321 centre,$
$\textit{(6)} NGC~3627 centre,$
$and \textit{(7)} NGC~7469 centre.$
$Conditions \textit{(1)}--\textit{(3)} correspond to the "SF disk" in Fig.~\ref{fig: line ratio vs CO}, and condition \textit{(4)} represents the "SB disk".$
$The last three conditions are for the "nuclei".$
$Under each ISM condition, we set a fixed \Tkin (in units of K), \dv (in units of km~s^{-1}), $\CO$abundance , and \Sigmol (in units of $\Mspc$2).$
$The adopted values are shown at the top of each panel in Figs.~\ref{fig: LTE RCICO} and \ref{fig: NonLTE RCICO}.$
$Our choice of \Tkin considered multi-line measurements in the literature, e.g., \citet{Heyer2015} for Galactic GMC ISM condition, and \citet{Teng2022} for NGC~3627 nucleus, as well as typical values in simulations (e.g., \citealt{Offner2014,Glover2015,Glover2016,Clark2019,HuChiaYu2021a}).$
$Our \dv is based on the observed line width in the highest-resolution ALMA data, i.e., from the original PHANGS-ALMA 1" data (\citealt{Leroy2021phangs}) or from \citet{Salak2019} and \citet{Izumi2020} for NGC~1808 and NGC~7469, respectively. The observed line width should at least new an upper limit, if not directly probing the turbulent \dv.$
$The $\CO$abundance  is more like a fiducial value and does not obviously affect later [$\CI$/CO] results.$
$We also take a fiducial \Sigmol which matches the observed CO(1--0) line brightness temperature and a CO-to-H_2 conversion factor \alphaCO \sim 4 \text{--} 0.8 \; \mathrm{M_{\odot}   (K   km   s^{-1})^{-1}} depending on the assumed conditions (e.g., \citealt{Bolatto2013}; and our paper~II).$
$This \Sigmol directly converts to an N_{\mathrm{H_2}} if assuming that 36\% of the mass is helium. Then N_{\mathrm{H_2}} and $\CO$abundance  together indicate a $\CO$columndensity . Only with these assumptions can we unambiguously link an abundance ratio [$\CI$/CO] (i.e., $\CI$columndensity/$\CO$columndensity) to a line flux ratio \RCICO .$
$Additionally, with the same filling factor assumption, \RCICOis also equivalent to the ratio of the CO-to-H_2 and $\CI$ -to-H_2 conversion factors, i.e., \RCICO = \alphaCO/\alphaCI, because they trace the same molecular gas at our resolution.$
$In a companion paper (paper II; Liu et al. 2022 in prep.), we study the behaviors of \alphaCO and \alphaCI in more details, also under both LTE and non-LTE conditions.$
$\subsection{LTE results}$
$\label{subsec: LTE}$
$\begin{figure*}[htb]$
$    \centering$
$    \includegraphics[width=0.99\textwidth]{Plot_LTE_RCI10CO21_vs_XCICO_varying_N_CO_T_kin}$
$    \caption{$
$        Observed/calculated \RCICO as a function of CO column density $\CO$columndensity  and [$\CI$/CO] abundance (column density) ratio \RNCINCO for 7 representative ISM conditions, based on LTE calculations.$
$        \textit{Upper panels}: Curved lines are \RCICO versus \RNCINCO for different $\CO$columndensity  as indicated by the color. Dashed means CO(2--1) optically thick and $\Remark${CI10} optically thin, dotted means both are both optically thin, and solid line means both optically thick.$
$        Horizontal thick line is the observed \RCICO value.$
$        \textit{Lower panels}: Black contours are \RCICO = 0.05,   0.10,   0.20 and 0.50 in the CO column density versus $\CI$/CO abundance ratio plane.$
$        Vertical color shading indicates the final \RNCINCO that can match both the observed \RCICOand the assumed \Sigmol value noted at the top of each panel.$
$    }$
$    \label{fig: LTE RCICO}$
$\end{figure*}$
$We determine the LTE solution of [$\CI$/CO] in Fig.~\ref{fig: LTE RCICO}.$
$To illustrate the complexity of this problem and the degeneracy of the \Tkin, $\CO$columndensity/\dv and [$\CI$/CO] parameters, we show how \RCICO (y-axis) varies with \RNCINCO (x-axis) and $\CO$columndensity  (color bar) in the upper panels, and how \RCICO (contour) can be pinpointed by determining the \Sigmol (right y-axis).$
$At the top of each column the key parameters of the aforementioned representative ISM conditions are shown as text. The assumed \Tkin and \dv generally increase from panels (1) to (7) mimicking real conditions. Observational-driven $\Remark${ICO10} ranges and the assumed \Sigmol for these ISM environments are also listed, and can be converted one into the other via \alphaCO. We also list the chosen $\CO$abundance  for clarity.$
$As shown in the upper panels, \RCICO increases with both \RNCINCO and $\CO$columndensity  under low-\Tkin conditions. However, a maximum \RCICO is reached in the high $\CI$  column density regime, where an increasing abundance ratio will no longer increase \RCICO. The line ratio saturates at about 0.5 when \Tkin \sim 15~K, or at about 1 for \Tkin \sim 100~K. The observed line ratios (horizontal dashed lines) are all lower than the saturation limit of \RCICO _{\mathrm{max}} \sim 0.7 \text{--} 0.9, and corresponds to a range of $\CO$columndensity or \Sigmol. This range is very wide in the low-\Tkin conditions, with \log $\CO$columndensity / \mathrm{cm^{-2}} \sim 17 - 19, but is very narrow at \Tkin \sim 100~K, that is, \RCICO can be determined from [$\CI$/CO] without much dependence on $\CO$columndensity.$
$In the lower panels, the contours of \RCICO = 0.05, 0.1, 0.2 and 0.5 are shown in the plane of CO column density $\CO$columndensity versus $\CI$/CO abundance ratio.$
$For a fixed \RCICO , a higher \RNCINCO corresponds to a lower $\CO$columndensity in the low-\Tkin regime, but \RCICO  becomes insensitive to $\CO$columndensity at a high \Tkin. In the latter case, \RNCINCO alone determines \RCICO .$
$By matching to the expected \Sigmol for each ISM condition, we can obtain an unambiguous \RNCINCO from the observed \RCICO  as shown in Fig.~\ref{fig: LTE RCICO}.$
$In this way, we determine the \RNCINCO to be around 0.1--0.2 for conditions \textit{(1)} and \textit{(2)}, and \sim 0.4 for conditions \textit{(3)} and \textit{(5)}.$
$The SB disk (condition \textit{4}) and AGN environments (conditions \textit{6}/\textit{7}) have \RNCINCO \sim 2 and \sim 1 \text{--} 10, respectively.$
$The strongly enhanced $\CI$/CO abundance ratios in SB disk and AGN environments are clear signatures of CRDR and XDR, respectively.$
$The NGC~4321 centre is a low-ionization nuclear emission-line region (LINER, e.g., \citealt{GarciaBurillo2005}), and given our assumptions it requires \RNCINCO \sim 0.4 in order to explain the observed low line ratio.$
$The gradually increasing $\CI$/CO abundance ratios in the nuclei of NGC~4321, NGC~3627, NGC~1808 and NGC~7469 agree well with the increasing power of AGN activities, e.g., their X-ray luminosities (Table~\ref{table: targets}).$
$Our assumptions for the Galactic cloud ISM condition are mainly based on \citet{Heyer2009}, \citet{Heyer2015} and \cite{RomanDuval2010}. The obtained [$\CI$/CO]~\sim 0.1 also agrees well with previous studies using $\CI$  and CO isotopologues (see Sect.~\ref{subsec: intro RCICO}.$
$For example, \citet{Ikeda1999} reported \RNCINCO \sim 0.05 \textnormal{--} 0.2 for the Orion cloud;$
$\citet{Oka2001} found \RNCINCO = 0.064 \pm 0.035 but a peak of \RNCINCO \sim 0.15 in the DR 15 $\HII$  region and two IR dark clouds;$
$\citet{Kramer2004} found \RNCINCO \sim 0.11 \textnormal{--} 0.12 in the massive SF region W3;$
$\citet{Genzel1988} found \RNCINCO \sim 0.2 for the massive SF region W51;$
$\citet{Zmuidzinas1988} reported \RNCINCO \sim 0.05 \textnormal{--} 0.15 in several dense clouds.$
$Recently, \citet{Izumi2021} derived an overall \RNCINCO \sim 0.1 in the massive SF region RCW38 at A_V \sim 10 \textnormal{--} 100, consistent with the Orion cloud, but also saw high [$\CI$/CO] \sim 0.2 \textnormal{--} 0.6 in some even lower column density regions.$
$Accurate determination of all the ISM properties instead of simple assumptions as in this work requires comprehensive, multi-J CO isotopologue observations (e.g., \citealt{Teng2022}).$
$Our representative ISM conditions only new a qualitative diagnostic to understand how to infer the $\CI$/CO abundance ratio from the observed \RCICO  quantity.$
$Their properties may not be very accurate, and are subject to change with future multi-tracer/CO isotopologue studies.$
$\begin{figure*}[htb]$
$\centering$
$\includegraphics[width=\textwidth]{Plot_RADEX_R_CI10CO21_sim_XCICO_NonLTE_varying_N_CO_T_kin_n_H2}$
$\caption{$
$Similar to Fig.~\ref{fig: LTE RCICO} but with non-LTE calculations.$
$See Fig.~\ref{fig: LTE RCICO} caption for the details.$
$}$
$\label{fig: NonLTE RCICO}$
$\end{figure*}$
$\subsection{Non-LTE results}$
$\label{subsec: non-LTE}$
$We perform the non-LTE calculations using the RADEX code (\citealt{vanderTak2007})$
$with the Large Velocity Gradient (LVG; i.e., expanding sphere) escape probability (\citealt{Scoville1974,Goldreich1974}).$
$Comparing to LTE, the volume density of H_2 is now an additional property that plays a role. The excitation temperature of each line no longer equals \Tkin.$
$We compute the non-LTE line ratios in a grid of ( \Tkin,   \dv,   $\CO$abundance,   \RNCINCO,   $\CO$columndensity,   \nHtwo ) parameter sets, then study the line and abundance ratios similarly as in the previous section.$
$In Fig.~\ref{fig: NonLTE RCICO} we show the non-LTE version of Fig.~\ref{fig: LTE RCICO}.$
$The non-LTE result does not qualitatively differ from that of LTE.$
$The most obvious difference is that at high temperature (\Tkin > 50~K, given our other assumed conditions), the required [$\CI$/CO] is lower by \times 2 for the same observed \RCICO, whereas at low temperature (\Tkin \sim 15~K) a \lesssim 2 \times [$\CI$/CO] is expected for the observed line ratio.$
$As listed in the title of each panel, we assume \log \nHtwo / \mathrm{cm^{-3}} = 2.5 to 3.0 for our conditions (1) to (7).$
$This is based on the fact that low-J CO and $\CI$  trace the bulk of molecular gas with \log \nHtwo / \mathrm{cm^{-3}} \sim 2--3 (e.g., \citealt{Leroy2017,Liudz2021a}), with a typical value \nHtwo \sim 300   \mathrm{cm^{-3}} widely adopted/found for Galactic and nearby galaxy GMCs (e.g., \citealt{Koda2012,Pety2017}, see also simulations of \citealt{Glover2012b}), whereas a smaller portion of their emission is from the dense gas with \log \nHtwo / \mathrm{cm^{-3}} \sim 4--6 (e.g., \citealt{Gao2004a}).$
$The fraction of the dense gas is expected to increase in more starbursting environment (\citealt{Gao2004b}), so does the mean gas density (\citealt{Liudz2021a}). Therefore, a \log \nHtwo / \mathrm{cm^{-3}} of 3.0 is perhaps a reasonable assumption, or at least a lower limit for our galaxy centres. A 10 \times higher density does not qualitatively change Fig.~\ref{fig: NonLTE RCICO} except for shifting the expected \RNCINCO to be 50\% higher in the \Tkin > 50~K conditions (making Fig.~\ref{fig: NonLTE RCICO} more like Fig.~\ref{fig: LTE RCICO}).$
$Therefore, our non-LTE result strongly confirms the LTE result in the previous section, that the intrinsic $\CI$/CO abundance has to vary significantly from about 0.1--0.2 to about 1--3, in order to explain the observed line ratios from Galactic GMCs (hundred pc scales) and NGC~3627/4321 SF disk GMCs to NGC~1808/7469 SB disk/AGN (r \lesssim 250~pc) environments.$
$\vspace{2ex}$
$\section{Discussion}$
$\label{sec: Discussion}$
$From this study we see that the line ratio \RCICO increases from 0.1 to 0.2 and > 0.5 from the SF disk to the SB disk and the vicinity (few hundred pc) of X-ray luminous AGN. Although we may be limited by our sample,$
$we suggest that this trend can be extended to general cases at both local and high redshift, because the $\CI$/CO abundance ratio enhancement is well-predicted by the CRDR and XDR theories.$
$\citet{Papadopoulos2010c} and \citet{Papadopoulos2011} pointed out the key role of CRDR in local (U)LIRGs. With UV photons alone, cold gas in starbursting environments can still be well shielded, hence as cold as \sim 10~K. However, with CRs produced from O, B star clusters and supernova remnants, the gas can be uniformly illuminated and heated up.$
$The extremely starbursty local ULIRGs have a CR density (\xi_{\mathrm{CR}}) 10^{3}--10^{4} times that of the Milky Way (\citealt{Papadopoulos2010c}), and thus will naturally dissociate more CO into $\CI$ .$
$\citet{Bisbas2017} and \citet{Papadopoulos2018} conducted simulations of molecular gas at tens of pc scale with varying CR strength, and found that when \xi_{\mathrm{CR}} is \sim 1 \textnormal{--} 100   \times Galactic value, the gas temperature in the dense gas remains largely unaffected (\sim 10~K).$
$However, when \xi_{\mathrm{CR}} reaches \sim 1000   \times Galactic value as in (U)LIRGs, the gas temperature increases to 30--50~K. The \citet{Bisbas2017} simulation also shows that the $\CI$  and $\ion$izedcarbon  fractional abundances at \log A_V \sim 0.5 increase by one and three orders of magnitude, respectively, and the CO fractional abundance decreases correspondingly by over two orders of magnitude, with an increasing \xi_{\mathrm{CR}} from the Galactic value to 100   \times of that.$
$These results are in line with our calculations in Sect.~\ref{subsec: LTE}.$
$Meanwhile, the AGN-driven XDR chemistry and molecule/atom excitation have been firstly modeled in great detail by \citet{Maloney1996}, \citet{Meijerink2005} and \citet{Meijerink2007}. In XDRs, photo-ionization becomes the dominant heating source instead of photo-electric emission from dust grains in PDRs, thus more molecules are dissociated and/or ionized. \citet{Meijerink2005}'s Model~1 shows that CO molecules are fully dissociated when N_{\mathrm{H}} < 10^{23}   \mathrm{cm^{-2}}, then the fractional abundance is gradually increased by six orders of magnitudes from N_{\mathrm{H}} = 10^{23}   \mathrm{cm^{-2}} to 10^{24}   \mathrm{cm^{-2}}. By contrast, the $\CI$  and $\ion$izedcarbon  abundances are about constant until reaching the densest part (N_{\mathrm{H}} > 10^{24.5}   \mathrm{cm^{-2}}). Unlike in PDRs, there is no clear $\ion$izedcarbon /$\CI$ /CO stratification in XDRs (\citealt{Meijerink2007,Wolfire2022}). Their Model~2 with a 100 \times higher energy deposition rate shows that the CO distribution contracts to the highest gas density spots, and the total cumulative [$\CI$/CO] is increased by about 0.2~dex, reaching \RNCINCO \sim 1.6.$
$This value agrees well with our calculations shown in Figs.~\ref{fig: LTE RCICO} and \ref{fig: NonLTE RCICO} (the ISM condition 7).$
$Therefore, both in theoretical and observational views, it is clear that the \RCICO line ratio is a good indicator of the starburst's CRDR and AGN's XDR.$
$If a \RCICO line ratio (to be specific, $\Remark${ICI10}/$\Remark${ICO21}) reaches 0.2--0.5, then the CRDR plays a crucial role and the abundance ratio [$\CI$/CO] could be \sim 1.$
$If it exceeds 0.5--1.0, then it may only be enhanced by an XDR, e.g., caused by an X-ray luminous AGN, which boosts the abundance ratio [$\CI$/CO] to \gtrsim 1--2.$
$\vspace{2ex}$
$\section{Conclusion}$
$\label{sec: Conclusion}$
$We presented new beam- and uv-coverage-matched $\Remark${CI10} and CO(2--1) maps in the galactocentric radius r \lesssim 7~kpc SF disk of NGC~3627 at \sim 190~pc resolution, and in the r \lesssim 3~kpc disk of NGC~4321 at \sim 270~pc resolution.$
$They are among the largest $\CI$  mosaic mappings available in nearby galaxies.$
$We combined the imaging of $\Remark${CI10} and CO(2--1) in two nearby, more starbursty and AGN-hosting galaxies, i.e., NGC~1808 central \sim1~kpc, and NGC~7469 central \sim1.5~kpc at 140--160~pc resolutions.$
$Together, we studied the spatial distributions of $\CI$  and CO, and their line ratio \RCICO as functions of various galaxy resolved properties, including CO brightness (Fig.~\ref{fig: line ratio vs CO}), ISRF \Umean (Fig.~\ref{fig: line ratio vs ISRF}), and metallicity (Fig.~\ref{fig: line ratio vs metallicity}).$
$In order to obtain the underlying intrinsic [$\CI$/CO] abundance ratio from the observed line ratio \RCICO, we assumed 7 representative ISM conditions and did both LTE and non-LTE calculations.$
$\vspace{1ex}$
$We summarize our findings as below:$
$\begin{itemize}[itemsep=0ex,topsep=1ex,leftmargin=4ex]$
$    \item The majority of the molecular gas in the SF disks of NGC~3627 and NGC~4321 has a uniform $\Remark${RCI10CO21} = 0.10 \pm 0.05.$
$    $\Remark${CI10} and $\Remark${CO21} exhibit very similar spatial distributions at our 140--270~pc resolution.$
$    \item The majority of the molecular gas in the SB disk of NGC~1808 has $\Remark${RCI10CO21} = 0.20 \pm 0.05.$
$    This elevated line ratio compared to the SF disk is likely caused by a \sim 3 \text{--} 5 \times increased [$\CI$/CO] abundance ratio, based on our LTE and non-LTE calculations and parameter assumptions (Figs.~\ref{fig: LTE RCICO} and \ref{fig: NonLTE RCICO}).$
$    \item The centres (r \lesssim 250~pc) of NGC~4321, NGC~3627, NGC~1808 and NGC~7469 show a strongly increasing trend in $\Remark${RCI10CO21}, from 0.05 to 0.5, as a function of I_{$\Remark${CO21}} --- i.e., $\Remark${RCI10CO21} \approx 0.035   (I_{$\Remark${CO21}}/[100\;$\Kkms$])^{0.8}.$
$    The inferred [$\CI$/CO] abundance ratio from our non-LTE calculation changes from \sim 0.2 to \sim 3, in line with the increasing X-ray luminosity of the centres/AGNs.$
$    \item The observed \RCICO versus ISRF trend is fairly flat, i.e., \RCICO  is insensitive to ISRF intensity changes, similar to the PDR model prediction calculated with PDRToolbox (although our resolution is much coarser than the physical scale of PDR models; Sect.~\ref{subsection: ISRF}).$
$    \item We find a \RCICO versus metallicity trend consistent with previous observations that covered lower-metallicity environments (\sim 0.2   \mathrm{Z_{\odot}}; \citealt{Bolatto2000b}), within the metallicity range (\sim 0.7 \text{--} 1   \mathrm{Z_{\odot}}) of our sample (Sect.~\ref{subsec: metallicity}).$
$    \item Our inferred [$\CI$/CO] abundance ratios agree well with the CRDR and XDR's theoretical studies.$
$    Given the drastically changed \RCICO , we propose that the $\CI$  to CO line ratio can be a promising indicator of CRDR (starbursting) and XDR (AGN). For instance, an observed line flux ratio of$
$    $\Remark${RCI10CO21} \sim 0.2 \textnormal{--} 0.5 and \gtrsim 0.5 effectively indicates CRDR and XDR, respectively.$
$    \item We do not find ubiquitous "CO-dark", $\CI$ -bright gas at the outer r\sim 1 \text{--} 3~kpc disks of NGC~3627 and NGC4321, nor systematic enhancement in \RCICO  at CO-faintest sightlines via stacking, mainly due to the incompleteness of our data.$
$    Nevertheless, a few sightlines, e.g., Region~22 in NGC~3627 show high \RCICO ~\sim 0.5 which do not fully comply with the noise behavior seen in our CASA simulations.$
$\end{itemize}$
$\vspace{2ex}$
$\begin{acknowledgements}$
$We thank the anonymous referee for very helpful comments.$
$ES, TS and TGW acknowledge funding from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme (grant agreement No. 694343).$
$ER acknowledges the support of the Natural Sciences and Engineering Research Council of Canada (NSERC), funding reference number RGPIN-2017-03987.$
$The work of AKL is partially supported by the National Science Foundation under Grants No. 1615105, 1615109, and 1653300.$
$AU acknowledges support from the Spanish grants PGC2018-094671-B-I00, funded by MCIN/AEI/10.13039/501100011033 and by "ERDF A way of making Europe", and PID2019-108765GB-I00, funded by MCIN/AEI/10.13039/501100011033.$
$RSK and SCOG acknowledge financial support from the German Research Foundation (DFG) via the collaborative research centre (SFB 881, Project-ID 138713538) “The Milky Way System” (subprojects A1, B1, B2, and B8). They also acknowledge funding from the Heidelberg Cluster of Excellence "STRUCTURES" in the framework of Germany’s Excellence Strategy (grant EXC-2181/1, Project-ID 390900948) and from the European Research Council via the ERC Synergy Grant "ECOGAL" (grant 855130).$
$FB and IB acknowledge funding from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme (grant agreement No.726384/Empire).$
$JC acknowledges funding from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme DustOrigin (ERC-2019-StG-851622).$
$MC gratefully acknowledges funding from the Deutsche Forschungsgemeinschaft (DFG) in the form of an Emmy Noether Research Group (grant number CH2137/1-1).$
$Y.G.’s work is partially supported by National Key Basic Research and Development Program of China (grant No. 2017YFA0402704), National Natural Science Foundation of China (NSFC, Nos.$
$12033004, and 11861131007), and Chinese Academy of Sciences Key Research$
$Program of Frontier Sciences (grant No. QYZDJ-SSW-SLH008).$
$AH was supported by the Programme National Cosmology et Galaxies (PNCG) of CNRS/INSU with INP and IN2P3, co-funded by CEA and CNES, and by the Programme National “Physique et Chimie du Milieu Interstellaire” (PCMI) of CNRS/INSU with INC/INP co-funded by CEA and CNES.$
$K.K. gratefully acknowledges funding from the German Research Foundation (DFG) in the form of an Emmy Noether Research Group (grant number KR4598/2-1, PI Kreckel)$
$JMDK and MC gratefully acknowledge funding from the Deutsche Forschungsgemeinschaft (DFG) in the form of an Emmy Noether Research Group (grant number KR4801/1-1) and the DFG Sachbeihilfe (grant number KR4801/2-1), and from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme via the ERC Starting Grant MUSTANG (grant agreement number 714907).$
$JP acknowledges support from the Programme National “Physique et Chimie du Milieu Interstellaire” (PCMI) of CNRS/INSU with INC/INP co-funded by CEA and CNES.$
$HAP acknowledges support by the Ministry of Science and Technology of Taiwan under grant 110-2112-M-032-020-MY3.$
$The work of JS is partially supported by the Natural Sciences and Engineering Research Council of Canada (NSERC) through the Canadian Institute for Theoretical Astrophysics (CITA) National Fellowship.$
$This paper makes use of the following ALMA data:$
$\incode{ADS/JAO.ALMA#2015.1.00902.S},$
$\incode{ADS/JAO.ALMA#2015.1.00956.S},$
$\incode{ADS/JAO.ALMA#2015.1.01191.S},$
$\incode{ADS/JAO.ALMA#2017.1.00078.S},$
$\incode{ADS/JAO.ALMA#2017.1.00984.S},$
$\incode{ADS/JAO.ALMA#2018.1.00994.S},$
$\incode{ADS/JAO.ALMA#2018.1.01290.S},$
$\incode{ADS/JAO.ALMA#2019.1.01635.S}.$
$ALMA is a partnership of ESO (representing its member states), NSF (USA) and NINS (Japan), together with NRC (Canada), MOST and ASIAA (Taiwan), and KASI (Republic of Korea), in cooperation with the Republic of Chile. The Joint ALMA Observatory is operated by ESO, AUI/NRAO and NAOJ.$
$\end{acknowledgements}$
$\bibliographystyle{aa_url}$
$\input{Article_CI_AA.bbl}$
$\begin{appendix}$
$\section{Data reduction, imaging and moment map generation}$
$\label{appendix: data reduction}$
$After downloading the raw data, i.e., 12m+7m for $\Remark${CO21} and 7m for $\Remark${CI10}, respectively, we ran the observatory calibration pipeline ("scriptForPI") to reduce the data.$
$As shown in \citet{Leroy2021pipeline}, the observatory calibration produces well-calibrated visibility data without further manual flagging.$
$We then split out the visibility data and proceeded to the imaging step.$
$In order to match the spatial resolution of $\Remark${CO21} and $\Remark${CI10}, we carefully matched their uv ranges.$
$We flagged the $\Remark${CO21} visibility data outside the uv range of 19--68~m (\sim15--52k\lambda) so as to match the uv range of the $\Remark${CI10} data (minimum to 80\% percentile: 8.9--32~m, or \sim15--52k\lambda).$
$Given that the $\Remark${CI10} uv plane distribution is elliptic, while the uv range selection in $\incode${CASA} is circular, our chosen thresholds are adjusted to make the final $\Remark${CO21} synthesized beam similar to that of $\Remark${CI10}.$
$Note that in a later step we also convolved both image cubes to the exact same rounded beam (3.46") before generating the moment maps.$
$The \textit{PHANGS-ALMA imaging \& post-processing pipeline} \footnote{\url{https://github.com/akleroy/phangs_imaging_scripts}; version 2.0.} developed by \citet{Leroy2021pipeline} is used for creating line cubes and moment maps. The code is suitable for general nearby galaxy strong line mapping interferometric and total power mosaic data. In brief, the visibility data are carefully transformed into the user-given channel width and frequency range corresponding to the line plus line-free channels, then continuum is subtracted by fitting an order of zero polynomial. Then, two stages of imaging ("cleaning") are performed: first a multi-scale cleaning down to \sim4\sigma, then a single-scale cleaning down to \sim1\sigma. At each stage, $\incode${CASA}'s $\incode${tclean} task is iteratively called so as to force many major cycles, which has proven to improve the accuracy of the deconvolution. Before and during the second stage cleaning, a clean mask is computed at each iteration according to the outcome of previous iteration by finding all >2\sigma pixels associated with >4\sigma peaks in a watershed algorithm (\citealt{Rosolowsky2006,Rosolowsky2021}). In this way the cleaning can include as much signal as possible and ensure a signal-free residual.$
$Then the image cubes are corrected for the primary beam attenuation, converted to Kelvin units, and convolved to a common beam, which is the coarser beam of $\Remark${CO21} and $\Remark${CI10}. Because of our uv range flagging before imaging, the directly $\incode${tclean}-imaged beam sizes are already close. This convolution further precisely matches $\Remark${CO21} and $\Remark${CI10} image cube beams.$
$Next, moment maps are generated with modules in the PHANGS-ALMA pipeline, consistently for CO and $\CI$ . This process includes firstly constructing noise and mask cubes from the image cubes, then combining the masks (i.e., where science signal should present), then computing moment 0 -- line integrated intensity, moment 1 -- line peak velocity, and moment 2 -- line width. The mask cubes are the key in producing moment maps. Here two masks are built: a "broad mask" which contains as much plausible science signal as possible, and a "strict mask" which is optimized for high-confidence signal. The strict mask is made by finding peaks that have >4\sigma over two consecutive channels then expanding down to 2\sigma also with the watershed algorithm. Then the broad mask is obtained by aggregating strict masks made from image cubes convolved to coarser resolutions (\sim30" in this work as well as in \citealt{Leroy2021pipeline}).$
$To make consistent moment maps for CO and $\CI$ , we aggregate their masks together as a joint mask, for broad and strict masks, respectively, then their final moment maps are computed within the joint mask.$
$The resulting $\Remark${CO21} and $\Remark${CI10} line intensity maps are then used for our next line ratio analysis. The line velocity and line width maps are not further analyzed, but are consistent with the products from the PHANGS-ALMA survey's data release as well as in \citet{Lang2020}. Given that the PHANGS-ALMA 12m+7m+tp $\Remark${CO21} has the full spatial information and are deeper, their line velocity and line width maps are more useful for kinematic studies than ours produced from this work.$
$\section{Assessing the missing flux in the NGC~3627 $\CI$  data}$
$\label{appendix: missing flux}$
$We assess the level of missing flux in the NGC~3627 $\CI$  data in three ways:$
$(a) comparing with the \textit{Herschel} SPIRE FTS $\Remark${CI10} observation in this galaxy,$
$(b) comparing the uv-trimmed CO(2-1) moment-0 map with the PHANGS-ALMA short-spacing corrected (12m+7m+tp) moment-0 map,$
$and (c) performing CASA simulation and comparing with the "true" moment-0 from the skymodel and the derived product from the simulated visibilities.$
$Method (a):$
$NGC~3627 has been observed with the \textit{Herschel} SPIRE (\citealt{Griffin2010}) Fourier Transform Spectrometer (FTS; \citealt{Naylor2010}) in mapping mode with nearly Nyquist sampling.$
$\cite{Liudz2015} reduced the archival data and extracted the line fluxes for each groups of pointings. $\Remark${CI10} is detected with \SNR\sim4-5 at three sightlines from the galaxy centre to the southern bar end, with R.A. and Dec. coordinates (J2000) 170.061905, +12.991336, 170.066361 +12.987091, and 170.071243 +12.981615, respectively. The $\Remark${CI10} luminosities are 1.44 \pm 0.31, 2.01 \pm 0.48, and 1.71 \pm 0.40 \times 10^{7} \; \mathrm{K km s^{-1} pc^{2}}, respectively, where the errors are purely from the RMS of line-free FTS spectra. By convolving our $\Remark${CI10} line cube to the FTS beam of \sim39" then extracting the moment maps, we measure a $\Remark${CI10} line luminosity of 0.52, 0.33 and 0.29 \times 10^{7} \; \mathrm{K km s^{-1} pc^{2}} at the above three positions, respectively, which means a missing flux of 36-75\% at the galaxy centre and 68-88\% at the southern bar end (at a 95\% confidence level).$
$Method (b):$
$We made ratio maps of our uv-trimmed CO(2-1) moment-0 map, and that of the original PHANGS-ALMA short-spacing-corrected (12m+7m+tp) CO(2-1) data but convolved to the corresponding resolution before making the moment maps. The ratio maps are made at two resolutions: the FTS 39" beam, and our working resolution of 3.46". The former leads to a missing flux of 26\%, 31\% and 36\% from the galaxy centre to the southern bar end. Note that the above \textit{Herschel} FTS analysis indicates in general a higher missing flux, but the FTS measurements are of low \SNR and errors purely from the RMS of the spectra can not fully account for other errors like pointing, spectral response, off-axis spaxel calibration, etc.$
$The latter ratio map shows that the missing flux at our working resolution and within the strict mask varies from \sim60\% to as low as 6\% at the brightest pixels. This is consistent with our next method based on simulation.$
$Method (c):$
$We performed detailed simulations of 7m-only $\Remark${CI10} and 12m+7m $\Remark${CO21} observations with the $\incode${CASA} $\incode${simobserve} task. We use the PHANGS-ALMA 12m+7m CO(2-1) clean model cube (one of the output products of $\incode${CASA} $\incode${tclean} runs) as the input $\incode${skymodel} for $\Remark${CO21}, and that scaled by a factor of 0.2 in Kelvin units for $\Remark${CI10} (representing a constant \RCICO=0.2).$
$Note that we do not use the short-spacing-corrected PHANGS-ALMA CO(2-1) cube because it has a finite beam size of \sim1", and simulation based on that may result in more extended emissions. On the contrary, the clean model cube has an infinite resolution, and because both "multiscale" and "singlescale" cleaning are used in the PHANGS-ALMA imaging, the clean model contains point as well as extended sources.$
$We also set the actual observing dates and integration times as the real observations.$
$Then we run $\incode${CASA} $\incode${simobserve} to simulate the visibilities of each observing block, concatenate them together, and process them into image cubes and moment maps in the same way as described in Appendix~\ref{appendix: data reduction}. Meanwhile, by convolving the input $\incode${skymodel} with a 2D Gaussian beam corresponding to our working resolution, and integrating over the spectral axis, we obtain a "true" $\incode${skymodel} moment-0 map which can be directly compared to the imaged moment-0 maps from the simulation. Their ratio map shows a missing flux of \sim50\% to \sim7\% in the strict mask, and more specifically, 24\%, 58\%, 85\% and 96\% pixels selected in our method as for real data have a missing flux < 20\%, < 30\%, < 40\% and < 50\%, respectively.$
$Fig.~\ref{fig: missing flux} illustrates the results from methods (b) (upper panels) and (c) (lower panels). Left panels show the $\Remark${CO21} moment-0 maps, analyzed at the same working resolution as our main analysis in Appendix~\ref{appendix: data reduction}. The simulated data show a fainter emission because we used the clean mode cube as the $\incode${skymodel}, which was made by cleaning the PHANGS-ALMA 12m+7m data down to 1~\sigma within the clean mask voxels. This will indeed lose some fluxes. However, the right panels show that the two methods lead to consistent missing flux results.$
$Note that in the last panel of Fig.~\ref{fig: missing flux}, some regions appear black in the simulation-based analysis because we require line ratios to be analyzed in the strict mask, while these regions are not in it anymore. This is because the $\incode${skymodel} loses some fluxes as mentioned above, thus the strict mask is smaller, and also because in the simulation we assumed a constant \RCICO which may lead to fainter $\CI$  than real data along faint sightlines, e.g., Region 22,  thus further shrinking the strict mask where both lines need to have sufficient \SNR.$
$Bearing these in mind, we conclude that methods (b) and (c) have consistent results on the missing flux, and method (a) is less accurate due to the unconsidered uncertainties.$
$These checks show that the overall missing flux in NGC~3627 is 31\% based on the full- and trimmed-uv $\Remark${CO21} analysis, or \sim36-88\% based on the more-uncertain \textit{Herschel} data. At the pixel level, it varies from 7\% to \sim50\% from bright to faint sightlines at our working resolution of 3.46".$
$These values are as expected given our ACA-only observation setup.$
$We emphasize again that by applying the uv-matching in our data reduction, our $\CI$ /$\CO$  line ratio study is not obviously affected by the missing flux issue, except perhaps in the faintest pixels or "cloud edges".$
$\begin{figure*}$
$\centering$
$\includegraphics[width=\textwidth, trim=0 7mm 0 0, clip]{Plot_missing_flux_NGC3627_CO21_uvtrimmed_to_CO21_full_uvrange}$
$\includegraphics[width=\textwidth, trim=0 0 0 5mm, clip]{Plot_missing_flux_NGC3627_CO21_uvtrimmed_to_CO21_skymodel_from_simulation}$
$\caption{$
$Upper left panel: short-spacing corrected (12m+7m+tp) PHANGS-ALMA CO(2-1) derived moment-0 map.$
$Upper right panel: ratio map of the uv-trimmed CO(2-1) moment-0 map used in this work to that in the left panel.$
$Lower left panel: the CO(2-1) moment-0 map derived from our simulation with uv trimming as for real data.$
$Lower right panel: ratio map of the uv-trimmed CO(2-1) moment-0 map based on the simulation to the "true" $\incode${skymodel} moment-0 map as shown left.$
$All moment maps are derived by first convolving cubes to our working resolution of 3.46" then performing masking analysis as described in Appendix~\ref{appendix: data reduction}, except for the "true" $\incode${skymodel} moment-0 map which is computed without any masking.$
$Note that in the simulation based analysis, some regions have too low \SNR due to their faint CO brightness or insufficient cleaning, thus they are out of the mask and show blank line ratios (instead of very low missing flux) in the lower right panel.$
$\label{fig: missing flux}$
$}$
$\end{figure*}$
$\section{Assessing noise-induced $\CI$  enhancement with simulation}$
$\label{appendix: simulation}$
$In Appendix~\ref{appendix: missing flux} we described our simulation. We also use it to assess how the noise can induce artificial [$\CI$/CO] line ratio variation.$
$In Fig.~\ref{fig: sim line ratio image} we show the line ratio map from the simulation with a constant \RCICO=0.2, and in Fig.~\ref{fig: sim line ratio scatter} we show the scatter plots with simulation results similar to Fig.~\ref{fig: line ratio vs CO}. Interestingly, we see certain $\CI$/CO variations, especially $\CI$  "enhancement" at the edge of galaxy centre's gas clump (region~1), and the edge of the northern spiral arm (Region 14). Nevertheless, the trend is very different from the real data, i.e., Fig.~\ref{fig: line ratio vs CO}.$
$First, galaxy centre has a much weaker [$\CI$/CO] in the real data, whereas in the simulation it is inverted.$
$Second, the scatter is much larger in real data, with a tail of $\CI$  enhanced regions which is not seen in the simulation.$
$Third, from the bottom panels of Fig.~\ref{fig: sim line ratio scatter}, where \SNR is shown, we see that \RCICO is biased to be lower in low-\SNR situation. Thus the high $\CI$  in real data is unlikely to be a bias of low \SNR.$
$\begin{figure*}$
$\centering$
$\includegraphics[width=\textwidth]{Plot_two_line_ratio_two_panel_image_NGC3627_CI10_CO21_with_regions_from_simulation}$
$\caption{$
$\CI  line flux and \RCICO line ratio image from the simulation mimicking real observation but with a constant input line ratio \RCICO=0.2.$
$}$
$\label{fig: sim line ratio image}$
$\end{figure*}$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.9\textwidth]{Plot_Simulation_Line_Ratio_CI10_CO21_vs_Line_Flux_CO21}$
$\includegraphics[width=0.9\textwidth, trim=0 0 0 13mm, clip]{Plot_Simulation_Line_Ratio_CI10_CO21_vs_Line_SNR_CI10}$
$\caption{$
$\textit{Upper panels}: $\Remark${CI10} and $\Remark${CO21} line brightness and ratio similar to Fig.\ref{fig: line ratio vs CO} but from our simulation (with a constant input \RCICO=0.2 as indicated by the horizontal shading in the right panels).$
$\textit{Lower panels}: Similar to the upper panels but showing line \SNR instead of brightness. This illustrates how \RCICO can be biased in low-\SNR situation.$
$}$
$\label{fig: sim line ratio scatter}$
$\end{figure*}$
$\section{$\CI$ and $\CO$ spectra of selected regions in NGC~3627}$
$\label{appendix: all regions}$
$We show the examples of $\Remark${CI10} and $\Remark${CO21} moment-0 maps and spectra for Regions 1, 4 \& 22 in Fig.~\ref{fig: NGC3627 regions}. They represent the NGC~3627 centre, off-centre location and the location at the outer disk of the gaseous spiral arm as discussed in Sect.~\ref{sec: Results}.$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.7\textwidth]{Plot_zoom_in_image_region_1}$
$\includegraphics[width=0.7\textwidth]{Plot_zoom_in_image_region_4}$
$\includegraphics[width=0.7\textwidth]{Plot_zoom_in_image_region_22}$
$\caption{$\Remark${CI10} and $\Remark${CO21} moment-0 maps and spectra of Regions 1, 4 and 22 as newcommandined in Fig.~\ref{fig: NGC3627 line ratio map}. All regions' spectra are available in our online data release (see Sect.~\ref{subsection: ALMA Band 8 Observation}).$
$\label{fig: NGC3627 regions}$
$}$
$\end{figure*}$
$\end{appendix}$
$\end{document}$
$\begin{appendix}$
$\section{Data reduction, imaging and moment map generation}$
$\label{appendix: data reduction}$
$After downloading the raw data, i.e., 12m+7m for $\Remark${CO21} and 7m for $\Remark${CI10}, respectively, we ran the observatory calibration pipeline ("scriptForPI") to reduce the data.$
$As shown in \citet{Leroy2021pipeline}, the observatory calibration produces well-calibrated visibility data without further manual flagging.$
$We then split out the visibility data and proceeded to the imaging step.$
$In order to match the spatial resolution of $\Remark${CO21} and $\Remark${CI10}, we carefully matched their uv ranges.$
$We flagged the $\Remark${CO21} visibility data outside the uv range of 19--68~m (\sim15--52k\lambda) so as to match the uv range of the $\Remark${CI10} data (minimum to 80\% percentile: 8.9--32~m, or \sim15--52k\lambda).$
$Given that the $\Remark${CI10} uv plane distribution is elliptic, while the uv range selection in $\incode${CASA} is circular, our chosen thresholds are adjusted to make the final $\Remark${CO21} synthesized beam similar to that of $\Remark${CI10}.$
$Note that in a later step we also convolved both image cubes to the exact same rounded beam (3.46") before generating the moment maps.$
$The \textit{PHANGS-ALMA imaging \& post-processing pipeline} \footnote{\url{https://github.com/akleroy/phangs_imaging_scripts}; version 2.0.} developed by \citet{Leroy2021pipeline} is used for creating line cubes and moment maps. The code is suitable for general nearby galaxy strong line mapping interferometric and total power mosaic data. In brief, the visibility data are carefully transformed into the user-given channel width and frequency range corresponding to the line plus line-free channels, then continuum is subtracted by fitting an order of zero polynomial. Then, two stages of imaging ("cleaning") are performed: first a multi-scale cleaning down to \sim4\sigma, then a single-scale cleaning down to \sim1\sigma. At each stage, $\incode${CASA}'s $\incode${tclean} task is iteratively called so as to force many major cycles, which has proven to improve the accuracy of the deconvolution. Before and during the second stage cleaning, a clean mask is computed at each iteration according to the outcome of previous iteration by finding all >2\sigma pixels associated with >4\sigma peaks in a watershed algorithm (\citealt{Rosolowsky2006,Rosolowsky2021}). In this way the cleaning can include as much signal as possible and ensure a signal-free residual.$
$Then the image cubes are corrected for the primary beam attenuation, converted to Kelvin units, and convolved to a common beam, which is the coarser beam of $\Remark${CO21} and $\Remark${CI10}. Because of our uv range flagging before imaging, the directly $\incode${tclean}-imaged beam sizes are already close. This convolution further precisely matches $\Remark${CO21} and $\Remark${CI10} image cube beams.$
$Next, moment maps are generated with modules in the PHANGS-ALMA pipeline, consistently for CO and $\CI$ . This process includes firstly constructing noise and mask cubes from the image cubes, then combining the masks (i.e., where science signal should present), then computing moment 0 -- line integrated intensity, moment 1 -- line peak velocity, and moment 2 -- line width. The mask cubes are the key in producing moment maps. Here two masks are built: a "broad mask" which contains as much plausible science signal as possible, and a "strict mask" which is optimized for high-confidence signal. The strict mask is made by finding peaks that have >4\sigma over two consecutive channels then expanding down to 2\sigma also with the watershed algorithm. Then the broad mask is obtained by aggregating strict masks made from image cubes convolved to coarser resolutions (\sim30" in this work as well as in \citealt{Leroy2021pipeline}).$
$To make consistent moment maps for CO and $\CI$ , we aggregate their masks together as a joint mask, for broad and strict masks, respectively, then their final moment maps are computed within the joint mask.$
$The resulting $\Remark${CO21} and $\Remark${CI10} line intensity maps are then used for our next line ratio analysis. The line velocity and line width maps are not further analyzed, but are consistent with the products from the PHANGS-ALMA survey's data release as well as in \citet{Lang2020}. Given that the PHANGS-ALMA 12m+7m+tp $\Remark${CO21} has the full spatial information and are deeper, their line velocity and line width maps are more useful for kinematic studies than ours produced from this work.$
$\section{Assessing the missing flux in the NGC~3627 $\CI$  data}$
$\label{appendix: missing flux}$
$We assess the level of missing flux in the NGC~3627 $\CI$  data in three ways:$
$(a) comparing with the \textit{Herschel} SPIRE FTS $\Remark${CI10} observation in this galaxy,$
$(b) comparing the uv-trimmed CO(2-1) moment-0 map with the PHANGS-ALMA short-spacing corrected (12m+7m+tp) moment-0 map,$
$and (c) performing CASA simulation and comparing with the "true" moment-0 from the skymodel and the derived product from the simulated visibilities.$
$Method (a):$
$NGC~3627 has been observed with the \textit{Herschel} SPIRE (\citealt{Griffin2010}) Fourier Transform Spectrometer (FTS; \citealt{Naylor2010}) in mapping mode with nearly Nyquist sampling.$
$\cite{Liudz2015} reduced the archival data and extracted the line fluxes for each groups of pointings. $\Remark${CI10} is detected with \SNR\sim4-5 at three sightlines from the galaxy centre to the southern bar end, with R.A. and Dec. coordinates (J2000) 170.061905, +12.991336, 170.066361 +12.987091, and 170.071243 +12.981615, respectively. The $\Remark${CI10} luminosities are 1.44 \pm 0.31, 2.01 \pm 0.48, and 1.71 \pm 0.40 \times 10^{7} \; \mathrm{K km s^{-1} pc^{2}}, respectively, where the errors are purely from the RMS of line-free FTS spectra. By convolving our $\Remark${CI10} line cube to the FTS beam of \sim39" then extracting the moment maps, we measure a $\Remark${CI10} line luminosity of 0.52, 0.33 and 0.29 \times 10^{7} \; \mathrm{K km s^{-1} pc^{2}} at the above three positions, respectively, which means a missing flux of 36-75\% at the galaxy centre and 68-88\% at the southern bar end (at a 95\% confidence level).$
$Method (b):$
$We made ratio maps of our uv-trimmed CO(2-1) moment-0 map, and that of the original PHANGS-ALMA short-spacing-corrected (12m+7m+tp) CO(2-1) data but convolved to the corresponding resolution before making the moment maps. The ratio maps are made at two resolutions: the FTS 39" beam, and our working resolution of 3.46". The former leads to a missing flux of 26\%, 31\% and 36\% from the galaxy centre to the southern bar end. Note that the above \textit{Herschel} FTS analysis indicates in general a higher missing flux, but the FTS measurements are of low \SNR and errors purely from the RMS of the spectra can not fully account for other errors like pointing, spectral response, off-axis spaxel calibration, etc.$
$The latter ratio map shows that the missing flux at our working resolution and within the strict mask varies from \sim60\% to as low as 6\% at the brightest pixels. This is consistent with our next method based on simulation.$
$Method (c):$
$We performed detailed simulations of 7m-only $\Remark${CI10} and 12m+7m $\Remark${CO21} observations with the $\incode${CASA} $\incode${simobserve} task. We use the PHANGS-ALMA 12m+7m CO(2-1) clean model cube (one of the output products of $\incode${CASA} $\incode${tclean} runs) as the input $\incode${skymodel} for $\Remark${CO21}, and that scaled by a factor of 0.2 in Kelvin units for $\Remark${CI10} (representing a constant \RCICO=0.2).$
$Note that we do not use the short-spacing-corrected PHANGS-ALMA CO(2-1) cube because it has a finite beam size of \sim1", and simulation based on that may result in more extended emissions. On the contrary, the clean model cube has an infinite resolution, and because both "multiscale" and "singlescale" cleaning are used in the PHANGS-ALMA imaging, the clean model contains point as well as extended sources.$
$We also set the actual observing dates and integration times as the real observations.$
$Then we run $\incode${CASA} $\incode${simobserve} to simulate the visibilities of each observing block, concatenate them together, and process them into image cubes and moment maps in the same way as described in Appendix~\ref{appendix: data reduction}. Meanwhile, by convolving the input $\incode${skymodel} with a 2D Gaussian beam corresponding to our working resolution, and integrating over the spectral axis, we obtain a "true" $\incode${skymodel} moment-0 map which can be directly compared to the imaged moment-0 maps from the simulation. Their ratio map shows a missing flux of \sim50\% to \sim7\% in the strict mask, and more specifically, 24\%, 58\%, 85\% and 96\% pixels selected in our method as for real data have a missing flux < 20\%, < 30\%, < 40\% and < 50\%, respectively.$
$Fig.~\ref{fig: missing flux} illustrates the results from methods (b) (upper panels) and (c) (lower panels). Left panels show the $\Remark${CO21} moment-0 maps, analyzed at the same working resolution as our main analysis in Appendix~\ref{appendix: data reduction}. The simulated data show a fainter emission because we used the clean mode cube as the $\incode${skymodel}, which was made by cleaning the PHANGS-ALMA 12m+7m data down to 1~\sigma within the clean mask voxels. This will indeed lose some fluxes. However, the right panels show that the two methods lead to consistent missing flux results.$
$Note that in the last panel of Fig.~\ref{fig: missing flux}, some regions appear black in the simulation-based analysis because we require line ratios to be analyzed in the strict mask, while these regions are not in it anymore. This is because the $\incode${skymodel} loses some fluxes as mentioned above, thus the strict mask is smaller, and also because in the simulation we assumed a constant \RCICO which may lead to fainter $\CI$  than real data along faint sightlines, e.g., Region 22,  thus further shrinking the strict mask where both lines need to have sufficient \SNR.$
$Bearing these in mind, we conclude that methods (b) and (c) have consistent results on the missing flux, and method (a) is less accurate due to the unconsidered uncertainties.$
$These checks show that the overall missing flux in NGC~3627 is 31\% based on the full- and trimmed-uv $\Remark${CO21} analysis, or \sim36-88\% based on the more-uncertain \textit{Herschel} data. At the pixel level, it varies from 7\% to \sim50\% from bright to faint sightlines at our working resolution of 3.46".$
$These values are as expected given our ACA-only observation setup.$
$We emphasize again that by applying the uv-matching in our data reduction, our $\CI$ /$\CO$  line ratio study is not obviously affected by the missing flux issue, except perhaps in the faintest pixels or "cloud edges".$
$\begin{figure*}$
$\centering$
$\includegraphics[width=\textwidth, trim=0 7mm 0 0, clip]{Plot_missing_flux_NGC3627_CO21_uvtrimmed_to_CO21_full_uvrange}$
$\includegraphics[width=\textwidth, trim=0 0 0 5mm, clip]{Plot_missing_flux_NGC3627_CO21_uvtrimmed_to_CO21_skymodel_from_simulation}$
$\caption{$
$Upper left panel: short-spacing corrected (12m+7m+tp) PHANGS-ALMA CO(2-1) derived moment-0 map.$
$Upper right panel: ratio map of the uv-trimmed CO(2-1) moment-0 map used in this work to that in the left panel.$
$Lower left panel: the CO(2-1) moment-0 map derived from our simulation with uv trimming as for real data.$
$Lower right panel: ratio map of the uv-trimmed CO(2-1) moment-0 map based on the simulation to the "true" $\incode${skymodel} moment-0 map as shown left.$
$All moment maps are derived by first convolving cubes to our working resolution of 3.46" then performing masking analysis as described in Appendix~\ref{appendix: data reduction}, except for the "true" $\incode${skymodel} moment-0 map which is computed without any masking.$
$Note that in the simulation based analysis, some regions have too low \SNR due to their faint CO brightness or insufficient cleaning, thus they are out of the mask and show blank line ratios (instead of very low missing flux) in the lower right panel.$
$\label{fig: missing flux}$
$}$
$\end{figure*}$
$\section{Assessing noise-induced $\CI$  enhancement with simulation}$
$\label{appendix: simulation}$
$In Appendix~\ref{appendix: missing flux} we described our simulation. We also use it to assess how the noise can induce artificial [$\CI$/CO] line ratio variation.$
$In Fig.~\ref{fig: sim line ratio image} we show the line ratio map from the simulation with a constant \RCICO=0.2, and in Fig.~\ref{fig: sim line ratio scatter} we show the scatter plots with simulation results similar to Fig.~\ref{fig: line ratio vs CO}. Interestingly, we see certain $\CI$/CO variations, especially $\CI$  "enhancement" at the edge of galaxy centre's gas clump (region~1), and the edge of the northern spiral arm (Region 14). Nevertheless, the trend is very different from the real data, i.e., Fig.~\ref{fig: line ratio vs CO}.$
$First, galaxy centre has a much weaker [$\CI$/CO] in the real data, whereas in the simulation it is inverted.$
$Second, the scatter is much larger in real data, with a tail of $\CI$  enhanced regions which is not seen in the simulation.$
$Third, from the bottom panels of Fig.~\ref{fig: sim line ratio scatter}, where \SNR is shown, we see that \RCICO is biased to be lower in low-\SNR situation. Thus the high $\CI$  in real data is unlikely to be a bias of low \SNR.$
$\begin{figure*}$
$\centering$
$\includegraphics[width=\textwidth]{Plot_two_line_ratio_two_panel_image_NGC3627_CI10_CO21_with_regions_from_simulation}$
$\caption{$
$\CI  line flux and \RCICO line ratio image from the simulation mimicking real observation but with a constant input line ratio \RCICO=0.2.$
$}$
$\label{fig: sim line ratio image}$
$\end{figure*}$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.9\textwidth]{Plot_Simulation_Line_Ratio_CI10_CO21_vs_Line_Flux_CO21}$
$\includegraphics[width=0.9\textwidth, trim=0 0 0 13mm, clip]{Plot_Simulation_Line_Ratio_CI10_CO21_vs_Line_SNR_CI10}$
$\caption{$
$\textit{Upper panels}: $\Remark${CI10} and $\Remark${CO21} line brightness and ratio similar to Fig.\ref{fig: line ratio vs CO} but from our simulation (with a constant input \RCICO=0.2 as indicated by the horizontal shading in the right panels).$
$\textit{Lower panels}: Similar to the upper panels but showing line \SNR instead of brightness. This illustrates how \RCICO can be biased in low-\SNR situation.$
$}$
$\label{fig: sim line ratio scatter}$
$\end{figure*}$
$\section{$\CI$ and $\CO$ spectra of selected regions in NGC~3627}$
$\label{appendix: all regions}$
$We show the examples of $\Remark${CI10} and $\Remark${CO21} moment-0 maps and spectra for Regions 1, 4 \& 22 in Fig.~\ref{fig: NGC3627 regions}. They represent the NGC~3627 centre, off-centre location and the location at the outer disk of the gaseous spiral arm as discussed in Sect.~\ref{sec: Results}.$
$\begin{figure*}$
$\centering$
$\includegraphics[width=0.7\textwidth]{Plot_zoom_in_image_region_1}$
$\includegraphics[width=0.7\textwidth]{Plot_zoom_in_image_region_4}$
$\includegraphics[width=0.7\textwidth]{Plot_zoom_in_image_region_22}$
$\caption{$\Remark${CI10} and $\Remark${CO21} moment-0 maps and spectra of Regions 1, 4 and 22 as newcommandined in Fig.~\ref{fig: NGC3627 line ratio map}. All regions' spectra are available in our online data release (see Sect.~\ref{subsection: ALMA Band 8 Observation}).$
$\label{fig: NGC3627 regions}$
$}$
$\end{figure*}$
$\end{appendix}$
$\end{document}}$
$\newcommand{\}{hyper}$
$\newcommand{\}{hyper}$
$\newcommand{\}{hyper}$
$\newcommand{\}{hyper}$</div>



<div id="title">

#  pc Scales

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.09661-b31b1b.svg)](https://arxiv.org/abs/2212.09661)<mark>Appeared on: 2022-12-20</mark> - _23 pages, 13 figures and one table in total (17 pages and 9 figures in main text). Accepted for publication in A&A. For associated data cubes and moment maps, see this https URL_

</div>
<div id="authors">

<mark><mark>Daizhong Liu</mark></mark>, et al. -- incl., <mark><mark>Eva Schinnerer</mark></mark>, <mark><mark>Frank Bigiel</mark></mark>, <mark><mark>Annie Hughes</mark></mark>, <mark><mark>Kathryn Kreckel</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present new neutral atomic carbon[C$\textsc${i}]($ ^3P_1 \to  ^3P_0$) mapping observations within the inner$\sim 7$kpc and$\sim 4$kpc of the disks of NGC 3627 and NGC 4321 at a spatial resolution of 190 pc and 270 pc, respectively, using the Atacama Large Millimeter/Submillimeter Array (ALMA) Atacama Compact Array (ACA).We combine these with the CO(2--1) data from PHANGS-ALMA, and literature[C$\textsc${i}]and CO data for two other starburst and/or active galactic nucleus (AGN) galaxies (NGC 1808, NGC 7469), to study:\textit{a)}the spatial distributions of C$\textsc${i}and CO emission;\textit{b)}the observed line ratio$\RCICO = I_{\mathrm{[C{\normalfont$\textsc${i}}]}(1\textnormal{--}0)} / I_{\mathrm{CO}(2\textnormal{--}1)}$as a function of various galactic properties; and\textit{c)}the abundance ratio of[C$\textsc${i}/CO].We find excellent spatial correspondence between C$\textsc${i}and CO emission and nearly uniform$\RCICO \sim 0.1$across the majority of the star-forming disks of NGC 3627 and NGC 4321.However,$\RCICO$strongly varies from$\sim 0.05$at the centre of NGC 4321 to$> 0.2 \text{--} 0.5$in NGC 1808's starbursting centre and NGC 7469's centre with an X-ray-luminous active galactic nucleus (AGN).Meanwhile,$\RCICO $does not obviously vary with$\Umean$, similar to the prediction of photodissociation dominated region (PDR) models.We also find a mildly decreasing$\RCICO $with an increasing metallicity over$0.7\text{--}0.85 \mathrm{Z_{\odot}}$, consistent with the literature.Assuming various typical ISM conditions representing giant molecular clouds, active star-forming regions and strong starbursting environments, we calculate the local-thermodynamic-equilibrium radiative transfer and estimate the[C$\textsc${i}/CO]abundance ratio to be$\sim 0.1$across the disks of NGC 3627 and NGC 4321, similar to previous large-scale findings in Galactic studies.However, this abundance ratio likely has a substantial increase to$\sim 1$and$\gtrsim 1$--5 in NGC 1808's starburst and NGC 7469's strong AGN environments, respectively, in line with the expectations for cosmic-ray dominated region (CRDR) and X-ray dominated region (XDR) chemistry.Finally, we do not find a robust evidence for a generally CO-dark, C$\textsc${i}-bright gas in the disk areas we probed.

</div>

<div id="div_fig1">

<img src="tmp_2212.09661/./Plot_Line_Ratio_CI10_CO21_New_surface_density.png" alt="Fig4" width="100%"/>

**Figure 4. -** \textit{Left}: $\Remark{ICI10}$ versus $\Remark{ICO21}$ in the four galaxies we studied: NGC 3627, NGC 4321, NGC 1808 and NGC 7469. $\Remark{ICI10}$ and $\Remark{ICO21}$ integrated fluxes are moment-0 intensities extracted from their beam and pixel size matched data cubes. Only data points with propagated moment-0 $\SNR \ge 5$ are shown.
        The blue (with crossed hatch), green (with horizontal line hatch), gray (with dotted hatch) and light gray shadings indicate the $5 \sigma$ thresholds for the four galaxies, respectively. Typical line widths of 10 km s$^{-1}$(NGC 3627 and NGC 4321) and 30 km s$^{-1}$(NGC 1808 and NGC 7469) in sigma are used to calculate the thresholds together with the channel rms in Table \ref{table: targets}.
        For NGC 3627 and NGC 4321, their ($uv$-matched) CO(2--1) $5 \sigma$ thresholds are out of the plotting range.
        Top axis shows the simply-converted H$_2$ surface density using a constant $\alphaCO = 4.3$ and CO excitation (i.e., CO(2--1) to CO(1--0) line ratio) $R_{21} = 0.65$.
        \textit{Right}: Same data but shown as the line ratio $\RCICO = $\Remark${ICI10}/$\Remark${ICO21}$ versus $\Remark{ICO21}$.
        The blue horizontal line indicates a star-forming (SF) disk-like $\RCICO \sim 0.1$, i.e., the spiral arms of NGC 3627 and NGC 4321. The gray horizontal line indicates a starburst (SB) disk-like $\RCICO \sim 0.2$, i.e., the $\sim 1$ kpc SB disk/ring in NGC 1808 and NGC 7469.
        The red line with a slope of 0.8 in logarithm ($0.035   (I_{$\Remark${CO21}}/[100 $\Kkms$])^{0.8}$) visually guides the trend of increasing $\RCICO$ in the central regions of these galaxies (see boxes in Figs. \ref{fig: NGC3627 line ratio map} to \ref{fig: other line ratio maps}, sizes $\sim 300$--500 pc).
     (*fig: line ratio vs CO*)

</div>
<div id="div_fig2">

<img src="tmp_2212.09661/./Plot_Line_Ratio_CI10_CO21_New_surface_density.png" alt="Fig13" width="100%"/>

**Figure 13. -** \textit{Left}: $\Remark{ICI10}$ versus $\Remark{ICO21}$ in the four galaxies we studied: NGC 3627, NGC 4321, NGC 1808 and NGC 7469. $\Remark{ICI10}$ and $\Remark{ICO21}$ integrated fluxes are moment-0 intensities extracted from their beam and pixel size matched data cubes. Only data points with propagated moment-0 $\SNR \ge 5$ are shown.
        The blue (with crossed hatch), green (with horizontal line hatch), gray (with dotted hatch) and light gray shadings indicate the $5 \sigma$ thresholds for the four galaxies, respectively. Typical line widths of 10 km s$^{-1}$(NGC 3627 and NGC 4321) and 30 km s$^{-1}$(NGC 1808 and NGC 7469) in sigma are used to calculate the thresholds together with the channel rms in Table \ref{table: targets}.
        For NGC 3627 and NGC 4321, their ($uv$-matched) CO(2--1) $5 \sigma$ thresholds are out of the plotting range.
        Top axis shows the simply-converted H$_2$ surface density using a constant $\alphaCO = 4.3$ and CO excitation (i.e., CO(2--1) to CO(1--0) line ratio) $R_{21} = 0.65$.
        \textit{Right}: Same data but shown as the line ratio $\RCICO = $\Remark${ICI10}/$\Remark${ICO21}$ versus $\Remark{ICO21}$.
        The blue horizontal line indicates a star-forming (SF) disk-like $\RCICO \sim 0.1$, i.e., the spiral arms of NGC 3627 and NGC 4321. The gray horizontal line indicates a starburst (SB) disk-like $\RCICO \sim 0.2$, i.e., the $\sim 1$ kpc SB disk/ring in NGC 1808 and NGC 7469.
        The red line with a slope of 0.8 in logarithm ($0.035   (I_{$\Remark${CO21}}/[100 $\Kkms$])^{0.8}$) visually guides the trend of increasing $\RCICO$ in the central regions of these galaxies (see boxes in Figs. \ref{fig: NGC3627 line ratio map} to \ref{fig: other line ratio maps}, sizes $\sim 300$--500 pc).
     (*fig: line ratio vs CO*)

</div>
<div id="div_fig3">

<img src="tmp_2212.09661/./Plot_LTE_RCI10CO21_vs_XCICO_varying_N_CO_T_kin.png" alt="Fig8" width="100%"/>

**Figure 8. -** 
        Observed/calculated $\RCICO$ as a function of CO column density $\CO$columndensity  and [$\CI$/CO] abundance (column density) ratio $\RNCINCO$ for 7 representative ISM conditions, based on LTE calculations.
        \textit{Upper panels}: Curved lines are $\RCICO$ versus $\RNCINCO$ for different $\CO$columndensity  as indicated by the color. Dashed means CO(2--1) optically thick and $\Remark{CI10}$ optically thin, dotted means both are both optically thin, and solid line means both optically thick.
        Horizontal thick line is the observed $\RCICO$ value.
        \textit{Lower panels}: Black contours are $\RCICO = 0.05,   0.10,   0.20$ and $0.50$ in the CO column density versus $\CI$/CO abundance ratio plane.
        Vertical color shading indicates the final $\RNCINCO$ that can match both the observed $\RCICO $ and the assumed $\Sigmol$ value noted at the top of each panel.
     (*fig: LTE RCICO*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\kms}{\ifmmode \mbox{\rm km s}^{-1}$
$	\else{\rm km s}^{-1}\ignorespaces$
$	\fi}$
$\newcommand{\mum}{\ifmmode \mu\mbox{\rm m}$
$	\else \mu{\rm m}$
$	\fi}$
$\newcommand{\Htwo}{\ifmmode \mbox{\rm H}$
$	\else{\rm H}_2$
$	\fi}$
$\newcommand{\OHp}{\ifmmode \mbox{\rm OH}$
$	\else{\rm OH^+}$
$	\fi}$
$\newcommand{\H2Op}{\ifmmode \mbox{\rm H}_2{\rm O}$
$	\else{\rm H}_2{\rm O}^+$
$	\fi}$
$\newcommand{\CHp}{\ifmmode \mbox{\rm CH}$
$	\else{\rm CH}^+$
$	\fi}$
$\newcommand{\Hp}{\ifmmode \mbox{\rm H}$
$	\else{\rm H}^+$
$	\fi}$
$\newcommand{\Hthreep}{\ifmmode \mbox{\rm H}$
$	\else{\rm H}^+_3$
$	\fi}$
$\newcommand{\}{mHI}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\kms}{\ifmmode \mbox{\rm km s}^{-1}$
$	\else{\rm km s}^{-1}\ignorespaces$
$	\fi}$
$\newcommand{\mum}{\ifmmode \mu\mbox{\rm m}$
$	\else \mu{\rm m}$
$	\fi}$
$\newcommand{\Htwo}{\ifmmode \mbox{\rm H}$
$	\else{\rm H}_2$
$	\fi}$
$\newcommand{\OHp}{\ifmmode \mbox{\rm OH}$
$	\else{\rm OH^+}$
$	\fi}$
$\newcommand{\H2Op}{\ifmmode \mbox{\rm H}_2{\rm O}$
$	\else{\rm H}_2{\rm O}^+$
$	\fi}$
$\newcommand{\CHp}{\ifmmode \mbox{\rm CH}$
$	\else{\rm CH}^+$
$	\fi}$
$\newcommand{\Hp}{\ifmmode \mbox{\rm H}$
$	\else{\rm H}^+$
$	\fi}$
$\newcommand{\Hthreep}{\ifmmode \mbox{\rm H}$
$	\else{\rm H}^+_3$
$	\fi}$
$\newcommand{\}{mHI}$</div>



<div id="title">

#  Unobscured QSO Hosts Driven by Star Formation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.09675-b31b1b.svg)](https://arxiv.org/abs/2212.09675)<mark>Appeared on: 2022-12-20</mark> - _19 pages, 7 figures, 3 tables_

</div>
<div id="authors">

Kirsty M. Butler, et al. -- incl., <mark>Fabian Walter</mark>

</div>
<div id="abstract">

**Abstract:** Feedback and outflows in galaxies that are associated with a quasar phase are expected to be pivotal in quenching the most massive galaxies. However, observations targeting the molecular outflow phase, which dominates both the mass and momentum and removes the immediate fuel for star formation, are limited in high--z QSO hosts. Massive quiescent galaxies found at$z\sim4$are predicted to have already quenched star formation by$z\sim5$and undergone their most intense growth at$z>6$. Here, we present two ALMA detections of molecular outflows, traced by blue--shifted absorption of the OH 119\mumdoublet, from a sample of three$z>6$infrared luminous QSO hosts: J2310+1855 and P183+05. OH 119\mumis also detected in emission in P183+05, and tentatively in the third source: P036+03. Using similar assumptions as for high--z Dusty Star--Forming Galaxy outflows, we find that our QSOs drive molecular outflows with comparable mass outflow rates, and that are comparably energetic except for J2310+1855's significantly lower outflow energy flux. We do not find evidence, nor require additional input from the central AGN to drive the molecular outflow in J2310+1855 but can not rule out an AGN contribution in P183+05 if a significant AGN contribution to$\rm L_{FIR}$is assumed and/or if the outflow covering fraction is high ($\geq53\%$), which evidence from the literature suggests is unlikely in these sources. Differences observed in the blue--shifted absorption spectral properties may instead be caused by the QSO hosts' more compact dust continuum, limiting observations to lower altitude and more central regions of the outflow.

</div>

<div id="div_fig1">

<img src="tmp_2212.09675/./figures/ParGridNoInc_FIR.png" alt="Fig6" width="100%"/>

**Figure 6. -** Grid of host galaxy and best fit OH 119 \mum absorption spectra properties. This includes: $\mathbf{v_{50}}$: mean velocity, $\mathbf{v_{84}}$: velocity above which 84\% of the absorption lies, $\mathbf{v_{max}}$: velocity above which 98\% of the absorption lies, \textbf{FWHM:} full width half maximum of the absorption, \textbf{EW:} equivalent width of the absorption, $\mathbf{L_{IR}}$: Far Infrared luminosity of the host galaxy, $\mathbf{\Sigma_{FIR}}$: Infrared surface brightness of the host galaxy given by $\rm L_{FIR}/(2\pi R_{1/2,cont}^2)$, $\mathbf{R_{cont}}$: Effective radius of the 119\mum continuum emission. We show low--z sources in grey, including both star forming and AGN host galaxies \citep{Veilleux2013}. The spectral properties are measurements of the total (systemic and blue shifted) absorption lines. In green we show the high--z DSFG sample studied by \cite{Spilker2020a,Spilker2020b}, further separating this sample into measurements of the total (systemic and blue shifted) absorption line (dark green crossed--circles), just the systemic components (dark green open circles) and just the outflowing components (light green filled). Sources with only an outflowing component will appear as a light green dot with a dark green outline. We display a fit to the outflow--only components in each panel (illustrative only) with a solid line and shade the $1\sigma$ scatter, both in light green. We include the tentative OH detection in the z = 6.13 QSO, ULAS J131911+095051 (\citealt{HerreraCamus2020}, purple triangle). J2310+1855 and P183+05 are shown by the filled orange square and triangle, respectively.). \label{fig:ParGrid} (*fig:ParGrid*)

</div>
<div id="div_fig2">

<img src="tmp_2212.09675/./figures/FIR0.5vsENoInc.png" alt="Fig3" width="100%"/>

**Figure 3. -** Energy (top) and momentum (bottom) flux as a function of $\rm L_{FIR}$(bottom axis) and SFR (top axis) derived for the comparison sample of high--z DSFGs \citep{Spilker2020a,Spilker2020b}(green) and J2310+1855 and P183+05 (orange squares and triangles, respectively). Filled symbols indicate upper limits, assuming an outflow covering fraction of 1. Hollow symbols indicate lower limits for J2310+1855 and P183+05, assuming a covering fraction equal to the fractional OH absorption depth (25\% and 9.4\%, respectively), and values derived assuming an average covering fraction of 0.3 \citep{GonzalezAlfonso2017}, for the DSFG sample. The smaller faint orange symbols indicate ranges for J2310+1855 and P183+05 assuming a 50\% AGN contribution to $\rm L_{FIR}$(see text). We indicate regions in the top panel where the energy injected via star formation, assuming a maximum coupling fraction to the ISM of 40\%\citep{Sharma2014,Fielding2018}, is equal to the energy flux of the outflow (white), where an unusually high coupling fraction or AGN contribution is required (light grey), and where an AGN is definitely required (darker grey). We indicate regions in the bottom panel where SNe ejecta provide sufficient momentum (white) and where radiation pressure on dust grains in an optically thin (light grey) or thick (darker grey) outflow is needed. \label{fig:FIRvsE} (*fig:FIRvsE*)

</div>
<div id="div_fig3">

<img src="tmp_2212.09675/./figures/Cartoon.png" alt="Fig4" width="100%"/>

**Figure 4. -** Idealised schematic of two equally luminous disky galaxies (left and right) seen at four different inclinations (from the top: $0^\circ$, $30^\circ$, $60^\circ$ and $90^\circ$). We consider a clumpy outflow ejected from the center of the galaxy, travelling radially in five angles covering the left hand side of the disk, and vertically through a galactic fountain trajectory. On the right we show this trajectory as a solid black arrow. Orange circles indicate clumps that intervene the observer's line of site and the background continuum, and thus contribute to the absorption signature of the outflow. Grey circles indicate when clumps do not intervene and are invisible to the observer via absorption.  \label{fig:cartoon} (*fig:cartoon*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

141  publications in the last 7 days.
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers